# [Датафреймы](https://drive.google.com/drive/folders/13K24GVmATfSxGF0K0LcWFHBiLxJRuRyN?usp=sharing)

*   [Headliner_df](https://drive.google.com/file/d/1-86JPhdac5Y-YIggqYWVh9rFCDoe8Uej/view?usp=sharing)
*   [Primavera_df](https://drive.google.com/file/d/16vejQ6aIN1VprZ8RuaG0Dx4SSj3eebbP/view?usp=sharing)
*   [Paveletskaya_city_df](https://drive.google.com/file/d/1-0RLElus9y3f3xJ7PovfTDFoLh_2HnPK/view?usp=sharing)
*   [Paveletskaya_df](https://drive.google.com/file/d/1-3V7Dyjq-N3m0dc-7FztObG4PeRrMfHl/view?usp=sharing)
*   [total_df](https://drive.google.com/file/d/18LdqvkasD_NMNbB025jQDyWmuTWfXrin/view?usp=sharing)
*   [total_not_match_df](https://drive.google.com/file/d/1-0C7m74ofUqLCH4C-eX3ARZMWRLOQeMJ/view?usp=sharing)
*   [total_match_df](https://drive.google.com/file/d/1-JerYBfZ95fZX698-N25-gC1HNC0jwTy/view?usp=sharing)

# [Таблицы](https://drive.google.com/drive/folders/1Wr3rQQkQtTuTFMRSdAbz1rMPHxdHCeLZ?usp=sharing)

*   [Записи звонков без информации в csv-файлах.xlsx](https://docs.google.com/spreadsheets/d/1-5hKNi2yH7Vy94H_LatiNmscocsK7U5m/edit?usp=sharing&ouid=103791142604649006389&rtpof=true&sd=true)
*   [Информация о звонках в csv-файлах без файлов записи звонков.xlsx](https://docs.google.com/spreadsheets/d/1-DpzEGSH02laE7cKinwxNzPj2zhXgN6J/edit?usp=sharing&ouid=103791142604649006389&rtpof=true&sd=true)
*   [Информация о звонках в csv-файлах с файлами записи звонков.xlsx](https://docs.google.com/spreadsheets/d/1-QR-Xng6RHZGsJCRVQQP3jCosBAQAIIF/edit?usp=sharing&ouid=103791142604649006389&rtpof=true&sd=true)

#[Документы](https://drive.google.com/drive/folders/1uHU0xgW0ztXk4gRE5qKr_rhnPNElqtJ0?usp=sharing)

[Проект письма заказчику](https://docs.google.com/document/d/1gO_o9NdiKts4QKQUXQJxy6pVkx40aM0uhf1DYFfudoM/edit?usp=sharing)

# 3 неделя

In [ ]:
!rm -r sample_data

In [ ]:
# Подключаем google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


## Библиотеки и переменные

In [ ]:
import os            # для работы с файловой системой
import pickle as pkl # для сохранения и загрузки переменных
import pandas as pd  # для работы с таблицами

# Параметры отображения датафреймов
pd.options.display.max_rows     = 99
pd.options.display.max_colwidth = 999

df_path           = my_drv_path + 'media108.ru/Датафреймы/'                             # путь к папке для датафреймов
table_path        = my_drv_path + 'media108.ru/Таблицы/'                                # путь к папке для таблиц
dataset_info_path = my_drv_path + 'media108.ru/Информация о звонках от Романа Петрова/' # путь к информации о звонках от Романа Петрова

# Пути к файлам unmatched_records
Paveletskaya_city_unmatched_path   = dataset_info_path + 'Paveletskaya_city_unmatched_records.txt'
Primavera_unmatched_path   = dataset_info_path + 'Primavera_unmatched_records.txt'

## Создание сводной таблицы записей звонков, о которых нет информации в csv-файлах

### Функции

In [ ]:
# Функция для создания датафрейма из списка записей звонков
def calls_list_to_df(calls_list):

  datetime_list    = [] # список для 'Дата и время'
  from_number_list = [] # список для 'Номер абонента'
  to_number_list   = [] # список для 'Номер сотрудника'
  session_list     = [] # список для 'Идентификатор сессии звонка'

  # Проходим по списку записей звонков
  for call in calls_list:

    call_item_list = call.split('_')
    time_item_list = call_item_list[1].split('.')
    time_item_list = time_item_list[0].split('-')

    # Заполняем списки для столбцов
    datetime_list.append(call_item_list[0] + ' ' + time_item_list[0] + ':' + time_item_list[1] + ':' + time_item_list[2])
    from_number_list.append(call_item_list[3])
    to_number_list.append(call_item_list[5])
    session_list.append(call_item_list[7])

  # Собираем созданные списки в словарь
  calls_dict = dict.fromkeys(['Дата и время', 'Номер абонента', 'Номер сотрудника', 'Идентификатор сессии звонка', 'Файл записи звонка'], [])
  calls_dict['Дата и время']                = datetime_list
  calls_dict['Номер абонента']              = from_number_list
  calls_dict['Номер сотрудника']            = to_number_list
  calls_dict['Идентификатор сессии звонка'] = session_list
  calls_dict['Файл записи звонка']          = calls_list

  # Возвращаем датафрейм
  return pd.DataFrame(calls_dict)

### Загрузка датафреймов со звонками, о которых нет информации в csv-файлах

In [ ]:
# Загружаем Headliner_not_match_df из файла
with open(df_path + 'Headliner_not_match_df.pkl', 'rb') as f:
  Headliner_not_match_df = pkl.load(f)

# Загружаем Primavera_not_match_df из файла
with open(df_path + 'Primavera_not_match_df.pkl', 'rb') as f:
  Primavera_not_match_df = pkl.load(f)

# Загружаем Paveletskaya_city_not_match_df из файла
with open(df_path + 'Paveletskaya_city_not_match_df.pkl', 'rb') as f:
  Paveletskaya_city_not_match_df = pkl.load(f)

### Создание датафреймов из файлов от Романа Петрова

In [ ]:
# Смотрим содержимое папки с информацией о звонках
os.listdir(dataset_info_path)

['Павелецкая (pavcity.turbo.site).csv',
 'Headliner.csv',
 'Павелецкая_ЖК_«Павелецкая_сити».csv',
 'Primavera.csv',
 '.ipynb_checkpoints',
 'Headliner_unmatched_records.txt',
 'Headliner_no_audio.csv',
 'Headliner_with_audio.csv',
 'Primavera_unmatched_records.txt',
 'Primavera_no_audio.csv',
 'Primavera_with_audio.csv']

In [ ]:
# Читаем файл 'Headliner_unmatched_records.txt'
with open(Headliner_unmatched_path, 'r') as f:

  # Список строк
  lines = f.readlines()

  print(f"Файл: {Headliner_unmatched_path}")
  print(f"Количество строк: {len(lines)}")
  print(f"Первые 10 строк:")
  for line in lines[:10]:
    print(line, end='')

Файл: /content/drive/MyDrive/media108.ru/Информация о звонках от Романа Петрова/Headliner_unmatched_records.txt
Количество строк: 0
Первые 10 строк:


In [ ]:
# Читаем файл 'Primavera_unmatched_records.txt'
with open(Primavera_unmatched_path, 'r') as f:

  # Список строк
  lines = f.readlines()

  print(f"Файл: {Primavera_unmatched_path}")
  print(f"Количество строк: {len(lines)}")
  print(f"Первые 10 строк:")
  for line in lines[:10]:
    print(line, end='')

Файл: /content/drive/MyDrive/media108.ru/Информация о звонках от Романа Петрова/Primavera_unmatched_records.txt
Количество строк: 466
Первые 10 строк:
2023-04-28_10-58-01.443973_from_79168623705_to_0111642_session_2901657750_talk.mp3
2023-04-07_15-14-26.286472_from_79035861746_to_0197872_session_2865992215_talk.mp3
2023-04-11_14-45-51.943888_from_79852229315_to_0111642_session_2868761754_talk.mp3
2023-04-07_18-25-46.218872_from_79165050055_to_0111639_session_2867030845_talk.mp3
2023-04-09_17-30-35.871150_from_79671795235_to_0150113_session_2869317077_talk.mp3
2023-04-10_13-51-42.116901_from_79015517628_to_0173695_session_2873854296_talk.mp3
2023-04-10_11-17-57.071026_from_74954322480_to_0197875_session_2873239392_talk.mp3
2023-04-10_17-57-19.695299_from_79161610659_to_0173695_session_2871197247_talk.mp3
2023-04-11_13-53-25.647196_from_74956756102_to_0111639_session_2876518680_talk.mp3
2023-04-10_16-44-33.432467_from_79037231781_to_0111639_session_2870987187_talk.mp3


In [ ]:
# Список файлов записей звонков, о которых нет информации в csv-файлах
calls_list = []
for l in lines:
  calls_list.append(l.rstrip('\n'))

In [ ]:
# Создаём Primavera_unmatched_records_df
Primavera_unmatched_records_df = calls_list_to_df(calls_list)
Primavera_unmatched_records_df

,Дата и время,Номер абонента,Номер сотрудника,Идентификатор сессии звонка,Файл записи звонка
0,2023-04-28 10:58:01,79168623705,0111642,2901657750,2023-04-28_10-58-01.443973_from_79168623705_to_0111642_session_2901657750_talk.mp3
1,2023-04-07 15:14:26,79035861746,0197872,2865992215,2023-04-07_15-14-26.286472_from_79035861746_to_0197872_session_2865992215_talk.mp3
2,2023-04-11 14:45:51,79852229315,0111642,2868761754,2023-04-11_14-45-51.943888_from_79852229315_to_0111642_session_2868761754_talk.mp3
3,2023-04-07 18:25:46,79165050055,0111639,2867030845,2023-04-07_18-25-46.218872_from_79165050055_to_0111639_session_2867030845_talk.mp3
4,2023-04-09 17:30:35,79671795235,0150113,2869317077,2023-04-09_17-30-35.871150_from_79671795235_to_0150113_session_2869317077_talk.mp3
...,...,...,...,...,...
461,2023-04-27 10:34:39,79104224055,0111639,2907316871,2023-04-27_10-34-39.993742_from_79104224055_to_0111639_session_2907316871_talk.mp3
462,2023-04-24 13:43:50,74950210291,0197875,2906918372,2023-04-24_13-43-50.370043_from_74950210291_to_0197875_session_2906918372_talk.mp3
463,2023-04-28 16:41:01,74951354274,0197872,2911788537,2023-04-28_16-41-01.978922_from_74951354274_to_0197872_session_2911788537_talk.mp3
464,2023-04-28 18:33:40,79992088941,0197879,2912083277,2023-04-28_18-33-40.333149_from_79992088941_to_0197879_session_2912083277_talk.mp3


### Соединение датафреймов

In [ ]:
# Смотрим размеры датафреймов
print(Headliner_not_match_df.shape)
print(Primavera_not_match_df.shape)
print(Paveletskaya_city_not_match_df.shape)

(0, 5)
(466, 5)
(0, 5)


In [ ]:
# Добавляем к файлам относительные пути
Primavera_not_match_df['Файл записи звонка'] = Primavera_not_match_df['Файл записи звонка'].apply(lambda x: 'Записи звонков/Записи звонков_primavera/' + x)

In [ ]:
# Создаём датафрейм записей звонков, о которых нет информации в csv-файлах
calls_not_match_info_df = Primavera_not_match_df
calls_not_match_info_df.index = range(calls_not_match_info_df.shape[0])
calls_not_match_info_df

,Дата и время,Номер абонента,Номер сотрудника,Идентификатор сессии звонка,Файл записи звонка
0,2023-04-07 08:37:12,79169245539,0197872,2861446698,Записи звонков/Записи звонков_primavera/2023-04-07_08-37-12.159811_from_79169245539_to_0197872_session_2861446698_talk.mp3
1,2023-04-07 11:11:00,79168630327,0197872,2861874202,Записи звонков/Записи звонков_primavera/2023-04-07_11-11-00.578033_from_79168630327_to_0197872_session_2861874202_talk.mp3
2,2023-04-07 11:39:20,79168630327,0197878,2861973030,Записи звонков/Записи звонков_primavera/2023-04-07_11-39-20.205581_from_79168630327_to_0197878_session_2861973030_talk.mp3
3,2023-04-07 13:16:04,79017716131,0197878,2862271810,Записи звонков/Записи звонков_primavera/2023-04-07_13-16-04.805425_from_79017716131_to_0197878_session_2862271810_talk.mp3
4,2023-04-07 15:06:47,79251823056,0111642,2862592034,Записи звонков/Записи звонков_primavera/2023-04-07_15-06-47.669158_from_79251823056_to_0111642_session_2862592034_talk.mp3
...,...,...,...,...,...
461,2023-04-28 20:50:31,74951826921,0197879,2919231016,Записи звонков/Записи звонков_primavera/2023-04-28_20-50-31.873100_from_74951826921_to_0197879_session_2919231016_talk.mp3
462,2023-04-29 12:04:10,79850679487,0111652,2919868308,Записи звонков/Записи звонков_primavera/2023-04-29_12-04-10.120306_from_79850679487_to_0111652_session_2919868308_talk.mp3
463,2023-04-29 16:48:42,79998432316,0197872,2920612188,Записи звонков/Записи звонков_primavera/2023-04-29_16-48-42.343865_from_79998432316_to_0197872_session_2920612188_talk.mp3
464,2023-04-29 18:41:51,79308170123,0111652,2920835588,Записи звонков/Записи звонков_primavera/2023-04-29_18-41-51.964538_from_79308170123_to_0111652_session_2920835588_talk.mp3


In [ ]:
# Приводим типы данных
calls_not_match_info_df['Дата и время']= pd.to_datetime(calls_not_match_info_df['Дата и время'])
calls_not_match_info_df = calls_not_match_info_df.astype({'Номер абонента': int})
calls_not_match_info_df = calls_not_match_info_df.astype({'Номер сотрудника': int})
calls_not_match_info_df = calls_not_match_info_df.astype({'Файл записи звонка': str})

In [ ]:
# Смотрим типы данных
calls_not_match_info_df.dtypes

Дата и время                   datetime64[ns]
Номер абонента                          int64
Номер сотрудника                        int64
Идентификатор сессии звонка             int64
Файл записи звонка                     object
dtype: object

In [ ]:
# Сохраняем calls_not_match_info_df в файл
with open(df_path + 'calls_not_match_info_df.pkl', 'wb') as f:
  pkl.dump(calls_not_match_info_df, f)

In [ ]:
# Сохраняем calls_not_match_info_df в файл Microsoft Excell
with pd.ExcelWriter(table_path + 'Записи звонков без информации в csv-файлах.xlsx') as excel_writer:
  calls_not_match_info_df.to_excel(excel_writer, sheet_name='Записи звонков без информации в csv-файлах', index=False)

## Создание сводной таблицы с информацией о звонках

### Загрузка датафреймов

In [ ]:
# Загружаем Headliner_df из файла
with open(df_path + 'Headliner_df.pkl', 'rb') as f:
  Headliner_df = pkl.load(f)

# Загружаем Headliner_calls_match_df из файла
with open(df_path + 'Headliner_calls_match_df.pkl', 'rb') as f:
  Headliner_calls_match_df = pkl.load(f)

# Загружаем Primavera_df из файла
with open(df_path + 'Primavera_df.pkl', 'rb') as f:
  Primavera_df = pkl.load(f)

# Загружаем Primavera_calls_match_df из файла
with open(df_path + 'Primavera_calls_match_df.pkl', 'rb') as f:
  Primavera_calls_match_df = pkl.load(f)

# Загружаем Paveletskaya_city_df из файла
with open(df_path + 'Paveletskaya_city_df.pkl', 'rb') as f:
  Paveletskaya_city_df = pkl.load(f)

# Загружаем Paveletskaya_city_calls_match_df из файла
with open(df_path + 'Paveletskaya_city_calls_match_df.pkl', 'rb') as f:
  Paveletskaya_city_calls_match_df = pkl.load(f)

# Загружаем Paveletskaya_df из файла
with open(df_path + 'Paveletskaya_df.pkl', 'rb') as f:
  Paveletskaya_df = pkl.load(f)

# Загружаем Paveletskaya_calls_match_df из файла
with open(df_path + 'Paveletskaya_calls_match_df.pkl', 'rb') as f:
  Paveletskaya_calls_match_df = pkl.load(f)

### Сравнение столбцов датафреймов

#### Headliner vs Primavera

In [ ]:
# Смотрим несовпадающие столбцы
for column in Headliner_df.columns.tolist():
  if column not in Primavera_df.columns.tolist():
    print(f'Headliner_df[{column}], {Headliner_df[column].shape[0]} строк, тип данных: {Headliner_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Primavera_df.columns.tolist():
  if column not in Headliner_df.columns.tolist():
    print(f'Primavera_df[{column}], {Primavera_df[column].shape[0]} строк, тип данных: {Primavera_df[column].dtypes}')

Primavera_df[Идентификатор звонка во внешней системе], 3506 строк, тип данных: int64
Primavera_df[Длительность обработки], 3506 строк, тип данных: int64


#### Headliner vs Paveletskaya_city

In [ ]:
# Смотрим несовпадающие столбцы
for column in Headliner_df.columns.tolist():
  if column not in Paveletskaya_city_df.columns.tolist():
    print(f'Headliner_df[{column}], {Headliner_df[column].shape[0]} строк, тип данных: {Headliner_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_city_df.columns.tolist():
  if column not in Headliner_df.columns.tolist():
    print(f'Paveletskaya_city_df[{column}], {Paveletskaya_city_df[column].shape[0]} строк, тип данных: {Paveletskaya_city_df[column].dtypes}')

Paveletskaya_city_df[Длительность обработки], 9327 строк, тип данных: int64
Paveletskaya_city_df[Идентификатор звонка во внешней системе], 9327 строк, тип данных: int64


#### Headliner vs Paveletskaya

In [ ]:
# Смотрим несовпадающие столбцы
for column in Headliner_df.columns.tolist():
  if column not in Paveletskaya_df.columns.tolist():
    print(f'Headliner_df[{column}], {Headliner_df[column].shape[0]} строк, тип данных: {Headliner_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_df.columns.tolist():
  if column not in Headliner_df.columns.tolist():
    print(f'Paveletskaya_df[{column}], {Paveletskaya_df[column].shape[0]} строк, тип данных: {Paveletskaya_df[column].dtypes}')

Paveletskaya_df[Длительность обработки], 1211 строк, тип данных: int64
Paveletskaya_df[Идентификатор звонка во внешней системе], 1211 строк, тип данных: int64


#### Primavera vs Paveletskaya_city

In [ ]:
# Смотрим несовпадающие столбцы
for column in Primavera_df.columns.tolist():
  if column not in Paveletskaya_city_df.columns.tolist():
    print(f'Primavera_df[{column}], {Primavera_df[column].shape[0]} строк, тип данных: {Primavera_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_city_df.columns.tolist():
  if column not in Primavera_df.columns.tolist():
    print(f'Paveletskaya_city_df[{column}], {Paveletskaya_city_df[column].shape[0]} строк, тип данных: {Paveletskaya_city_df[column].dtypes}')

#### Primavera vs Paveletskaya

In [ ]:
# Смотрим несовпадающие столбцы
for column in Primavera_df.columns.tolist():
  if column not in Paveletskaya_df.columns.tolist():
    print(f'Primavera_df[{column}], {Primavera_df[column].shape[0]} строк, тип данных: {Primavera_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_df.columns.tolist():
  if column not in Primavera_df.columns.tolist():
    print(f'Paveletskaya_df[{column}], {Paveletskaya_df[column].shape[0]} строк, тип данных: {Paveletskaya_df[column].dtypes}')

#### Paveletskaya_city vs Paveletskaya

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_city_df.columns.tolist():
  if column not in Paveletskaya_df.columns.tolist():
    print(f'Paveletskaya_city_df[{column}], {Paveletskaya_city_df[column].shape[0]} строк, тип данных: {Paveletskaya_city_df[column].dtypes}')

In [ ]:
# Смотрим несовпадающие столбцы
for column in Paveletskaya_df.columns.tolist():
  if column not in Paveletskaya_city_df.columns.tolist():
    print(f'Paveletskaya_df[{column}], {Paveletskaya_df[column].shape[0]} строк, тип данных: {Paveletskaya_df[column].dtypes}')

### Приведение типов данных и предобработка датафреймов

#### Headliner

##### Статус

In [ ]:
Headliner_df['Статус'].head()

0      Принятый
1      Принятый
2      Принятый
3    Потерянный
4      Принятый
Name: Статус, dtype: object

In [ ]:
print(Headliner_df['Статус'].count())
print(Headliner_df['Статус'].shape[0])

4759
4759


In [ ]:
Headliner_df['Статус'].tail(20)

4739      Принятый
4740      Принятый
4741    Потерянный
4742    Потерянный
4743    Потерянный
4744      Принятый
4745      Принятый
4746    Потерянный
4747    Потерянный
4748      Принятый
4749    Потерянный
4750      Принятый
4751      Принятый
4752      Принятый
4753      Принятый
4754      Принятый
4755    Потерянный
4756      Принятый
4757      Принятый
4758      Принятый
Name: Статус, dtype: object

In [ ]:
Headliner_df['Статус'][Headliner_df['Статус'] == ''].count()

0

##### Тип

In [ ]:
Headliner_df['Тип'].head()

0                 Звонок ВАТС
1                 Звонок ВАТС
2                 Звонок ВАТС
3                 Звонок ВАТС
4    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
print(Headliner_df['Тип'].count())
print(Headliner_df['Тип'].shape[0])

4759
4759


In [ ]:
Headliner_df['Тип'].tail(20)

4739                 Звонок ВАТС
4740                 Звонок ВАТС
4741                 Звонок ВАТС
4742    Динамический коллтрекинг
4743    Динамический коллтрекинг
4744                 Звонок ВАТС
4745                 Звонок ВАТС
4746    Динамический коллтрекинг
4747    Динамический коллтрекинг
4748                 Звонок ВАТС
4749    Динамический коллтрекинг
4750                 Звонок ВАТС
4751                 Звонок ВАТС
4752     Автоперезвон по заявкам
4753    Динамический коллтрекинг
4754    Динамический коллтрекинг
4755    Динамический коллтрекинг
4756    Динамический коллтрекинг
4757    Динамический коллтрекинг
4758    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
Headliner_df['Тип'][Headliner_df['Тип'] == ''].count()

0

##### Дата и время

In [ ]:
Headliner_df['Дата и время'].head()

0    2023-07-31 20:51:16
1    2023-07-31 19:57:08
2    2023-07-31 19:06:54
3    2023-07-31 17:52:01
4    2023-07-31 17:31:22
Name: Дата и время, dtype: object

In [ ]:
print(Headliner_df['Дата и время'].count())
print(Headliner_df['Дата и время'].shape[0])

4759
4759


In [ ]:
Headliner_calls_match_df['Дата и время'].head()

956    2023-05-08 16:14:10
997    2023-05-09 11:53:48
974    2023-05-09 17:51:30
984    2023-05-09 18:34:04
965    2023-05-09 20:19:21
Name: Дата и время, dtype: object

In [ ]:
print(Headliner_calls_match_df['Дата и время'].count())
print(Headliner_calls_match_df['Дата и время'].shape[0])

2045
2045


In [ ]:
# Приводим тип данных столбца 'Дата и время'
Headliner_df['Дата и время'] = pd.to_datetime(Headliner_df['Дата и время'])
Headliner_calls_match_df['Дата и время']= pd.to_datetime(Headliner_df['Дата и время'])

In [ ]:
Headliner_df['Дата и время'].head()

0   2023-07-31 20:51:16
1   2023-07-31 19:57:08
2   2023-07-31 19:06:54
3   2023-07-31 17:52:01
4   2023-07-31 17:31:22
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Headliner_calls_match_df['Дата и время'].head()

956   2023-07-12 10:27:43
997   2023-07-11 14:10:18
974   2023-07-11 17:38:21
984   2023-07-11 15:48:01
965   2023-07-12 09:41:40
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Headliner_df['Дата и время'].tail(20)

4739   2023-05-01 11:54:21
4740   2023-05-01 11:45:00
4741   2023-05-01 11:44:30
4742   2023-05-01 11:33:54
4743   2023-05-01 11:33:24
4744   2023-05-01 11:24:16
4745   2023-05-01 11:22:21
4746   2023-05-01 11:15:49
4747   2023-05-01 11:15:19
4748   2023-05-01 11:10:12
4749   2023-05-01 09:52:15
4750   2023-05-01 09:44:05
4751   2023-05-01 09:42:34
4752   2023-07-29 11:54:25
4753   2023-07-21 11:51:29
4754   2023-07-18 20:17:31
4755   2023-06-29 14:16:26
4756   2023-06-29 13:58:53
4757   2023-06-23 11:49:20
4758   2023-05-03 15:40:48
Name: Дата и время, dtype: datetime64[ns]

##### Номер абонента

In [ ]:
Headliner_df['Номер абонента'].head()

0    79990016333.0
1    79662014286.0
2    79257835550.0
3    74957457444.0
4    79167708081.0
Name: Номер абонента, dtype: object

In [ ]:
print(Headliner_df['Номер абонента'].count())
print(Headliner_df['Номер абонента'].shape[0])

4758
4759


In [ ]:
# Избавляемся от пустот в столбце 'Номер абонента'
Headliner_df['Номер абонента'] = Headliner_df['Номер абонента'].fillna('0')

In [ ]:
Headliner_calls_match_df['Номер абонента'].head()

956    74955199580
997    79910665524
974    79670165153
984    79274431546
965    79274432048
Name: Номер абонента, dtype: object

In [ ]:
print(Headliner_calls_match_df['Номер абонента'].count())
print(Headliner_calls_match_df['Номер абонента'].shape[0])

2045
2045


In [ ]:
# Приводим тип данных столбца 'Номер абонента'
Headliner_df = Headliner_df.astype({'Номер абонента': int})
Headliner_calls_match_df = Headliner_calls_match_df.astype({'Номер абонента': int})

In [ ]:
Headliner_df['Номер абонента'].head()

0    79990016333
1    79662014286
2    79257835550
3    74957457444
4    79167708081
Name: Номер абонента, dtype: int64

In [ ]:
Headliner_calls_match_df['Номер абонента'].head()

956    74955199580
997    79910665524
974    79670165153
984    79274431546
965    79274432048
Name: Номер абонента, dtype: int64

In [ ]:
Headliner_df['Номер абонента'].tail(20)

4739    79343337554
4740    79303162665
4741    79303162665
4742    79584102096
4743    79584102096
4744    79643463441
4745    79170778599
4746    79344000790
4747    79344000790
4748    79170778599
4749    78005115537
4750    79777662131
4751    79777662131
4752    79222406766
4753    79257753217
4754    79265889402
4755    79112719262
4756    79636447113
4757    79031531020
4758    79874881362
Name: Номер абонента, dtype: int64

##### Виртуальный номер

In [ ]:
Headliner_df['Виртуальный номер'].head()

0    74951520757
1    74953202993
2    74951810000
3    74950212365
4    74951825873
Name: Виртуальный номер, dtype: object

In [ ]:
print(Headliner_df['Виртуальный номер'].count())
print(Headliner_df['Виртуальный номер'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Виртуальный номер'
Headliner_df = Headliner_df.astype({'Виртуальный номер': int})

In [ ]:
Headliner_df['Виртуальный номер'].head()

0    74951520757
1    74953202993
2    74951810000
3    74950212365
4    74951825873
Name: Виртуальный номер, dtype: int64

In [ ]:
Headliner_df['Виртуальный номер'].tail(20)

4739    74951519845
4740    74951520757
4741    74951520757
4742    74951825864
4743    74951825864
4744    74953202993
4745    74950854281
4746    74951353644
4747    74951353644
4748    74950854281
4749    74953202985
4750    74951060379
4751    74951060379
4752    74951626925
4753    74952555682
4754    74952368074
4755    74951825867
4756    74951825867
4757    74951814638
4758    74951250252
Name: Виртуальный номер, dtype: int64

##### Рекламная кампания

In [ ]:
Headliner_df['Рекламная кампания'].head()

0    Google мой бизнес
1        Базы // Avito
2       Звонки_оффлайн
3       Номер на сайте
4        Яндекс Директ
Name: Рекламная кампания, dtype: object

In [ ]:
print(Headliner_df['Рекламная кампания'].count())
print(Headliner_df['Рекламная кампания'].shape[0])

4759
4759


In [ ]:
Headliner_df['Рекламная кампания'].tail(20)

4739                   Геосервисы // Яндекс Навигатор
4740                                Google мой бизнес
4741                                Google мой бизнес
4742                                    Яндекс Директ
4743                                    Яндекс Директ
4744                                    Базы // Avito
4745                         Офис заселения Хедлайнер
4746                Посетители без рекламной кампании
4747                Посетители без рекламной кампании
4748                         Офис заселения Хедлайнер
4749                                    Яндекс Директ
4750    Медийная реклама // kvartirny-control // Спец
4751    Медийная реклама // kvartirny-control // Спец
4752                                    Яндекс Директ
4753                                    Яндекс Директ
4754                                    Яндекс Директ
4755                                    Яндекс Директ
4756                                    Яндекс Директ
4757                        

In [ ]:
Headliner_df['Рекламная кампания'][Headliner_df['Рекламная кампания'] == ''].count()

0

##### Длительность звонка

In [ ]:
Headliner_df['Длительность звонка'].head()

0    00:02:50
1    00:00:25
2    00:03:12
3    00:00:22
4    00:03:09
Name: Длительность звонка, dtype: object

In [ ]:
print(Headliner_df['Длительность звонка'].count())
print(Headliner_df['Длительность звонка'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Длительность звонка'
Headliner_df['Длительность звонка'] = pd.to_datetime(Headliner_df['Длительность звонка']).dt.time

In [ ]:
Headliner_df['Длительность звонка'].head()

0    00:02:50
1    00:00:25
2    00:03:12
3    00:00:22
4    00:03:09
Name: Длительность звонка, dtype: object

In [ ]:
Headliner_df['Длительность звонка'].tail(20)

4739    00:00:24
4740    00:00:17
4741    00:00:27
4742    00:00:27
4743    00:00:27
4744    00:00:35
4745    00:01:44
4746    00:00:28
4747    00:00:27
4748    00:05:20
4749    00:00:03
4750    00:08:33
4751    00:00:48
4752    00:04:40
4753    00:07:29
4754    00:02:13
4755    00:00:17
4756    00:03:07
4757    00:01:57
4758    00:09:07
Name: Длительность звонка, dtype: object

##### Длительность ожидания ответа

In [ ]:
Headliner_df['Длительность ожидания ответа'].head()

0    00:00:00
1    00:00:01
2    00:00:10
3    00:00:22
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
print(Headliner_df['Длительность ожидания ответа'].count())
print(Headliner_df['Длительность ожидания ответа'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Длительность ожидания ответа'
Headliner_df['Длительность ожидания ответа'] = pd.to_datetime(Headliner_df['Длительность ожидания ответа']).dt.time

In [ ]:
Headliner_df['Длительность ожидания ответа'].head()

0    00:00:00
1    00:00:01
2    00:00:10
3    00:00:22
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
Headliner_df['Длительность ожидания ответа'].tail(20)

4739    00:00:09
4740    00:00:05
4741    00:00:27
4742    00:00:27
4743    00:00:27
4744    00:00:31
4745    00:00:38
4746    00:00:28
4747    00:00:27
4748    00:00:10
4749    00:00:03
4750    00:00:01
4751    00:00:31
4752    00:00:34
4753    00:00:46
4754    00:00:50
4755    00:00:17
4756    00:00:09
4757    00:00:08
4758    00:00:00
Name: Длительность ожидания ответа, dtype: object

##### Причина завершения

In [ ]:
Headliner_df['Причина завершения'].head()

0    Сотрудник разорвал соединение
1      Абонент разорвал соединение
2      Абонент разорвал соединение
3     Не дозвонились до сотрудника
4    Сотрудник разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
print(Headliner_df['Причина завершения'].count())
print(Headliner_df['Причина завершения'].shape[0])

4759
4759


In [ ]:
Headliner_df['Причина завершения'].tail(20)

4739    Сотрудник разорвал соединение
4740    Сотрудник разорвал соединение
4741      Абонент разорвал соединение
4742      Абонент разорвал соединение
4743      Абонент разорвал соединение
4744      Абонент разорвал соединение
4745      Абонент разорвал соединение
4746      Абонент разорвал соединение
4747      Абонент разорвал соединение
4748      Абонент разорвал соединение
4749      Абонент разорвал соединение
4750      Абонент разорвал соединение
4751    Сотрудник разорвал соединение
4752    Сотрудник разорвал соединение
4753    Сотрудник разорвал соединение
4754      Абонент разорвал соединение
4755      Абонент разорвал соединение
4756      Абонент разорвал соединение
4757    Сотрудник разорвал соединение
4758    Сотрудник разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
Headliner_df['Причина завершения'][Headliner_df['Причина завершения'] == ''].count()

0

##### Сценарий

In [ ]:
Headliner_df['Сценарий'].head()

0    .Последовательный обзвон
1                     Кортрос
2    .Последовательный обзвон
3    .Последовательный обзвон
4    .Последовательный обзвон
Name: Сценарий, dtype: object

In [ ]:
print(Headliner_df['Сценарий'].count())
print(Headliner_df['Сценарий'].shape[0])

4752
4759


In [ ]:
# Избавляемся от пустот в столбце 'Сценарий'
Headliner_df['Сценарий'] = Headliner_df['Сценарий'].fillna('нет сценария')

In [ ]:
print(Headliner_df['Сценарий'].count())
print(Headliner_df['Сценарий'].shape[0])

4759
4759


In [ ]:
# Убираем '.' в столбце 'Сценарий'
Headliner_df['Сценарий'] = Headliner_df['Сценарий'].apply(lambda x: x.lstrip('.'))

In [ ]:
Headliner_df['Сценарий'].head()

0    Последовательный обзвон
1                    Кортрос
2    Последовательный обзвон
3    Последовательный обзвон
4    Последовательный обзвон
Name: Сценарий, dtype: object

In [ ]:
Headliner_df['Сценарий'].tail(20)

4739     Последовательный обзвон
4740     Последовательный обзвон
4741     Последовательный обзвон
4742     Последовательный обзвон
4743     Последовательный обзвон
4744     Последовательный обзвон
4745    Офис заселения Хедлайнер
4746     Последовательный обзвон
4747     Последовательный обзвон
4748    Офис заселения Хедлайнер
4749     Последовательный обзвон
4750     Последовательный обзвон
4751     Последовательный обзвон
4752                Автоперезвон
4753     Последовательный обзвон
4754     Последовательный обзвон
4755     Последовательный обзвон
4756     Последовательный обзвон
4757     Последовательный обзвон
4758     Последовательный обзвон
Name: Сценарий, dtype: object

In [ ]:
Headliner_df['Сценарий'][Headliner_df['Сценарий'] == ''].count()

0

##### Сотрудник

In [ ]:
Headliner_df['Сотрудник'].head()

0      Кортрос
1      Кортрос
2    Capital 2
3          NaN
4      Кортрос
Name: Сотрудник, dtype: object

In [ ]:
print(Headliner_df['Сотрудник'].count())
print(Headliner_df['Сотрудник'].shape[0])

2947
4759


In [ ]:
# Избавляемся от пустот в столбце 'Сотрудник'
Headliner_df['Сотрудник'] = Headliner_df['Сотрудник'].fillna('нет сотрудника')

In [ ]:
print(Headliner_df['Сотрудник'].count())
print(Headliner_df['Сотрудник'].shape[0])

4759
4759


In [ ]:
Headliner_df['Сотрудник'].head()

0           Кортрос
1           Кортрос
2         Capital 2
3    нет сотрудника
4           Кортрос
Name: Сотрудник, dtype: object

In [ ]:
Headliner_df['Сотрудник'].tail(20)

4739                       Capital 2
4740                       Capital 2
4741                  нет сотрудника
4742                  нет сотрудника
4743                  нет сотрудника
4744    Метриум Вертушка 74997552563
4745                         Копылец
4746                  нет сотрудника
4747                  нет сотрудника
4748                         Копылец
4749                  нет сотрудника
4750    Метриум Вертушка 74997552563
4751    Метриум Вертушка 74997552563
4752                         Кортрос
4753                         Кортрос
4754                         Кортрос
4755                                
4756       Capital 7 (495) 960 77 74
4757                   Бест-вертушка
4758                         Кортрос
Name: Сотрудник, dtype: object

In [ ]:
Headliner_df['Сотрудник'][Headliner_df['Сотрудник'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет сотрудника' в столбце 'Сотрудник'
Headliner_df['Сотрудник'] = Headliner_df['Сотрудник'].apply(lambda x: 'нет сотрудника' if x == '' else x)

In [ ]:
Headliner_df['Сотрудник'][Headliner_df['Сотрудник'] == ''].count()

0

In [ ]:
Headliner_df['Сотрудник'].tail(20)

4739                       Capital 2
4740                       Capital 2
4741                  нет сотрудника
4742                  нет сотрудника
4743                  нет сотрудника
4744    Метриум Вертушка 74997552563
4745                         Копылец
4746                  нет сотрудника
4747                  нет сотрудника
4748                         Копылец
4749                  нет сотрудника
4750    Метриум Вертушка 74997552563
4751    Метриум Вертушка 74997552563
4752                         Кортрос
4753                         Кортрос
4754                         Кортрос
4755                  нет сотрудника
4756       Capital 7 (495) 960 77 74
4757                   Бест-вертушка
4758                         Кортрос
Name: Сотрудник, dtype: object

##### Комментарий

In [ ]:
Headliner_df['Комментарий'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Комментарий, dtype: object

In [ ]:
print(Headliner_df['Комментарий'].count())
print(Headliner_df['Комментарий'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Комментарий'
Headliner_df['Комментарий'] = Headliner_df['Комментарий'].fillna('нет комментария')

In [ ]:
print(Headliner_df['Комментарий'].count())
print(Headliner_df['Комментарий'].shape[0])

4759
4759


In [ ]:
Headliner_df['Комментарий'].head()

0    нет комментария
1    нет комментария
2    нет комментария
3    нет комментария
4    нет комментария
Name: Комментарий, dtype: object

In [ ]:
Headliner_df['Комментарий'].tail(20)

4739    нет комментария
4740    нет комментария
4741    нет комментария
4742    нет комментария
4743    нет комментария
4744    нет комментария
4745    нет комментария
4746    нет комментария
4747    нет комментария
4748    нет комментария
4749    нет комментария
4750    нет комментария
4751    нет комментария
4752                   
4753                   
4754                   
4755                   
4756                   
4757                   
4758                   
Name: Комментарий, dtype: object

In [ ]:
Headliner_df['Комментарий'][Headliner_df['Комментарий'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет комментария' в столбце 'Комментарий'
Headliner_df['Комментарий'] = Headliner_df['Комментарий'].apply(lambda x: 'нет комментария' if x == '' else x)

In [ ]:
Headliner_df['Комментарий'][Headliner_df['Комментарий'] == ''].count()

0

In [ ]:
Headliner_df['Комментарий'].tail(20)

4739    нет комментария
4740    нет комментария
4741    нет комментария
4742    нет комментария
4743    нет комментария
4744    нет комментария
4745    нет комментария
4746    нет комментария
4747    нет комментария
4748    нет комментария
4749    нет комментария
4750    нет комментария
4751    нет комментария
4752    нет комментария
4753    нет комментария
4754    нет комментария
4755    нет комментария
4756    нет комментария
4757    нет комментария
4758    нет комментария
Name: Комментарий, dtype: object

##### UTM-метка Source

In [ ]:
Headliner_df['UTM-метка Source'].head()

0       NaN
1       NaN
2       NaN
3       NaN
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Source'].count())
print(Headliner_df['UTM-метка Source'].shape[0])

607
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Source'
Headliner_df['UTM-метка Source'] = Headliner_df['UTM-метка Source'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Source'].count())
print(Headliner_df['UTM-метка Source'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Source'].head()

0       нет
1       нет
2       нет
3       нет
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
Headliner_df['UTM-метка Source'].tail(20)

4739       нет
4740       нет
4741       нет
4742       нет
4743       нет
4744       нет
4745       нет
4746       нет
4747       нет
4748       нет
4749    yandex
4750       нет
4751       нет
4752    yandex
4753    yandex
4754    yandex
4755    yandex
4756    yandex
4757    yandex
4758          
Name: UTM-метка Source, dtype: object

In [ ]:
Headliner_df['UTM-метка Source'][Headliner_df['UTM-метка Source'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Source'
Headliner_df['UTM-метка Source'] = Headliner_df['UTM-метка Source'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Source'][Headliner_df['UTM-метка Source'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Source'].tail(20)

4739       нет
4740       нет
4741       нет
4742       нет
4743       нет
4744       нет
4745       нет
4746       нет
4747       нет
4748       нет
4749    yandex
4750       нет
4751       нет
4752    yandex
4753    yandex
4754    yandex
4755    yandex
4756    yandex
4757    yandex
4758       нет
Name: UTM-метка Source, dtype: object

##### ID посетителя

In [ ]:
Headliner_df['ID посетителя'].head()

0             NaN
1             NaN
2             NaN
3    7414204397.0
4    7459754252.0
Name: ID посетителя, dtype: object

In [ ]:
print(Headliner_df['ID посетителя'].count())
print(Headliner_df['ID посетителя'].shape[0])

2159
4759


In [ ]:
# Избавляемся от пустот в столбце 'ID посетителя'
Headliner_df['ID посетителя'] = Headliner_df['ID посетителя'].fillna('0')

In [ ]:
print(Headliner_df['ID посетителя'].count())
print(Headliner_df['ID посетителя'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'ID посетителя'
Headliner_df = Headliner_df.astype({'ID посетителя': int})

In [ ]:
Headliner_df['ID посетителя'].head()

0             0
1             0
2             0
3    7414204397
4    7459754252
Name: ID посетителя, dtype: int64

In [ ]:
Headliner_df['ID посетителя'].tail(20)

4739             0
4740             0
4741             0
4742    7167081560
4743    7167081560
4744             0
4745    4507529727
4746    7169248096
4747    7169248096
4748    4507529727
4749    7167674108
4750             0
4751             0
4752    7438708987
4753    7438053217
4754    7432149392
4755    7390973987
4756    7390973987
4757    7377885782
4758    7179161056
Name: ID посетителя, dtype: int64

##### Номер обращения

In [ ]:
Headliner_df['Номер обращения'].head()

0      1
1      1
2      1
3    209
4      1
Name: Номер обращения, dtype: object

In [ ]:
print(Headliner_df['Номер обращения'].count())
print(Headliner_df['Номер обращения'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Номер обращения'
Headliner_df = Headliner_df.astype({'Номер обращения': int})

In [ ]:
Headliner_df['Номер обращения'].head()

0      1
1      1
2      1
3    209
4      1
Name: Номер обращения, dtype: int64

In [ ]:
Headliner_df['Номер обращения'].tail(20)

4739     1
4740     2
4741     1
4742     2
4743     1
4744     1
4745     4
4746     2
4747     1
4748     3
4749     1
4750     2
4751     1
4752     3
4753     1
4754     1
4755     7
4756     6
4757     1
4758    23
Name: Номер обращения, dtype: int64

##### Поисковый запрос

In [ ]:
Headliner_df['Поисковый запрос'].head(20)

0                             NaN
1                             NaN
2                             NaN
3     {{44083963558_44083963558}}
4         46117472944_46117472944
5                             NaN
6                             NaN
7          {{+хедлайнер +москва}}
8          {{+хедлайнер +москва}}
9                             NaN
10                            NaN
11                            NaN
12                            NaN
13                            NaN
14                            NaN
15                            NaN
16                            NaN
17        46117476237_46117476237
18                            NaN
19                            NaN
Name: Поисковый запрос, dtype: object

In [ ]:
print(Headliner_df['Поисковый запрос'].count())
print(Headliner_df['Поисковый запрос'].shape[0])

942
4759


In [ ]:
# Избавляемся от пустот в столбце 'Поисковый запрос'
Headliner_df['Поисковый запрос'] = Headliner_df['Поисковый запрос'].fillna('нет запроса')

In [ ]:
Headliner_df['Поисковый запрос'].head(20)

0                     нет запроса
1                     нет запроса
2                     нет запроса
3     {{44083963558_44083963558}}
4         46117472944_46117472944
5                     нет запроса
6                     нет запроса
7          {{+хедлайнер +москва}}
8          {{+хедлайнер +москва}}
9                     нет запроса
10                    нет запроса
11                    нет запроса
12                    нет запроса
13                    нет запроса
14                    нет запроса
15                    нет запроса
16                    нет запроса
17        46117476237_46117476237
18                    нет запроса
19                    нет запроса
Name: Поисковый запрос, dtype: object

In [ ]:
Headliner_df['Поисковый запрос'].tail(20)

4739                     нет запроса
4740                     нет запроса
4741                     нет запроса
4742       {44083962236_44083962236}
4743       {44083962236_44083962236}
4744                     нет запроса
4745    {{хедлайнер жилой комплекс}}
4746                     нет запроса
4747                     нет запроса
4748    {{хедлайнер жилой комплекс}}
4749         44083962195_44083962195
4750                     нет запроса
4751                     нет запроса
4752         46118338566_46118338566
4753         46117476238_46117476238
4754         46117472903_46117472903
4755         44083963615_44083963615
4756         44083963615_44083963615
4757         44084907889_44084907889
4758                                
Name: Поисковый запрос, dtype: object

In [ ]:
Headliner_df['Поисковый запрос'][Headliner_df['Поисковый запрос'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет запроса' в столбце 'Поисковый запрос'
Headliner_df['Поисковый запрос'] = Headliner_df['Поисковый запрос'].apply(lambda x: 'нет запроса' if x == '' else x)

In [ ]:
Headliner_df['Поисковый запрос'][Headliner_df['Поисковый запрос'] == ''].count()

0

In [ ]:
Headliner_df['Поисковый запрос'].tail(20)

4739                     нет запроса
4740                     нет запроса
4741                     нет запроса
4742       {44083962236_44083962236}
4743       {44083962236_44083962236}
4744                     нет запроса
4745    {{хедлайнер жилой комплекс}}
4746                     нет запроса
4747                     нет запроса
4748    {{хедлайнер жилой комплекс}}
4749         44083962195_44083962195
4750                     нет запроса
4751                     нет запроса
4752         46118338566_46118338566
4753         46117476238_46117476238
4754         46117472903_46117472903
4755         44083963615_44083963615
4756         44083963615_44083963615
4757         44084907889_44084907889
4758                     нет запроса
Name: Поисковый запрос, dtype: object

##### Теги

In [ ]:
Headliner_df['Теги'].head()

0       Коммерческая недвижимость - запись на просмотр - Условно целевой
1                                               спорный - нет соединения
2    Первичный целевой - 2 очередь - 3 очередь - 1-ком - цена за кв метр
3                                               спорный - нет соединения
4              интересуют скидки - Первичный целевой - 3 очередь - 2-ком
Name: Теги, dtype: object

In [ ]:
print(Headliner_df['Теги'].count())
print(Headliner_df['Теги'].shape[0])

4759
4759


In [ ]:
Headliner_df['Теги'].tail(20)

4739                                                     Спам - возможно фрод
4740                                                                     Спам
4741                                                 спорный - нет соединения
4742                                                 спорный - нет соединения
4743                                                 спорный - нет соединения
4744                                                 спорный - нет соединения
4745                           постпродажное обслуживание - Вторичный целевой
4746                                                 спорный - нет соединения
4747                                                 спорный - нет соединения
4748                           постпродажное обслуживание - Вторичный целевой
4749                                                 спорный - нет соединения
4750              Первичный целевой - 2 очередь - ипотека - 3 очередь - 2-ком
4751                                                 спорный - н

In [ ]:
Headliner_df['Теги'][Headliner_df['Теги'] == ''].count()

0

##### UTM-метка Campaign

In [ ]:
Headliner_df['UTM-метка Campaign'].head()

0                                  NaN
1                                  NaN
2                                  NaN
3                                  NaN
4    brend_search_mmo_neutral|91303138
Name: UTM-метка Campaign, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Campaign'].count())
print(Headliner_df['UTM-метка Campaign'].shape[0])

550
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Campaign'
Headliner_df['UTM-метка Campaign'] = Headliner_df['UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Campaign'].count())
print(Headliner_df['UTM-метка Campaign'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Campaign'].head()

0                                  нет
1                                  нет
2                                  нет
3                                  нет
4    brend_search_mmo_neutral|91303138
Name: UTM-метка Campaign, dtype: object

In [ ]:
Headliner_df['UTM-метка Campaign'].tail(20)

4739                                                    нет
4740                                                    нет
4741                                                    нет
4742                                                    нет
4743                                                    нет
4744                                                    нет
4745                                                    нет
4746                                                    нет
4747                                                    нет
4748                                                    нет
4749                    brend_search_mmo_main_site|85809500
4750                                                    нет
4751                                                    нет
4752                          brend_search_rf_room|91308635
4753                         brend_search_mmo_room|91303199
4754                      brend_search_mmo_neutral|91303138
4755               brend_search_mmo_apar

In [ ]:
Headliner_df['UTM-метка Campaign'][Headliner_df['UTM-метка Campaign'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Campaign'
Headliner_df['UTM-метка Campaign'] = Headliner_df['UTM-метка Campaign'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Campaign'][Headliner_df['UTM-метка Campaign'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Campaign'].tail(20)

4739                                                    нет
4740                                                    нет
4741                                                    нет
4742                                                    нет
4743                                                    нет
4744                                                    нет
4745                                                    нет
4746                                                    нет
4747                                                    нет
4748                                                    нет
4749                    brend_search_mmo_main_site|85809500
4750                                                    нет
4751                                                    нет
4752                          brend_search_rf_room|91308635
4753                         brend_search_mmo_room|91303199
4754                      brend_search_mmo_neutral|91303138
4755               brend_search_mmo_apar

##### Номер сотрудника

In [ ]:
Headliner_df['Номер сотрудника'].head()

0        0188880
1        0188880
2    74957898990
3    74959339929
4        0188880
Name: Номер сотрудника, dtype: object

In [ ]:
print(Headliner_df['Номер сотрудника'].count())
print(Headliner_df['Номер сотрудника'].shape[0])

4735
4759


In [ ]:
# Избавляемся от пустот в столбце 'Номер сотрудника'
Headliner_df['Номер сотрудника'] = Headliner_df['Номер сотрудника'].fillna('0')

In [ ]:
print(Headliner_df['Номер сотрудника'].count())
print(Headliner_df['Номер сотрудника'].shape[0])

4759
4759


In [ ]:
# Обрезаем '<5' в столбце 'Номер сотрудника'
Headliner_df['Номер сотрудника'] = Headliner_df['Номер сотрудника'].apply(lambda x: x.rstrip('<5'))

In [ ]:
Headliner_calls_match_df['Номер сотрудника'].head()

956    74997552563
997    74997552563
974        0188880
984    74997552563
965    74997552563
Name: Номер сотрудника, dtype: object

In [ ]:
# Приводим тип данных столбца 'Номер сотрудника'
Headliner_df = Headliner_df.astype({'Номер сотрудника': int})
Headliner_calls_match_df = Headliner_calls_match_df.astype({'Номер сотрудника': int})

In [ ]:
Headliner_df['Номер сотрудника'].head()

0         188880
1         188880
2    74957898990
3    74959339929
4         188880
Name: Номер сотрудника, dtype: int64

In [ ]:
Headliner_calls_match_df['Номер сотрудника'].head()

956    74997552563
997    74997552563
974         188880
984    74997552563
965    74997552563
Name: Номер сотрудника, dtype: int64

In [ ]:
Headliner_df['Номер сотрудника'].tail(20)

4739    74957898990
4740    74957898990
4741    74957898990
4742    74957898990
4743    74957898990
4744    74997552563
4745    79154084359
4746    74957898990
4747    74957898990
4748    79154084359
4749    74957898990
4750    74997552563
4751    74997552563
4752         188880
4753         188880
4754    74959339902
4755    74957898990
4756    74959339902
4757    74959339902
4758         188880
Name: Номер сотрудника, dtype: int64

In [ ]:
Headliner_calls_match_df['Номер сотрудника'].tail(20)

1035    74959339902
1048    74959339929
1043    74959339902
1028    74959339929
1027         188880
1024    74957898990
1037         188880
1017    74959339902
2035    74959339929
2044    74959339929
1007    74959339902
1009    74959339929
2040    74957898990
2048    74957898990
2023    74957898990
1002    74959339902
2004    74957898990
2002    74959339929
2001    74957898990
2013    74959339929
Name: Номер сотрудника, dtype: int64

##### UTM-метка Content

In [ ]:
Headliner_df['UTM-метка Content'].head()

0                                                                                                                              NaN
1                                                                                                                              NaN
2                                                                                                                              NaN
3                                                                                                                              NaN
4    id|46117472944_46117472944|cid|91303138|gid|5246793501|aid|14682291928|adp|no|pos|premium1|src|search_none|dvc|desktop|coef|0
Name: UTM-метка Content, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Content'].count())
print(Headliner_df['UTM-метка Content'].shape[0])

549
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Content'
Headliner_df['UTM-метка Content'] = Headliner_df['UTM-метка Content'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Content'].count())
print(Headliner_df['UTM-метка Content'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Content'].head()

0                                                                                                                              нет
1                                                                                                                              нет
2                                                                                                                              нет
3                                                                                                                              нет
4    id|46117472944_46117472944|cid|91303138|gid|5246793501|aid|14682291928|adp|no|pos|premium1|src|search_none|dvc|desktop|coef|0
Name: UTM-метка Content, dtype: object

In [ ]:
Headliner_df['UTM-метка Content'].tail(20)

4739                                                                                                                              нет
4740                                                                                                                              нет
4741                                                                                                                              нет
4742                                                                                                                              нет
4743                                                                                                                              нет
4744                                                                                                                              нет
4745                                                                                                                              нет
4746                                                          

In [ ]:
Headliner_df['UTM-метка Content'][Headliner_df['UTM-метка Content'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Content'
Headliner_df['UTM-метка Content'] = Headliner_df['UTM-метка Content'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Content'][Headliner_df['UTM-метка Content'] == ''].count()

0

##### Client ID Яндекс.Метрика

In [ ]:
Headliner_df['Client ID Яндекс.Метрика'].head()

0                      
1                      
2                      
3                      
4    169081372023684422
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
print(Headliner_df['Client ID Яндекс.Метрика'].count())
print(Headliner_df['Client ID Яндекс.Метрика'].shape[0])

4759
4759


In [ ]:
# Меняем '' на '0' в столбце 'Client ID Яндекс.Метрика'
Headliner_df['Client ID Яндекс.Метрика'] = Headliner_df['Client ID Яндекс.Метрика'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['Client ID Яндекс.Метрика'].head()

0                     0
1                     0
2                     0
3                     0
4    169081372023684422
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
Headliner_df['Client ID Яндекс.Метрика'].tail(20)

4739                      0
4740                      0
4741                      0
4742    1682867536964025912
4743    1682867536964025912
4744                      0
4745                      0
4746    1682927484192081419
4747    1682927484192081419
4748                      0
4749    1682878397478068708
4750                      0
4751                      0
4752    1689948653852395413
4753    1689929456200822551
4754    1689698870809062270
4755    1688036128756567826
4756    1688036128756567826
4757     167048556979834633
4758    1683117090700449615
Name: Client ID Яндекс.Метрика, dtype: object

##### UTM-метка Medium

In [ ]:
Headliner_df['UTM-метка Medium'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Medium'].count())
print(Headliner_df['UTM-метка Medium'].shape[0])

578
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Medium'
Headliner_df['UTM-метка Medium'] = Headliner_df['UTM-метка Medium'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Medium'].count())
print(Headliner_df['UTM-метка Medium'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Medium'].head()

0    нет
1    нет
2    нет
3    нет
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
Headliner_df['UTM-метка Medium'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    cpc
4750    нет
4751    нет
4752    cpc
4753    cpc
4754    cpc
4755    cpc
4756    cpc
4757    cpc
4758       
Name: UTM-метка Medium, dtype: object

In [ ]:
Headliner_df['UTM-метка Medium'][Headliner_df['UTM-метка Medium'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Medium'
Headliner_df['UTM-метка Medium'] = Headliner_df['UTM-метка Medium'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Medium'][Headliner_df['UTM-метка Medium'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Medium'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    cpc
4750    нет
4751    нет
4752    cpc
4753    cpc
4754    cpc
4755    cpc
4756    cpc
4757    cpc
4758    нет
Name: UTM-метка Medium, dtype: object

##### Операции

In [ ]:
Headliner_df['Операции'].head()

0    Переадресация на последнего менеджера 1, Переадресация 1
1                                             Переадресация 1
2    Переадресация на последнего менеджера 1, Переадресация 1
3                     Переадресация на последнего менеджера 1
4    Переадресация на последнего менеджера 1, Переадресация 1
Name: Операции, dtype: object

In [ ]:
print(Headliner_df['Операции'].count())
print(Headliner_df['Операции'].shape[0])

4751
4759


In [ ]:
# Избавляемся от пустот в столбце 'Операции'
Headliner_df['Операции'] = Headliner_df['Операции'].fillna('нет операций')

In [ ]:
print(Headliner_df['Операции'].count())
print(Headliner_df['Операции'].shape[0])

4759
4759


In [ ]:
Headliner_df['Операции'].head()

0    Переадресация на последнего менеджера 1, Переадресация 1
1                                             Переадресация 1
2    Переадресация на последнего менеджера 1, Переадресация 1
3                     Переадресация на последнего менеджера 1
4    Переадресация на последнего менеджера 1, Переадресация 1
Name: Операции, dtype: object

In [ ]:
Headliner_df['Операции'].tail(20)

4739    Переадресация на последнего менеджера 1, Переадресация 1
4740    Переадресация на последнего менеджера 1, Переадресация 1
4741    Переадресация на последнего менеджера 1, Переадресация 1
4742    Переадресация на последнего менеджера 1, Переадресация 1
4743    Переадресация на последнего менеджера 1, Переадресация 1
4744    Переадресация на последнего менеджера 1, Переадресация 1
4745                                             Переадресация 1
4746    Переадресация на последнего менеджера 1, Переадресация 1
4747    Переадресация на последнего менеджера 1, Переадресация 1
4748                                             Переадресация 1
4749    Переадресация на последнего менеджера 1, Переадресация 1
4750                     Переадресация на последнего менеджера 1
4751    Переадресация на последнего менеджера 1, Переадресация 1
4752                                             Переадресация 1
4753    Переадресация на последнего менеджера 1, Переадресация 1
4754    Переадресация на 

In [ ]:
Headliner_df['Операции'][Headliner_df['Операции'] == ''].count()

0

##### Трансфер

In [ ]:
Headliner_df['Трансфер'].head()

0    Нет
1    Нет
2    Нет
3    Нет
4    Нет
Name: Трансфер, dtype: object

In [ ]:
print(Headliner_df['Трансфер'].count())
print(Headliner_df['Трансфер'].shape[0])

4759
4759


In [ ]:
Headliner_df['Трансфер'].tail(20)

4739    Нет
4740    Нет
4741    Нет
4742    Нет
4743    Нет
4744    Нет
4745    Нет
4746    Нет
4747    Нет
4748    Нет
4749    Нет
4750    Нет
4751    Нет
4752    Нет
4753    Нет
4754    Нет
4755    Нет
4756    Нет
4757    Нет
4758    Нет
Name: Трансфер, dtype: object

In [ ]:
Headliner_df['Трансфер'][Headliner_df['Трансфер'] == ''].count()

0

##### Идентификатор сессии звонка

In [ ]:
Headliner_df['Идентификатор сессии звонка'].head()

0    3128665492
1    3124867081
2    3128534127
3    3126905895
4    3123899209
Name: Идентификатор сессии звонка, dtype: object

In [ ]:
print(Headliner_df['Идентификатор сессии звонка'].count())
print(Headliner_df['Идентификатор сессии звонка'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Идентификатор сессии звонка'
Headliner_df = Headliner_df.astype({'Идентификатор сессии звонка': int})

In [ ]:
Headliner_df['Идентификатор сессии звонка'].head()

0    3128665492
1    3124867081
2    3128534127
3    3126905895
4    3123899209
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Headliner_calls_match_df['Идентификатор сессии звонка'].head()

956    2918538922
997    2919161802
974    2919520238
984    2919550530
965    2919615074
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Headliner_df['Идентификатор сессии звонка'].tail(20)

4739    2914908443
4740    2915509617
4741    2914884735
4742    2915490217
4743    2922924204
4744    2906191794
4745    2922899180
4746    2922887860
4747    2915462945
4748    2922880964
4749    2915327261
4750    2906054630
4751    2922722228
4752    3123710942
4753    3102250329
4754    3102760810
4755    3056058896
4756    3055826489
4757    3040446925
4758    2928515476
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Headliner_calls_match_df['Идентификатор сессии звонка'].tail(20)

1035    3102222555
1048    3102235995
1043    3102413220
1028    3102488970
1027    3102538210
1024    3102545410
1037    3102759850
1017    3102760810
2035    3103186655
2044    3103283965
1007    3103319325
1009    3103392850
2040    3103779895
2048    3103904545
2023    3104061795
1002    3104067170
2004    3104358905
2002    3104418070
2001    3104422845
2013    3104540010
Name: Идентификатор сессии звонка, dtype: int64

##### Полная длительность ожидания

In [ ]:
Headliner_df['Полная длительность ожидания'].head()

0    00:00:00
1    00:00:01
2    00:00:11
3    00:00:22
4    00:00:01
Name: Полная длительность ожидания, dtype: object

In [ ]:
print(Headliner_df['Полная длительность ожидания'].count())
print(Headliner_df['Полная длительность ожидания'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Полная длительность ожидания'
Headliner_df['Полная длительность ожидания'] = pd.to_datetime(Headliner_df['Полная длительность ожидания']).dt.time

In [ ]:
Headliner_df['Полная длительность ожидания'].head()

0    00:00:00
1    00:00:01
2    00:00:11
3    00:00:22
4    00:00:01
Name: Полная длительность ожидания, dtype: object

In [ ]:
Headliner_df['Полная длительность ожидания'].tail(20)

4739    00:00:09
4740    00:00:05
4741    00:00:27
4742    00:00:27
4743    00:00:27
4744    00:00:31
4745    00:00:38
4746    00:00:28
4747    00:00:27
4748    00:00:11
4749    00:00:03
4750    00:00:02
4751    00:00:32
4752    00:00:34
4753    00:00:47
4754    00:00:50
4755    00:00:17
4756    00:00:10
4757    00:00:08
4758    00:00:01
Name: Полная длительность ожидания, dtype: object

##### Расширенная UTM-метка Source

In [ ]:
Headliner_df['Расширенная UTM-метка Source'].head()

0    Google_My_Business 
1                    NaN
2                    NaN
3                    NaN
4                 yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Source'].count())
print(Headliner_df['Расширенная UTM-метка Source'].shape[0])

851
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Source'
Headliner_df['Расширенная UTM-метка Source'] = Headliner_df['Расширенная UTM-метка Source'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Source'].count())
print(Headliner_df['Расширенная UTM-метка Source'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Source'].head()

0    Google_My_Business 
1                    нет
2                    нет
3                    нет
4                 yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Source'].tail(20)

4739                    нет
4740    Google_My_Business 
4741    Google_My_Business 
4742                    нет
4743                    нет
4744                    нет
4745                    нет
4746                    нет
4747                    нет
4748                    нет
4749                 yandex
4750                    нет
4751                    нет
4752                 yandex
4753                 yandex
4754                 yandex
4755                 yandex
4756                 yandex
4757                 yandex
4758                       
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Source'][Headliner_df['Расширенная UTM-метка Source'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Source'
Headliner_df['Расширенная UTM-метка Source'] = Headliner_df['Расширенная UTM-метка Source'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Source'][Headliner_df['Расширенная UTM-метка Source'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Source'].tail(20)

4739                    нет
4740    Google_My_Business 
4741    Google_My_Business 
4742                    нет
4743                    нет
4744                    нет
4745                    нет
4746                    нет
4747                    нет
4748                    нет
4749                 yandex
4750                    нет
4751                    нет
4752                 yandex
4753                 yandex
4754                 yandex
4755                 yandex
4756                 yandex
4757                 yandex
4758                    нет
Name: Расширенная UTM-метка Source, dtype: object

##### Расширенная UTM-метка Campaign

In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'].head()

0                                  NaN
1                                  NaN
2                                  NaN
3                                  NaN
4    brend_search_mmo_neutral|91303138
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Campaign'].count())
print(Headliner_df['Расширенная UTM-метка Campaign'].shape[0])

550
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Campaign'
Headliner_df['Расширенная UTM-метка Campaign'] = Headliner_df['Расширенная UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Campaign'].count())
print(Headliner_df['Расширенная UTM-метка Campaign'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'].head()

0                                  нет
1                                  нет
2                                  нет
3                                  нет
4    brend_search_mmo_neutral|91303138
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'].tail(20)

4739                                                    нет
4740                                                    нет
4741                                                    нет
4742                                                    нет
4743                                                    нет
4744                                                    нет
4745                                                    нет
4746                                                    нет
4747                                                    нет
4748                                                    нет
4749                    brend_search_mmo_main_site|85809500
4750                                                    нет
4751                                                    нет
4752                          brend_search_rf_room|91308635
4753                         brend_search_mmo_room|91303199
4754                      brend_search_mmo_neutral|91303138
4755               brend_search_mmo_apar

In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'][Headliner_df['Расширенная UTM-метка Campaign'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Campaign'
Headliner_df['Расширенная UTM-метка Campaign'] = Headliner_df['Расширенная UTM-метка Campaign'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'][Headliner_df['Расширенная UTM-метка Campaign'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Campaign'].tail(20)

4739                                                    нет
4740                                                    нет
4741                                                    нет
4742                                                    нет
4743                                                    нет
4744                                                    нет
4745                                                    нет
4746                                                    нет
4747                                                    нет
4748                                                    нет
4749                    brend_search_mmo_main_site|85809500
4750                                                    нет
4751                                                    нет
4752                          brend_search_rf_room|91308635
4753                         brend_search_mmo_room|91303199
4754                      brend_search_mmo_neutral|91303138
4755               brend_search_mmo_apar

##### Расширенная UTM-метка Medium

In [ ]:
Headliner_df['Расширенная UTM-метка Medium'].head()

0    cpc
1    NaN
2    NaN
3    NaN
4    cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Medium'].count())
print(Headliner_df['Расширенная UTM-метка Medium'].shape[0])

708
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Medium'
Headliner_df['Расширенная UTM-метка Medium'] = Headliner_df['Расширенная UTM-метка Medium'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Medium'].count())
print(Headliner_df['Расширенная UTM-метка Medium'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Medium'].head()

0    cpc
1    нет
2    нет
3    нет
4    cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Medium'].tail(20)

4739    нет
4740    cpc
4741    cpc
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    cpc
4750    нет
4751    нет
4752    cpc
4753    cpc
4754    cpc
4755    cpc
4756    cpc
4757    cpc
4758       
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Medium'][Headliner_df['Расширенная UTM-метка Medium'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Medium'
Headliner_df['Расширенная UTM-метка Medium'] = Headliner_df['Расширенная UTM-метка Medium'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Medium'][Headliner_df['Расширенная UTM-метка Medium'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Medium'].tail(20)

4739    нет
4740    cpc
4741    cpc
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    cpc
4750    нет
4751    нет
4752    cpc
4753    cpc
4754    cpc
4755    cpc
4756    cpc
4757    cpc
4758    нет
Name: Расширенная UTM-метка Medium, dtype: object

##### Расширенная UTM-метка Content

In [ ]:
Headliner_df['Расширенная UTM-метка Content'].head()

0                                                                                                                              NaN
1                                                                                                                              NaN
2                                                                                                                              NaN
3                                                                                                                              NaN
4    id|46117472944_46117472944|cid|91303138|gid|5246793501|aid|14682291928|adp|no|pos|premium1|src|search_none|dvc|desktop|coef|0
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Content'].count())
print(Headliner_df['Расширенная UTM-метка Content'].shape[0])

549
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Content'
Headliner_df['Расширенная UTM-метка Content'] = Headliner_df['Расширенная UTM-метка Content'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Content'].count())
print(Headliner_df['Расширенная UTM-метка Content'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Content'].head()

0                                                                                                                              нет
1                                                                                                                              нет
2                                                                                                                              нет
3                                                                                                                              нет
4    id|46117472944_46117472944|cid|91303138|gid|5246793501|aid|14682291928|adp|no|pos|premium1|src|search_none|dvc|desktop|coef|0
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Content'].tail(20)

4739                                                                                                                              нет
4740                                                                                                                              нет
4741                                                                                                                              нет
4742                                                                                                                              нет
4743                                                                                                                              нет
4744                                                                                                                              нет
4745                                                                                                                              нет
4746                                                          

In [ ]:
Headliner_df['Расширенная UTM-метка Content'][Headliner_df['Расширенная UTM-метка Content'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Content'
Headliner_df['Расширенная UTM-метка Content'] = Headliner_df['Расширенная UTM-метка Content'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Content'][Headliner_df['Расширенная UTM-метка Content'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Content'].tail(20)

4739                                                                                                                              нет
4740                                                                                                                              нет
4741                                                                                                                              нет
4742                                                                                                                              нет
4743                                                                                                                              нет
4744                                                                                                                              нет
4745                                                                                                                              нет
4746                                                          

##### Расширенная UTM-метка Term

In [ ]:
Headliner_df['Расширенная UTM-метка Term'].head()

0                        NaN
1                        NaN
2                        NaN
3                        NaN
4    46117472944_46117472944
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Term'].count())
print(Headliner_df['Расширенная UTM-метка Term'].shape[0])

478
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Term'
Headliner_df['Расширенная UTM-метка Term'] = Headliner_df['Расширенная UTM-метка Term'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Term'].count())
print(Headliner_df['Расширенная UTM-метка Term'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Term'].head()

0                        нет
1                        нет
2                        нет
3                        нет
4    46117472944_46117472944
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Term'].tail(20)

4739                        нет
4740                        нет
4741                        нет
4742                        нет
4743                        нет
4744                        нет
4745                        нет
4746                        нет
4747                        нет
4748                        нет
4749    44083962195_44083962195
4750                        нет
4751                        нет
4752    46118338566_46118338566
4753    46117476238_46117476238
4754    46117472903_46117472903
4755    44083963615_44083963615
4756    44083963615_44083963615
4757    44084907889_44084907889
4758                           
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Term'][Headliner_df['Расширенная UTM-метка Term'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Term'
Headliner_df['Расширенная UTM-метка Term'] = Headliner_df['Расширенная UTM-метка Term'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Term'][Headliner_df['Расширенная UTM-метка Term'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Term'].tail(20)

4739                        нет
4740                        нет
4741                        нет
4742                        нет
4743                        нет
4744                        нет
4745                        нет
4746                        нет
4747                        нет
4748                        нет
4749    44083962195_44083962195
4750                        нет
4751                        нет
4752    46118338566_46118338566
4753    46117476238_46117476238
4754    46117472903_46117472903
4755    44083963615_44083963615
4756    44083963615_44083963615
4757    44084907889_44084907889
4758                        нет
Name: Расширенная UTM-метка Term, dtype: object

##### Расширенная UTM-метка Referrer

In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Referrer'].count())
print(Headliner_df['Расширенная UTM-метка Referrer'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Referrer'
Headliner_df['Расширенная UTM-метка Referrer'] = Headliner_df['Расширенная UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Referrer'].count())
print(Headliner_df['Расширенная UTM-метка Referrer'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'][Headliner_df['Расширенная UTM-метка Referrer'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Referrer'
Headliner_df['Расширенная UTM-метка Referrer'] = Headliner_df['Расширенная UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'][Headliner_df['Расширенная UTM-метка Referrer'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Referrer'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Расширенная UTM-метка Referrer, dtype: object

##### Расширенная UTM-метка Expid

In [ ]:
Headliner_df['Расширенная UTM-метка Expid'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
print(Headliner_df['Расширенная UTM-метка Expid'].count())
print(Headliner_df['Расширенная UTM-метка Expid'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Expid'
Headliner_df['Расширенная UTM-метка Expid'] = Headliner_df['Расширенная UTM-метка Expid'].fillna('нет')

In [ ]:
print(Headliner_df['Расширенная UTM-метка Expid'].count())
print(Headliner_df['Расширенная UTM-метка Expid'].shape[0])

4759
4759


In [ ]:
Headliner_df['Расширенная UTM-метка Expid'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Expid'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
Headliner_df['Расширенная UTM-метка Expid'][Headliner_df['Расширенная UTM-метка Expid'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Expid'
Headliner_df['Расширенная UTM-метка Expid'] = Headliner_df['Расширенная UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Расширенная UTM-метка Expid'][Headliner_df['Расширенная UTM-метка Expid'] == ''].count()

0

In [ ]:
Headliner_df['Расширенная UTM-метка Expid'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Расширенная UTM-метка Expid, dtype: object

##### Длительность разговора

In [ ]:
Headliner_df['Длительность разговора'].head()

0    00:02:50
1    00:00:24
2    00:03:02
3    00:00:00
4    00:03:08
Name: Длительность разговора, dtype: object

In [ ]:
print(Headliner_df['Длительность разговора'].count())
print(Headliner_df['Длительность разговора'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Длительность разговора'
Headliner_df['Длительность разговора'] = pd.to_datetime(Headliner_df['Длительность разговора']).dt.time

In [ ]:
Headliner_df['Длительность разговора'].head()

0    00:02:50
1    00:00:24
2    00:03:02
3    00:00:00
4    00:03:08
Name: Длительность разговора, dtype: object

In [ ]:
Headliner_df['Длительность разговора'].tail(20)

4739    00:00:15
4740    00:00:12
4741    00:00:00
4742    00:00:00
4743    00:00:00
4744    00:00:04
4745    00:01:06
4746    00:00:00
4747    00:00:00
4748    00:05:10
4749    00:00:00
4750    00:08:32
4751    00:00:17
4752    00:04:06
4753    00:06:43
4754    00:01:23
4755    00:00:00
4756    00:02:58
4757    00:01:49
4758    00:09:07
Name: Длительность разговора, dtype: object

##### Длительность постобработки

In [ ]:
Headliner_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
print(Headliner_df['Длительность постобработки'].count())
print(Headliner_df['Длительность постобработки'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Длительность постобработки'
Headliner_df['Длительность постобработки'] = pd.to_datetime(Headliner_df['Длительность постобработки']).dt.time

In [ ]:
Headliner_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
Headliner_df['Длительность постобработки'].tail(20)

4739    00:00:00
4740    00:00:00
4741    00:00:00
4742    00:00:00
4743    00:00:00
4744    00:00:00
4745    00:00:00
4746    00:00:00
4747    00:00:00
4748    00:00:00
4749    00:00:00
4750    00:00:00
4751    00:00:00
4752    00:00:00
4753    00:00:00
4754    00:00:00
4755    00:00:00
4756    00:00:00
4757    00:00:00
4758    00:00:00
Name: Длительность постобработки, dtype: object

##### Чистая длительность разговора

In [ ]:
Headliner_df['Чистая длительность разговора'].head()

0    00:02:50
1    00:00:24
2    00:03:01
3    00:00:00
4    00:03:08
Name: Чистая длительность разговора, dtype: object

In [ ]:
print(Headliner_df['Чистая длительность разговора'].count())
print(Headliner_df['Чистая длительность разговора'].shape[0])

4759
4759


In [ ]:
# Приводим тип данных столбца 'Чистая длительность разговора'
Headliner_df['Чистая длительность разговора'] = pd.to_datetime(Headliner_df['Чистая длительность разговора']).dt.time

In [ ]:
Headliner_df['Чистая длительность разговора'].head()

0    00:02:50
1    00:00:24
2    00:03:01
3    00:00:00
4    00:03:08
Name: Чистая длительность разговора, dtype: object

In [ ]:
Headliner_df['Чистая длительность разговора'].tail(20)

4739    00:00:15
4740    00:00:12
4741    00:00:00
4742    00:00:00
4743    00:00:00
4744    00:00:04
4745    00:01:06
4746    00:00:00
4747    00:00:00
4748    00:05:09
4749    00:00:00
4750    00:08:31
4751    00:00:16
4752    00:04:06
4753    00:06:42
4754    00:01:23
4755    00:00:00
4756    00:02:57
4757    00:01:49
4758    00:09:06
Name: Чистая длительность разговора, dtype: object

##### AdLense-метка ef_id

In [ ]:
Headliner_df['AdLense-метка ef_id'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: AdLense-метка ef_id, dtype: object

In [ ]:
print(Headliner_df['AdLense-метка ef_id'].count())
print(Headliner_df['AdLense-метка ef_id'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'AdLense-метка ef_id'
Headliner_df['AdLense-метка ef_id'] = Headliner_df['AdLense-метка ef_id'].fillna('нет')

In [ ]:
print(Headliner_df['AdLense-метка ef_id'].count())
print(Headliner_df['AdLense-метка ef_id'].shape[0])

4759
4759


In [ ]:
Headliner_df['AdLense-метка ef_id'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Headliner_df['AdLense-метка ef_id'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Headliner_df['AdLense-метка ef_id'][Headliner_df['AdLense-метка ef_id'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'AdLense-метка ef_id'
Headliner_df['AdLense-метка ef_id'] = Headliner_df['AdLense-метка ef_id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['AdLense-метка ef_id'][Headliner_df['AdLense-метка ef_id'] == ''].count()

0

In [ ]:
Headliner_df['AdLense-метка ef_id'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Headliner_df['AdLense-метка ef_id'][Headliner_df['AdLense-метка ef_id'] == 'нет'].count()

4759

In [ ]:
Headliner_df['AdLense-метка ef_id'][Headliner_df['AdLense-метка ef_id'] != 'нет'].count()

0

##### Метка yclid

In [ ]:
Headliner_df['Метка yclid'].head()

0                       
1                       
2                       
3                       
4    4362316331213197338
Name: Метка yclid, dtype: object

In [ ]:
print(Headliner_df['Метка yclid'].count())
print(Headliner_df['Метка yclid'].shape[0])

4759
4759


In [ ]:
# Меняем '' на '0' в столбце 'Метка yclid'
Headliner_df['Метка yclid'] = Headliner_df['Метка yclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['Метка yclid'].head()

0                      0
1                      0
2                      0
3                      0
4    4362316331213197338
Name: Метка yclid, dtype: object

In [ ]:
Headliner_df['Метка yclid'].tail(20)

4739                       0
4740                       0
4741                       0
4742                       0
4743                       0
4744                       0
4745                       0
4746                       0
4747                       0
4748                       0
4749    17806418073444155391
4750                       0
4751                       0
4752     4303265393218755384
4753     4130512236344056618
4754     4070033519714573204
4755     3634145067036383429
4756     3634145067036383429
4757     3496300806044719944
4758                       0
Name: Метка yclid, dtype: object

In [ ]:
Headliner_df['Метка yclid'][Headliner_df['Метка yclid'] == ''].count()

0

##### Метка ymclid

In [ ]:
Headliner_df['Метка ymclid'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Метка ymclid, dtype: object

In [ ]:
print(Headliner_df['Метка ymclid'].count())
print(Headliner_df['Метка ymclid'].shape[0])

7
4759


In [ ]:
# Меняем '' на '0' в столбце 'Метка ymclid'
Headliner_df['Метка ymclid'] = Headliner_df['Метка ymclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['Метка ymclid'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752      0
4753      0
4754      0
4755      0
4756      0
4757      0
4758      0
Name: Метка ymclid, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'Метка ymclid'
Headliner_df['Метка ymclid'] = Headliner_df['Метка ymclid'].fillna('0')

In [ ]:
print(Headliner_df['Метка ymclid'].count())
print(Headliner_df['Метка ymclid'].shape[0])

4759
4759


In [ ]:
Headliner_df['Метка ymclid'].head()

0    0
1    0
2    0
3    0
4    0
Name: Метка ymclid, dtype: object

In [ ]:
Headliner_df['Метка ymclid'].tail(20)

4739    0
4740    0
4741    0
4742    0
4743    0
4744    0
4745    0
4746    0
4747    0
4748    0
4749    0
4750    0
4751    0
4752    0
4753    0
4754    0
4755    0
4756    0
4757    0
4758    0
Name: Метка ymclid, dtype: object

In [ ]:
Headliner_df['Метка ymclid'][Headliner_df['Метка ymclid'] == ''].count()

0

In [ ]:
Headliner_df['Метка ymclid'][Headliner_df['Метка ymclid'] == '0'].count()

4759

In [ ]:
Headliner_df['Метка ymclid'][Headliner_df['Метка ymclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка ymclid'
Headliner_df['Метка ymclid'] = Headliner_df['Метка ymclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Headliner_df['Метка ymclid'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Метка ymclid, dtype: object

##### Метка gclid

In [ ]:
Headliner_df['Метка gclid'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Метка gclid, dtype: object

In [ ]:
print(Headliner_df['Метка gclid'].count())
print(Headliner_df['Метка gclid'].shape[0])

7
4759


In [ ]:
# Меняем '' на '0' в столбце 'Метка gclid'
Headliner_df['Метка gclid'] = Headliner_df['Метка gclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['Метка gclid'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752      0
4753      0
4754      0
4755      0
4756      0
4757      0
4758      0
Name: Метка gclid, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'Метка gclid'
Headliner_df['Метка gclid'] = Headliner_df['Метка gclid'].fillna('0')

In [ ]:
print(Headliner_df['Метка gclid'].count())
print(Headliner_df['Метка gclid'].shape[0])

4759
4759


In [ ]:
Headliner_df['Метка gclid'].tail(20)

4739    0
4740    0
4741    0
4742    0
4743    0
4744    0
4745    0
4746    0
4747    0
4748    0
4749    0
4750    0
4751    0
4752    0
4753    0
4754    0
4755    0
4756    0
4757    0
4758    0
Name: Метка gclid, dtype: object

In [ ]:
Headliner_df['Метка gclid'][Headliner_df['Метка gclid'] == ''].count()

0

In [ ]:
Headliner_df['Метка gclid'][Headliner_df['Метка gclid'] == '0'].count()

4759

In [ ]:
Headliner_df['Метка gclid'][Headliner_df['Метка gclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка gclid'
Headliner_df['Метка gclid'] = Headliner_df['Метка gclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Headliner_df['Метка gclid'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Метка gclid, dtype: object

##### Метка from

In [ ]:
Headliner_df['Метка from'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Метка from, dtype: object

In [ ]:
print(Headliner_df['Метка from'].count())
print(Headliner_df['Метка from'].shape[0])

7
4759


In [ ]:
# Меняем '' на '0' в столбце 'Метка from'
Headliner_df['Метка from'] = Headliner_df['Метка from'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['Метка from'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752      0
4753      0
4754      0
4755      0
4756      0
4757      0
4758      0
Name: Метка from, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'Метка from'
Headliner_df['Метка from'] = Headliner_df['Метка from'].fillna('0')

In [ ]:
print(Headliner_df['Метка from'].count())
print(Headliner_df['Метка from'].shape[0])

4759
4759


In [ ]:
Headliner_df['Метка from'].tail(20)

4739    0
4740    0
4741    0
4742    0
4743    0
4744    0
4745    0
4746    0
4747    0
4748    0
4749    0
4750    0
4751    0
4752    0
4753    0
4754    0
4755    0
4756    0
4757    0
4758    0
Name: Метка from, dtype: object

In [ ]:
Headliner_df['Метка from'][Headliner_df['Метка from'] == ''].count()

0

In [ ]:
Headliner_df['Метка from'][Headliner_df['Метка from'] == '0'].count()

4759

In [ ]:
Headliner_df['Метка from'][Headliner_df['Метка from'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка from'
Headliner_df['Метка from'] = Headliner_df['Метка from'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Headliner_df['Метка from'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Метка from, dtype: object

##### OS-метка service-name

In [ ]:
Headliner_df['OS-метка service-name'].tail(20)

4739                 NaN
4740                 NaN
4741                 NaN
4742                 NaN
4743                 NaN
4744                 NaN
4745                 NaN
4746                 NaN
4747                 NaN
4748                 NaN
4749    direct.yandex.ru
4750                 NaN
4751                 NaN
4752                    
4753                    
4754                    
4755                    
4756                    
4757                    
4758                    
Name: OS-метка service-name, dtype: object

In [ ]:
print(Headliner_df['OS-метка service-name'].count())
print(Headliner_df['OS-метка service-name'].shape[0])

36
4759


In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка service-name'
Headliner_df['OS-метка service-name'] = Headliner_df['OS-метка service-name'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['OS-метка service-name'].tail(20)

4739                 NaN
4740                 NaN
4741                 NaN
4742                 NaN
4743                 NaN
4744                 NaN
4745                 NaN
4746                 NaN
4747                 NaN
4748                 NaN
4749    direct.yandex.ru
4750                 NaN
4751                 NaN
4752                 нет
4753                 нет
4754                 нет
4755                 нет
4756                 нет
4757                 нет
4758                 нет
Name: OS-метка service-name, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка service-name'
Headliner_df['OS-метка service-name'] = Headliner_df['OS-метка service-name'].fillna('нет')

In [ ]:
Headliner_df['OS-метка service-name'].tail(20)

4739                 нет
4740                 нет
4741                 нет
4742                 нет
4743                 нет
4744                 нет
4745                 нет
4746                 нет
4747                 нет
4748                 нет
4749    direct.yandex.ru
4750                 нет
4751                 нет
4752                 нет
4753                 нет
4754                 нет
4755                 нет
4756                 нет
4757                 нет
4758                 нет
Name: OS-метка service-name, dtype: object

In [ ]:
Headliner_df['OS-метка service-name'][Headliner_df['OS-метка service-name'] == ''].count()

0

In [ ]:
Headliner_df['OS-метка service-name'][Headliner_df['OS-метка service-name'] == 'нет'].count()

4730

In [ ]:
Headliner_df['OS-метка service-name'][Headliner_df['OS-метка service-name'] != 'нет'].count()

29

##### OS-метка campaign-id

In [ ]:
Headliner_df['OS-метка campaign-id'].tail(20)

4739           NaN
4740           NaN
4741           NaN
4742           NaN
4743           NaN
4744           NaN
4745           NaN
4746           NaN
4747           NaN
4748           NaN
4749    85809500.0
4750           NaN
4751           NaN
4752              
4753              
4754              
4755              
4756              
4757              
4758              
Name: OS-метка campaign-id, dtype: object

In [ ]:
print(Headliner_df['OS-метка campaign-id'].count())
print(Headliner_df['OS-метка campaign-id'].shape[0])

36
4759


In [ ]:
# Меняем '' на '0' в столбце 'OS-метка campaign-id'
Headliner_df['OS-метка campaign-id'] = Headliner_df['OS-метка campaign-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['OS-метка campaign-id'].tail(20)

4739           NaN
4740           NaN
4741           NaN
4742           NaN
4743           NaN
4744           NaN
4745           NaN
4746           NaN
4747           NaN
4748           NaN
4749    85809500.0
4750           NaN
4751           NaN
4752             0
4753             0
4754             0
4755             0
4756             0
4757             0
4758             0
Name: OS-метка campaign-id, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка campaign-id'
Headliner_df['OS-метка campaign-id'] = Headliner_df['OS-метка campaign-id'].fillna('0')

In [ ]:
print(Headliner_df['OS-метка campaign-id'].count())
print(Headliner_df['OS-метка campaign-id'].shape[0])

4759
4759


In [ ]:
Headliner_df['OS-метка campaign-id'].tail(20)

4739             0
4740             0
4741             0
4742             0
4743             0
4744             0
4745             0
4746             0
4747             0
4748             0
4749    85809500.0
4750             0
4751             0
4752             0
4753             0
4754             0
4755             0
4756             0
4757             0
4758             0
Name: OS-метка campaign-id, dtype: object

In [ ]:
# Приводим тип данных столбца 'OS-метка campaign-id'
Headliner_df = Headliner_df.astype({'OS-метка campaign-id': int})

In [ ]:
Headliner_df['OS-метка campaign-id'].tail(20)

4739           0
4740           0
4741           0
4742           0
4743           0
4744           0
4745           0
4746           0
4747           0
4748           0
4749    85809500
4750           0
4751           0
4752           0
4753           0
4754           0
4755           0
4756           0
4757           0
4758           0
Name: OS-метка campaign-id, dtype: int64

##### OS-метка ad-id

In [ ]:
Headliner_df['OS-метка ad-id'].tail(20)

4739              NaN
4740              NaN
4741              NaN
4742              NaN
4743              NaN
4744              NaN
4745              NaN
4746              NaN
4747              NaN
4748              NaN
4749    13836463555.0
4750              NaN
4751              NaN
4752                 
4753                 
4754                 
4755                 
4756                 
4757                 
4758                 
Name: OS-метка ad-id, dtype: object

In [ ]:
print(Headliner_df['OS-метка ad-id'].count())
print(Headliner_df['OS-метка ad-id'].shape[0])

36
4759


In [ ]:
# Меняем '' на '0' в столбце 'OS-метка ad-id'
Headliner_df['OS-метка ad-id'] = Headliner_df['OS-метка ad-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Headliner_df['OS-метка ad-id'].tail(20)

4739              NaN
4740              NaN
4741              NaN
4742              NaN
4743              NaN
4744              NaN
4745              NaN
4746              NaN
4747              NaN
4748              NaN
4749    13836463555.0
4750              NaN
4751              NaN
4752                0
4753                0
4754                0
4755                0
4756                0
4757                0
4758                0
Name: OS-метка ad-id, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка ad-id'
Headliner_df['OS-метка ad-id'] = Headliner_df['OS-метка ad-id'].fillna('0')

In [ ]:
print(Headliner_df['OS-метка ad-id'].count())
print(Headliner_df['OS-метка ad-id'].shape[0])

4759
4759


In [ ]:
Headliner_df['OS-метка ad-id'].tail(20)

4739                0
4740                0
4741                0
4742                0
4743                0
4744                0
4745                0
4746                0
4747                0
4748                0
4749    13836463555.0
4750                0
4751                0
4752                0
4753                0
4754                0
4755                0
4756                0
4757                0
4758                0
Name: OS-метка ad-id, dtype: object

In [ ]:
# Приводим тип данных столбца 'OS-метка ad-id'
Headliner_df = Headliner_df.astype({'OS-метка ad-id': int})

In [ ]:
Headliner_df['OS-метка ad-id'].tail(20)

4739              0
4740              0
4741              0
4742              0
4743              0
4744              0
4745              0
4746              0
4747              0
4748              0
4749    13836463555
4750              0
4751              0
4752              0
4753              0
4754              0
4755              0
4756              0
4757              0
4758              0
Name: OS-метка ad-id, dtype: int64

##### OS-метка source-id

In [ ]:
Headliner_df['OS-метка source-id'].tail(20)

4739                  NaN
4740                  NaN
4741                  NaN
4742                  NaN
4743                  NaN
4744                  NaN
4745                  NaN
4746                  NaN
4747                  NaN
4748                  NaN
4749    yandex.ru:premium
4750                  NaN
4751                  NaN
4752                     
4753                     
4754                     
4755                     
4756                     
4757                     
4758                     
Name: OS-метка source-id, dtype: object

In [ ]:
print(Headliner_df['OS-метка source-id'].count())
print(Headliner_df['OS-метка source-id'].shape[0])

36
4759


In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка source-id'
Headliner_df['OS-метка source-id'] = Headliner_df['OS-метка source-id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['OS-метка source-id'].tail(20)

4739                  NaN
4740                  NaN
4741                  NaN
4742                  NaN
4743                  NaN
4744                  NaN
4745                  NaN
4746                  NaN
4747                  NaN
4748                  NaN
4749    yandex.ru:premium
4750                  NaN
4751                  NaN
4752                  нет
4753                  нет
4754                  нет
4755                  нет
4756                  нет
4757                  нет
4758                  нет
Name: OS-метка source-id, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка source-id'
Headliner_df['OS-метка source-id'] = Headliner_df['OS-метка source-id'].fillna('нет')

In [ ]:
print(Headliner_df['OS-метка source-id'].count())
print(Headliner_df['OS-метка source-id'].shape[0])

4759
4759


In [ ]:
Headliner_df['OS-метка source-id'].tail(20)

4739                  нет
4740                  нет
4741                  нет
4742                  нет
4743                  нет
4744                  нет
4745                  нет
4746                  нет
4747                  нет
4748                  нет
4749    yandex.ru:premium
4750                  нет
4751                  нет
4752                  нет
4753                  нет
4754                  нет
4755                  нет
4756                  нет
4757                  нет
4758                  нет
Name: OS-метка source-id, dtype: object

In [ ]:
Headliner_df['OS-метка source-id'][Headliner_df['OS-метка source-id'] == ''].count()

0

In [ ]:
Headliner_df['OS-метка source-id'][Headliner_df['OS-метка source-id'] == 'нет'].count()

4730

In [ ]:
Headliner_df['OS-метка source-id'][Headliner_df['OS-метка source-id'] != 'нет'].count()

29

##### UTM-метка Term

In [ ]:
Headliner_df['UTM-метка Term'].tail(20)

4739                        NaN
4740                        NaN
4741                        NaN
4742                        NaN
4743                        NaN
4744                        NaN
4745                        NaN
4746                        NaN
4747                        NaN
4748                        NaN
4749    44083962195_44083962195
4750                        NaN
4751                        NaN
4752    46118338566_46118338566
4753    46117476238_46117476238
4754    46117472903_46117472903
4755    44083963615_44083963615
4756    44083963615_44083963615
4757    44084907889_44084907889
4758                           
Name: UTM-метка Term, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Term'].count())
print(Headliner_df['UTM-метка Term'].shape[0])

478
4759


In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Term'
Headliner_df['UTM-метка Term'] = Headliner_df['UTM-метка Term'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Term'].tail(20)

4739                        NaN
4740                        NaN
4741                        NaN
4742                        NaN
4743                        NaN
4744                        NaN
4745                        NaN
4746                        NaN
4747                        NaN
4748                        NaN
4749    44083962195_44083962195
4750                        NaN
4751                        NaN
4752    46118338566_46118338566
4753    46117476238_46117476238
4754    46117472903_46117472903
4755    44083963615_44083963615
4756    44083963615_44083963615
4757    44084907889_44084907889
4758                        нет
Name: UTM-метка Term, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Term'
Headliner_df['UTM-метка Term'] = Headliner_df['UTM-метка Term'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Term'].count())
print(Headliner_df['UTM-метка Term'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Term'].tail(20)

4739                        нет
4740                        нет
4741                        нет
4742                        нет
4743                        нет
4744                        нет
4745                        нет
4746                        нет
4747                        нет
4748                        нет
4749    44083962195_44083962195
4750                        нет
4751                        нет
4752    46118338566_46118338566
4753    46117476238_46117476238
4754    46117472903_46117472903
4755    44083963615_44083963615
4756    44083963615_44083963615
4757    44084907889_44084907889
4758                        нет
Name: UTM-метка Term, dtype: object

In [ ]:
Headliner_df['UTM-метка Term'][Headliner_df['UTM-метка Term'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Term'][Headliner_df['UTM-метка Term'] == 'нет'].count()

4282

In [ ]:
Headliner_df['UTM-метка Term'][Headliner_df['UTM-метка Term'] != 'нет'].count()

477

##### UTM-метка Referrer

In [ ]:
Headliner_df['UTM-метка Referrer'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: UTM-метка Referrer, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Referrer'].count())
print(Headliner_df['UTM-метка Referrer'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Referrer'
Headliner_df['UTM-метка Referrer'] = Headliner_df['UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Referrer'].count())
print(Headliner_df['UTM-метка Referrer'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Referrer'][Headliner_df['UTM-метка Referrer'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Referrer'
Headliner_df['UTM-метка Referrer'] = Headliner_df['UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Referrer'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: UTM-метка Referrer, dtype: object

In [ ]:
Headliner_df['UTM-метка Referrer'][Headliner_df['UTM-метка Referrer'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Referrer'][Headliner_df['UTM-метка Referrer'] == 'нет'].count()

4759

In [ ]:
Headliner_df['UTM-метка Referrer'][Headliner_df['UTM-метка Referrer'] != 'нет'].count()

0

##### UTM-метка Expid

In [ ]:
Headliner_df['UTM-метка Expid'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: UTM-метка Expid, dtype: object

In [ ]:
print(Headliner_df['UTM-метка Expid'].count())
print(Headliner_df['UTM-метка Expid'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Expid'
Headliner_df['UTM-метка Expid'] = Headliner_df['UTM-метка Expid'].fillna('нет')

In [ ]:
print(Headliner_df['UTM-метка Expid'].count())
print(Headliner_df['UTM-метка Expid'].shape[0])

4759
4759


In [ ]:
Headliner_df['UTM-метка Expid'][Headliner_df['UTM-метка Expid'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Expid'
Headliner_df['UTM-метка Expid'] = Headliner_df['UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['UTM-метка Expid'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: UTM-метка Expid, dtype: object

In [ ]:
Headliner_df['UTM-метка Expid'][Headliner_df['UTM-метка Expid'] == ''].count()

0

In [ ]:
Headliner_df['UTM-метка Expid'][Headliner_df['UTM-метка Expid'] == 'нет'].count()

4759

In [ ]:
Headliner_df['UTM-метка Expid'][Headliner_df['UTM-метка Expid'] != 'нет'].count()

0

##### Посадочная страница

In [ ]:
Headliner_df['Посадочная страница'].tail(20)

4739                                                                                                                                                                                                                                                                                                                                   NaN
4740                                                                                                                                                                                                                                                                                                                                   NaN
4741                                                                                                                                                                                                                                                                                                                                   NaN
4742   

In [ ]:
print(Headliner_df['Посадочная страница'].count())
print(Headliner_df['Посадочная страница'].shape[0])

1189
4759


In [ ]:
# Избавляемся от пустот в столбце 'Посадочная страница'
Headliner_df['Посадочная страница'] = Headliner_df['Посадочная страница'].fillna('нет')

In [ ]:
print(Headliner_df['Посадочная страница'].count())
print(Headliner_df['Посадочная страница'].shape[0])

4759
4759


In [ ]:
Headliner_df['Посадочная страница'][Headliner_df['Посадочная страница'] == ''].count()

0

In [ ]:
Headliner_df['Посадочная страница'].tail(20)

4739                                                                                                                                                                                                                                                                                                                                   нет
4740                                                                                                                                                                                                                                                                                                                                   нет
4741                                                                                                                                                                                                                                                                                                                                   нет
4742   

##### Первая рекламная кампания

In [ ]:
Headliner_df['Первая рекламная кампания'].tail(20)

4739    Посетители без рекламной кампании
4740    Посетители без рекламной кампании
4741    Посетители без рекламной кампании
4742                        Яндекс Директ
4743                        Яндекс Директ
4744    Посетители без рекламной кампании
4745                      {Яндекс Директ}
4746    Посетители без рекламной кампании
4747    Посетители без рекламной кампании
4748                      {Яндекс Директ}
4749                        Яндекс Директ
4750    Посетители без рекламной кампании
4751    Посетители без рекламной кампании
4752                        Яндекс Директ
4753                        Яндекс Директ
4754                        Яндекс Директ
4755                       Google Визитка
4756                        Яндекс Директ
4757                        Яндекс Директ
4758                Поисковое продвижение
Name: Первая рекламная кампания, dtype: object

In [ ]:
print(Headliner_df['Первая рекламная кампания'].count())
print(Headliner_df['Первая рекламная кампания'].shape[0])

4759
4759


In [ ]:
Headliner_df['Первая рекламная кампания'][Headliner_df['Первая рекламная кампания'] == ''].count()

0

##### Количество посещений

In [ ]:
Headliner_df['Количество посещений'].tail(20)

4739     NaN
4740     NaN
4741     NaN
4742       3
4743       3
4744     NaN
4745    {12}
4746       1
4747       1
4748    {12}
4749       1
4750     NaN
4751     NaN
4752       5
4753       1
4754       1
4755       1
4756       1
4757       1
4758       1
Name: Количество посещений, dtype: object

In [ ]:
print(Headliner_df['Количество посещений'].count())
print(Headliner_df['Количество посещений'].shape[0])

2044
4759


In [ ]:
# Избавляемся от пустот в столбце 'Количество посещений'
Headliner_df['Количество посещений'] = Headliner_df['Количество посещений'].fillna('0')

In [ ]:
print(Headliner_df['Количество посещений'].count())
print(Headliner_df['Количество посещений'].shape[0])

4759
4759


In [ ]:
Headliner_df['Количество посещений'][Headliner_df['Количество посещений'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Количество посещений'
Headliner_df['Количество посещений'] = Headliner_df['Количество посещений'].apply(lambda x: x.strip('{}'))

In [ ]:
Headliner_df['Количество посещений'].tail(20)

4739     0
4740     0
4741     0
4742     3
4743     3
4744     0
4745    12
4746     1
4747     1
4748    12
4749     1
4750     0
4751     0
4752     5
4753     1
4754     1
4755     1
4756     1
4757     1
4758     1
Name: Количество посещений, dtype: object

In [ ]:
# Приводим тип данных столбца 'Количество посещений'
Headliner_df = Headliner_df.astype({'Количество посещений': int})

In [ ]:
Headliner_df['Количество посещений'].tail(20)

4739     0
4740     0
4741     0
4742     3
4743     3
4744     0
4745    12
4746     1
4747     1
4748    12
4749     1
4750     0
4751     0
4752     5
4753     1
4754     1
4755     1
4756     1
4757     1
4758     1
Name: Количество посещений, dtype: int64

##### Тип посетителя

In [ ]:
Headliner_df['Тип посетителя'].tail(20)

4739      Не заполнен
4740      Не заполнен
4741      Не заполнен
4742      Вернувшийся
4743      Вернувшийся
4744      Не заполнен
4745    {Не заполнен}
4746            Новый
4747            Новый
4748    {Не заполнен}
4749            Новый
4750      Не заполнен
4751      Не заполнен
4752      Вернувшийся
4753            Новый
4754            Новый
4755            Новый
4756            Новый
4757            Новый
4758            Новый
Name: Тип посетителя, dtype: object

In [ ]:
print(Headliner_df['Тип посетителя'].count())
print(Headliner_df['Тип посетителя'].shape[0])

4759
4759


In [ ]:
Headliner_df['Тип посетителя'][Headliner_df['Тип посетителя'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Тип посетителя'
Headliner_df['Тип посетителя'] = Headliner_df['Тип посетителя'].apply(lambda x: x.strip('{}'))

In [ ]:
Headliner_df['Тип посетителя'].tail(20)

4739    Не заполнен
4740    Не заполнен
4741    Не заполнен
4742    Вернувшийся
4743    Вернувшийся
4744    Не заполнен
4745    Не заполнен
4746          Новый
4747          Новый
4748    Не заполнен
4749          Новый
4750    Не заполнен
4751    Не заполнен
4752    Вернувшийся
4753          Новый
4754          Новый
4755          Новый
4756          Новый
4757          Новый
4758          Новый
Name: Тип посетителя, dtype: object

##### Тип устройства

In [ ]:
Headliner_df['Тип устройства'].tail(20)

4739      Прочее
4740      Прочее
4741      Прочее
4742    Смартфон
4743    Смартфон
4744      Прочее
4745      Прочее
4746          ПК
4747          ПК
4748      Прочее
4749    Смартфон
4750      Прочее
4751      Прочее
4752          ПК
4753          ПК
4754          ПК
4755          ПК
4756          ПК
4757          ПК
4758          ПК
Name: Тип устройства, dtype: object

In [ ]:
print(Headliner_df['Тип устройства'].count())
print(Headliner_df['Тип устройства'].shape[0])

4759
4759


In [ ]:
Headliner_df['Тип устройства'][Headliner_df['Тип устройства'] == ''].count()

0

##### Client ID Google Analytics

In [ ]:
Headliner_df['Client ID Google Analytics'].tail(20)

4739                      NaN
4740                      NaN
4741                      NaN
4742         141701139.168287
4743         141701139.168287
4744                      NaN
4745                      NaN
4746         907380207.168293
4747         907380207.168293
4748                      NaN
4749         556792878.168288
4750                      NaN
4751                      NaN
4752    1132129771.1689948652
4753     263963569.1689929455
4754     298334486.1689698873
4755    1279923173.1688035965
4756    1279923173.1688035965
4757     193897556.1670485568
4758    2130376127.1683117084
Name: Client ID Google Analytics, dtype: object

In [ ]:
print(Headliner_df['Client ID Google Analytics'].count())
print(Headliner_df['Client ID Google Analytics'].shape[0])

1111
4759


In [ ]:
# Избавляемся от пустот в столбце 'Client ID Google Analytics'
Headliner_df['Client ID Google Analytics'] = Headliner_df['Client ID Google Analytics'].fillna('0')

In [ ]:
print(Headliner_df['Client ID Google Analytics'].count())
print(Headliner_df['Client ID Google Analytics'].shape[0])

4759
4759


In [ ]:
Headliner_df['Client ID Google Analytics'][Headliner_df['Client ID Google Analytics'] == ''].count()

0

In [ ]:
Headliner_df['Client ID Google Analytics'].tail(20)

4739                        0
4740                        0
4741                        0
4742         141701139.168287
4743         141701139.168287
4744                        0
4745                        0
4746         907380207.168293
4747         907380207.168293
4748                        0
4749         556792878.168288
4750                        0
4751                        0
4752    1132129771.1689948652
4753     263963569.1689929455
4754     298334486.1689698873
4755    1279923173.1688035965
4756    1279923173.1688035965
4757     193897556.1670485568
4758    2130376127.1683117084
Name: Client ID Google Analytics, dtype: object

##### ID ресурса Google Analytics

In [ ]:
Headliner_df['ID ресурса Google Analytics'].tail(20)

4739               NaN
4740               NaN
4741               NaN
4742    UA-111037154-1
4743    UA-111037154-1
4744               NaN
4745               NaN
4746    UA-111037154-1
4747    UA-111037154-1
4748               NaN
4749    UA-111037154-1
4750               NaN
4751               NaN
4752    UA-111037154-1
4753    UA-111037154-1
4754    UA-111037154-1
4755    UA-111037154-1
4756    UA-111037154-1
4757    UA-111037154-1
4758    UA-111037154-1
Name: ID ресурса Google Analytics, dtype: object

In [ ]:
print(Headliner_df['ID ресурса Google Analytics'].count())
print(Headliner_df['ID ресурса Google Analytics'].shape[0])

1249
4759


In [ ]:
# Избавляемся от пустот в столбце 'ID ресурса Google Analytics'
Headliner_df['ID ресурса Google Analytics'] = Headliner_df['ID ресурса Google Analytics'].fillna('нет')

In [ ]:
print(Headliner_df['ID ресурса Google Analytics'].count())
print(Headliner_df['ID ресурса Google Analytics'].shape[0])

4759
4759


In [ ]:
Headliner_df['ID ресурса Google Analytics'][Headliner_df['ID ресурса Google Analytics'] == ''].count()

0

In [ ]:
Headliner_df['ID ресурса Google Analytics'].tail(20)

4739               нет
4740               нет
4741               нет
4742    UA-111037154-1
4743    UA-111037154-1
4744               нет
4745               нет
4746    UA-111037154-1
4747    UA-111037154-1
4748               нет
4749    UA-111037154-1
4750               нет
4751               нет
4752    UA-111037154-1
4753    UA-111037154-1
4754    UA-111037154-1
4755    UA-111037154-1
4756    UA-111037154-1
4757    UA-111037154-1
4758    UA-111037154-1
Name: ID ресурса Google Analytics, dtype: object

##### ID счетчика Яндекс.Метрика

In [ ]:
Headliner_df['ID счетчика Яндекс.Метрика'].tail(20)

4739           NaN
4740           NaN
4741           NaN
4742    46973610.0
4743    46973610.0
4744           NaN
4745           NaN
4746    46973610.0
4747    46973610.0
4748           NaN
4749    46973610.0
4750           NaN
4751           NaN
4752      46973610
4753      46973610
4754      46973610
4755      46973610
4756      46973610
4757      46973610
4758      46973610
Name: ID счетчика Яндекс.Метрика, dtype: object

In [ ]:
print(Headliner_df['ID счетчика Яндекс.Метрика'].count())
print(Headliner_df['ID счетчика Яндекс.Метрика'].shape[0])

1249
4759


In [ ]:
# Избавляемся от пустот в столбце 'ID счетчика Яндекс.Метрика'
Headliner_df['ID счетчика Яндекс.Метрика'] = Headliner_df['ID счетчика Яндекс.Метрика'].fillna('0')

In [ ]:
print(Headliner_df['ID счетчика Яндекс.Метрика'].count())
print(Headliner_df['ID счетчика Яндекс.Метрика'].shape[0])

4759
4759


In [ ]:
Headliner_df['ID счетчика Яндекс.Метрика'][Headliner_df['ID счетчика Яндекс.Метрика'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'ID счетчика Яндекс.Метрика'
Headliner_df = Headliner_df.astype({'ID счетчика Яндекс.Метрика': int})

In [ ]:
Headliner_df['ID счетчика Яндекс.Метрика'].tail(20)

4739           0
4740           0
4741           0
4742    46973610
4743    46973610
4744           0
4745           0
4746    46973610
4747    46973610
4748           0
4749    46973610
4750           0
4751           0
4752    46973610
4753    46973610
4754    46973610
4755    46973610
4756    46973610
4757    46973610
4758    46973610
Name: ID счетчика Яндекс.Метрика, dtype: int64

##### URL страницы обращения

In [ ]:
Headliner_df['URL страницы обращения'].tail(20)

4739                                                                                                                                                                                             NaN
4740                                                                                                                                                                                             NaN
4741                                                                                                                                                                                             NaN
4742                                                                                                                                                                    https://head-liner.ru/about/
4743                                                                                                                                                                    https://head-liner.ru/about/
4744           

In [ ]:
print(Headliner_df['URL страницы обращения'].count())
print(Headliner_df['URL страницы обращения'].shape[0])

1249
4759


In [ ]:
# Избавляемся от пустот в столбце 'URL страницы обращения'
Headliner_df['URL страницы обращения'] = Headliner_df['URL страницы обращения'].fillna('нет')

In [ ]:
print(Headliner_df['URL страницы обращения'].count())
print(Headliner_df['URL страницы обращения'].shape[0])

4759
4759


In [ ]:
Headliner_df['URL страницы обращения'][Headliner_df['URL страницы обращения'] == ''].count()

0

In [ ]:
Headliner_df['URL страницы обращения'] = Headliner_df['URL страницы обращения'].apply(lambda x: x.strip())

In [ ]:
Headliner_df['URL страницы обращения'].tail(20)

4739                                                                                                                                                                                             нет
4740                                                                                                                                                                                             нет
4741                                                                                                                                                                                             нет
4742                                                                                                                                                                    https://head-liner.ru/about/
4743                                                                                                                                                                    https://head-liner.ru/about/
4744           

##### Страна

In [ ]:
Headliner_df['Страна'].tail(20)

4739       NaN
4740       NaN
4741       NaN
4742    Россия
4743    Россия
4744       NaN
4745       NaN
4746    Россия
4747    Россия
4748       NaN
4749    Россия
4750       NaN
4751       NaN
4752    Россия
4753    Россия
4754    Россия
4755    Россия
4756    Россия
4757    Россия
4758    Россия
Name: Страна, dtype: object

In [ ]:
print(Headliner_df['Страна'].count())
print(Headliner_df['Страна'].shape[0])

1184
4759


In [ ]:
# Избавляемся от пустот в столбце 'Страна'
Headliner_df['Страна'] = Headliner_df['Страна'].fillna('нет')

In [ ]:
print(Headliner_df['Страна'].count())
print(Headliner_df['Страна'].shape[0])

4759
4759


In [ ]:
Headliner_df['Страна'][Headliner_df['Страна'] == ''].count()

0

In [ ]:
Headliner_df['Страна'].tail(20)

4739       нет
4740       нет
4741       нет
4742    Россия
4743    Россия
4744       нет
4745       нет
4746    Россия
4747    Россия
4748       нет
4749    Россия
4750       нет
4751       нет
4752    Россия
4753    Россия
4754    Россия
4755    Россия
4756    Россия
4757    Россия
4758    Россия
Name: Страна, dtype: object

##### Область

In [ ]:
Headliner_df['Область'].tail(20)

4739                    NaN
4740                    NaN
4741                    NaN
4742                 Москва
4743                 Москва
4744                    NaN
4745                    NaN
4746                 Москва
4747                 Москва
4748                    NaN
4749    Ярославская область
4750                    NaN
4751                    NaN
4752        Санкт-Петербург
4753     Московская область
4754    Саратовская область
4755                 Москва
4756                 Москва
4757                 Москва
4758           Башкортостан
Name: Область, dtype: object

In [ ]:
print(Headliner_df['Область'].count())
print(Headliner_df['Область'].shape[0])

1090
4759


In [ ]:
# Избавляемся от пустот в столбце 'Область'
Headliner_df['Область'] = Headliner_df['Область'].fillna('нет')

In [ ]:
print(Headliner_df['Область'].count())
print(Headliner_df['Область'].shape[0])

4759
4759


In [ ]:
Headliner_df['Область'][Headliner_df['Область'] == ''].count()

0

In [ ]:
Headliner_df['Область'].tail(20)

4739                    нет
4740                    нет
4741                    нет
4742                 Москва
4743                 Москва
4744                    нет
4745                    нет
4746                 Москва
4747                 Москва
4748                    нет
4749    Ярославская область
4750                    нет
4751                    нет
4752        Санкт-Петербург
4753     Московская область
4754    Саратовская область
4755                 Москва
4756                 Москва
4757                 Москва
4758           Башкортостан
Name: Область, dtype: object

##### Город

In [ ]:
Headliner_df['Город'].tail(20)

4739                NaN
4740                NaN
4741                NaN
4742             Москва
4743             Москва
4744                NaN
4745                NaN
4746             Москва
4747             Москва
4748                NaN
4749            Рыбинск
4750                NaN
4751                NaN
4752    Санкт-Петербург
4753           Протвино
4754            Саратов
4755             Москва
4756             Москва
4757             Москва
4758            Салават
Name: Город, dtype: object

In [ ]:
print(Headliner_df['Город'].count())
print(Headliner_df['Город'].shape[0])

1076
4759


In [ ]:
# Избавляемся от пустот в столбце 'Город'
Headliner_df['Город'] = Headliner_df['Город'].fillna('нет')

In [ ]:
print(Headliner_df['Город'].count())
print(Headliner_df['Город'].shape[0])

4759
4759


In [ ]:
Headliner_df['Город'][Headliner_df['Город'] == ''].count()

0

In [ ]:
Headliner_df['Город'].tail(20)

4739                нет
4740                нет
4741                нет
4742             Москва
4743             Москва
4744                нет
4745                нет
4746             Москва
4747             Москва
4748                нет
4749            Рыбинск
4750                нет
4751                нет
4752    Санкт-Петербург
4753           Протвино
4754            Саратов
4755             Москва
4756             Москва
4757             Москва
4758            Салават
Name: Город, dtype: object

##### Поисковая система

In [ ]:
Headliner_df['Поисковая система'].tail(20)

4739       NaN
4740       NaN
4741       NaN
4742       NaN
4743       NaN
4744       NaN
4745       NaN
4746       NaN
4747       NaN
4748       NaN
4749       NaN
4750       NaN
4751       NaN
4752    yandex
4753    yandex
4754    yandex
4755    yandex
4756    yandex
4757    yandex
4758    yandex
Name: Поисковая система, dtype: object

In [ ]:
print(Headliner_df['Поисковая система'].count())
print(Headliner_df['Поисковая система'].shape[0])

505
4759


In [ ]:
# Избавляемся от пустот в столбце 'Поисковая система'
Headliner_df['Поисковая система'] = Headliner_df['Поисковая система'].fillna('нет')

In [ ]:
print(Headliner_df['Поисковая система'].count())
print(Headliner_df['Поисковая система'].shape[0])

4759
4759


In [ ]:
Headliner_df['Поисковая система'][Headliner_df['Поисковая система'] == ''].count()

0

In [ ]:
Headliner_df['Поисковая система'].tail(20)

4739       нет
4740       нет
4741       нет
4742       нет
4743       нет
4744       нет
4745       нет
4746       нет
4747       нет
4748       нет
4749       нет
4750       нет
4751       нет
4752    yandex
4753    yandex
4754    yandex
4755    yandex
4756    yandex
4757    yandex
4758    yandex
Name: Поисковая система, dtype: object

##### Реферер

In [ ]:
Headliner_df['Реферер'].tail(20)

4739                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
print(Headliner_df['Реферер'].count())
print(Headliner_df['Реферер'].shape[0])

716
4759


In [ ]:
# Избавляемся от пустот в столбце 'Реферер'
Headliner_df['Реферер'] = Headliner_df['Реферер'].fillna('нет')

In [ ]:
print(Headliner_df['Реферер'].count())
print(Headliner_df['Реферер'].shape[0])

4759
4759


In [ ]:
Headliner_df['Реферер'][Headliner_df['Реферер'] == ''].count()

0

In [ ]:
Headliner_df['Реферер'].tail(20)

4739                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

##### Домен реферера

In [ ]:
Headliner_df['Домен реферера'].tail(20)

4739              NaN
4740              NaN
4741              NaN
4742    head-liner.ru
4743    head-liner.ru
4744              NaN
4745              NaN
4746    head-liner.ru
4747    head-liner.ru
4748              NaN
4749      away.vk.com
4750              NaN
4751              NaN
4752    www.yandex.ru
4753        yandex.ru
4754        yandex.ru
4755        yandex.ru
4756        yandex.ru
4757        yandex.ru
4758        yandex.ru
Name: Домен реферера, dtype: object

In [ ]:
print(Headliner_df['Домен реферера'].count())
print(Headliner_df['Домен реферера'].shape[0])

716
4759


In [ ]:
# Избавляемся от пустот в столбце 'Домен реферера'
Headliner_df['Домен реферера'] = Headliner_df['Домен реферера'].fillna('нет')

In [ ]:
print(Headliner_df['Домен реферера'].count())
print(Headliner_df['Домен реферера'].shape[0])

4759
4759


In [ ]:
Headliner_df['Домен реферера'][Headliner_df['Домен реферера'] == ''].count()

0

In [ ]:
Headliner_df['Домен реферера'].tail(20)

4739              нет
4740              нет
4741              нет
4742    head-liner.ru
4743    head-liner.ru
4744              нет
4745              нет
4746    head-liner.ru
4747    head-liner.ru
4748              нет
4749      away.vk.com
4750              нет
4751              нет
4752    www.yandex.ru
4753        yandex.ru
4754        yandex.ru
4755        yandex.ru
4756        yandex.ru
4757        yandex.ru
4758        yandex.ru
Name: Домен реферера, dtype: object

##### Сайт

In [ ]:
Headliner_df['Сайт'].tail(20)

4739    head-liner.ru
4740    head-liner.ru
4741    head-liner.ru
4742    head-liner.ru
4743    head-liner.ru
4744    head-liner.ru
4745    head-liner.ru
4746    head-liner.ru
4747    head-liner.ru
4748    head-liner.ru
4749    head-liner.ru
4750    head-liner.ru
4751    head-liner.ru
4752    head-liner.ru
4753    head-liner.ru
4754    head-liner.ru
4755    head-liner.ru
4756    head-liner.ru
4757    head-liner.ru
4758    head-liner.ru
Name: Сайт, dtype: object

In [ ]:
print(Headliner_df['Сайт'].count())
print(Headliner_df['Сайт'].shape[0])

4759
4759


In [ ]:
Headliner_df['Сайт'][Headliner_df['Сайт'] == ''].count()

0

##### Оценка сотрудника

In [ ]:
Headliner_df['Оценка сотрудника'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Оценка сотрудника, dtype: object

In [ ]:
print(Headliner_df['Оценка сотрудника'].count())
print(Headliner_df['Оценка сотрудника'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Оценка сотрудника'
Headliner_df['Оценка сотрудника'] = Headliner_df['Оценка сотрудника'].fillna('нет')

In [ ]:
print(Headliner_df['Оценка сотрудника'].count())
print(Headliner_df['Оценка сотрудника'].shape[0])

4759
4759


In [ ]:
Headliner_df['Оценка сотрудника'][Headliner_df['Оценка сотрудника'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Оценка сотрудника'
Headliner_df['Оценка сотрудника'] = Headliner_df['Оценка сотрудника'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Оценка сотрудника'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Оценка сотрудника, dtype: object

In [ ]:
Headliner_df['Оценка сотрудника'][Headliner_df['Оценка сотрудника'] == ''].count()

0

In [ ]:
Headliner_df['Оценка сотрудника'][Headliner_df['Оценка сотрудника'] == 'нет'].count()

4759

In [ ]:
Headliner_df['Оценка сотрудника'][Headliner_df['Оценка сотрудника'] != 'нет'].count()

0

##### Внутренний звонок

In [ ]:
Headliner_df['Внутренний звонок'].tail(20)

4739    Нет
4740    Нет
4741    Нет
4742    Нет
4743    Нет
4744    Нет
4745    Нет
4746    Нет
4747    Нет
4748    Нет
4749    Нет
4750    Нет
4751    Нет
4752    Нет
4753    Нет
4754    Нет
4755    Нет
4756    Нет
4757    Нет
4758    Нет
Name: Внутренний звонок, dtype: object

In [ ]:
print(Headliner_df['Внутренний звонок'].count())
print(Headliner_df['Внутренний звонок'].shape[0])

4759
4759


In [ ]:
Headliner_df['Внутренний звонок'][Headliner_df['Внутренний звонок'] == ''].count()

0

In [ ]:
Headliner_df['Внутренний звонок'][Headliner_df['Внутренний звонок'] == 'Нет'].count()

4757

In [ ]:
Headliner_df['Внутренний звонок'][Headliner_df['Внутренний звонок'] != 'Нет'].count()

2

In [ ]:
Headliner_df['Внутренний звонок'][Headliner_df['Внутренний звонок'] != 'Нет']

2617    Да
2618    Да
Name: Внутренний звонок, dtype: object

##### ФИО контакта

In [ ]:
Headliner_df['ФИО контакта'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: ФИО контакта, dtype: object

In [ ]:
print(Headliner_df['ФИО контакта'].count())
print(Headliner_df['ФИО контакта'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'ФИО контакта'
Headliner_df['ФИО контакта'] = Headliner_df['ФИО контакта'].fillna('нет')

In [ ]:
print(Headliner_df['ФИО контакта'].count())
print(Headliner_df['ФИО контакта'].shape[0])

4759
4759


In [ ]:
Headliner_df['ФИО контакта'][Headliner_df['ФИО контакта'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'ФИО контакта'
Headliner_df['ФИО контакта'] = Headliner_df['ФИО контакта'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['ФИО контакта'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: ФИО контакта, dtype: object

In [ ]:
Headliner_df['ФИО контакта'][Headliner_df['ФИО контакта'] == ''].count()

0

In [ ]:
Headliner_df['ФИО контакта'][Headliner_df['ФИО контакта'] == 'нет'].count()

4759

In [ ]:
Headliner_df['ФИО контакта'][Headliner_df['ФИО контакта'] != 'нет'].count()

0

##### Имя тренера

In [ ]:
Headliner_df['Имя тренера'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Имя тренера, dtype: object

In [ ]:
print(Headliner_df['Имя тренера'].count())
print(Headliner_df['Имя тренера'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Имя тренера'
Headliner_df['Имя тренера'] = Headliner_df['Имя тренера'].fillna('нет')

In [ ]:
print(Headliner_df['Имя тренера'].count())
print(Headliner_df['Имя тренера'].shape[0])

4759
4759


In [ ]:
Headliner_df['Имя тренера'][Headliner_df['Имя тренера'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Имя тренера'
Headliner_df['Имя тренера'] = Headliner_df['Имя тренера'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Имя тренера'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Имя тренера, dtype: object

In [ ]:
Headliner_df['Имя тренера'][Headliner_df['Имя тренера'] == ''].count()

0

In [ ]:
Headliner_df['Имя тренера'][Headliner_df['Имя тренера'] == 'нет'].count()

4759

In [ ]:
Headliner_df['Имя тренера'][Headliner_df['Имя тренера'] != 'нет'].count()

0

##### Группа, выбранная посетителем

In [ ]:
Headliner_df['Группа, выбранная посетителем'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
print(Headliner_df['Группа, выбранная посетителем'].count())
print(Headliner_df['Группа, выбранная посетителем'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Группа, выбранная посетителем'
Headliner_df['Группа, выбранная посетителем'] = Headliner_df['Группа, выбранная посетителем'].fillna('нет')

In [ ]:
print(Headliner_df['Группа, выбранная посетителем'].count())
print(Headliner_df['Группа, выбранная посетителем'].shape[0])

4759
4759


In [ ]:
Headliner_df['Группа, выбранная посетителем'][Headliner_df['Группа, выбранная посетителем'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Группа, выбранная посетителем'
Headliner_df['Группа, выбранная посетителем'] = Headliner_df['Группа, выбранная посетителем'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Группа, выбранная посетителем'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
Headliner_df['Группа, выбранная посетителем'][Headliner_df['Группа, выбранная посетителем'] == ''].count()

0

In [ ]:
Headliner_df['Группа, выбранная посетителем'][Headliner_df['Группа, выбранная посетителем'] == 'нет'].count()

4759

In [ ]:
Headliner_df['Группа, выбранная посетителем'][Headliner_df['Группа, выбранная посетителем'] != 'нет'].count()

0

##### Регион номера абонента

In [ ]:
Headliner_df['Регион номера абонента'].tail(20)

4739                     г. Москва
4740            РФ / не определено
4741            РФ / не определено
4742            г. Санкт-Петербург
4743            г. Санкт-Петербург
4744            г. Санкт-Петербург
4745            Республика Чувашия
4746            РФ / не определено
4747            РФ / не определено
4748            Республика Чувашия
4749    Российская Федерация 8-800
4750                     г. Москва
4751                     г. Москва
4752                 Пермский край
4753                     г. Москва
4754                     г. Москва
4755            г. Санкт-Петербург
4756                     г. Москва
4757                     г. Москва
4758       Республика Башкортостан
Name: Регион номера абонента, dtype: object

In [ ]:
print(Headliner_df['Регион номера абонента'].count())
print(Headliner_df['Регион номера абонента'].shape[0])

4630
4759


In [ ]:
# Избавляемся от пустот в столбце 'Регион номера абонента'
Headliner_df['Регион номера абонента'] = Headliner_df['Регион номера абонента'].fillna('не определено')

In [ ]:
print(Headliner_df['Регион номера абонента'].count())
print(Headliner_df['Регион номера абонента'].shape[0])

4759
4759


In [ ]:
Headliner_df['Регион номера абонента'][Headliner_df['Регион номера абонента'] == ''].count()

0

##### Страна номера абонента

In [ ]:
Headliner_df['Страна номера абонента'].tail(20)

4739    РФ
4740    РФ
4741    РФ
4742    РФ
4743    РФ
4744    РФ
4745    РФ
4746    РФ
4747    РФ
4748    РФ
4749    РФ
4750    РФ
4751    РФ
4752    РФ
4753    РФ
4754    РФ
4755    РФ
4756    РФ
4757    РФ
4758    РФ
Name: Страна номера абонента, dtype: object

In [ ]:
print(Headliner_df['Страна номера абонента'].count())
print(Headliner_df['Страна номера абонента'].shape[0])

4630
4759


In [ ]:
# Избавляемся от пустот в столбце 'Страна номера абонента'
Headliner_df['Страна номера абонента'] = Headliner_df['Страна номера абонента'].fillna('не определено')

In [ ]:
print(Headliner_df['Страна номера абонента'].count())
print(Headliner_df['Страна номера абонента'].shape[0])

4759
4759


In [ ]:
Headliner_df['Страна номера абонента'][Headliner_df['Страна номера абонента'] == ''].count()

0

##### Контакт из CRM

In [ ]:
Headliner_df['Контакт из CRM'].tail(20)

4739    NaN
4740    NaN
4741    NaN
4742    NaN
4743    NaN
4744    NaN
4745    NaN
4746    NaN
4747    NaN
4748    NaN
4749    NaN
4750    NaN
4751    NaN
4752       
4753       
4754       
4755       
4756       
4757       
4758       
Name: Контакт из CRM, dtype: object

In [ ]:
print(Headliner_df['Контакт из CRM'].count())
print(Headliner_df['Контакт из CRM'].shape[0])

7
4759


In [ ]:
# Избавляемся от пустот в столбце 'Контакт из CRM'
Headliner_df['Контакт из CRM'] = Headliner_df['Контакт из CRM'].fillna('нет')

In [ ]:
print(Headliner_df['Контакт из CRM'].count())
print(Headliner_df['Контакт из CRM'].shape[0])

4759
4759


In [ ]:
Headliner_df['Контакт из CRM'][Headliner_df['Контакт из CRM'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Контакт из CRM'
Headliner_df['Контакт из CRM'] = Headliner_df['Контакт из CRM'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Headliner_df['Контакт из CRM'].tail(20)

4739    нет
4740    нет
4741    нет
4742    нет
4743    нет
4744    нет
4745    нет
4746    нет
4747    нет
4748    нет
4749    нет
4750    нет
4751    нет
4752    нет
4753    нет
4754    нет
4755    нет
4756    нет
4757    нет
4758    нет
Name: Контакт из CRM, dtype: object

In [ ]:
Headliner_df['Контакт из CRM'][Headliner_df['Контакт из CRM'] == ''].count()

0

In [ ]:
Headliner_df['Контакт из CRM'][Headliner_df['Контакт из CRM'] == 'нет'].count()

4759

In [ ]:
Headliner_df['Контакт из CRM'][Headliner_df['Контакт из CRM'] != 'нет'].count()

0

##### Типы данных

In [ ]:
# Смотрим типы данных
Headliner_calls_match_df.dtypes

Дата и время                   datetime64[ns]
Номер абонента                          int64
Номер сотрудника                        int64
Идентификатор сессии звонка             int64
Файл записи звонка                     object
dtype: object

In [ ]:
# Смотрим типы данных
Headliner_df.dtypes

Статус                                    object
Тип                                       object
Дата и время                      datetime64[ns]
Номер абонента                             int64
Виртуальный номер                          int64
Рекламная кампания                        object
Длительность звонка                       object
Длительность ожидания ответа              object
Причина завершения                        object
Сценарий                                  object
Сотрудник                                 object
Комментарий                               object
UTM-метка Source                          object
ID посетителя                              int64
Номер обращения                            int64
Поисковый запрос                          object
Теги                                      object
UTM-метка Campaign                        object
Номер сотрудника                           int64
UTM-метка Content                         object
Client ID Яндекс.Мет

##### Добавление столбцов

In [ ]:
# Смотрим несовпадающие столбцы Primavera_df vs Headliner_df
for column in Primavera_df.columns.tolist():
  if column not in Headliner_df.columns.tolist():
    print(f'Primavera_df[{column}], тип данных: {Primavera_df[column].dtypes}')

Primavera_df[Идентификатор звонка во внешней системе], тип данных: int64
Primavera_df[Длительность обработки], тип данных: int64


In [ ]:
# Добавляем столбец 'Идентификатор звонка во внешней системе' в Headliner_df
Headliner_df['Идентификатор звонка во внешней системе'] = 0

In [ ]:
# Добавляем столбец 'Длительность обработки' в Headliner_df
Headliner_df['Длительность обработки'] = 0

In [ ]:
# Упорядочиваем столбцы в Headliner_df как в Primavera_df
Headliner_df = Headliner_df[Primavera_df.columns]

In [ ]:
# Смотрим несовпадающие столбцы Headliner_calls_match_df vs Headliner_df
for column in Headliner_calls_match_df.columns.tolist():
  if column not in Headliner_df.columns.tolist():
    print(f'Headliner_calls_match_df[{column}], {Headliner_calls_match_df[column].shape[0]} строк, тип данных: {Headliner_calls_match_df[column].dtypes}')

Headliner_calls_match_df[Файл записи звонка], 2045 строк, тип данных: object


In [ ]:
# Сортируем Headliner_df по столбцу 'Идентификатор сессии звонка'
Headliner_df = Headliner_df.sort_values('Идентификатор сессии звонка')

In [ ]:
# Индексируем по порядку
Headliner_df.index = range(Headliner_df.shape[0])

In [ ]:
# Добавляем относительные пути в столбец 'Файл записи звонка' в Headliner_calls_match_df
Headliner_calls_match_df['Файл записи звонка'] = Headliner_calls_match_df['Файл записи звонка'].apply(lambda x: 'Записи звонков/Записи звонков_хедлайнер/' + x)
Headliner_calls_match_df['Файл записи звонка']

956     Записи звонков/Записи звонков_хедлайнер/2023-05-08_16-14-10.247769_from_74955199580_to_74997552563_session_2918538922_talk.mp3
997     Записи звонков/Записи звонков_хедлайнер/2023-05-09_11-53-48.291185_from_79910665524_to_74997552563_session_2919161802_talk.mp3
974         Записи звонков/Записи звонков_хедлайнер/2023-05-09_17-51-30.193682_from_79670165153_to_0188880_session_2919520238_talk.mp3
984     Записи звонков/Записи звонков_хедлайнер/2023-05-09_18-34-04.371381_from_79274431546_to_74997552563_session_2919550530_talk.mp3
965     Записи звонков/Записи звонков_хедлайнер/2023-05-09_20-19-21.050197_from_79274432048_to_74997552563_session_2919615074_talk.mp3
                                                                     ...                                                              
1002    Записи звонков/Записи звонков_хедлайнер/2023-07-19_14-07-03.063955_from_79054502244_to_74959339902_session_3104067170_talk.mp3
2004    Записи звонков/Записи звонков_хедлайнер/2023-07

In [ ]:
# Индексируем по порядку
Headliner_calls_match_df.index = range(Headliner_calls_match_df.shape[0])

In [ ]:
# Добавляем столбец 'Файл записи звонка' в Headliner_df
Headliner_df = Headliner_df.merge(Headliner_calls_match_df[['Идентификатор сессии звонка','Файл записи звонка']], how='outer', on=['Идентификатор сессии звонка'])

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Headliner_df['Файл записи звонка'].count())
print(Headliner_df['Файл записи звонка'].shape[0])

2045
4759


In [ ]:
# Запоняем пустоты в столбце 'Файл записи звонка' значением 'нет файла'
Headliner_df['Файл записи звонка'] = Headliner_df['Файл записи звонка'].fillna('нет файла')

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Headliner_df['Файл записи звонка'].count())
print(Headliner_df['Файл записи звонка'].shape[0])

4759
4759


In [ ]:
# Нет файлов
Headliner_df['Файл записи звонка'][Headliner_df['Файл записи звонка'] == 'нет файла'].count()

2714

In [ ]:
# Есть файлы
Headliner_df['Файл записи звонка'][Headliner_df['Файл записи звонка'] != 'нет файла'].count()

2045

##### Сохранение датафреймов

In [ ]:
# Сохраняем Headliner_df в файл
with open(df_path + 'Headliner_df.pkl', 'wb') as f:
  pkl.dump(Headliner_df, f)

In [ ]:
# Сохраняем Headliner_calls_match_df в файл
with open(df_path + 'Headliner_calls_match_df.pkl', 'wb') as f:
  pkl.dump(Headliner_calls_match_df, f)

#### Primavera

##### Статус

In [ ]:
Primavera_df['Статус'].head()

0      Принятый
1      Принятый
2      Принятый
3    Потерянный
4    Потерянный
Name: Статус, dtype: object

In [ ]:
print(Primavera_df['Статус'].count())
print(Primavera_df['Статус'].shape[0])

3506
3506


In [ ]:
Primavera_df['Статус'].tail(20)

3486    Потерянный
3487    Потерянный
3488    Потерянный
3489    Потерянный
3490    Потерянный
3491    Потерянный
3492    Потерянный
3493    Потерянный
3494    Потерянный
3495    Потерянный
3496      Принятый
3497    Потерянный
3498    Потерянный
3499    Потерянный
3500    Потерянный
3501      Принятый
3502      Принятый
3503    Потерянный
3504    Потерянный
3505    Потерянный
Name: Статус, dtype: object

In [ ]:
Primavera_df['Статус'][Primavera_df['Статус'] == ''].count()

0

##### Тип

In [ ]:
Primavera_df['Тип'].head()

0             Звонок ВАТС
1               Аналитика
2               Аналитика
3    Омниканальный виджет
4    Омниканальный виджет
Name: Тип, dtype: object

In [ ]:
print(Primavera_df['Тип'].count())
print(Primavera_df['Тип'].shape[0])

3506
3506


In [ ]:
Primavera_df['Тип'].tail(20)

3486        Омниканальный виджет
3487        Омниканальный виджет
3488        Омниканальный виджет
3489        Омниканальный виджет
3490        Омниканальный виджет
3491        Омниканальный виджет
3492        Омниканальный виджет
3493    Динамический коллтрекинг
3494    Динамический коллтрекинг
3495        Омниканальный виджет
3496        Омниканальный виджет
3497        Омниканальный виджет
3498        Омниканальный виджет
3499        Омниканальный виджет
3500        Омниканальный виджет
3501    Динамический коллтрекинг
3502    Динамический коллтрекинг
3503        Омниканальный виджет
3504        Омниканальный виджет
3505        Омниканальный виджет
Name: Тип, dtype: object

In [ ]:
Primavera_df['Тип'][Primavera_df['Тип'] == ''].count()

0

##### Дата и время

In [ ]:
Primavera_df['Дата и время'].head()

0    2023-07-31 20:01:25
1    2023-07-31 19:30:00
2    2023-07-31 19:29:51
3    2023-07-31 19:20:01
4    2023-07-31 19:04:58
Name: Дата и время, dtype: object

In [ ]:
print(Primavera_df['Дата и время'].count())
print(Primavera_df['Дата и время'].shape[0])

3506
3506


In [ ]:
Primavera_calls_match_df['Дата и время'].head()

328    2023-05-01 10:25:48
297    2023-05-01 12:48:55
312    2023-05-01 17:32:54
329    2023-05-01 17:47:27
290    2023-05-02 13:20:04
Name: Дата и время, dtype: object

In [ ]:
print(Primavera_calls_match_df['Дата и время'].count())
print(Primavera_calls_match_df['Дата и время'].shape[0])

1517
1517


In [ ]:
# Приводим тип данных столбца 'Дата и время'
Primavera_df['Дата и время'] = pd.to_datetime(Primavera_df['Дата и время'])
Primavera_calls_match_df['Дата и время']= pd.to_datetime(Primavera_df['Дата и время'])

In [ ]:
Primavera_df['Дата и время'].head()

0   2023-07-31 20:01:25
1   2023-07-31 19:30:00
2   2023-07-31 19:29:51
3   2023-07-31 19:20:01
4   2023-07-31 19:04:58
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Primavera_calls_match_df['Дата и время'].head()

328   2023-07-23 16:56:51
297   2023-07-24 14:01:03
312   2023-07-24 10:18:25
329   2023-07-23 16:28:04
290   2023-07-24 15:36:55
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Primavera_df['Дата и время'].tail(20)

3486   2023-05-17 16:31:08
3487   2023-05-17 13:54:55
3488   2023-05-17 13:39:37
3489   2023-05-17 13:34:15
3490   2023-05-17 13:19:14
3491   2023-05-17 13:03:58
3492   2023-05-17 12:58:41
3493   2023-05-15 11:42:57
3494   2023-05-12 16:24:54
3495   2023-05-10 16:30:39
3496   2023-05-10 15:11:10
3497   2023-05-10 15:05:28
3498   2023-05-05 15:15:12
3499   2023-05-05 14:59:45
3500   2023-05-05 14:54:14
3501   2023-05-04 16:57:14
3502   2023-05-02 13:25:21
3503   2023-05-02 10:54:09
3504   2023-05-02 10:38:48
3505   2023-05-02 10:33:29
Name: Дата и время, dtype: datetime64[ns]

##### Номер абонента

In [ ]:
Primavera_df['Номер абонента'].head()

0    79687877778
1    79017951654
2    79660174449
3    79878367573
4    79878367573
Name: Номер абонента, dtype: object

In [ ]:
print(Primavera_df['Номер абонента'].count())
print(Primavera_df['Номер абонента'].shape[0])

3506
3506


In [ ]:
Primavera_calls_match_df['Номер абонента'].head()

328    79605015550
297    79372616437
312    79169640642
329    79099933333
290    79113699353
Name: Номер абонента, dtype: object

In [ ]:
print(Primavera_calls_match_df['Номер абонента'].count())
print(Primavera_calls_match_df['Номер абонента'].shape[0])

1517
1517


In [ ]:
# Приводим тип данных столбца 'Номер абонента'
Primavera_df = Primavera_df.astype({'Номер абонента': int})
Primavera_calls_match_df = Primavera_calls_match_df.astype({'Номер абонента': int})

In [ ]:
Primavera_df['Номер абонента'].head()

0    79687877778
1    79017951654
2    79660174449
3    79878367573
4    79878367573
Name: Номер абонента, dtype: int64

In [ ]:
Primavera_calls_match_df['Номер абонента'].head()

328    79605015550
297    79372616437
312    79169640642
329    79099933333
290    79113699353
Name: Номер абонента, dtype: int64

In [ ]:
Primavera_df['Номер абонента'].tail(20)

3486    79231966939
3487    79139158969
3488    79139158969
3489    79139158969
3490    79812586547
3491    79812586547
3492    79812586547
3493    74997530634
3494    79060205393
3495    79265448217
3496    79108390195
3497    79108390195
3498    79101653089
3499    79101653089
3500    79101653089
3501    79854440211
3502    79037788030
3503    79887865643
3504    79887865643
3505    79887865643
Name: Номер абонента, dtype: int64

##### Рекламная кампания

In [ ]:
Primavera_df['Рекламная кампания'].head()

0    СМС рассылка по внешнему сегменту
1                 Novostroy-m | Звонок
2                 Циан | Базы | Звонок
3                        Яндекс Директ
4                        Яндекс Директ
Name: Рекламная кампания, dtype: object

In [ ]:
print(Primavera_df['Рекламная кампания'].count())
print(Primavera_df['Рекламная кампания'].shape[0])

3506
3506


In [ ]:
Primavera_df['Рекламная кампания'].tail(20)

3486    Яндекс Директ
3487    Яндекс Директ
3488    Яндекс Директ
3489    Яндекс Директ
3490    Яндекс Директ
3491    Яндекс Директ
3492    Яндекс Директ
3493    Яндекс Директ
3494    Яндекс Директ
3495    Яндекс Директ
3496    Яндекс Директ
3497    Яндекс Директ
3498    Яндекс Директ
3499    Яндекс Директ
3500    Яндекс Директ
3501    Яндекс Директ
3502    Яндекс Директ
3503    Яндекс Директ
3504    Яндекс Директ
3505    Яндекс Директ
Name: Рекламная кампания, dtype: object

In [ ]:
Primavera_df['Рекламная кампания'][Primavera_df['Рекламная кампания'] == ''].count()

0

##### Длительность звонка

In [ ]:
Primavera_df['Длительность звонка'].head()

0    00:33:32
1    00:00:58
2    00:01:53
3    00:00:03
4    00:00:02
Name: Длительность звонка, dtype: object

In [ ]:
print(Primavera_df['Длительность звонка'].count())
print(Primavera_df['Длительность звонка'].shape[0])

3506
3506


In [ ]:
Primavera_df['Длительность звонка'].loc[Primavera_df['Длительность звонка'] == '-1д 23:59:59']

1853    -1д 23:59:59
1854    -1д 23:59:59
Name: Длительность звонка, dtype: object

In [ ]:
# Меняем '-1д 23:59:59' на '23:59:59' в столбце 'Длительность звонка'
Primavera_df['Длительность звонка'] = Primavera_df['Длительность звонка'].apply(lambda x: '23:59:59' if x == '-1д 23:59:59' else x)

In [ ]:
# Приводим тип данных столбца 'Длительность звонка'
Primavera_df['Длительность звонка'] = pd.to_datetime(Primavera_df['Длительность звонка']).dt.time

In [ ]:
Primavera_df['Длительность звонка'].head()

0    00:33:32
1    00:00:58
2    00:01:53
3    00:00:03
4    00:00:02
Name: Длительность звонка, dtype: object

In [ ]:
Primavera_df['Длительность звонка'].tail(20)

3486    00:01:02
3487    00:00:18
3488    00:00:17
3489    00:00:17
3490    00:00:12
3491    00:00:12
3492    00:00:12
3493    00:01:03
3494    00:00:04
3495    00:00:14
3496    00:00:40
3497    00:00:36
3498    00:00:26
3499    00:00:26
3500    00:00:26
3501    00:01:27
3502    00:00:46
3503    00:00:19
3504    00:00:19
3505    00:00:18
Name: Длительность звонка, dtype: object

##### Длительность ожидания ответа

In [ ]:
Primavera_df['Длительность ожидания ответа'].head()

0    00:00:30
1    00:00:08
2    00:00:27
3    00:00:03
4    00:00:02
Name: Длительность ожидания ответа, dtype: object

In [ ]:
print(Primavera_df['Длительность ожидания ответа'].count())
print(Primavera_df['Длительность ожидания ответа'].shape[0])

3506
3506


In [ ]:
Primavera_df['Длительность ожидания ответа'].loc[Primavera_df['Длительность ожидания ответа'] == '-1д 23:59:59']

1853    -1д 23:59:59
1854    -1д 23:59:59
Name: Длительность ожидания ответа, dtype: object

In [ ]:
# Меняем '-1д 23:59:59' на '23:59:59' в столбце 'Длительность ожидания ответа'
Primavera_df['Длительность ожидания ответа'] = Primavera_df['Длительность ожидания ответа'].apply(lambda x: '23:59:59' if x == '-1д 23:59:59' else x)

In [ ]:
# Приводим тип данных столбца 'Длительность ожидания ответа'
Primavera_df['Длительность ожидания ответа'] = pd.to_datetime(Primavera_df['Длительность ожидания ответа']).dt.time

In [ ]:
Primavera_df['Длительность ожидания ответа'].head()

0    00:00:30
1    00:00:08
2    00:00:27
3    00:00:03
4    00:00:02
Name: Длительность ожидания ответа, dtype: object

In [ ]:
Primavera_df['Длительность ожидания ответа'].tail(20)

3486    00:01:02
3487    00:00:18
3488    00:00:17
3489    00:00:17
3490    00:00:12
3491    00:00:12
3492    00:00:12
3493    00:01:03
3494    00:00:04
3495    00:00:14
3496    00:00:34
3497    00:00:36
3498    00:00:26
3499    00:00:26
3500    00:00:26
3501    00:00:06
3502    00:00:06
3503    00:00:19
3504    00:00:19
3505    00:00:18
Name: Длительность ожидания ответа, dtype: object

##### Сотрудник

In [ ]:
Primavera_df['Сотрудник'].head()

0    Aleksandr Kuleshov
1      Tatyana Denisova
2      Operator5Metrium
3                   NaN
4                   NaN
Name: Сотрудник, dtype: object

In [ ]:
print(Primavera_df['Сотрудник'].count())
print(Primavera_df['Сотрудник'].shape[0])

2073
3506


In [ ]:
# Избавляемся от пустот в столбце 'Сотрудник'
Primavera_df['Сотрудник'] = Primavera_df['Сотрудник'].fillna('нет сотрудника')

In [ ]:
print(Primavera_df['Сотрудник'].count())
print(Primavera_df['Сотрудник'].shape[0])

3506
3506


In [ ]:
Primavera_df['Сотрудник'].head()

0    Aleksandr Kuleshov
1      Tatyana Denisova
2      Operator5Metrium
3        нет сотрудника
4        нет сотрудника
Name: Сотрудник, dtype: object

In [ ]:
Primavera_df['Сотрудник'].tail(20)

3486                     
3487                     
3488                     
3489                     
3490                     
3491                     
3492                     
3493                     
3494     Operator5Metrium
3495                     
3496    Manager2OneMoscow
3497                     
3498                     
3499                     
3500                     
3501         Operator4VSN
3502     Operator6Metrium
3503                     
3504                     
3505                     
Name: Сотрудник, dtype: object

In [ ]:
Primavera_df['Сотрудник'][Primavera_df['Сотрудник'] == ''].count()

70

In [ ]:
# Меняем '' на 'нет сотрудника' в столбце 'Сотрудник'
Primavera_df['Сотрудник'] = Primavera_df['Сотрудник'].apply(lambda x: 'нет сотрудника' if x == '' else x)

In [ ]:
Primavera_df['Сотрудник'][Primavera_df['Сотрудник'] == ''].count()

0

In [ ]:
Primavera_df['Сотрудник'].tail(20)

3486       нет сотрудника
3487       нет сотрудника
3488       нет сотрудника
3489       нет сотрудника
3490       нет сотрудника
3491       нет сотрудника
3492       нет сотрудника
3493       нет сотрудника
3494     Operator5Metrium
3495       нет сотрудника
3496    Manager2OneMoscow
3497       нет сотрудника
3498       нет сотрудника
3499       нет сотрудника
3500       нет сотрудника
3501         Operator4VSN
3502     Operator6Metrium
3503       нет сотрудника
3504       нет сотрудника
3505       нет сотрудника
Name: Сотрудник, dtype: object

##### UTM-метка Campaign

In [ ]:
Primavera_df['UTM-метка Campaign'].head()

0                                             NaN
1                                             NaN
2                                             NaN
3    m108|retargeting_new_rsy_rf|92517773|context
4    m108|retargeting_new_rsy_rf|92517773|context
Name: UTM-метка Campaign, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Campaign'].count())
print(Primavera_df['UTM-метка Campaign'].shape[0])

1566
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Campaign'
Primavera_df['UTM-метка Campaign'] = Primavera_df['UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Campaign'].count())
print(Primavera_df['UTM-метка Campaign'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Campaign'].head()

0                                             нет
1                                             нет
2                                             нет
3    m108|retargeting_new_rsy_rf|92517773|context
4    m108|retargeting_new_rsy_rf|92517773|context
Name: UTM-метка Campaign, dtype: object

In [ ]:
Primavera_df['UTM-метка Campaign'].tail(20)

3486    m108|profile_competitors_rsy_rf|87621747|context
3487    m108|profile_competitors_rsy_rf|87621747|context
3488    m108|profile_competitors_rsy_rf|87621747|context
3489    m108|profile_competitors_rsy_rf|87621747|context
3490        m108|interes_realty_rsy_mmo|82276325|context
3491        m108|interes_realty_rsy_mmo|82276325|context
3492        m108|interes_realty_rsy_mmo|82276325|context
3493       m108|interes_ipoteka_rsy_mmo|85569957|context
3494         m108|start+kotlovan_rsy_rf|84726293|context
3495        m108|interes_realty_rsy_mmo|82276325|context
3496        m108|interes_realty_rsy_mmo|82276325|context
3497        m108|interes_realty_rsy_mmo|82276325|context
3498         m108|interes_realty_rsy_rf|82276333|context
3499         m108|interes_realty_rsy_rf|82276333|context
3500         m108|interes_realty_rsy_rf|82276333|context
3501       m108|brand_podarok_search_mmo|87274641|search
3502         m108|start+kotlovan_rsy_rf|84726293|context
3503         m108|start+kotlova

In [ ]:
Primavera_df['UTM-метка Campaign'][Primavera_df['UTM-метка Campaign'] == ''].count()

0

In [ ]:
Primavera_df['UTM-метка Campaign'].tail(20)

3486    m108|profile_competitors_rsy_rf|87621747|context
3487    m108|profile_competitors_rsy_rf|87621747|context
3488    m108|profile_competitors_rsy_rf|87621747|context
3489    m108|profile_competitors_rsy_rf|87621747|context
3490        m108|interes_realty_rsy_mmo|82276325|context
3491        m108|interes_realty_rsy_mmo|82276325|context
3492        m108|interes_realty_rsy_mmo|82276325|context
3493       m108|interes_ipoteka_rsy_mmo|85569957|context
3494         m108|start+kotlovan_rsy_rf|84726293|context
3495        m108|interes_realty_rsy_mmo|82276325|context
3496        m108|interes_realty_rsy_mmo|82276325|context
3497        m108|interes_realty_rsy_mmo|82276325|context
3498         m108|interes_realty_rsy_rf|82276333|context
3499         m108|interes_realty_rsy_rf|82276333|context
3500         m108|interes_realty_rsy_rf|82276333|context
3501       m108|brand_podarok_search_mmo|87274641|search
3502         m108|start+kotlovan_rsy_rf|84726293|context
3503         m108|start+kotlova

##### Теги

In [ ]:
Primavera_df['Теги'].head()

0                                       Целевой_М108
1                             Нецелевой контакт_М108
2                             Нецелевой контакт_М108
3    Не обработано - Нецелевой контакт - Повтор_М108
4    Не обработано - Нецелевой контакт - Повтор_М108
Name: Теги, dtype: object

In [ ]:
print(Primavera_df['Теги'].count())
print(Primavera_df['Теги'].shape[0])

3506
3506


In [ ]:
Primavera_df['Теги'].tail(20)

3486    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3487               Не обработано - Нецелевой контакт - Повтор_М108
3488               Не обработано - Нецелевой контакт - Повтор_М108
3489    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3490               Не обработано - Нецелевой контакт - Повтор_М108
3491               Не обработано - Нецелевой контакт - Повтор_М108
3492    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3493    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3494    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3495    Не обработано - Нецелевой контакт - Нецелевой контакт_М108
3496                                                   Повтор_М108
3497       Обработано - Нецелевой контакт - Нецелевой контакт_М108
3498               Не обработано - Нецелевой контакт - Повтор_М108
3499               Не обработано - Нецелевой контакт - Повтор_М108
3500    Не обработано - Нецелевой контакт - Нецелевой контакт_

In [ ]:
Primavera_df['Теги'][Primavera_df['Теги'] == ''].count()

0

##### Номер обращения

In [ ]:
Primavera_df['Номер обращения'].head()

0       1
1       1
2       1
3    1140
4    1139
Name: Номер обращения, dtype: object

In [ ]:
print(Primavera_df['Номер обращения'].count())
print(Primavera_df['Номер обращения'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Номер обращения'
Primavera_df = Primavera_df.astype({'Номер обращения': int})

In [ ]:
Primavera_df['Номер обращения'].head()

0       1
1       1
2       1
3    1140
4    1139
Name: Номер обращения, dtype: int64

In [ ]:
Primavera_df['Номер обращения'].tail(20)

3486     67
3487     21
3488     20
3489     19
3490     17
3491     16
3492     15
3493     58
3494     24
3495      4
3496     11
3497     10
3498     15
3499     14
3500     13
3501     43
3502    216
3503     15
3504     14
3505     13
Name: Номер обращения, dtype: int64

##### UTM-метка Term

In [ ]:
Primavera_df['UTM-метка Term'].tail(20)

3486                                                       
3487                                                       
3488                                                       
3489                                                       
3490                                                       
3491                                                       
3492                                                       
3493                                                       
3494    новостройки москвы циан от застройщика старт продаж
3495                                                       
3496                                                       
3497                                                       
3498                                                       
3499                                                       
3500                                                       
3501                                           жк примавера
3502               старт продаж новая мо

In [ ]:
print(Primavera_df['UTM-метка Term'].count())
print(Primavera_df['UTM-метка Term'].shape[0])

747
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Term'
Primavera_df['UTM-метка Term'] = Primavera_df['UTM-метка Term'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Term'].count())
print(Primavera_df['UTM-метка Term'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Term'][Primavera_df['UTM-метка Term'] == ''].count()

50

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Term'
Primavera_df['UTM-метка Term'] = Primavera_df['UTM-метка Term'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['UTM-метка Term'][Primavera_df['UTM-метка Term'] == ''].count()

0

In [ ]:
Primavera_df['UTM-метка Term'].tail(20)

3486                                                    нет
3487                                                    нет
3488                                                    нет
3489                                                    нет
3490                                                    нет
3491                                                    нет
3492                                                    нет
3493                                                    нет
3494    новостройки москвы циан от застройщика старт продаж
3495                                                    нет
3496                                                    нет
3497                                                    нет
3498                                                    нет
3499                                                    нет
3500                                                    нет
3501                                           жк примавера
3502               старт продаж новая мо

In [ ]:
Primavera_df['UTM-метка Term'][Primavera_df['UTM-метка Term'] == 'нет'].count()

2809

In [ ]:
Primavera_df['UTM-метка Term'][Primavera_df['UTM-метка Term'] != 'нет'].count()

697

##### Причина завершения

In [ ]:
Primavera_df['Причина завершения'].head()

0      Абонент разорвал соединение
1    Сотрудник разорвал соединение
2    Сотрудник разорвал соединение
3              Абонент не отвечает
4              Абонент не отвечает
Name: Причина завершения, dtype: object

In [ ]:
print(Primavera_df['Причина завершения'].count())
print(Primavera_df['Причина завершения'].shape[0])

3506
3506


In [ ]:
Primavera_df['Причина завершения'].tail(20)

3486            Время дозвона истекло
3487              Абонент не отвечает
3488              Абонент не отвечает
3489              Абонент не отвечает
3490              Абонент не отвечает
3491              Абонент не отвечает
3492              Абонент не отвечает
3493     Не дозвонились до сотрудника
3494    Сотрудник разорвал соединение
3495       Нет операции для обработки
3496      Абонент разорвал соединение
3497     Не дозвонились до сотрудника
3498              Абонент не отвечает
3499              Абонент не отвечает
3500              Абонент не отвечает
3501      Абонент разорвал соединение
3502    Сотрудник разорвал соединение
3503              Абонент не отвечает
3504              Абонент не отвечает
3505              Абонент не отвечает
Name: Причина завершения, dtype: object

In [ ]:
Primavera_df['Причина завершения'][Primavera_df['Причина завершения'] == ''].count()

0

##### Виртуальный номер

In [ ]:
Primavera_df['Виртуальный номер'].head()

0    74951252537
1    74954320952
2    74951068605
3    74950210291
4    74950210291
Name: Виртуальный номер, dtype: object

In [ ]:
print(Primavera_df['Виртуальный номер'].count())
print(Primavera_df['Виртуальный номер'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Виртуальный номер'
Primavera_df = Primavera_df.astype({'Виртуальный номер': int})

In [ ]:
Primavera_df['Виртуальный номер'].head()

0    74951252537
1    74954320952
2    74951068605
3    74950210291
4    74950210291
Name: Виртуальный номер, dtype: int64

In [ ]:
Primavera_df['Виртуальный номер'].tail(20)

3486    74950210291
3487    74950210291
3488    74950210291
3489    74950210291
3490    74950210291
3491    74950210291
3492    74950210291
3493    74951625279
3494    74951068390
3495    74950210291
3496    74950210291
3497    74950210291
3498    74950210291
3499    74950210291
3500    74950210291
3501    74953246192
3502    74959662305
3503    74950210291
3504    74950210291
3505    74950210291
Name: Виртуальный номер, dtype: int64

##### UTM-метка Source

In [ ]:
Primavera_df['UTM-метка Source'].head()

0       NaN
1       NaN
2       NaN
3    yandex
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Source'].count())
print(Primavera_df['UTM-метка Source'].shape[0])

1566
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Source'
Primavera_df['UTM-метка Source'] = Primavera_df['UTM-метка Source'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Source'].count())
print(Primavera_df['UTM-метка Source'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Source'].head()

0       нет
1       нет
2       нет
3    yandex
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
Primavera_df['UTM-метка Source'].tail(20)

3486    yandex
3487    yandex
3488    yandex
3489    yandex
3490    yandex
3491    yandex
3492    yandex
3493    yandex
3494    yandex
3495    yandex
3496    yandex
3497    yandex
3498    yandex
3499    yandex
3500    yandex
3501    yandex
3502    yandex
3503    yandex
3504    yandex
3505    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
Primavera_df['UTM-метка Source'][Primavera_df['UTM-метка Source'] == ''].count()

0

##### UTM-метка Medium

In [ ]:
Primavera_df['UTM-метка Medium'].head()

0    NaN
1    NaN
2    NaN
3    cpc
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Medium'].count())
print(Primavera_df['UTM-метка Medium'].shape[0])

1564
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Medium'
Primavera_df['UTM-метка Medium'] = Primavera_df['UTM-метка Medium'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Medium'].count())
print(Primavera_df['UTM-метка Medium'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Medium'].head()

0    нет
1    нет
2    нет
3    cpc
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
Primavera_df['UTM-метка Medium'].tail(20)

3486    cpc
3487    cpc
3488    cpc
3489    cpc
3490    cpc
3491    cpc
3492    cpc
3493    cpc
3494    cpc
3495    cpc
3496    cpc
3497    cpc
3498    cpc
3499    cpc
3500    cpc
3501    cpc
3502    cpc
3503    cpc
3504    cpc
3505    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
Primavera_df['UTM-метка Medium'][Primavera_df['UTM-метка Medium'] == ''].count()

0

##### UTM-метка Content

In [ ]:
Primavera_df['UTM-метка Content'].head()

0                                                                                                          NaN
1                                                                                                          NaN
2                                                                                                          NaN
3    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
4    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
Name: UTM-метка Content, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Content'].count())
print(Primavera_df['UTM-метка Content'].shape[0])

1522
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Content'
Primavera_df['UTM-метка Content'] = Primavera_df['UTM-метка Content'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Content'].count())
print(Primavera_df['UTM-метка Content'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Content'].head()

0                                                                                                          нет
1                                                                                                          нет
2                                                                                                          нет
3    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
4    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
Name: UTM-метка Content, dtype: object

In [ ]:
Primavera_df['UTM-метка Content'].tail(20)

3486              5192314923|14128426025|47642247|dzen.ru|mobile|14128426025|type1|47642247|47642247|1|none|11316
3487                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3488                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3489                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3490               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3491               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3492               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3493               5162529018|13794196700|45369764|dzen.ru|desktop|13794196700|type1|45369764|45369764|1|none|213
3494      5151257235|13677074587|43745648635|dzen.ru|desktop|13677074587|type1|437456486

In [ ]:
Primavera_df['UTM-метка Content'][Primavera_df['UTM-метка Content'] == ''].count()

0

##### Client ID Яндекс.Метрика

In [ ]:
Primavera_df['Client ID Яндекс.Метрика'].head()

0                       
1                       
2                       
3    1688055753141902417
4    1688055753141902417
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
print(Primavera_df['Client ID Яндекс.Метрика'].count())
print(Primavera_df['Client ID Яндекс.Метрика'].shape[0])

3506
3506


In [ ]:
Primavera_df['Client ID Яндекс.Метрика'][Primavera_df['Client ID Яндекс.Метрика'] == ''].count()

1790

In [ ]:
# Меняем '' на '0' в столбце 'Client ID Яндекс.Метрика'
Primavera_df['Client ID Яндекс.Метрика'] = Primavera_df['Client ID Яндекс.Метрика'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Client ID Яндекс.Метрика'][Primavera_df['Client ID Яндекс.Метрика'] == ''].count()

0

In [ ]:
Primavera_df['Client ID Яндекс.Метрика'].head()

0                      0
1                      0
2                      0
3    1688055753141902417
4    1688055753141902417
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
Primavera_df['Client ID Яндекс.Метрика'].tail(20)

3486    1684319514919464753
3487    1684319514919464753
3488    1684319514919464753
3489    1684319514919464753
3490    1684317012512055305
3491    1684317012512055305
3492    1684317012512055305
3493    1684139479297764452
3494     168389740142599584
3495    1683725379884562615
3496    1683720218463117973
3497    1683720218463117973
3498     168328758691002773
3499     168328758691002773
3500     168328758691002773
3501    1683204564311032733
3502    1680714986757669589
3503    1682498792635913810
3504    1682498792635913810
3505    1682498792635913810
Name: Client ID Яндекс.Метрика, dtype: object

##### Первая рекламная кампания

In [ ]:
Primavera_df['Первая рекламная кампания'].tail(20)

3486    Яндекс Директ
3487    Яндекс Директ
3488    Яндекс Директ
3489    Яндекс Директ
3490    Яндекс Директ
3491    Яндекс Директ
3492    Яндекс Директ
3493    Яндекс Директ
3494    Яндекс Директ
3495    Яндекс Директ
3496    Яндекс Директ
3497    Яндекс Директ
3498    Яндекс Директ
3499    Яндекс Директ
3500    Яндекс Директ
3501    Яндекс Директ
3502    Яндекс Директ
3503    Яндекс Директ
3504    Яндекс Директ
3505    Яндекс Директ
Name: Первая рекламная кампания, dtype: object

In [ ]:
print(Primavera_df['Первая рекламная кампания'].count())
print(Primavera_df['Первая рекламная кампания'].shape[0])

3506
3506


In [ ]:
Primavera_df['Первая рекламная кампания'][Primavera_df['Первая рекламная кампания'] == ''].count()

0

##### ID посетителя

In [ ]:
Primavera_df['ID посетителя'].head()

0             NaN
1             NaN
2             NaN
3    7391725482.0
4    7391725482.0
Name: ID посетителя, dtype: object

In [ ]:
print(Primavera_df['ID посетителя'].count())
print(Primavera_df['ID посетителя'].shape[0])

1860
3506


In [ ]:
# Избавляемся от пустот в столбце 'ID посетителя'
Primavera_df['ID посетителя'] = Primavera_df['ID посетителя'].fillna('0')

In [ ]:
print(Primavera_df['ID посетителя'].count())
print(Primavera_df['ID посетителя'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'ID посетителя'
Primavera_df = Primavera_df.astype({'ID посетителя': int})

In [ ]:
Primavera_df['ID посетителя'].head()

0             0
1             0
2             0
3    7391725482
4    7391725482
Name: ID посетителя, dtype: int64

In [ ]:
Primavera_df['ID посетителя'].tail(20)

3486    7235857668
3487    7235857668
3488    7235857668
3489    7235857668
3490    7235664688
3491    7235664688
3492    7235664688
3493    7225909228
3494    7215437828
3495    7206493048
3496    7206099080
3497    7206099080
3498    7187794184
3499    7187794184
3500    7187794184
3501    7183724300
3502    7050708872
3503    7149243064
3504    7149243064
3505    7149243064
Name: ID посетителя, dtype: int64

##### Страна номера абонента

In [ ]:
Primavera_df['Страна номера абонента'].tail(20)

3486    РФ
3487    РФ
3488    РФ
3489    РФ
3490    РФ
3491    РФ
3492    РФ
3493    РФ
3494    РФ
3495    РФ
3496    РФ
3497    РФ
3498    РФ
3499    РФ
3500    РФ
3501    РФ
3502    РФ
3503    РФ
3504    РФ
3505    РФ
Name: Страна номера абонента, dtype: object

In [ ]:
print(Primavera_df['Страна номера абонента'].count())
print(Primavera_df['Страна номера абонента'].shape[0])

3506
3506


In [ ]:
Primavera_df['Страна номера абонента'][Primavera_df['Страна номера абонента'] == ''].count()

0

##### Регион номера абонента

In [ ]:
Primavera_df['Регион номера абонента'].tail(20)

3486     Новосибирская обл.
3487     Новосибирская обл.
3488     Новосибирская обл.
3489     Новосибирская обл.
3490     г. Санкт-Петербург
3491     г. Санкт-Петербург
3492     г. Санкт-Петербург
3493              г. Москва
3494              г. Москва
3495              г. Москва
3496          Тверская обл.
3497          Тверская обл.
3498          Тульская обл.
3499          Тульская обл.
3500          Тульская обл.
3501              г. Москва
3502              г. Москва
3503    Республика Дагестан
3504    Республика Дагестан
3505    Республика Дагестан
Name: Регион номера абонента, dtype: object

In [ ]:
print(Primavera_df['Регион номера абонента'].count())
print(Primavera_df['Регион номера абонента'].shape[0])

3506
3506


In [ ]:
Primavera_df['Регион номера абонента'][Primavera_df['Регион номера абонента'] == ''].count()

0

##### Идентификатор звонка во внешней системе

In [ ]:
Primavera_df['Идентификатор звонка во внешней системе'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Идентификатор звонка во внешней системе, dtype: object

In [ ]:
print(Primavera_df['Идентификатор звонка во внешней системе'].count())
print(Primavera_df['Идентификатор звонка во внешней системе'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Идентификатор звонка во внешней системе'
Primavera_df['Идентификатор звонка во внешней системе'] = Primavera_df['Идентификатор звонка во внешней системе'].fillna('0')

In [ ]:
print(Primavera_df['Идентификатор звонка во внешней системе'].count())
print(Primavera_df['Идентификатор звонка во внешней системе'].shape[0])

3506
3506


In [ ]:
Primavera_df['Идентификатор звонка во внешней системе'][Primavera_df['Идентификатор звонка во внешней системе'] == ''].count()

81

In [ ]:
# Меняем '' на '0' в столбце 'Идентификатор звонка во внешней системе'
Primavera_df['Идентификатор звонка во внешней системе'] = Primavera_df['Идентификатор звонка во внешней системе'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Идентификатор звонка во внешней системе'][Primavera_df['Идентификатор звонка во внешней системе'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'Идентификатор звонка во внешней системе'
Primavera_df = Primavera_df.astype({'Идентификатор звонка во внешней системе': int})

In [ ]:
Primavera_df['Идентификатор звонка во внешней системе'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Идентификатор звонка во внешней системе, dtype: int64

##### Тип устройства

In [ ]:
Primavera_df['Тип устройства'].tail(20)

3486    Смартфон
3487    Смартфон
3488    Смартфон
3489    Смартфон
3490          ПК
3491          ПК
3492          ПК
3493          ПК
3494          ПК
3495          ПК
3496          ПК
3497          ПК
3498          ПК
3499          ПК
3500          ПК
3501          ПК
3502          ПК
3503          ПК
3504          ПК
3505          ПК
Name: Тип устройства, dtype: object

In [ ]:
print(Primavera_df['Тип устройства'].count())
print(Primavera_df['Тип устройства'].shape[0])

3506
3506


In [ ]:
Primavera_df['Тип устройства'][Primavera_df['Тип устройства'] == ''].count()

0

##### URL страницы обращения

In [ ]:
Primavera_df['URL страницы обращения'].tail(20)

3486                              https://primavera.moscow/
3487                              https://primavera.moscow/
3488                              https://primavera.moscow/
3489                              https://primavera.moscow/
3490                              https://primavera.moscow/
3491                              https://primavera.moscow/
3492                              https://primavera.moscow/
3493                       https://primavera.moscow/#about=
3494    https://primavera.moscow/kvartiry/vivaldi/flat-1271
3495                              https://primavera.moscow/
3496                              https://primavera.moscow/
3497                              https://primavera.moscow/
3498                        https://primavera.moscow/search
3499                        https://primavera.moscow/search
3500                        https://primavera.moscow/search
3501                      https://primavera.moscow/contacts
3502                              https:

In [ ]:
print(Primavera_df['URL страницы обращения'].count())
print(Primavera_df['URL страницы обращения'].shape[0])

1794
3506


In [ ]:
# Избавляемся от пустот в столбце 'URL страницы обращения'
Primavera_df['URL страницы обращения'] = Primavera_df['URL страницы обращения'].fillna('нет')

In [ ]:
print(Primavera_df['URL страницы обращения'].count())
print(Primavera_df['URL страницы обращения'].shape[0])

3506
3506


In [ ]:
Primavera_df['URL страницы обращения'][Primavera_df['URL страницы обращения'] == ''].count()

0

##### Город

In [ ]:
Primavera_df['Город'].tail(20)

3486    Новосибирск
3487    Новосибирск
3488    Новосибирск
3489    Новосибирск
3490               
3491               
3492               
3493         Москва
3494    Севастополь
3495         Москва
3496         Москва
3497         Москва
3498         Сургут
3499         Сургут
3500         Сургут
3501         Москва
3502         Липецк
3503               
3504               
3505               
Name: Город, dtype: object

In [ ]:
print(Primavera_df['Город'].count())
print(Primavera_df['Город'].shape[0])

1595
3506


In [ ]:
# Избавляемся от пустот в столбце 'Город'
Primavera_df['Город'] = Primavera_df['Город'].fillna('нет')

In [ ]:
print(Primavera_df['Город'].count())
print(Primavera_df['Город'].shape[0])

3506
3506


In [ ]:
Primavera_df['Город'][Primavera_df['Город'] == ''].count()

11

In [ ]:
# Меняем '' на 'нет' в столбце 'Город'
Primavera_df['Город'] = Primavera_df['Город'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Город'].tail(20)

3486    Новосибирск
3487    Новосибирск
3488    Новосибирск
3489    Новосибирск
3490            нет
3491            нет
3492            нет
3493         Москва
3494    Севастополь
3495         Москва
3496         Москва
3497         Москва
3498         Сургут
3499         Сургут
3500         Сургут
3501         Москва
3502         Липецк
3503            нет
3504            нет
3505            нет
Name: Город, dtype: object

##### Страна

In [ ]:
Primavera_df['Страна'].tail(20)

3486      Россия
3487      Россия
3488      Россия
3489      Россия
3490    Германия
3491    Германия
3492    Германия
3493      Россия
3494      Россия
3495      Россия
3496      Россия
3497      Россия
3498      Россия
3499      Россия
3500      Россия
3501      Россия
3502      Россия
3503      Россия
3504      Россия
3505      Россия
Name: Страна, dtype: object

In [ ]:
print(Primavera_df['Страна'].count())
print(Primavera_df['Страна'].shape[0])

1776
3506


In [ ]:
# Избавляемся от пустот в столбце 'Страна'
Primavera_df['Страна'] = Primavera_df['Страна'].fillna('нет')

In [ ]:
print(Primavera_df['Страна'].count())
print(Primavera_df['Страна'].shape[0])

3506
3506


In [ ]:
Primavera_df['Страна'][Primavera_df['Страна'] == ''].count()

0

In [ ]:
Primavera_df['Страна'].head(20)

0                    нет
1                    нет
2                    нет
3                 Россия
4                 Россия
5                 Россия
6     Чешская Республика
7                 Россия
8                 Россия
9                 Россия
10                   нет
11                   нет
12                Россия
13                Россия
14                Россия
15                Россия
16                   нет
17                   нет
18                   нет
19                   нет
Name: Страна, dtype: object

##### Сайт

In [ ]:
Primavera_df['Сайт'].tail(20)

3486    primavera.moscow
3487    primavera.moscow
3488    primavera.moscow
3489    primavera.moscow
3490    primavera.moscow
3491    primavera.moscow
3492    primavera.moscow
3493    primavera.moscow
3494    primavera.moscow
3495    primavera.moscow
3496    primavera.moscow
3497    primavera.moscow
3498    primavera.moscow
3499    primavera.moscow
3500    primavera.moscow
3501    primavera.moscow
3502    primavera.moscow
3503    primavera.moscow
3504    primavera.moscow
3505    primavera.moscow
Name: Сайт, dtype: object

In [ ]:
print(Primavera_df['Сайт'].count())
print(Primavera_df['Сайт'].shape[0])

3506
3506


In [ ]:
Primavera_df['Сайт'][Primavera_df['Сайт'] == ''].count()

0

##### Оценка сотрудника

In [ ]:
Primavera_df['Оценка сотрудника'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Оценка сотрудника, dtype: object

In [ ]:
print(Primavera_df['Оценка сотрудника'].count())
print(Primavera_df['Оценка сотрудника'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Оценка сотрудника'
Primavera_df['Оценка сотрудника'] = Primavera_df['Оценка сотрудника'].fillna('нет')

In [ ]:
print(Primavera_df['Оценка сотрудника'].count())
print(Primavera_df['Оценка сотрудника'].shape[0])

3506
3506


In [ ]:
Primavera_df['Оценка сотрудника'][Primavera_df['Оценка сотрудника'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Оценка сотрудника'
Primavera_df['Оценка сотрудника'] = Primavera_df['Оценка сотрудника'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Оценка сотрудника'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Оценка сотрудника, dtype: object

In [ ]:
Primavera_df['Оценка сотрудника'][Primavera_df['Оценка сотрудника'] == ''].count()

0

In [ ]:
Primavera_df['Оценка сотрудника'][Primavera_df['Оценка сотрудника'] == 'нет'].count()

3506

In [ ]:
Primavera_df['Оценка сотрудника'][Primavera_df['Оценка сотрудника'] != 'нет'].count()

0

##### Номер сотрудника

In [ ]:
Primavera_df['Номер сотрудника'].head()

0    253192.0
1    253208.0
2    111642.0
3         NaN
4         NaN
Name: Номер сотрудника, dtype: object

In [ ]:
print(Primavera_df['Номер сотрудника'].count())
print(Primavera_df['Номер сотрудника'].shape[0])

2850
3506


In [ ]:
# Избавляемся от пустот в столбце 'Номер сотрудника'
Primavera_df['Номер сотрудника'] = Primavera_df['Номер сотрудника'].fillna('0')

In [ ]:
print(Primavera_df['Номер сотрудника'].count())
print(Primavera_df['Номер сотрудника'].shape[0])

3506
3506


In [ ]:
Primavera_df['Номер сотрудника'][Primavera_df['Номер сотрудника'] == ''].count()

50

In [ ]:
# Меняем '' на '0' в столбце 'Номер сотрудника'
Primavera_df['Номер сотрудника'] = Primavera_df['Номер сотрудника'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Номер сотрудника'][Primavera_df['Номер сотрудника'] == ''].count()

0

In [ ]:
Primavera_calls_match_df['Номер сотрудника'].head()

328    0197879
297    0150111
312    0197872
329    0111651
290    0150108
Name: Номер сотрудника, dtype: object

In [ ]:
# Приводим тип данных столбца 'Номер сотрудника'
Primavera_df = Primavera_df.astype({'Номер сотрудника': int})
Primavera_calls_match_df = Primavera_calls_match_df.astype({'Номер сотрудника': int})

In [ ]:
Primavera_df['Номер сотрудника'].head()

0    253192
1    253208
2    111642
3         0
4         0
Name: Номер сотрудника, dtype: int64

In [ ]:
Primavera_calls_match_df['Номер сотрудника'].head()

328    197879
297    150111
312    197872
329    111651
290    150108
Name: Номер сотрудника, dtype: int64

In [ ]:
Primavera_df['Номер сотрудника'].tail(20)

3486         0
3487         0
3488         0
3489         0
3490         0
3491         0
3492         0
3493    173695
3494    111642
3495         0
3496    150108
3497    173695
3498         0
3499         0
3500         0
3501    197878
3502    111651
3503         0
3504         0
3505         0
Name: Номер сотрудника, dtype: int64

In [ ]:
Primavera_calls_match_df['Номер сотрудника'].tail(20)

1022    253211
1051    253211
1019    253206
1094    111652
1038    150111
1026    111631
1030    253208
1091    150113
1083    150113
1096    111652
1002    253211
1077    253208
1014    111652
1007    253211
1031    111631
1050    150108
1047    111631
1009    111631
1039    111631
1018    111639
Name: Номер сотрудника, dtype: int64

##### Трансфер

In [ ]:
Primavera_df['Трансфер'].head()

0    Нет
1    Нет
2    Нет
3    Нет
4    Нет
Name: Трансфер, dtype: object

In [ ]:
print(Primavera_df['Трансфер'].count())
print(Primavera_df['Трансфер'].shape[0])

3506
3506


In [ ]:
Primavera_df['Трансфер'].tail(20)

3486    Нет
3487    Нет
3488    Нет
3489    Нет
3490    Нет
3491    Нет
3492    Нет
3493    Нет
3494    Нет
3495    Нет
3496    Нет
3497    Нет
3498    Нет
3499    Нет
3500    Нет
3501    Нет
3502    Нет
3503    Нет
3504    Нет
3505    Нет
Name: Трансфер, dtype: object

In [ ]:
Primavera_df['Трансфер'][Primavera_df['Трансфер'] == ''].count()

0

##### Внутренний звонок

In [ ]:
Primavera_df['Внутренний звонок'].tail(20)

3486    Нет
3487    Нет
3488    Нет
3489    Нет
3490    Нет
3491    Нет
3492    Нет
3493    Нет
3494    Нет
3495    Нет
3496    Нет
3497    Нет
3498    Нет
3499    Нет
3500    Нет
3501    Нет
3502    Нет
3503    Нет
3504    Нет
3505    Нет
Name: Внутренний звонок, dtype: object

In [ ]:
print(Primavera_df['Внутренний звонок'].count())
print(Primavera_df['Внутренний звонок'].shape[0])

3506
3506


In [ ]:
Primavera_df['Внутренний звонок'][Primavera_df['Внутренний звонок'] == ''].count()

0

In [ ]:
Primavera_df['Внутренний звонок'][Primavera_df['Внутренний звонок'] == 'Нет'].count()

3494

In [ ]:
Primavera_df['Внутренний звонок'][Primavera_df['Внутренний звонок'] != 'Нет'].count()

12

In [ ]:
Primavera_df['Внутренний звонок'][Primavera_df['Внутренний звонок'] != 'Нет']

383     Да
670     Да
672     Да
892     Да
1113    Да
1114    Да
1115    Да
1116    Да
1117    Да
2375    Да
2933    Да
2965    Да
Name: Внутренний звонок, dtype: object

##### ФИО контакта

In [ ]:
Primavera_df['ФИО контакта'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: ФИО контакта, dtype: object

In [ ]:
print(Primavera_df['ФИО контакта'].count())
print(Primavera_df['ФИО контакта'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'ФИО контакта'
Primavera_df['ФИО контакта'] = Primavera_df['ФИО контакта'].fillna('нет')

In [ ]:
print(Primavera_df['ФИО контакта'].count())
print(Primavera_df['ФИО контакта'].shape[0])

3506
3506


In [ ]:
Primavera_df['ФИО контакта'][Primavera_df['ФИО контакта'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'ФИО контакта'
Primavera_df['ФИО контакта'] = Primavera_df['ФИО контакта'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['ФИО контакта'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: ФИО контакта, dtype: object

In [ ]:
Primavera_df['ФИО контакта'][Primavera_df['ФИО контакта'] == ''].count()

0

In [ ]:
Primavera_df['ФИО контакта'][Primavera_df['ФИО контакта'] == 'нет'].count()

3506

In [ ]:
Primavera_df['ФИО контакта'][Primavera_df['ФИО контакта'] != 'нет'].count()

0

##### Имя тренера

In [ ]:
Primavera_df['Имя тренера'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Имя тренера, dtype: object

In [ ]:
print(Primavera_df['Имя тренера'].count())
print(Primavera_df['Имя тренера'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Имя тренера'
Primavera_df['Имя тренера'] = Primavera_df['Имя тренера'].fillna('нет')

In [ ]:
print(Primavera_df['Имя тренера'].count())
print(Primavera_df['Имя тренера'].shape[0])

3506
3506


In [ ]:
Primavera_df['Имя тренера'][Primavera_df['Имя тренера'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Имя тренера'
Primavera_df['Имя тренера'] = Primavera_df['Имя тренера'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Имя тренера'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Имя тренера, dtype: object

In [ ]:
Primavera_df['Имя тренера'][Primavera_df['Имя тренера'] == ''].count()

0

In [ ]:
Primavera_df['Имя тренера'][Primavera_df['Имя тренера'] == 'нет'].count()

3506

In [ ]:
Primavera_df['Имя тренера'][Primavera_df['Имя тренера'] != 'нет'].count()

0

##### Сценарий

In [ ]:
Primavera_df['Сценарий'].head()

0                 Обработка вызова 2023
1                 Обработка вызова 2023
2                 Обработка вызова 2023
3    Обработка вызова 2023 для колбэков
4    Обработка вызова 2023 для колбэков
Name: Сценарий, dtype: object

In [ ]:
print(Primavera_df['Сценарий'].count())
print(Primavera_df['Сценарий'].shape[0])

3455
3506


In [ ]:
# Избавляемся от пустот в столбце 'Сценарий'
Primavera_df['Сценарий'] = Primavera_df['Сценарий'].fillna('нет сценария')

In [ ]:
print(Primavera_df['Сценарий'].count())
print(Primavera_df['Сценарий'].shape[0])

3506
3506


In [ ]:
Primavera_df['Сценарий'].tail(20)

3486    Обработка вызова для колбэка 2023
3487    Обработка вызова для колбэка 2023
3488    Обработка вызова для колбэка 2023
3489    Обработка вызова для колбэка 2023
3490    Обработка вызова для колбэка 2023
3491    Обработка вызова для колбэка 2023
3492    Обработка вызова для колбэка 2023
3493       Интерактивная обработка вызова
3494       Интерактивная обработка вызова
3495    Обработка вызова для колбэка 2023
3496    Обработка вызова для колбэка 2023
3497    Обработка вызова для колбэка 2023
3498    Обработка вызова для колбэка 2023
3499    Обработка вызова для колбэка 2023
3500    Обработка вызова для колбэка 2023
3501       Интерактивная обработка вызова
3502       Интерактивная обработка вызова
3503    Обработка вызова для колбэка 2023
3504    Обработка вызова для колбэка 2023
3505    Обработка вызова для колбэка 2023
Name: Сценарий, dtype: object

In [ ]:
Primavera_df['Сценарий'][Primavera_df['Сценарий'] == ''].count()

0

##### Операции

In [ ]:
Primavera_df['Операции'].head()

0    Интерактивная обработка вызова 1, Интерактивная обработка вызова 2
1                                      Интерактивная обработка вызова 1
2    Интерактивная обработка вызова 1, Интерактивная обработка вызова 2
3                                      Интерактивная обработка вызова 1
4                                      Интерактивная обработка вызова 1
Name: Операции, dtype: object

In [ ]:
print(Primavera_df['Операции'].count())
print(Primavera_df['Операции'].shape[0])

3454
3506


In [ ]:
# Избавляемся от пустот в столбце 'Операции'
Primavera_df['Операции'] = Primavera_df['Операции'].fillna('нет операций')

In [ ]:
print(Primavera_df['Операции'].count())
print(Primavera_df['Операции'].shape[0])

3506
3506


In [ ]:
Primavera_df['Операции'].tail(20)

3486                                      Интерактивная обработка вызова 1
3487                                      Интерактивная обработка вызова 1
3488                                      Интерактивная обработка вызова 1
3489                                      Интерактивная обработка вызова 1
3490                                      Интерактивная обработка вызова 1
3491                                      Интерактивная обработка вызова 1
3492                                      Интерактивная обработка вызова 1
3493                                      Интерактивная обработка вызова 1
3494                                      Интерактивная обработка вызова 1
3495    Интерактивная обработка вызова 1, Интерактивная обработка вызова 1
3496    Интерактивная обработка вызова 1, Интерактивная обработка вызова 1
3497    Интерактивная обработка вызова 1, Интерактивная обработка вызова 1
3498                                      Интерактивная обработка вызова 1
3499                     

In [ ]:
Primavera_df['Операции'][Primavera_df['Операции'] == ''].count()

0

##### Идентификатор сессии звонка

In [ ]:
Primavera_df['Идентификатор сессии звонка'].head()

0    3128610272
1    3124137714
2    3124136879
3    3128578812
4    3128558172
Name: Идентификатор сессии звонка, dtype: object

In [ ]:
print(Primavera_df['Идентификатор сессии звонка'].count())
print(Primavera_df['Идентификатор сессии звонка'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Идентификатор сессии звонка'
Primavera_df = Primavera_df.astype({'Идентификатор сессии звонка': int})

In [ ]:
Primavera_df['Идентификатор сессии звонка'].head()

0    3128610272
1    3124137714
2    3124136879
3    3128578812
4    3128558172
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Primavera_calls_match_df['Идентификатор сессии звонка'].head()

328    2906114070
297    2906349026
312    2906829010
329    2906845626
290    2908089834
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Primavera_df['Идентификатор сессии звонка'].tail(20)

3486    2958584576
3487    2957932368
3488    2957868984
3489    2957846304
3490    2957786408
3491    2957727700
3492    2957707280
3493    2929114554
3494    2925525778
3495    2942033560
3496    2941706348
3497    2941687072
3498    2933902212
3499    2933839984
3500    2933799548
3501    2931643480
3502    2917601233
3503    2924662976
3504    2924590524
3505    2924569648
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Primavera_calls_match_df['Идентификатор сессии звонка'].tail(20)

1022    3091814387
1051    3092340177
1019    3092727687
1094    3092943870
1038    3092993577
1026    3093360602
1030    3093398932
1091    3093539570
1083    3093550380
1096    3093561345
1002    3093871612
1077    3093954200
1014    3094818180
1007    3095079880
1031    3095337940
1050    3095965625
1047    3096335875
1009    3097006075
1039    3097217120
1018    3098000195
Name: Идентификатор сессии звонка, dtype: int64

##### Группа, выбранная посетителем

In [ ]:
Primavera_df['Группа, выбранная посетителем'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
print(Primavera_df['Группа, выбранная посетителем'].count())
print(Primavera_df['Группа, выбранная посетителем'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Группа, выбранная посетителем'
Primavera_df['Группа, выбранная посетителем'] = Primavera_df['Группа, выбранная посетителем'].fillna('нет')

In [ ]:
print(Primavera_df['Группа, выбранная посетителем'].count())
print(Primavera_df['Группа, выбранная посетителем'].shape[0])

3506
3506


In [ ]:
Primavera_df['Группа, выбранная посетителем'][Primavera_df['Группа, выбранная посетителем'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Группа, выбранная посетителем'
Primavera_df['Группа, выбранная посетителем'] = Primavera_df['Группа, выбранная посетителем'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Группа, выбранная посетителем'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
Primavera_df['Группа, выбранная посетителем'][Primavera_df['Группа, выбранная посетителем'] == ''].count()

0

In [ ]:
Primavera_df['Группа, выбранная посетителем'][Primavera_df['Группа, выбранная посетителем'] == 'нет'].count()

3506

In [ ]:
Primavera_df['Группа, выбранная посетителем'][Primavera_df['Группа, выбранная посетителем'] != 'нет'].count()

0

##### Длительность обработки

In [ ]:
Primavera_df['Длительность обработки'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Длительность обработки, dtype: object

In [ ]:
print(Primavera_df['Длительность обработки'].count())
print(Primavera_df['Длительность обработки'].shape[0])

324
3506


In [ ]:
# Избавляемся от пустот в столбце 'Длительность обработки'
Primavera_df['Длительность обработки'] = Primavera_df['Длительность обработки'].fillna('0')

In [ ]:
print(Primavera_df['Длительность обработки'].count())
print(Primavera_df['Длительность обработки'].shape[0])

3506
3506


In [ ]:
Primavera_df['Длительность обработки'][Primavera_df['Длительность обработки'] == ''].count()

79

In [ ]:
# Меняем '' на '0' в столбце 'Длительность обработки'
Primavera_df['Длительность обработки'] = Primavera_df['Длительность обработки'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Длительность обработки'][Primavera_df['Длительность обработки'] == ''].count()

0

In [ ]:
Primavera_df['Длительность обработки'][Primavera_df['Длительность обработки'] != '0']

28       1608.0
30        412.0
31        983.0
32       1188.0
33       1277.0
         ...   
3417      131.0
3418      159.0
3424      102.0
3464    2785968
3497        382
Name: Длительность обработки, Length: 245, dtype: object

In [ ]:
# Приводим тип данных столбца 'Длительность обработки'
Primavera_df = Primavera_df.astype({'Длительность обработки': int})

In [ ]:
Primavera_df['Длительность обработки'].tail(20)

3486      0
3487      0
3488      0
3489      0
3490      0
3491      0
3492      0
3493      0
3494      0
3495      0
3496      0
3497    382
3498      0
3499      0
3500      0
3501      0
3502      0
3503      0
3504      0
3505      0
Name: Длительность обработки, dtype: int64

##### Комментарий

In [ ]:
Primavera_df['Комментарий'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Комментарий, dtype: object

In [ ]:
print(Primavera_df['Комментарий'].count())
print(Primavera_df['Комментарий'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Комментарий'
Primavera_df['Комментарий'] = Primavera_df['Комментарий'].fillna('нет комментария')

In [ ]:
print(Primavera_df['Комментарий'].count())
print(Primavera_df['Комментарий'].shape[0])

3506
3506


In [ ]:
Primavera_df['Комментарий'].head()

0    нет комментария
1    нет комментария
2    нет комментария
3    нет комментария
4    нет комментария
Name: Комментарий, dtype: object

In [ ]:
Primavera_df['Комментарий'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Комментарий, dtype: object

In [ ]:
Primavera_df['Комментарий'][Primavera_df['Комментарий'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет комментария' в столбце 'Комментарий'
Primavera_df['Комментарий'] = Primavera_df['Комментарий'].apply(lambda x: 'нет комментария' if x == '' else x)

In [ ]:
Primavera_df['Комментарий'][Primavera_df['Комментарий'] == ''].count()

0

In [ ]:
Primavera_df['Комментарий'].tail(20)

3486    нет комментария
3487    нет комментария
3488    нет комментария
3489    нет комментария
3490    нет комментария
3491    нет комментария
3492    нет комментария
3493    нет комментария
3494    нет комментария
3495    нет комментария
3496    нет комментария
3497    нет комментария
3498    нет комментария
3499    нет комментария
3500    нет комментария
3501    нет комментария
3502    нет комментария
3503    нет комментария
3504    нет комментария
3505    нет комментария
Name: Комментарий, dtype: object

##### Контакт из CRM

In [ ]:
Primavera_df['Контакт из CRM'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Контакт из CRM, dtype: object

In [ ]:
print(Primavera_df['Контакт из CRM'].count())
print(Primavera_df['Контакт из CRM'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Контакт из CRM'
Primavera_df['Контакт из CRM'] = Primavera_df['Контакт из CRM'].fillna('нет')

In [ ]:
print(Primavera_df['Контакт из CRM'].count())
print(Primavera_df['Контакт из CRM'].shape[0])

3506
3506


In [ ]:
Primavera_df['Контакт из CRM'][Primavera_df['Контакт из CRM'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Контакт из CRM'
Primavera_df['Контакт из CRM'] = Primavera_df['Контакт из CRM'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Контакт из CRM'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Контакт из CRM, dtype: object

In [ ]:
Primavera_df['Контакт из CRM'][Primavera_df['Контакт из CRM'] == ''].count()

0

In [ ]:
Primavera_df['Контакт из CRM'][Primavera_df['Контакт из CRM'] == 'нет'].count()

3506

In [ ]:
Primavera_df['Контакт из CRM'][Primavera_df['Контакт из CRM'] != 'нет'].count()

0

##### Поисковая система

In [ ]:
Primavera_df['Поисковая система'].tail(20)

3486          
3487          
3488          
3489          
3490          
3491          
3492          
3493          
3494          
3495          
3496          
3497          
3498          
3499          
3500          
3501    yandex
3502          
3503          
3504          
3505          
Name: Поисковая система, dtype: object

In [ ]:
print(Primavera_df['Поисковая система'].count())
print(Primavera_df['Поисковая система'].shape[0])

562
3506


In [ ]:
# Избавляемся от пустот в столбце 'Поисковая система'
Primavera_df['Поисковая система'] = Primavera_df['Поисковая система'].fillna('нет')

In [ ]:
print(Primavera_df['Поисковая система'].count())
print(Primavera_df['Поисковая система'].shape[0])

3506
3506


In [ ]:
Primavera_df['Поисковая система'][Primavera_df['Поисковая система'] == ''].count()

78

In [ ]:
# Меняем '' на 'нет' в столбце 'Поисковая система'
Primavera_df['Поисковая система'] = Primavera_df['Поисковая система'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Поисковая система'][Primavera_df['Поисковая система'] == ''].count()

0

In [ ]:
Primavera_df['Поисковая система'].tail(20)

3486       нет
3487       нет
3488       нет
3489       нет
3490       нет
3491       нет
3492       нет
3493       нет
3494       нет
3495       нет
3496       нет
3497       нет
3498       нет
3499       нет
3500       нет
3501    yandex
3502       нет
3503       нет
3504       нет
3505       нет
Name: Поисковая система, dtype: object

##### Поисковый запрос

In [ ]:
Primavera_df['Поисковый запрос'].head(20)

0                                    NaN
1                                    NaN
2                                    NaN
3                          {новостройка}
4                          {новостройка}
5                          {новостройка}
6                                    NaN
7     жилой комплекс волоколамское шоссе
8     жилой комплекс волоколамское шоссе
9                                    NaN
10                                   NaN
11                                   NaN
12    жилой комплекс волоколамское шоссе
13    жилой комплекс волоколамское шоссе
14    жилой комплекс волоколамское шоссе
15    жилой комплекс волоколамское шоссе
16                                   NaN
17                                   NaN
18                                   NaN
19                                   NaN
Name: Поисковый запрос, dtype: object

In [ ]:
print(Primavera_df['Поисковый запрос'].count())
print(Primavera_df['Поисковый запрос'].shape[0])

908
3506


In [ ]:
# Избавляемся от пустот в столбце 'Поисковый запрос'
Primavera_df['Поисковый запрос'] = Primavera_df['Поисковый запрос'].fillna('нет запроса')

In [ ]:
Primavera_df['Поисковый запрос'].head(20)

0                            нет запроса
1                            нет запроса
2                            нет запроса
3                          {новостройка}
4                          {новостройка}
5                          {новостройка}
6                            нет запроса
7     жилой комплекс волоколамское шоссе
8     жилой комплекс волоколамское шоссе
9                            нет запроса
10                           нет запроса
11                           нет запроса
12    жилой комплекс волоколамское шоссе
13    жилой комплекс волоколамское шоссе
14    жилой комплекс волоколамское шоссе
15    жилой комплекс волоколамское шоссе
16                           нет запроса
17                           нет запроса
18                           нет запроса
19                           нет запроса
Name: Поисковый запрос, dtype: object

In [ ]:
Primavera_df['Поисковый запрос'].tail(20)

3486                                                       
3487                                                       
3488                                                       
3489                                                       
3490                                                       
3491                                                       
3492                                                       
3493                                                       
3494    новостройки москвы циан от застройщика старт продаж
3495                                                       
3496                                                       
3497                                                       
3498                                                       
3499                                                       
3500                                                       
3501                                           жк примавера
3502               старт продаж новая мо

In [ ]:
Primavera_df['Поисковый запрос'][Primavera_df['Поисковый запрос'] == ''].count()

44

In [ ]:
# Меняем '' на 'нет запроса' в столбце 'Поисковый запрос'
Primavera_df['Поисковый запрос'] = Primavera_df['Поисковый запрос'].apply(lambda x: 'нет запроса' if x == '' else x)

In [ ]:
Primavera_df['Поисковый запрос'][Primavera_df['Поисковый запрос'] == ''].count()

0

In [ ]:
Primavera_df['Поисковый запрос'].tail(20)

3486                                            нет запроса
3487                                            нет запроса
3488                                            нет запроса
3489                                            нет запроса
3490                                            нет запроса
3491                                            нет запроса
3492                                            нет запроса
3493                                            нет запроса
3494    новостройки москвы циан от застройщика старт продаж
3495                                            нет запроса
3496                                            нет запроса
3497                                            нет запроса
3498                                            нет запроса
3499                                            нет запроса
3500                                            нет запроса
3501                                           жк примавера
3502               старт продаж новая мо

In [ ]:
# Убираем '{}' в столбце 'Поисковый запрос'
Primavera_df['Поисковый запрос'] = Primavera_df['Поисковый запрос'].apply(lambda x: x.strip('{}'))

In [ ]:
Primavera_df['Поисковый запрос'].head(20)

0                            нет запроса
1                            нет запроса
2                            нет запроса
3                            новостройка
4                            новостройка
5                            новостройка
6                            нет запроса
7     жилой комплекс волоколамское шоссе
8     жилой комплекс волоколамское шоссе
9                            нет запроса
10                           нет запроса
11                           нет запроса
12    жилой комплекс волоколамское шоссе
13    жилой комплекс волоколамское шоссе
14    жилой комплекс волоколамское шоссе
15    жилой комплекс волоколамское шоссе
16                           нет запроса
17                           нет запроса
18                           нет запроса
19                           нет запроса
Name: Поисковый запрос, dtype: object

##### Реферер

In [ ]:
Primavera_df['Реферер'].tail(20)

3486    https://yandex.ru/an/count/Wj0ejI_zO1K2DHO0L2WyRG_dACYDc0K05GCnDTPaP000000uWjRi0K08u8lOWT__Ylsf0O01WwEL28W1X-tR5f01qh3reUc0W802c07IiFMXQR01fBERiGYu0TR0uzScm0760TW1veNtiW7W0QxCdAS1w06A0WEW0jxNZndm0lNFm945xl0Q-0JzmTSlY0NzmTSla0NzmTSle0MwYyOEg0NmteyCi0NmteyCk0NmteyCm0M0fiHmo0M0ZiuSq0NnrRe5e0Rei06e1kYm0RW6Tia62F06tDWREXMf1u437Q4nglCak0Uq1j32a806u0ZlwySCw0a7y0c07lW9y1202WRu2e2r680BhkLmY0i6c0lMyJBe39S2c0tLvJBW3OQ0WO60c828G9WEWS_qi-7uZ8jTa12pcidgeeAycYkQ40dfFyWGmQ0Gc16LxowXkUaIPZDuaB_hCswe4yFie9NSh_MB5z0KdBUrV-WKZ0AO5f2lagC6eCaMq1Qklzw-0PWNlxUIBwWN2S0NjTO1mXWCTdHkD5GvGqTvCKHLy1ZS_06W6UYm0R0PYE2Ru8txXfqL-1d_tCWfWHh___lsyGbYQh0Qyj_4ZiATzEr4mXeaD3bXPMGrC6GjDpSpE2qqOJaqBJarCMKjC3HaPZ4nOsGnEJOnoXeaD3b1HKGrC4H1HIqmD4H6CJ53H88S3KbeH3evD51hP5f8QJUe7W6m7m787y30fcUI8AaW65g9DBbC3q2u8Fs6Bf0YbUyka2AMxowG8hMGBv0Ykf0lrIB__t__WIE9u3-G8y-3sUIjqOZ9eG6O8_UDgjtynfIRRl8Z4Ack_7E10m40BiQLfUFx1G2e9E41i2G1k2JX0S8a0J7A90CoCZKJ0Aqifh14SMOkWz1WC4-f7F57vF9F9kqgWswAORhamWxfoR_CS78HhHsZ0kMnobGSFQ8_Sb66gKCNmv09c0S0~

In [ ]:
print(Primavera_df['Реферер'].count())
print(Primavera_df['Реферер'].shape[0])

1227
3506


In [ ]:
# Избавляемся от пустот в столбце 'Реферер'
Primavera_df['Реферер'] = Primavera_df['Реферер'].fillna('нет')

In [ ]:
print(Primavera_df['Реферер'].count())
print(Primavera_df['Реферер'].shape[0])

3506
3506


In [ ]:
Primavera_df['Реферер'][Primavera_df['Реферер'] == ''].count()

0

In [ ]:
Primavera_df['Реферер'].tail(20)

3486    https://yandex.ru/an/count/Wj0ejI_zO1K2DHO0L2WyRG_dACYDc0K05GCnDTPaP000000uWjRi0K08u8lOWT__Ylsf0O01WwEL28W1X-tR5f01qh3reUc0W802c07IiFMXQR01fBERiGYu0TR0uzScm0760TW1veNtiW7W0QxCdAS1w06A0WEW0jxNZndm0lNFm945xl0Q-0JzmTSlY0NzmTSla0NzmTSle0MwYyOEg0NmteyCi0NmteyCk0NmteyCm0M0fiHmo0M0ZiuSq0NnrRe5e0Rei06e1kYm0RW6Tia62F06tDWREXMf1u437Q4nglCak0Uq1j32a806u0ZlwySCw0a7y0c07lW9y1202WRu2e2r680BhkLmY0i6c0lMyJBe39S2c0tLvJBW3OQ0WO60c828G9WEWS_qi-7uZ8jTa12pcidgeeAycYkQ40dfFyWGmQ0Gc16LxowXkUaIPZDuaB_hCswe4yFie9NSh_MB5z0KdBUrV-WKZ0AO5f2lagC6eCaMq1Qklzw-0PWNlxUIBwWN2S0NjTO1mXWCTdHkD5GvGqTvCKHLy1ZS_06W6UYm0R0PYE2Ru8txXfqL-1d_tCWfWHh___lsyGbYQh0Qyj_4ZiATzEr4mXeaD3bXPMGrC6GjDpSpE2qqOJaqBJarCMKjC3HaPZ4nOsGnEJOnoXeaD3b1HKGrC4H1HIqmD4H6CJ53H88S3KbeH3evD51hP5f8QJUe7W6m7m787y30fcUI8AaW65g9DBbC3q2u8Fs6Bf0YbUyka2AMxowG8hMGBv0Ykf0lrIB__t__WIE9u3-G8y-3sUIjqOZ9eG6O8_UDgjtynfIRRl8Z4Ack_7E10m40BiQLfUFx1G2e9E41i2G1k2JX0S8a0J7A90CoCZKJ0Aqifh14SMOkWz1WC4-f7F57vF9F9kqgWswAORhamWxfoR_CS78HhHsZ0kMnobGSFQ8_Sb66gKCNmv09c0S0~

##### Домен реферера

In [ ]:
Primavera_df['Домен реферера'].tail(20)

3486        yandex.ru
3487        yandex.ru
3488        yandex.ru
3489        yandex.ru
3490        yandex.ru
3491        yandex.ru
3492        yandex.ru
3493        yandex.ru
3494        yandex.ru
3495        yandex.ru
3496        yandex.ru
3497        yandex.ru
3498        yandex.ru
3499        yandex.ru
3500        yandex.ru
3501    www.yandex.ru
3502        yandex.ru
3503        yandex.ru
3504        yandex.ru
3505        yandex.ru
Name: Домен реферера, dtype: object

In [ ]:
print(Primavera_df['Домен реферера'].count())
print(Primavera_df['Домен реферера'].shape[0])

1227
3506


In [ ]:
# Избавляемся от пустот в столбце 'Домен реферера'
Primavera_df['Домен реферера'] = Primavera_df['Домен реферера'].fillna('нет')

In [ ]:
print(Primavera_df['Домен реферера'].count())
print(Primavera_df['Домен реферера'].shape[0])

3506
3506


In [ ]:
Primavera_df['Домен реферера'][Primavera_df['Домен реферера'] == ''].count()

0

In [ ]:
Primavera_df['Домен реферера'].tail(20)

3486        yandex.ru
3487        yandex.ru
3488        yandex.ru
3489        yandex.ru
3490        yandex.ru
3491        yandex.ru
3492        yandex.ru
3493        yandex.ru
3494        yandex.ru
3495        yandex.ru
3496        yandex.ru
3497        yandex.ru
3498        yandex.ru
3499        yandex.ru
3500        yandex.ru
3501    www.yandex.ru
3502        yandex.ru
3503        yandex.ru
3504        yandex.ru
3505        yandex.ru
Name: Домен реферера, dtype: object

##### Область

In [ ]:
Primavera_df['Область'].tail(20)

3486    Новосибирская область
3487    Новосибирская область
3488    Новосибирская область
3489    Новосибирская область
3490                         
3491                         
3492                         
3493                   Москва
3494        Город Севастополь
3495                   Москва
3496                   Москва
3497                   Москва
3498      Ханты-Мансийский АО
3499      Ханты-Мансийский АО
3500      Ханты-Мансийский АО
3501                   Москва
3502         Липецкая область
3503                         
3504                         
3505                         
Name: Область, dtype: object

In [ ]:
print(Primavera_df['Область'].count())
print(Primavera_df['Область'].shape[0])

1635
3506


In [ ]:
# Избавляемся от пустот в столбце 'Область'
Primavera_df['Область'] = Primavera_df['Область'].fillna('нет')

In [ ]:
print(Primavera_df['Область'].count())
print(Primavera_df['Область'].shape[0])

3506
3506


In [ ]:
Primavera_df['Область'][Primavera_df['Область'] == ''].count()

11

In [ ]:
# Меняем '' на 'нет' в столбце 'Область'
Primavera_df['Область'] = Primavera_df['Область'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Область'][Primavera_df['Область'] == ''].count()

0

In [ ]:
Primavera_df['Область'].tail(20)

3486    Новосибирская область
3487    Новосибирская область
3488    Новосибирская область
3489    Новосибирская область
3490                      нет
3491                      нет
3492                      нет
3493                   Москва
3494        Город Севастополь
3495                   Москва
3496                   Москва
3497                   Москва
3498      Ханты-Мансийский АО
3499      Ханты-Мансийский АО
3500      Ханты-Мансийский АО
3501                   Москва
3502         Липецкая область
3503                      нет
3504                      нет
3505                      нет
Name: Область, dtype: object

##### Посадочная страница

In [ ]:
Primavera_df['Посадочная страница'].tail(20)

3486                                                                                                                         https://primavera.moscow/
3487                                                                                                                         https://primavera.moscow/
3488                                                                                                                         https://primavera.moscow/
3489                                                                                                                         https://primavera.moscow/
3490                                                                                                                         https://primavera.moscow/
3491                                                                                                                         https://primavera.moscow/
3492                                                                                          

In [ ]:
print(Primavera_df['Посадочная страница'].count())
print(Primavera_df['Посадочная страница'].shape[0])

1778
3506


In [ ]:
# Избавляемся от пустот в столбце 'Посадочная страница'
Primavera_df['Посадочная страница'] = Primavera_df['Посадочная страница'].fillna('нет')

In [ ]:
print(Primavera_df['Посадочная страница'].count())
print(Primavera_df['Посадочная страница'].shape[0])

3506
3506


In [ ]:
Primavera_df['Посадочная страница'][Primavera_df['Посадочная страница'] == ''].count()

0

In [ ]:
Primavera_df['Посадочная страница'].tail(20)

3486                                                                                                                         https://primavera.moscow/
3487                                                                                                                         https://primavera.moscow/
3488                                                                                                                         https://primavera.moscow/
3489                                                                                                                         https://primavera.moscow/
3490                                                                                                                         https://primavera.moscow/
3491                                                                                                                         https://primavera.moscow/
3492                                                                                          

##### Количество посещений

In [ ]:
Primavera_df['Количество посещений'].tail(20)

3486     2
3487     1
3488     1
3489     1
3490     1
3491     1
3492     1
3493     1
3494     1
3495     1
3496     1
3497     1
3498     1
3499     1
3500     1
3501     4
3502    25
3503     2
3504     2
3505     2
Name: Количество посещений, dtype: object

In [ ]:
print(Primavera_df['Количество посещений'].count())
print(Primavera_df['Количество посещений'].shape[0])

1838
3506


In [ ]:
# Избавляемся от пустот в столбце 'Количество посещений'
Primavera_df['Количество посещений'] = Primavera_df['Количество посещений'].fillna('0')

In [ ]:
print(Primavera_df['Количество посещений'].count())
print(Primavera_df['Количество посещений'].shape[0])

3506
3506


In [ ]:
Primavera_df['Количество посещений'][Primavera_df['Количество посещений'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Количество посещений'
Primavera_df['Количество посещений'] = Primavera_df['Количество посещений'].apply(lambda x: x.strip('{}'))

In [ ]:
# Приводим тип данных столбца 'Количество посещений'
Primavera_df = Primavera_df.astype({'Количество посещений': int})

In [ ]:
Primavera_df['Количество посещений'].tail(20)

3486     2
3487     1
3488     1
3489     1
3490     1
3491     1
3492     1
3493     1
3494     1
3495     1
3496     1
3497     1
3498     1
3499     1
3500     1
3501     4
3502    25
3503     2
3504     2
3505     2
Name: Количество посещений, dtype: int64

##### Тип посетителя

In [ ]:
Primavera_df['Тип посетителя'].tail(20)

3486    Вернувшийся
3487          Новый
3488          Новый
3489          Новый
3490          Новый
3491          Новый
3492          Новый
3493          Новый
3494          Новый
3495          Новый
3496          Новый
3497          Новый
3498          Новый
3499          Новый
3500          Новый
3501    Вернувшийся
3502    Вернувшийся
3503    Вернувшийся
3504    Вернувшийся
3505    Вернувшийся
Name: Тип посетителя, dtype: object

In [ ]:
print(Primavera_df['Тип посетителя'].count())
print(Primavera_df['Тип посетителя'].shape[0])

3506
3506


In [ ]:
Primavera_df['Тип посетителя'][Primavera_df['Тип посетителя'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Тип посетителя'
Primavera_df['Тип посетителя'] = Primavera_df['Тип посетителя'].apply(lambda x: x.strip('{}'))

In [ ]:
Primavera_df['Тип посетителя'].tail(20)

3486    Вернувшийся
3487          Новый
3488          Новый
3489          Новый
3490          Новый
3491          Новый
3492          Новый
3493          Новый
3494          Новый
3495          Новый
3496          Новый
3497          Новый
3498          Новый
3499          Новый
3500          Новый
3501    Вернувшийся
3502    Вернувшийся
3503    Вернувшийся
3504    Вернувшийся
3505    Вернувшийся
Name: Тип посетителя, dtype: object

##### Client ID Google Analytics

In [ ]:
Primavera_df['Client ID Google Analytics'].tail(20)

3486    1728746961.1684319515
3487    1728746961.1684319515
3488    1728746961.1684319515
3489    1728746961.1684319515
3490    1107347558.1684317012
3491    1107347558.1684317012
3492    1107347558.1684317012
3493    1888886513.1684139480
3494     552762533.1683897404
3495    2120857893.1683725381
3496    1524441474.1683720219
3497    1524441474.1683720219
3498     840248104.1683287587
3499     840248104.1683287587
3500     840248104.1683287587
3501      89941127.1683204565
3502     176861434.1680714987
3503    2121878252.1682498793
3504    2121878252.1682498793
3505    2121878252.1682498793
Name: Client ID Google Analytics, dtype: object

In [ ]:
print(Primavera_df['Client ID Google Analytics'].count())
print(Primavera_df['Client ID Google Analytics'].shape[0])

1677
3506


In [ ]:
# Избавляемся от пустот в столбце 'Client ID Google Analytics'
Primavera_df['Client ID Google Analytics'] = Primavera_df['Client ID Google Analytics'].fillna('0')

In [ ]:
print(Primavera_df['Client ID Google Analytics'].count())
print(Primavera_df['Client ID Google Analytics'].shape[0])

3506
3506


In [ ]:
Primavera_df['Client ID Google Analytics'][Primavera_df['Client ID Google Analytics'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Client ID Google Analytics'
Primavera_df['Client ID Google Analytics'] = Primavera_df['Client ID Google Analytics'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Client ID Google Analytics'][Primavera_df['Client ID Google Analytics'] == ''].count()

0

In [ ]:
Primavera_df['Client ID Google Analytics'].tail(20)

3486    1728746961.1684319515
3487    1728746961.1684319515
3488    1728746961.1684319515
3489    1728746961.1684319515
3490    1107347558.1684317012
3491    1107347558.1684317012
3492    1107347558.1684317012
3493    1888886513.1684139480
3494     552762533.1683897404
3495    2120857893.1683725381
3496    1524441474.1683720219
3497    1524441474.1683720219
3498     840248104.1683287587
3499     840248104.1683287587
3500     840248104.1683287587
3501      89941127.1683204565
3502     176861434.1680714987
3503    2121878252.1682498793
3504    2121878252.1682498793
3505    2121878252.1682498793
Name: Client ID Google Analytics, dtype: object

##### ID ресурса Google Analytics

In [ ]:
Primavera_df['ID ресурса Google Analytics'].tail(20)

3486    UA-211696639-1
3487    UA-211696639-1
3488    UA-211696639-1
3489    UA-211696639-1
3490    UA-211696639-1
3491    UA-211696639-1
3492    UA-211696639-1
3493    UA-211696639-1
3494    UA-211696639-1
3495    UA-211696639-1
3496    UA-211696639-1
3497    UA-211696639-1
3498    UA-211696639-1
3499    UA-211696639-1
3500    UA-211696639-1
3501    UA-211696639-1
3502    UA-211696639-1
3503    UA-211696639-1
3504    UA-211696639-1
3505    UA-211696639-1
Name: ID ресурса Google Analytics, dtype: object

In [ ]:
print(Primavera_df['ID ресурса Google Analytics'].count())
print(Primavera_df['ID ресурса Google Analytics'].shape[0])

1794
3506


In [ ]:
# Избавляемся от пустот в столбце 'ID ресурса Google Analytics'
Primavera_df['ID ресурса Google Analytics'] = Primavera_df['ID ресурса Google Analytics'].fillna('нет')

In [ ]:
print(Primavera_df['ID ресурса Google Analytics'].count())
print(Primavera_df['ID ресурса Google Analytics'].shape[0])

3506
3506


In [ ]:
Primavera_df['ID ресурса Google Analytics'][Primavera_df['ID ресурса Google Analytics'] == ''].count()

0

In [ ]:
Primavera_df['ID ресурса Google Analytics'].tail(20)

3486    UA-211696639-1
3487    UA-211696639-1
3488    UA-211696639-1
3489    UA-211696639-1
3490    UA-211696639-1
3491    UA-211696639-1
3492    UA-211696639-1
3493    UA-211696639-1
3494    UA-211696639-1
3495    UA-211696639-1
3496    UA-211696639-1
3497    UA-211696639-1
3498    UA-211696639-1
3499    UA-211696639-1
3500    UA-211696639-1
3501    UA-211696639-1
3502    UA-211696639-1
3503    UA-211696639-1
3504    UA-211696639-1
3505    UA-211696639-1
Name: ID ресурса Google Analytics, dtype: object

##### ID счетчика Яндекс.Метрика

In [ ]:
Primavera_df['ID счетчика Яндекс.Метрика'].tail(20)

3486    86217072
3487    86217072
3488    86217072
3489    86217072
3490    86217072
3491    86217072
3492    86217072
3493    86217072
3494    86217072
3495    86217072
3496    86217072
3497    86217072
3498    86217072
3499    86217072
3500    86217072
3501    86217072
3502    86217072
3503    86217072
3504    86217072
3505    86217072
Name: ID счетчика Яндекс.Метрика, dtype: object

In [ ]:
print(Primavera_df['ID счетчика Яндекс.Метрика'].count())
print(Primavera_df['ID счетчика Яндекс.Метрика'].shape[0])

1794
3506


In [ ]:
# Избавляемся от пустот в столбце 'ID счетчика Яндекс.Метрика'
Primavera_df['ID счетчика Яндекс.Метрика'] = Primavera_df['ID счетчика Яндекс.Метрика'].fillna('0')

In [ ]:
print(Primavera_df['ID счетчика Яндекс.Метрика'].count())
print(Primavera_df['ID счетчика Яндекс.Метрика'].shape[0])

3506
3506


In [ ]:
Primavera_df['ID счетчика Яндекс.Метрика'][Primavera_df['ID счетчика Яндекс.Метрика'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'ID счетчика Яндекс.Метрика'
Primavera_df = Primavera_df.astype({'ID счетчика Яндекс.Метрика': int})

In [ ]:
Primavera_df['ID счетчика Яндекс.Метрика'].tail(20)

3486    86217072
3487    86217072
3488    86217072
3489    86217072
3490    86217072
3491    86217072
3492    86217072
3493    86217072
3494    86217072
3495    86217072
3496    86217072
3497    86217072
3498    86217072
3499    86217072
3500    86217072
3501    86217072
3502    86217072
3503    86217072
3504    86217072
3505    86217072
Name: ID счетчика Яндекс.Метрика, dtype: int64

##### UTM-метка Referrer

In [ ]:
Primavera_df['UTM-метка Referrer'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: UTM-метка Referrer, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Referrer'].count())
print(Primavera_df['UTM-метка Referrer'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Referrer'
Primavera_df['UTM-метка Referrer'] = Primavera_df['UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Referrer'].count())
print(Primavera_df['UTM-метка Referrer'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Referrer'][Primavera_df['UTM-метка Referrer'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Referrer'
Primavera_df['UTM-метка Referrer'] = Primavera_df['UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['UTM-метка Referrer'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: UTM-метка Referrer, dtype: object

In [ ]:
Primavera_df['UTM-метка Referrer'][Primavera_df['UTM-метка Referrer'] == ''].count()

0

In [ ]:
Primavera_df['UTM-метка Referrer'][Primavera_df['UTM-метка Referrer'] == 'нет'].count()

3506

In [ ]:
Primavera_df['UTM-метка Referrer'][Primavera_df['UTM-метка Referrer'] != 'нет'].count()

0

##### UTM-метка Expid

In [ ]:
Primavera_df['UTM-метка Expid'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: UTM-метка Expid, dtype: object

In [ ]:
print(Primavera_df['UTM-метка Expid'].count())
print(Primavera_df['UTM-метка Expid'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Expid'
Primavera_df['UTM-метка Expid'] = Primavera_df['UTM-метка Expid'].fillna('нет')

In [ ]:
print(Primavera_df['UTM-метка Expid'].count())
print(Primavera_df['UTM-метка Expid'].shape[0])

3506
3506


In [ ]:
Primavera_df['UTM-метка Expid'][Primavera_df['UTM-метка Expid'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Expid'
Primavera_df['UTM-метка Expid'] = Primavera_df['UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['UTM-метка Expid'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: UTM-метка Expid, dtype: object

In [ ]:
Primavera_df['UTM-метка Expid'][Primavera_df['UTM-метка Expid'] == ''].count()

0

In [ ]:
Primavera_df['UTM-метка Expid'][Primavera_df['UTM-метка Expid'] == 'нет'].count()

3506

In [ ]:
Primavera_df['UTM-метка Expid'][Primavera_df['UTM-метка Expid'] != 'нет'].count()

0

##### AdLense-метка ef_id

In [ ]:
Primavera_df['AdLense-метка ef_id'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: AdLense-метка ef_id, dtype: object

In [ ]:
print(Primavera_df['AdLense-метка ef_id'].count())
print(Primavera_df['AdLense-метка ef_id'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'AdLense-метка ef_id'
Primavera_df['AdLense-метка ef_id'] = Primavera_df['AdLense-метка ef_id'].fillna('нет')

In [ ]:
print(Primavera_df['AdLense-метка ef_id'].count())
print(Primavera_df['AdLense-метка ef_id'].shape[0])

3506
3506


In [ ]:
Primavera_df['AdLense-метка ef_id'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Primavera_df['AdLense-метка ef_id'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Primavera_df['AdLense-метка ef_id'][Primavera_df['AdLense-метка ef_id'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'AdLense-метка ef_id'
Primavera_df['AdLense-метка ef_id'] = Primavera_df['AdLense-метка ef_id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['AdLense-метка ef_id'][Primavera_df['AdLense-метка ef_id'] == ''].count()

0

In [ ]:
Primavera_df['AdLense-метка ef_id'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Primavera_df['AdLense-метка ef_id'][Primavera_df['AdLense-метка ef_id'] == 'нет'].count()

3506

In [ ]:
Primavera_df['AdLense-метка ef_id'][Primavera_df['AdLense-метка ef_id'] != 'нет'].count()

0

##### Метка yclid

In [ ]:
Primavera_df['Метка yclid'].head()

0                    NaN
1                    NaN
2                    NaN
3    4363725577004324171
4    4363725577004324171
Name: Метка yclid, dtype: object

In [ ]:
print(Primavera_df['Метка yclid'].count())
print(Primavera_df['Метка yclid'].shape[0])

1390
3506


In [ ]:
# Меняем '' на '0' в столбце 'Метка yclid'
Primavera_df['Метка yclid'] = Primavera_df['Метка yclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Метка yclid'].head()

0                    NaN
1                    NaN
2                    NaN
3    4363725577004324171
4    4363725577004324171
Name: Метка yclid, dtype: object

In [ ]:
Primavera_df['Метка yclid'].tail(20)

3486    2662663404380815959
3487    2659899654995247703
3488    2659899654995247703
3489    2659899654995247703
3490    2659243462674672393
3491    2659243462674672393
3492    2659243462674672393
3493    2612704797570172083
3494    2549244662505596079
3495    2504151094701384814
3496    2502798362189828695
3497    2502798362189828695
3498    2389386309354710364
3499    2389386309354710364
3500    2389386309354710364
3501    2367698927704662526
3502    2319550148502164195
3503    2317316041848198448
3504    2317316041848198448
3505    2317316041848198448
Name: Метка yclid, dtype: object

In [ ]:
Primavera_df['Метка yclid'][Primavera_df['Метка yclid'] == ''].count()

0

##### Метка ymclid

In [ ]:
Primavera_df['Метка ymclid'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Метка ymclid, dtype: object

In [ ]:
print(Primavera_df['Метка ymclid'].count())
print(Primavera_df['Метка ymclid'].shape[0])

81
3506


In [ ]:
# Меняем '' на '0' в столбце 'Метка ymclid'
Primavera_df['Метка ymclid'] = Primavera_df['Метка ymclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Метка ymclid'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: Метка ymclid, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'Метка ymclid'
Primavera_df['Метка ymclid'] = Primavera_df['Метка ymclid'].fillna('0')

In [ ]:
print(Primavera_df['Метка ymclid'].count())
print(Primavera_df['Метка ymclid'].shape[0])

3506
3506


In [ ]:
Primavera_df['Метка ymclid'].head()

0    0
1    0
2    0
3    0
4    0
Name: Метка ymclid, dtype: object

In [ ]:
Primavera_df['Метка ymclid'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: Метка ymclid, dtype: object

In [ ]:
Primavera_df['Метка ymclid'][Primavera_df['Метка ymclid'] == ''].count()

0

In [ ]:
Primavera_df['Метка ymclid'][Primavera_df['Метка ymclid'] == '0'].count()

3506

In [ ]:
Primavera_df['Метка ymclid'][Primavera_df['Метка ymclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка ymclid'
Primavera_df['Метка ymclid'] = Primavera_df['Метка ymclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Primavera_df['Метка ymclid'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Метка ymclid, dtype: object

##### Метка gclid

In [ ]:
Primavera_df['Метка gclid'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Метка gclid, dtype: object

In [ ]:
print(Primavera_df['Метка gclid'].count())
print(Primavera_df['Метка gclid'].shape[0])

92
3506


In [ ]:
# Меняем '' на '0' в столбце 'Метка gclid'
Primavera_df['Метка gclid'] = Primavera_df['Метка gclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Метка gclid'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: Метка gclid, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'Метка gclid'
Primavera_df['Метка gclid'] = Primavera_df['Метка gclid'].fillna('0')

In [ ]:
print(Primavera_df['Метка gclid'].count())
print(Primavera_df['Метка gclid'].shape[0])

3506
3506


In [ ]:
Primavera_df['Метка gclid'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: Метка gclid, dtype: object

In [ ]:
Primavera_df['Метка gclid'][Primavera_df['Метка gclid'] == ''].count()

0

In [ ]:
Primavera_df['Метка gclid'][Primavera_df['Метка gclid'] == '0'].count()

3495

In [ ]:
Primavera_df['Метка gclid'][Primavera_df['Метка gclid'] != '0'].count()

11

In [ ]:
Primavera_df['Метка gclid'][Primavera_df['Метка gclid'] != '0']

779                                          EAIaIQobChMI9o6nzu6GgAMVQRAGAB3f1giVEAAYASAAEgJ6f_D_BwE
781                                          EAIaIQobChMI9o6nzu6GgAMVQRAGAB3f1giVEAAYASAAEgJ6f_D_BwE
782                                          EAIaIQobChMI9o6nzu6GgAMVQRAGAB3f1giVEAAYASAAEgJ6f_D_BwE
783                                          EAIaIQobChMI9o6nzu6GgAMVQRAGAB3f1giVEAAYASAAEgJ6f_D_BwE
784                                          EAIaIQobChMI9o6nzu6GgAMVQRAGAB3f1giVEAAYASAAEgJ6f_D_BwE
1399     CjwKCAjwkeqkBhAnEiwA5U-uM9j91GzbZLtrgUaeFiGnW3raFhorCnwuDcs59gUfu-qsJ30W_BVozhoCUMIQAvD_BwE
1663    Cj0KCQjwnMWkBhDLARIsAHBOftrQoB6Yjgur2TCZeVdGDhAClSle1aqbV9Y2xomSpPUOHOgk5e1g9u8aArDTEALw_wcB
1664    Cj0KCQjwnMWkBhDLARIsAHBOftrQoB6Yjgur2TCZeVdGDhAClSle1aqbV9Y2xomSpPUOHOgk5e1g9u8aArDTEALw_wcB
1665    Cj0KCQjwnMWkBhDLARIsAHBOftrQoB6Yjgur2TCZeVdGDhAClSle1aqbV9Y2xomSpPUOHOgk5e1g9u8aArDTEALw_wcB
2926     CjwKCAjw9pGjBhB-EiwAa5jl3BWxmdyiATWYNjnu5xGZ1-UbVYiLv9jrwMywQMGWSStmVIZOCwXapBoC_R

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка gclid'
Primavera_df['Метка gclid'] = Primavera_df['Метка gclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Primavera_df['Метка gclid'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Метка gclid, dtype: object

##### Метка from

In [ ]:
Primavera_df['Метка from'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Метка from, dtype: object

In [ ]:
print(Primavera_df['Метка from'].count())
print(Primavera_df['Метка from'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Метка from'
Primavera_df['Метка from'] = Primavera_df['Метка from'].fillna('0')

In [ ]:
print(Primavera_df['Метка from'].count())
print(Primavera_df['Метка from'].shape[0])

3506
3506


In [ ]:
# Меняем '' на '0' в столбце 'Метка from'
Primavera_df['Метка from'] = Primavera_df['Метка from'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['Метка from'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: Метка from, dtype: object

In [ ]:
Primavera_df['Метка from'][Primavera_df['Метка from'] == ''].count()

0

In [ ]:
Primavera_df['Метка from'][Primavera_df['Метка from'] == '0'].count()

3506

In [ ]:
Primavera_df['Метка from'][Primavera_df['Метка from'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка from'
Primavera_df['Метка from'] = Primavera_df['Метка from'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Primavera_df['Метка from'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Метка from, dtype: object

##### Длительность разговора

In [ ]:
Primavera_df['Длительность разговора'].head()

0    00:33:02
1    00:00:50
2    00:01:26
3    00:00:00
4    00:00:00
Name: Длительность разговора, dtype: object

In [ ]:
print(Primavera_df['Длительность разговора'].count())
print(Primavera_df['Длительность разговора'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Длительность разговора'
Primavera_df['Длительность разговора'] = pd.to_datetime(Primavera_df['Длительность разговора']).dt.time

In [ ]:
Primavera_df['Длительность разговора'].head()

0    00:33:02
1    00:00:50
2    00:01:26
3    00:00:00
4    00:00:00
Name: Длительность разговора, dtype: object

In [ ]:
Primavera_df['Длительность разговора'].tail(20)

3486    00:00:00
3487    00:00:00
3488    00:00:00
3489    00:00:00
3490    00:00:00
3491    00:00:00
3492    00:00:00
3493    00:00:00
3494    00:00:00
3495    00:00:00
3496    00:00:06
3497    00:00:00
3498    00:00:00
3499    00:00:00
3500    00:00:00
3501    00:01:21
3502    00:00:40
3503    00:00:00
3504    00:00:00
3505    00:00:00
Name: Длительность разговора, dtype: object

##### Длительность постобработки

In [ ]:
Primavera_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
print(Primavera_df['Длительность постобработки'].count())
print(Primavera_df['Длительность постобработки'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Длительность постобработки'
Primavera_df['Длительность постобработки'] = pd.to_datetime(Primavera_df['Длительность постобработки']).dt.time

In [ ]:
Primavera_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
Primavera_df['Длительность постобработки'].tail(20)

3486    00:00:00
3487    00:00:00
3488    00:00:00
3489    00:00:00
3490    00:00:00
3491    00:00:00
3492    00:00:00
3493    00:00:00
3494    00:00:00
3495    00:00:00
3496    00:00:00
3497    00:00:00
3498    00:00:00
3499    00:00:00
3500    00:00:00
3501    00:00:00
3502    00:00:00
3503    00:00:00
3504    00:00:00
3505    00:00:00
Name: Длительность постобработки, dtype: object

##### Чистая длительность разговора

In [ ]:
Primavera_df['Чистая длительность разговора'].head()

0    00:33:01
1    00:00:50
2    00:01:26
3    00:00:00
4    00:00:00
Name: Чистая длительность разговора, dtype: object

In [ ]:
print(Primavera_df['Чистая длительность разговора'].count())
print(Primavera_df['Чистая длительность разговора'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'Чистая длительность разговора'
Primavera_df['Чистая длительность разговора'] = pd.to_datetime(Primavera_df['Чистая длительность разговора']).dt.time

In [ ]:
Primavera_df['Чистая длительность разговора'].head()

0    00:33:01
1    00:00:50
2    00:01:26
3    00:00:00
4    00:00:00
Name: Чистая длительность разговора, dtype: object

In [ ]:
Primavera_df['Чистая длительность разговора'].tail(20)

3486    00:00:00
3487    00:00:00
3488    00:00:00
3489    00:00:00
3490    00:00:00
3491    00:00:00
3492    00:00:00
3493    00:00:00
3494    00:00:00
3495    00:00:00
3496    00:00:06
3497    00:00:00
3498    00:00:00
3499    00:00:00
3500    00:00:00
3501    00:01:21
3502    00:00:40
3503    00:00:00
3504    00:00:00
3505    00:00:00
Name: Чистая длительность разговора, dtype: object

##### Полная длительность ожидания

In [ ]:
Primavera_df['Полная длительность ожидания'].head()

0    00:00:31
1    00:00:08
2    00:00:27
3    00:00:03
4    00:00:02
Name: Полная длительность ожидания, dtype: object

In [ ]:
print(Primavera_df['Полная длительность ожидания'].count())
print(Primavera_df['Полная длительность ожидания'].shape[0])

3506
3506


In [ ]:
Primavera_df['Полная длительность ожидания'].loc[Primavera_df['Полная длительность ожидания'] == '-1д 23:59:59']

1853    -1д 23:59:59
1854    -1д 23:59:59
Name: Полная длительность ожидания, dtype: object

In [ ]:
# Меняем '-1д 23:59:59' на '23:59:59' в столбце 'Полная длительность ожидания'
Primavera_df['Полная длительность ожидания'] = Primavera_df['Полная длительность ожидания'].apply(lambda x: '23:59:59' if x == '-1д 23:59:59' else x)

In [ ]:
# Приводим тип данных столбца 'Полная длительность ожидания'
Primavera_df['Полная длительность ожидания'] = pd.to_datetime(Primavera_df['Полная длительность ожидания']).dt.time

In [ ]:
Primavera_df['Полная длительность ожидания'].head()

0    00:00:31
1    00:00:08
2    00:00:27
3    00:00:03
4    00:00:02
Name: Полная длительность ожидания, dtype: object

In [ ]:
Primavera_df['Полная длительность ожидания'].tail(20)

3486    00:01:02
3487    00:00:18
3488    00:00:17
3489    00:00:17
3490    00:00:12
3491    00:00:12
3492    00:00:12
3493    00:01:03
3494    00:00:04
3495    00:00:14
3496    00:00:34
3497    00:00:36
3498    00:00:26
3499    00:00:26
3500    00:00:26
3501    00:00:06
3502    00:00:06
3503    00:00:19
3504    00:00:19
3505    00:00:18
Name: Полная длительность ожидания, dtype: object

##### Расширенная UTM-метка Source

In [ ]:
Primavera_df['Расширенная UTM-метка Source'].head()

0            NaN
1    novostroy_m
2            NaN
3         yandex
4         yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Source'].count())
print(Primavera_df['Расширенная UTM-метка Source'].shape[0])

1826
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Source'
Primavera_df['Расширенная UTM-метка Source'] = Primavera_df['Расширенная UTM-метка Source'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Source'].count())
print(Primavera_df['Расширенная UTM-метка Source'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Source'].head()

0            нет
1    novostroy_m
2            нет
3         yandex
4         yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Source'].tail(20)

3486    yandex
3487    yandex
3488    yandex
3489    yandex
3490    yandex
3491    yandex
3492    yandex
3493    yandex
3494    yandex
3495    yandex
3496    yandex
3497    yandex
3498    yandex
3499    yandex
3500    yandex
3501    yandex
3502    yandex
3503    yandex
3504    yandex
3505    yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Source'][Primavera_df['Расширенная UTM-метка Source'] == ''].count()

0

##### Расширенная UTM-метка Campaign

In [ ]:
Primavera_df['Расширенная UTM-метка Campaign'].head()

0                                             NaN
1                                             NaN
2                                             NaN
3    m108|retargeting_new_rsy_rf|92517773|context
4    m108|retargeting_new_rsy_rf|92517773|context
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Campaign'].count())
print(Primavera_df['Расширенная UTM-метка Campaign'].shape[0])

1568
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Campaign'
Primavera_df['Расширенная UTM-метка Campaign'] = Primavera_df['Расширенная UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Campaign'].count())
print(Primavera_df['Расширенная UTM-метка Campaign'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Campaign'].head()

0                                             нет
1                                             нет
2                                             нет
3    m108|retargeting_new_rsy_rf|92517773|context
4    m108|retargeting_new_rsy_rf|92517773|context
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Campaign'].tail(20)

3486    m108|profile_competitors_rsy_rf|87621747|context
3487    m108|profile_competitors_rsy_rf|87621747|context
3488    m108|profile_competitors_rsy_rf|87621747|context
3489    m108|profile_competitors_rsy_rf|87621747|context
3490        m108|interes_realty_rsy_mmo|82276325|context
3491        m108|interes_realty_rsy_mmo|82276325|context
3492        m108|interes_realty_rsy_mmo|82276325|context
3493       m108|interes_ipoteka_rsy_mmo|85569957|context
3494         m108|start+kotlovan_rsy_rf|84726293|context
3495        m108|interes_realty_rsy_mmo|82276325|context
3496        m108|interes_realty_rsy_mmo|82276325|context
3497        m108|interes_realty_rsy_mmo|82276325|context
3498         m108|interes_realty_rsy_rf|82276333|context
3499         m108|interes_realty_rsy_rf|82276333|context
3500         m108|interes_realty_rsy_rf|82276333|context
3501       m108|brand_podarok_search_mmo|87274641|search
3502         m108|start+kotlovan_rsy_rf|84726293|context
3503         m108|start+kotlova

In [ ]:
Primavera_df['Расширенная UTM-метка Campaign'][Primavera_df['Расширенная UTM-метка Campaign'] == ''].count()

0

##### Расширенная UTM-метка Medium

In [ ]:
Primavera_df['Расширенная UTM-метка Medium'].head()

0     NaN
1    week
2     NaN
3     cpc
4     cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Medium'].count())
print(Primavera_df['Расширенная UTM-метка Medium'].shape[0])

1660
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Medium'
Primavera_df['Расширенная UTM-метка Medium'] = Primavera_df['Расширенная UTM-метка Medium'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Medium'].count())
print(Primavera_df['Расширенная UTM-метка Medium'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Medium'].head()

0     нет
1    week
2     нет
3     cpc
4     cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Medium'].tail(20)

3486    cpc
3487    cpc
3488    cpc
3489    cpc
3490    cpc
3491    cpc
3492    cpc
3493    cpc
3494    cpc
3495    cpc
3496    cpc
3497    cpc
3498    cpc
3499    cpc
3500    cpc
3501    cpc
3502    cpc
3503    cpc
3504    cpc
3505    cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Medium'][Primavera_df['Расширенная UTM-метка Medium'] == ''].count()

0

##### Расширенная UTM-метка Content

In [ ]:
Primavera_df['Расширенная UTM-метка Content'].head()

0                                                                                                          NaN
1                                                                                                          NaN
2                                                                                                          NaN
3    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
4    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Content'].count())
print(Primavera_df['Расширенная UTM-метка Content'].shape[0])

1522
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Content'
Primavera_df['Расширенная UTM-метка Content'] = Primavera_df['Расширенная UTM-метка Content'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Content'].count())
print(Primavera_df['Расширенная UTM-метка Content'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Content'].head()

0                                                                                                          нет
1                                                                                                          нет
2                                                                                                          нет
3    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
4    5255653129|14772857680|52421796|proantibiotik.ru|desktop|14772857680|type1|52421796|52421796|0|none|11156
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Content'].tail(20)

3486              5192314923|14128426025|47642247|dzen.ru|mobile|14128426025|type1|47642247|47642247|1|none|11316
3487                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3488                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3489                 5192314923|14128426028|47642247|dzen.ru|mobile|14128426028|type1|47642247|47642247|1|none|65
3490               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3491               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3492               5115655687|13488200564|41721602|dzen.ru|desktop|13488200564|type1|41721602|41721602|2|none|213
3493               5162529018|13794196700|45369764|dzen.ru|desktop|13794196700|type1|45369764|45369764|1|none|213
3494      5151257235|13677074587|43745648635|dzen.ru|desktop|13677074587|type1|437456486

In [ ]:
Primavera_df['Расширенная UTM-метка Content'][Primavera_df['Расширенная UTM-метка Content'] == ''].count()

0

##### Расширенная UTM-метка Term

In [ ]:
Primavera_df['Расширенная UTM-метка Term'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Term'].count())
print(Primavera_df['Расширенная UTM-метка Term'].shape[0])

747
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Term'
Primavera_df['Расширенная UTM-метка Term'] = Primavera_df['Расширенная UTM-метка Term'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Term'].count())
print(Primavera_df['Расширенная UTM-метка Term'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Term'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Term'].tail(20)

3486                                                       
3487                                                       
3488                                                       
3489                                                       
3490                                                       
3491                                                       
3492                                                       
3493                                                       
3494    новостройки москвы циан от застройщика старт продаж
3495                                                       
3496                                                       
3497                                                       
3498                                                       
3499                                                       
3500                                                       
3501                                           жк примавера
3502               старт продаж новая мо

In [ ]:
Primavera_df['Расширенная UTM-метка Term'][Primavera_df['Расширенная UTM-метка Term'] == ''].count()

50

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Term'
Primavera_df['Расширенная UTM-метка Term'] = Primavera_df['Расширенная UTM-метка Term'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Расширенная UTM-метка Term'][Primavera_df['Расширенная UTM-метка Term'] == ''].count()

0

In [ ]:
Primavera_df['Расширенная UTM-метка Term'].tail(20)

3486                                                    нет
3487                                                    нет
3488                                                    нет
3489                                                    нет
3490                                                    нет
3491                                                    нет
3492                                                    нет
3493                                                    нет
3494    новостройки москвы циан от застройщика старт продаж
3495                                                    нет
3496                                                    нет
3497                                                    нет
3498                                                    нет
3499                                                    нет
3500                                                    нет
3501                                           жк примавера
3502               старт продаж новая мо

##### Расширенная UTM-метка Referrer

In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Referrer'].count())
print(Primavera_df['Расширенная UTM-метка Referrer'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Referrer'
Primavera_df['Расширенная UTM-метка Referrer'] = Primavera_df['Расширенная UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Referrer'].count())
print(Primavera_df['Расширенная UTM-метка Referrer'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'][Primavera_df['Расширенная UTM-метка Referrer'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Referrer'
Primavera_df['Расширенная UTM-метка Referrer'] = Primavera_df['Расширенная UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'][Primavera_df['Расширенная UTM-метка Referrer'] == ''].count()

0

In [ ]:
Primavera_df['Расширенная UTM-метка Referrer'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Расширенная UTM-метка Referrer, dtype: object

##### Расширенная UTM-метка Expid

In [ ]:
Primavera_df['Расширенная UTM-метка Expid'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
print(Primavera_df['Расширенная UTM-метка Expid'].count())
print(Primavera_df['Расширенная UTM-метка Expid'].shape[0])

81
3506


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Expid'
Primavera_df['Расширенная UTM-метка Expid'] = Primavera_df['Расширенная UTM-метка Expid'].fillna('нет')

In [ ]:
print(Primavera_df['Расширенная UTM-метка Expid'].count())
print(Primavera_df['Расширенная UTM-метка Expid'].shape[0])

3506
3506


In [ ]:
Primavera_df['Расширенная UTM-метка Expid'].head()

0    нет
1    нет
2    нет
3    нет
4    нет
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Expid'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
Primavera_df['Расширенная UTM-метка Expid'][Primavera_df['Расширенная UTM-метка Expid'] == ''].count()

81

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Expid'
Primavera_df['Расширенная UTM-метка Expid'] = Primavera_df['Расширенная UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['Расширенная UTM-метка Expid'][Primavera_df['Расширенная UTM-метка Expid'] == ''].count()

0

In [ ]:
Primavera_df['Расширенная UTM-метка Expid'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: Расширенная UTM-метка Expid, dtype: object

##### OS-метка service-name

In [ ]:
Primavera_df['OS-метка service-name'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: OS-метка service-name, dtype: object

In [ ]:
print(Primavera_df['OS-метка service-name'].count())
print(Primavera_df['OS-метка service-name'].shape[0])

141
3506


In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка service-name'
Primavera_df['OS-метка service-name'] = Primavera_df['OS-метка service-name'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['OS-метка service-name'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: OS-метка service-name, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка service-name'
Primavera_df['OS-метка service-name'] = Primavera_df['OS-метка service-name'].fillna('нет')

In [ ]:
print(Primavera_df['OS-метка service-name'].count())
print(Primavera_df['OS-метка service-name'].shape[0])

3506
3506


In [ ]:
Primavera_df['OS-метка service-name'][Primavera_df['OS-метка service-name'] == ''].count()

0

In [ ]:
Primavera_df['OS-метка service-name'][Primavera_df['OS-метка service-name'] == 'нет'].count()

3446

In [ ]:
Primavera_df['OS-метка service-name'][Primavera_df['OS-метка service-name'] != 'нет'].count()

60

##### OS-метка campaign-id

In [ ]:
Primavera_df['OS-метка campaign-id'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: OS-метка campaign-id, dtype: object

In [ ]:
print(Primavera_df['OS-метка campaign-id'].count())
print(Primavera_df['OS-метка campaign-id'].shape[0])

141
3506


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка campaign-id'
Primavera_df['OS-метка campaign-id'] = Primavera_df['OS-метка campaign-id'].fillna('0')

In [ ]:
print(Primavera_df['OS-метка campaign-id'].count())
print(Primavera_df['OS-метка campaign-id'].shape[0])

3506
3506


In [ ]:
# Меняем '' на '0' в столбце 'OS-метка campaign-id'
Primavera_df['OS-метка campaign-id'] = Primavera_df['OS-метка campaign-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['OS-метка campaign-id'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: OS-метка campaign-id, dtype: object

In [ ]:
# Приводим тип данных столбца 'OS-метка campaign-id'
Primavera_df = Primavera_df.astype({'OS-метка campaign-id': int})

In [ ]:
Primavera_df['OS-метка campaign-id'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: OS-метка campaign-id, dtype: int64

##### OS-метка ad-id

In [ ]:
Primavera_df['OS-метка ad-id'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: OS-метка ad-id, dtype: object

In [ ]:
print(Primavera_df['OS-метка ad-id'].count())
print(Primavera_df['OS-метка ad-id'].shape[0])

141
3506


In [ ]:
# Меняем '' на '0' в столбце 'OS-метка ad-id'
Primavera_df['OS-метка ad-id'] = Primavera_df['OS-метка ad-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Primavera_df['OS-метка ad-id'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: OS-метка ad-id, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка ad-id'
Primavera_df['OS-метка ad-id'] = Primavera_df['OS-метка ad-id'].fillna('0')

In [ ]:
print(Primavera_df['OS-метка ad-id'].count())
print(Primavera_df['OS-метка ad-id'].shape[0])

3506
3506


In [ ]:
# Приводим тип данных столбца 'OS-метка ad-id'
Primavera_df = Primavera_df.astype({'OS-метка ad-id': int})

In [ ]:
Primavera_df['OS-метка ad-id'].tail(20)

3486    0
3487    0
3488    0
3489    0
3490    0
3491    0
3492    0
3493    0
3494    0
3495    0
3496    0
3497    0
3498    0
3499    0
3500    0
3501    0
3502    0
3503    0
3504    0
3505    0
Name: OS-метка ad-id, dtype: int64

##### OS-метка source-id

In [ ]:
Primavera_df['OS-метка source-id'].tail(20)

3486    
3487    
3488    
3489    
3490    
3491    
3492    
3493    
3494    
3495    
3496    
3497    
3498    
3499    
3500    
3501    
3502    
3503    
3504    
3505    
Name: OS-метка source-id, dtype: object

In [ ]:
print(Primavera_df['OS-метка source-id'].count())
print(Primavera_df['OS-метка source-id'].shape[0])

141
3506


In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка source-id'
Primavera_df['OS-метка source-id'] = Primavera_df['OS-метка source-id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Primavera_df['OS-метка source-id'].tail(20)

3486    нет
3487    нет
3488    нет
3489    нет
3490    нет
3491    нет
3492    нет
3493    нет
3494    нет
3495    нет
3496    нет
3497    нет
3498    нет
3499    нет
3500    нет
3501    нет
3502    нет
3503    нет
3504    нет
3505    нет
Name: OS-метка source-id, dtype: object

In [ ]:
# Избавляемся от пустот в столбце 'OS-метка source-id'
Primavera_df['OS-метка source-id'] = Primavera_df['OS-метка source-id'].fillna('нет')

In [ ]:
print(Primavera_df['OS-метка source-id'].count())
print(Primavera_df['OS-метка source-id'].shape[0])

3506
3506


In [ ]:
Primavera_df['OS-метка source-id'][Primavera_df['OS-метка source-id'] == ''].count()

0

In [ ]:
Primavera_df['OS-метка source-id'][Primavera_df['OS-метка source-id'] == 'нет'].count()

3446

In [ ]:
Primavera_df['OS-метка source-id'][Primavera_df['OS-метка source-id'] != 'нет'].count()

60

##### Типы данных

In [ ]:
# Смотрим типы данных
Primavera_calls_match_df.dtypes

Дата и время                   datetime64[ns]
Номер абонента                          int64
Номер сотрудника                        int64
Идентификатор сессии звонка             int64
Файл записи звонка                     object
dtype: object

In [ ]:
# Смотрим типы данных
Primavera_df.dtypes

Статус                                             object
Тип                                                object
Дата и время                               datetime64[ns]
Номер абонента                                      int64
Рекламная кампания                                 object
Длительность звонка                                object
Длительность ожидания ответа                       object
Сотрудник                                          object
UTM-метка Campaign                                 object
Теги                                               object
Номер обращения                                     int64
UTM-метка Term                                     object
Причина завершения                                 object
Виртуальный номер                                   int64
UTM-метка Source                                   object
UTM-метка Medium                                   object
UTM-метка Content                                  object
Client ID Янде

##### Добавление столбцов

In [ ]:
# Смотрим несовпадающие столбцы Primavera_calls_match_df vs Primavera_df
for column in Primavera_calls_match_df.columns.tolist():
  if column not in Primavera_df.columns.tolist():
    print(f'Primavera_calls_match_df[{column}], {Primavera_calls_match_df[column].shape[0]} строк, тип данных: {Primavera_calls_match_df[column].dtypes}')

Primavera_calls_match_df[Файл записи звонка], 1517 строк, тип данных: object


In [ ]:
# Сортируем Primavera_df по столбцу 'Идентификатор сессии звонка'
Primavera_df = Primavera_df.sort_values('Идентификатор сессии звонка')

In [ ]:
# Индексируем по порядку
Primavera_df.index = range(Primavera_df.shape[0])

In [ ]:
# Добавляем относительные пути в столбец 'Файл записи звонка' в Primavera_calls_match_df
Primavera_calls_match_df['Файл записи звонка'] = Primavera_calls_match_df['Файл записи звонка'].apply(lambda x: 'Записи звонков/Записи звонков_primavera/' + x)
Primavera_calls_match_df['Файл записи звонка']

328     Записи звонков/Записи звонков_primavera/2023-05-01_10-25-48.968195_from_79605015550_to_0197879_session_2906114070_talk.mp3
297     Записи звонков/Записи звонков_primavera/2023-05-01_12-48-55.165895_from_79372616437_to_0150111_session_2906349026_talk.mp3
312     Записи звонков/Записи звонков_primavera/2023-05-01_17-32-54.319258_from_79169640642_to_0197872_session_2906829010_talk.mp3
329     Записи звонков/Записи звонков_primavera/2023-05-01_17-47-27.496928_from_79099933333_to_0111651_session_2906845626_talk.mp3
290     Записи звонков/Записи звонков_primavera/2023-05-02_13-20-04.263031_from_79113699353_to_0150108_session_2908089834_talk.mp3
                                                                   ...                                                            
1050    Записи звонков/Записи звонков_primavera/2023-07-14_18-42-23.856843_from_79605340557_to_0150108_session_3095965625_talk.mp3
1047    Записи звонков/Записи звонков_primavera/2023-07-15_09-04-06.416803_from_791

In [ ]:
# Индексируем по порядку
Primavera_calls_match_df.index = range(Primavera_calls_match_df.shape[0])

In [ ]:
# Добавляем столбец 'Файл записи звонка' в Primavera_df
Primavera_df = Primavera_df.merge(Primavera_calls_match_df[['Идентификатор сессии звонка','Файл записи звонка']], how='outer', on=['Идентификатор сессии звонка'])

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Primavera_df['Файл записи звонка'].count())
print(Primavera_df['Файл записи звонка'].shape[0])

1517
3506


In [ ]:
# Запоняем пустоты в столбце 'Файл записи звонка' значением 'нет файла'
Primavera_df['Файл записи звонка'] = Primavera_df['Файл записи звонка'].fillna('нет файла')

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Primavera_df['Файл записи звонка'].count())
print(Primavera_df['Файл записи звонка'].shape[0])

3506
3506


In [ ]:
# Нет файлов
Primavera_df['Файл записи звонка'][Primavera_df['Файл записи звонка'] == 'нет файла'].count()

1989

In [ ]:
# Есть файлы
Primavera_df['Файл записи звонка'][Primavera_df['Файл записи звонка'] != 'нет файла'].count()

1517

##### Сохранение датафреймов

In [ ]:
# Сохраняем Primavera_df в файл
with open(df_path + 'Primavera_df.pkl', 'wb') as f:
  pkl.dump(Primavera_df, f)

In [ ]:
# Сохраняем Primavera_calls_match_df в файл
with open(df_path + 'Primavera_calls_match_df.pkl', 'wb') as f:
  pkl.dump(Primavera_calls_match_df, f)

#### Paveletskaya_city

##### Статус

In [ ]:
Paveletskaya_city_df['Статус'].head()

0    Потерянный
1      Принятый
2    Потерянный
3      Принятый
4      Принятый
Name: Статус, dtype: object

In [ ]:
print(Paveletskaya_city_df['Статус'].count())
print(Paveletskaya_city_df['Статус'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Статус'].tail(20)

9307      Принятый
9308      Принятый
9309      Принятый
9310      Принятый
9311      Принятый
9312      Принятый
9313      Принятый
9314      Принятый
9315      Принятый
9316      Принятый
9317      Принятый
9318      Принятый
9319    Потерянный
9320      Принятый
9321      Принятый
9322      Принятый
9323      Принятый
9324      Принятый
9325      Принятый
9326      Принятый
Name: Статус, dtype: object

In [ ]:
Paveletskaya_city_df['Статус'][Paveletskaya_city_df['Статус'] == ''].count()

0

##### Тип

In [ ]:
Paveletskaya_city_df['Тип'].head()

0     Автоперезвон по заявкам
1                   Аналитика
2     Автоперезвон по заявкам
3     Автоперезвон по заявкам
4    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
print(Paveletskaya_city_df['Тип'].count())
print(Paveletskaya_city_df['Тип'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Тип'].tail(20)

9307                   Аналитика
9308                   Аналитика
9309                   Аналитика
9310                   Аналитика
9311                   Аналитика
9312                   Аналитика
9313                   Аналитика
9314                   Аналитика
9315                   Аналитика
9316                   Аналитика
9317                   Аналитика
9318     Автоперезвон по заявкам
9319     Автоперезвон по заявкам
9320     Автоперезвон по заявкам
9321    Динамический коллтрекинг
9322    Динамический коллтрекинг
9323    Динамический коллтрекинг
9324    Динамический коллтрекинг
9325    Динамический коллтрекинг
9326    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
Paveletskaya_city_df['Тип'][Paveletskaya_city_df['Тип'] == ''].count()

0

##### Дата и время

In [ ]:
Paveletskaya_city_df['Дата и время'].head()

0    2023-07-31 20:35:42
1    2023-07-31 19:52:57
2    2023-07-31 19:21:06
3    2023-07-31 19:18:04
4    2023-07-31 19:16:16
Name: Дата и время, dtype: object

In [ ]:
print(Paveletskaya_city_df['Дата и время'].count())
print(Paveletskaya_city_df['Дата и время'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_calls_match_df['Дата и время'].head()

959    2023-06-17 13:44:17
961    2023-06-17 14:08:51
897    2023-06-17 16:16:05
876    2023-06-17 12:54:28
982    2023-06-17 14:21:31
Name: Дата и время, dtype: object

In [ ]:
print(Paveletskaya_city_calls_match_df['Дата и время'].count())
print(Paveletskaya_city_calls_match_df['Дата и время'].shape[0])

992
992


In [ ]:
# Приводим тип данных столбца 'Дата и время'
Paveletskaya_city_df['Дата и время'] = pd.to_datetime(Paveletskaya_city_df['Дата и время'])
Paveletskaya_city_calls_match_df['Дата и время']= pd.to_datetime(Paveletskaya_city_df['Дата и время'])

In [ ]:
Paveletskaya_city_df['Дата и время'].head()

0   2023-07-31 20:35:42
1   2023-07-31 19:52:57
2   2023-07-31 19:21:06
3   2023-07-31 19:18:04
4   2023-07-31 19:16:16
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Paveletskaya_city_calls_match_df['Дата и время'].head()

959   2023-07-02 17:34:27
961   2023-07-02 13:52:24
897   2023-07-04 10:35:05
876   2023-07-04 14:32:19
982   2023-07-01 20:38:53
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Paveletskaya_city_df['Дата и время'].tail(20)

9307   2023-05-01 16:53:07
9308   2023-05-01 16:13:31
9309   2023-05-01 15:07:21
9310   2023-05-01 14:47:31
9311   2023-05-01 14:27:49
9312   2023-05-01 14:19:05
9313   2023-05-01 14:15:40
9314   2023-05-01 13:54:40
9315   2023-05-01 12:24:40
9316   2023-05-01 10:48:24
9317   2023-05-01 10:45:48
9318   2023-07-31 15:35:21
9319   2023-07-31 15:13:33
9320   2023-07-26 14:11:49
9321   2023-07-18 12:43:15
9322   2023-07-11 11:41:21
9323   2023-07-09 09:07:41
9324   2023-06-02 13:03:49
9325   2023-05-31 11:52:06
9326   2023-05-31 11:47:40
Name: Дата и время, dtype: datetime64[ns]

##### Номер абонента

In [ ]:
Paveletskaya_city_df['Номер абонента'].head()

0    79847034099.0
1    79055235742.0
2    79993480367.0
3    79993482013.0
4    79037201544.0
Name: Номер абонента, dtype: object

In [ ]:
print(Paveletskaya_city_df['Номер абонента'].count())
print(Paveletskaya_city_df['Номер абонента'].shape[0])

9324
9327


In [ ]:
# Избавляемся от пустот в столбце 'Номер абонента'
Paveletskaya_city_df['Номер абонента'] = Paveletskaya_city_df['Номер абонента'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Номер абонента'].count())
print(Paveletskaya_city_df['Номер абонента'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_calls_match_df['Номер абонента'].head()

959    79066743768
961    74993205445
897    79680303488
876    79803286601
982    79259041087
Name: Номер абонента, dtype: object

In [ ]:
print(Paveletskaya_city_calls_match_df['Номер абонента'].count())
print(Paveletskaya_city_calls_match_df['Номер абонента'].shape[0])

992
992


In [ ]:
# Приводим тип данных столбца 'Номер абонента'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Номер абонента': int})
Paveletskaya_city_calls_match_df = Paveletskaya_city_calls_match_df.astype({'Номер абонента': int})

In [ ]:
Paveletskaya_city_df['Номер абонента'].head()

0    79847034099
1    79055235742
2    79993480367
3    79993482013
4    79037201544
Name: Номер абонента, dtype: int64

In [ ]:
Paveletskaya_city_calls_match_df['Номер абонента'].head()

959    79066743768
961    74993205445
897    79680303488
876    79803286601
982    79259041087
Name: Номер абонента, dtype: int64

In [ ]:
Paveletskaya_city_df['Номер абонента'].tail(20)

9307    79031575733
9308    79032807272
9309    79601679630
9310    79651980098
9311    79652478156
9312    79615506325
9313    79037453850
9314    79263355183
9315    79775196373
9316    77055259984
9317    79263888778
9318    79033154042
9319    79852334204
9320    79177230322
9321    79104878388
9322    79031327722
9323    79138625625
9324    79656863828
9325    79263068269
9326    79263068269
Name: Номер абонента, dtype: int64

##### Рекламная кампания

In [ ]:
Paveletskaya_city_df['Рекламная кампания'].head()

0    Media108 | Павелецкая Сити | direct | Яндекс.Директ
1         MRGroup | Павелецкая Сити | seo | Google Поиск
2    Media108 | Павелецкая Сити | direct | Яндекс.Директ
3    Media108 | Павелецкая Сити | direct | Яндекс.Директ
4    Media108 | Павелецкая Сити | direct | Яндекс.Директ
Name: Рекламная кампания, dtype: object

In [ ]:
print(Paveletskaya_city_df['Рекламная кампания'].count())
print(Paveletskaya_city_df['Рекламная кампания'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Рекламная кампания'].tail(20)

9307                                                   Artics | Павелецкая Сити | xml | cian.ru
9308                                    MRGroup | Павелецкая Сити | sms | СМС рассылка (ручная)
9309                                                Artics | Павелецкая Сити | cpa | reffection
9310                                                Artics | Павелецкая Сити | cpa | reffection
9311                                                Artics | Павелецкая Сити | cpa | reffection
9312                                                 Artics | Павелецкая Сити | media | 2realty
9313                                                Artics | Павелецкая Сити | cpa | reffection
9314                                          Artics | Павелецкая Сити | xml | yandex.realty.ru
9315                                             Artics | Павелецкая Сити | maps | Яндекс.Карты
9316       MRGroup | Павелецкая Сити | direct | Яндекс.Директ Турбо ya_paveletskaya_msk_n_brand
9317                                    

In [ ]:
Paveletskaya_city_df['Рекламная кампания'][Paveletskaya_city_df['Рекламная кампания'] == ''].count()

0

##### Длительность звонка

In [ ]:
Paveletskaya_city_df['Длительность звонка'].head()

0    00:00:37
1    00:04:06
2    00:00:36
3    00:00:45
4    00:05:08
Name: Длительность звонка, dtype: object

In [ ]:
print(Paveletskaya_city_df['Длительность звонка'].count())
print(Paveletskaya_city_df['Длительность звонка'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Длительность звонка'
Paveletskaya_city_df['Длительность звонка'] = pd.to_datetime(Paveletskaya_city_df['Длительность звонка']).dt.time

In [ ]:
Paveletskaya_city_df['Длительность звонка'].head()

0    00:00:37
1    00:04:06
2    00:00:36
3    00:00:45
4    00:05:08
Name: Длительность звонка, dtype: object

In [ ]:
Paveletskaya_city_df['Длительность звонка'].tail(20)

9307    00:04:24
9308    00:07:00
9309    00:03:58
9310    00:09:03
9311    00:00:50
9312    00:05:04
9313    00:07:49
9314    00:00:42
9315    00:05:21
9316    00:00:18
9317    00:17:41
9318    00:06:33
9319    00:00:36
9320    00:30:18
9321    00:10:12
9322    00:00:33
9323    00:00:56
9324    00:05:22
9325    00:03:00
9326    00:00:53
Name: Длительность звонка, dtype: object

##### Длительность ожидания ответа

In [ ]:
Paveletskaya_city_df['Длительность ожидания ответа'].head()

0    00:00:37
1    00:00:02
2    00:00:36
3    00:00:40
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
print(Paveletskaya_city_df['Длительность ожидания ответа'].count())
print(Paveletskaya_city_df['Длительность ожидания ответа'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Длительность ожидания ответа'
Paveletskaya_city_df['Длительность ожидания ответа'] = pd.to_datetime(Paveletskaya_city_df['Длительность ожидания ответа']).dt.time

In [ ]:
Paveletskaya_city_df['Длительность ожидания ответа'].head()

0    00:00:37
1    00:00:02
2    00:00:36
3    00:00:40
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
Paveletskaya_city_df['Длительность ожидания ответа'].tail(20)

9307    00:00:01
9308    00:00:01
9309    00:00:01
9310    00:00:01
9311    00:00:01
9312    00:00:01
9313    00:00:01
9314    00:00:01
9315    00:00:01
9316    00:00:03
9317    00:00:01
9318    00:00:35
9319    00:00:36
9320    00:00:39
9321    00:00:01
9322    00:00:01
9323    00:00:01
9324    00:00:02
9325    00:00:02
9326    00:00:01
Name: Длительность ожидания ответа, dtype: object

##### Сотрудник

In [ ]:
Paveletskaya_city_df['Сотрудник'].head()

0    Обратный звонок (SIP<1)
1                74955141111
2    Обратный звонок (SIP<1)
3    Обратный звонок (SIP<1)
4                74955141111
Name: Сотрудник, dtype: object

In [ ]:
print(Paveletskaya_city_df['Сотрудник'].count())
print(Paveletskaya_city_df['Сотрудник'].shape[0])

2660
9327


In [ ]:
# Избавляемся от пустот в столбце 'Сотрудник'
Paveletskaya_city_df['Сотрудник'] = Paveletskaya_city_df['Сотрудник'].fillna('нет сотрудника')

In [ ]:
print(Paveletskaya_city_df['Сотрудник'].count())
print(Paveletskaya_city_df['Сотрудник'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Сотрудник'].head()

0    Обратный звонок (SIP<1)
1                74955141111
2    Обратный звонок (SIP<1)
3    Обратный звонок (SIP<1)
4                74955141111
Name: Сотрудник, dtype: object

In [ ]:
Paveletskaya_city_df['Сотрудник'].tail(20)

9307                    SIP URI
9308                74955141111
9309                    SIP URI
9310                    SIP URI
9311                    SIP URI
9312                    SIP URI
9313                    SIP URI
9314                    SIP URI
9315                74955141111
9316                74955141111
9317                    SIP URI
9318    Обратный звонок (SIP<1)
9319    Обратный звонок (SIP<1)
9320    Обратный звонок (SIP<1)
9321                74955141111
9322                74955141111
9323                74955141111
9324                74955141111
9325                74955141111
9326                74955141111
Name: Сотрудник, dtype: object

In [ ]:
Paveletskaya_city_df['Сотрудник'][Paveletskaya_city_df['Сотрудник'] == ''].count()

0

##### UTM-метка Campaign

In [ ]:
Paveletskaya_city_df['UTM-метка Campaign'].head()

0       m108_pavcity_brand_net_mmo|88281925
1                                       NaN
2       m108_pavcity_brand_net_mmo|88281925
3       m108_pavcity_brand_net_mmo|88281925
4    m108_pavcity_brand_search_mmo|87584869
Name: UTM-метка Campaign, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Campaign'].count())
print(Paveletskaya_city_df['UTM-метка Campaign'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Campaign'
Paveletskaya_city_df['UTM-метка Campaign'] = Paveletskaya_city_df['UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Campaign'].count())
print(Paveletskaya_city_df['UTM-метка Campaign'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Campaign'].head()

0       m108_pavcity_brand_net_mmo|88281925
1                                       нет
2       m108_pavcity_brand_net_mmo|88281925
3       m108_pavcity_brand_net_mmo|88281925
4    m108_pavcity_brand_search_mmo|87584869
Name: UTM-метка Campaign, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Campaign'].tail(20)

9307                                        нет
9308                                        нет
9309                                        нет
9310                                        нет
9311                                        нет
9312                                        нет
9313                                        нет
9314                                        нет
9315                                        нет
9316                                        нет
9317                                        нет
9318      m108_pavcity_brand_search_rf|88705684
9319     m108_pavcity_loc_alll_net_mmo|88841281
9320    m108_pavelcity_brand_search_rf|88705684
9321                  brand_search_mmo|87584869
9322                   brand_search_rf|88705684
9323                  brand_search_mmo|87584869
9324                  brand_search_mmo|87584869
9325                ya_paveletskaya_msk_s_brand
9326                ya_paveletskaya_msk_s_brand
Name: UTM-метка Campaign, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Campaign'][Paveletskaya_city_df['UTM-метка Campaign'] == ''].count()

0

##### Теги

In [ ]:
Paveletskaya_city_df['Теги'].head()

0    Не обработано - Нецелевой контакт
1                         Целевой_М108
2    Не обработано - Нецелевой контакт
3                                  NaN
4                                  NaN
Name: Теги, dtype: object

In [ ]:
print(Paveletskaya_city_df['Теги'].count())
print(Paveletskaya_city_df['Теги'].shape[0])

7895
9327


In [ ]:
# Избавляемся от пустот в столбце 'Теги'
Paveletskaya_city_df['Теги'] = Paveletskaya_city_df['Теги'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Теги'].count())
print(Paveletskaya_city_df['Теги'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Теги'].tail(20)

9307                                  нет
9308                                  нет
9309                                  нет
9310                                  нет
9311                                  нет
9312                                  нет
9313                                  нет
9314                                  нет
9315                                  нет
9316                    Нецелевой контакт
9317                                  нет
9318                                     
9319    Не обработано - Нецелевой контакт
9320                         Целевой_М108
9321                                     
9322                                     
9323                                     
9324                                     
9325                                     
9326                                     
Name: Теги, dtype: object

In [ ]:
Paveletskaya_city_df['Теги'][Paveletskaya_city_df['Теги'] == ''].count()

7

In [ ]:
# Меняем '' на 'нет' в столбце 'Теги'
Paveletskaya_city_df['Теги'] = Paveletskaya_city_df['Теги'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Теги'][Paveletskaya_city_df['Теги'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Теги'].tail(20)

9307                                  нет
9308                                  нет
9309                                  нет
9310                                  нет
9311                                  нет
9312                                  нет
9313                                  нет
9314                                  нет
9315                                  нет
9316                    Нецелевой контакт
9317                                  нет
9318                                  нет
9319    Не обработано - Нецелевой контакт
9320                         Целевой_М108
9321                                  нет
9322                                  нет
9323                                  нет
9324                                  нет
9325                                  нет
9326                                  нет
Name: Теги, dtype: object

##### Номер обращения

In [ ]:
Paveletskaya_city_df['Номер обращения'].head()

0     3.0
1     1.0
2    63.0
3    60.0
4     9.0
Name: Номер обращения, dtype: object

In [ ]:
print(Paveletskaya_city_df['Номер обращения'].count())
print(Paveletskaya_city_df['Номер обращения'].shape[0])

9324
9327


In [ ]:
# Избавляемся от пустот в столбце 'Номер обращения'
Paveletskaya_city_df['Номер обращения'] = Paveletskaya_city_df['Номер обращения'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Номер обращения'].count())
print(Paveletskaya_city_df['Номер обращения'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Номер обращения'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Номер обращения': int})

In [ ]:
Paveletskaya_city_df['Номер обращения'].head()

0     3
1     1
2    63
3    60
4     9
Name: Номер обращения, dtype: int64

In [ ]:
Paveletskaya_city_df['Номер обращения'].tail(20)

9307     1
9308     3
9309     1
9310     1
9311     1
9312     1
9313     1
9314     1
9315     1
9316     1
9317     1
9318    23
9319     3
9320    19
9321     2
9322     2
9323     2
9324     2
9325     3
9326     2
Name: Номер обращения, dtype: int64

##### UTM-метка Term

In [ ]:
Paveletskaya_city_df['UTM-метка Term'].tail(20)

9307                                    NaN
9308                                    NaN
9309                                    NaN
9310                                    NaN
9311                                    NaN
9312                                    NaN
9313                                    NaN
9314                                    NaN
9315                                    NaN
9316                                    NaN
9317                                    NaN
9318                     жк павелецкая сити
9319    купить квартиру серпуховская москва
9320        павелецкая сити купить квартиру
9321                     жк павелецкая сити
9322              москва жк павелецкая сити
9323                     жк павелецкая сити
9324                        павелецкий сити
9325              жк павелецкий сити москва
9326              жк павелецкий сити москва
Name: UTM-метка Term, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Term'].count())
print(Paveletskaya_city_df['UTM-метка Term'].shape[0])

2791
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Term'
Paveletskaya_city_df['UTM-метка Term'] = Paveletskaya_city_df['UTM-метка Term'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Term'].count())
print(Paveletskaya_city_df['UTM-метка Term'].shape[0])

9327
9327


In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Term'
Paveletskaya_city_df['UTM-метка Term'] = Paveletskaya_city_df['UTM-метка Term'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['UTM-метка Term'].tail(20)

9307                                    нет
9308                                    нет
9309                                    нет
9310                                    нет
9311                                    нет
9312                                    нет
9313                                    нет
9314                                    нет
9315                                    нет
9316                                    нет
9317                                    нет
9318                     жк павелецкая сити
9319    купить квартиру серпуховская москва
9320        павелецкая сити купить квартиру
9321                     жк павелецкая сити
9322              москва жк павелецкая сити
9323                     жк павелецкая сити
9324                        павелецкий сити
9325              жк павелецкий сити москва
9326              жк павелецкий сити москва
Name: UTM-метка Term, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Term'][Paveletskaya_city_df['UTM-метка Term'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['UTM-метка Term'][Paveletskaya_city_df['UTM-метка Term'] == 'нет'].count()

6536

In [ ]:
Paveletskaya_city_df['UTM-метка Term'][Paveletskaya_city_df['UTM-метка Term'] != 'нет'].count()

2791

##### Причина завершения

In [ ]:
Paveletskaya_city_df['Причина завершения'].head()

0       Не дозвонились до абонента
1      Абонент разорвал соединение
2       Не дозвонились до абонента
3    Сотрудник разорвал соединение
4      Абонент разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
print(Paveletskaya_city_df['Причина завершения'].count())
print(Paveletskaya_city_df['Причина завершения'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Причина завершения'].tail(20)

9307      Абонент разорвал соединение
9308      Абонент разорвал соединение
9309      Абонент разорвал соединение
9310    Сотрудник разорвал соединение
9311    Сотрудник разорвал соединение
9312      Абонент разорвал соединение
9313      Абонент разорвал соединение
9314    Сотрудник разорвал соединение
9315      Абонент разорвал соединение
9316      Абонент разорвал соединение
9317    Сотрудник разорвал соединение
9318    Сотрудник разорвал соединение
9319       Не дозвонились до абонента
9320      Абонент разорвал соединение
9321      Абонент разорвал соединение
9322      Абонент разорвал соединение
9323    Сотрудник разорвал соединение
9324    Сотрудник разорвал соединение
9325      Абонент разорвал соединение
9326    Сотрудник разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
Paveletskaya_city_df['Причина завершения'][Paveletskaya_city_df['Причина завершения'] == ''].count()

0

##### Виртуальный номер

In [ ]:
Paveletskaya_city_df['Виртуальный номер'].head()

0    74951069644
1    74954871142
2    74951069644
3    74951069644
4    74951069937
Name: Виртуальный номер, dtype: object

In [ ]:
print(Paveletskaya_city_df['Виртуальный номер'].count())
print(Paveletskaya_city_df['Виртуальный номер'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Виртуальный номер'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Виртуальный номер': int})

In [ ]:
Paveletskaya_city_df['Виртуальный номер'].head()

0    74951069644
1    74954871142
2    74951069644
3    74951069644
4    74951069937
Name: Виртуальный номер, dtype: int64

In [ ]:
Paveletskaya_city_df['Виртуальный номер'].tail(20)

9307    74954871066
9308    74950855916
9309    74954871137
9310    74954871137
9311    74954871137
9312    74951048628
9313    74954871137
9314    74954871105
9315    74954871139
9316    74952560836
9317    74954871067
9318    74951069644
9319    74951069644
9320    74951069644
9321    74951251120
9322    74951252568
9323    74951251120
9324    74951321692
9325    74951519981
9326    74951519981
Name: Виртуальный номер, dtype: int64

##### UTM-метка Source

In [ ]:
Paveletskaya_city_df['UTM-метка Source'].head()

0    yandex
1       NaN
2    yandex
3    yandex
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Source'].count())
print(Paveletskaya_city_df['UTM-метка Source'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Source'
Paveletskaya_city_df['UTM-метка Source'] = Paveletskaya_city_df['UTM-метка Source'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Source'].count())
print(Paveletskaya_city_df['UTM-метка Source'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Source'].head()

0    yandex
1       нет
2    yandex
3    yandex
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Source'].tail(20)

9307       нет
9308       нет
9309       нет
9310       нет
9311       нет
9312       нет
9313       нет
9314       нет
9315       нет
9316       нет
9317       нет
9318    yandex
9319    yandex
9320    yandex
9321    yandex
9322    yandex
9323    yandex
9324    yandex
9325    yandex
9326    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Source'][Paveletskaya_city_df['UTM-метка Source'] == ''].count()

0

##### UTM-метка Medium

In [ ]:
Paveletskaya_city_df['UTM-метка Medium'].head()

0    cpc
1    NaN
2    cpc
3    cpc
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Medium'].count())
print(Paveletskaya_city_df['UTM-метка Medium'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Medium'
Paveletskaya_city_df['UTM-метка Medium'] = Paveletskaya_city_df['UTM-метка Medium'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Medium'].count())
print(Paveletskaya_city_df['UTM-метка Medium'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Medium'].head()

0    cpc
1    нет
2    cpc
3    cpc
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Medium'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    cpc
9319    cpc
9320    cpc
9321    cpc
9322    cpc
9323    cpc
9324    cpc
9325    cpc
9326    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Medium'][Paveletskaya_city_df['UTM-метка Medium'] == ''].count()

0

##### UTM-метка Content

In [ ]:
Paveletskaya_city_df['UTM-метка Content'].head()

0    id|44896430327_44896430327|cid|88281925|gid|5203088556|aid|14221067105|adp|no|pos|none0|src|context_fashionpronews.ru|dvc|mobile|coef|0
1                                                                                                                                        NaN
2          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
3          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
4               id|44890473306_44890473306|cid|87584869|gid|5202782758|aid|14219372961|adp|no|pos|premium1|src|search_none|dvc|mobile|coef|0
Name: UTM-метка Content, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Content'].count())
print(Paveletskaya_city_df['UTM-метка Content'].shape[0])

4677
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Content'
Paveletskaya_city_df['UTM-метка Content'] = Paveletskaya_city_df['UTM-метка Content'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Content'].count())
print(Paveletskaya_city_df['UTM-метка Content'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Content'].head()

0    id|44896430327_44896430327|cid|88281925|gid|5203088556|aid|14221067105|adp|no|pos|none0|src|context_fashionpronews.ru|dvc|mobile|coef|0
1                                                                                                                                        нет
2          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
3          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
4               id|44890473306_44890473306|cid|87584869|gid|5202782758|aid|14219372961|adp|no|pos|premium1|src|search_none|dvc|mobile|coef|0
Name: UTM-метка Content, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Content'].tail(20)

9307                                                                                                                               нет
9308                                                                                                                               нет
9309                                                                                                                               нет
9310                                                                                                                               нет
9311                                                                                                                               нет
9312                                                                                                                               нет
9313                                                                                                                               нет
9314                                                   

In [ ]:
Paveletskaya_city_df['UTM-метка Content'][Paveletskaya_city_df['UTM-метка Content'] == ''].count()

0

##### Client ID Яндекс.Метрика

In [ ]:
Paveletskaya_city_df['Client ID Яндекс.Метрика'].head()

0    1690824885484886343
1                       
2    1690287528185718606
3    1690287528185718606
4    1595961781574341891
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
print(Paveletskaya_city_df['Client ID Яндекс.Метрика'].count())
print(Paveletskaya_city_df['Client ID Яндекс.Метрика'].shape[0])

9327
9327


In [ ]:
# Меняем '' на '0' в столбце 'Client ID Яндекс.Метрика'
Paveletskaya_city_df['Client ID Яндекс.Метрика'] = Paveletskaya_city_df['Client ID Яндекс.Метрика'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Client ID Яндекс.Метрика'].head()

0    1690824885484886343
1                      0
2    1690287528185718606
3    1690287528185718606
4    1595961781574341891
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
Paveletskaya_city_df['Client ID Яндекс.Метрика'].tail(20)

9307                       0
9308                       0
9309                       0
9310                       0
9311                       0
9312                       0
9313                       0
9314                       0
9315                       0
9316                       0
9317                       0
9318      167767170677521641
9319     1690805516166145820
9320      167767170677521641
9321     1689673358830550169
9322     1689024387362961387
9323      168884843644744796
9324    15965692761071430182
9325     1685522740892549681
9326     1685522740892549681
Name: Client ID Яндекс.Метрика, dtype: object

In [ ]:
Paveletskaya_city_df['Client ID Яндекс.Метрика'][Paveletskaya_city_df['Client ID Яндекс.Метрика'] == ''].count()

0

##### Первая рекламная кампания

In [ ]:
Paveletskaya_city_df['Первая рекламная кампания'].tail(20)

9307                                                          Посетители без рекламной кампании
9308                                                          Посетители без рекламной кампании
9309                                                          Посетители без рекламной кампании
9310                                                          Посетители без рекламной кампании
9311                                                          Посетители без рекламной кампании
9312                                                          Посетители без рекламной кампании
9313                                                          Посетители без рекламной кампании
9314                                                          Посетители без рекламной кампании
9315                                                          Посетители без рекламной кампании
9316                                                          Посетители без рекламной кампании
9317                                    

In [ ]:
print(Paveletskaya_city_df['Первая рекламная кампания'].count())
print(Paveletskaya_city_df['Первая рекламная кампания'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Первая рекламная кампания'][Paveletskaya_city_df['Первая рекламная кампания'] == ''].count()

0

##### ID посетителя

In [ ]:
Paveletskaya_city_df['ID посетителя'].head()

0    7506916149.0
1             NaN
2    7487205504.0
3    7487205504.0
4    7439533739.0
Name: ID посетителя, dtype: object

In [ ]:
print(Paveletskaya_city_df['ID посетителя'].count())
print(Paveletskaya_city_df['ID посетителя'].shape[0])

7522
9327


In [ ]:
# Избавляемся от пустот в столбце 'ID посетителя'
Paveletskaya_city_df['ID посетителя'] = Paveletskaya_city_df['ID посетителя'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['ID посетителя'].count())
print(Paveletskaya_city_df['ID посетителя'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'ID посетителя'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'ID посетителя': int})

In [ ]:
Paveletskaya_city_df['ID посетителя'].head()

0    7506916149
1             0
2    7487205504
3    7487205504
4    7439533739
Name: ID посетителя, dtype: int64

In [ ]:
Paveletskaya_city_df['ID посетителя'].tail(20)

9307             0
9308             0
9309             0
9310             0
9311             0
9312             0
9313             0
9314             0
9315             0
9316             0
9317             0
9318    7420127004
9319    7505946019
9320    7420127004
9321    7463823464
9322    7440208409
9323    7433870749
9324    7174252369
9325    7168554117
9326    7168554117
Name: ID посетителя, dtype: int64

##### Страна номера абонента

In [ ]:
Paveletskaya_city_df['Страна номера абонента'].tail(20)

9307           РФ
9308           РФ
9309           РФ
9310           РФ
9311           РФ
9312           РФ
9313           РФ
9314           РФ
9315           РФ
9316    Казахстан
9317           РФ
9318           РФ
9319           РФ
9320           РФ
9321           РФ
9322           РФ
9323           РФ
9324           РФ
9325           РФ
9326           РФ
Name: Страна номера абонента, dtype: object

In [ ]:
print(Paveletskaya_city_df['Страна номера абонента'].count())
print(Paveletskaya_city_df['Страна номера абонента'].shape[0])

9324
9327


In [ ]:
# Избавляемся от пустот в столбце 'Страна номера абонента'
Paveletskaya_city_df['Страна номера абонента'] = Paveletskaya_city_df['Страна номера абонента'].fillna('не определено')

In [ ]:
print(Paveletskaya_city_df['Страна номера абонента'].count())
print(Paveletskaya_city_df['Страна номера абонента'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Страна номера абонента'][Paveletskaya_city_df['Страна номера абонента'] == ''].count()

0

##### Регион номера абонента

In [ ]:
Paveletskaya_city_df['Регион номера абонента'].tail(20)

9307                    г. Москва
9308                    г. Москва
9309           Нижегородская обл.
9310                    г. Москва
9311                    г. Москва
9312            Ямало-Ненецкий АО
9313                    г. Москва
9314                    г. Москва
9315                    г. Москва
9316    Казахстан / не определено
9317                    г. Москва
9318           Волгоградская обл.
9319                    г. Москва
9320           Волгоградская обл.
9321                    г. Москва
9322                    г. Москва
9323                 Томская обл.
9324           Республика Чувашия
9325                    г. Москва
9326                    г. Москва
Name: Регион номера абонента, dtype: object

In [ ]:
print(Paveletskaya_city_df['Регион номера абонента'].count())
print(Paveletskaya_city_df['Регион номера абонента'].shape[0])

9324
9327


In [ ]:
# Избавляемся от пустот в столбце 'Регион номера абонента'
Paveletskaya_city_df['Регион номера абонента'] = Paveletskaya_city_df['Регион номера абонента'].fillna('не определено')

In [ ]:
print(Paveletskaya_city_df['Регион номера абонента'].count())
print(Paveletskaya_city_df['Регион номера абонента'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Регион номера абонента'][Paveletskaya_city_df['Регион номера абонента'] == ''].count()

0

##### Идентификатор звонка во внешней системе

In [ ]:
Paveletskaya_city_df['Идентификатор звонка во внешней системе'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Идентификатор звонка во внешней системе, dtype: object

In [ ]:
print(Paveletskaya_city_df['Идентификатор звонка во внешней системе'].count())
print(Paveletskaya_city_df['Идентификатор звонка во внешней системе'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Идентификатор звонка во внешней системе'
Paveletskaya_city_df['Идентификатор звонка во внешней системе'] = Paveletskaya_city_df['Идентификатор звонка во внешней системе'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Идентификатор звонка во внешней системе'].count())
print(Paveletskaya_city_df['Идентификатор звонка во внешней системе'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Идентификатор звонка во внешней системе'][Paveletskaya_city_df['Идентификатор звонка во внешней системе'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Идентификатор звонка во внешней системе'
Paveletskaya_city_df['Идентификатор звонка во внешней системе'] = Paveletskaya_city_df['Идентификатор звонка во внешней системе'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Идентификатор звонка во внешней системе'][Paveletskaya_city_df['Идентификатор звонка во внешней системе'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'Идентификатор звонка во внешней системе'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Идентификатор звонка во внешней системе': int})

In [ ]:
Paveletskaya_city_df['Идентификатор звонка во внешней системе'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Идентификатор звонка во внешней системе, dtype: int64

##### Тип устройства

In [ ]:
Paveletskaya_city_df['Тип устройства'].tail(20)

9307    Прочее
9308    Прочее
9309    Прочее
9310    Прочее
9311    Прочее
9312    Прочее
9313    Прочее
9314    Прочее
9315    Прочее
9316    Прочее
9317    Прочее
9318        ПК
9319        ПК
9320        ПК
9321        ПК
9322        ПК
9323        ПК
9324        ПК
9325        ПК
9326        ПК
Name: Тип устройства, dtype: object

In [ ]:
print(Paveletskaya_city_df['Тип устройства'].count())
print(Paveletskaya_city_df['Тип устройства'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Тип устройства'][Paveletskaya_city_df['Тип устройства'] == ''].count()

0

##### URL страницы обращения

In [ ]:
Paveletskaya_city_df['URL страницы обращения'].tail(20)

9307                                                                                                                                                                                                         NaN
9308                                                                                                                                                                                                         NaN
9309                                                                                                                                                                                                         NaN
9310                                                                                                                                                                                                         NaN
9311                                                                                                                                                                

In [ ]:
print(Paveletskaya_city_df['URL страницы обращения'].count())
print(Paveletskaya_city_df['URL страницы обращения'].shape[0])

4896
9327


In [ ]:
# Избавляемся от пустот в столбце 'URL страницы обращения'
Paveletskaya_city_df['URL страницы обращения'] = Paveletskaya_city_df['URL страницы обращения'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['URL страницы обращения'].count())
print(Paveletskaya_city_df['URL страницы обращения'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['URL страницы обращения'][Paveletskaya_city_df['URL страницы обращения'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['URL страницы обращения'].tail(20)

9307                                                                                                                                                                                                         нет
9308                                                                                                                                                                                                         нет
9309                                                                                                                                                                                                         нет
9310                                                                                                                                                                                                         нет
9311                                                                                                                                                                

##### Город

In [ ]:
Paveletskaya_city_df['Город'].tail(20)

9307                NaN
9308                NaN
9309                NaN
9310                NaN
9311                NaN
9312                NaN
9313                NaN
9314                NaN
9315                NaN
9316                NaN
9317                NaN
9318             Москва
9319             Москва
9320             Москва
9321    Санкт-Петербург
9322               Сочи
9323             Москва
9324             Москва
9325             Москва
9326             Москва
Name: Город, dtype: object

In [ ]:
print(Paveletskaya_city_df['Город'].count())
print(Paveletskaya_city_df['Город'].shape[0])

4668
9327


In [ ]:
# Избавляемся от пустот в столбце 'Город'
Paveletskaya_city_df['Город'] = Paveletskaya_city_df['Город'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Город'].count())
print(Paveletskaya_city_df['Город'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Город'][Paveletskaya_city_df['Город'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Город'].tail(20)

9307                нет
9308                нет
9309                нет
9310                нет
9311                нет
9312                нет
9313                нет
9314                нет
9315                нет
9316                нет
9317                нет
9318             Москва
9319             Москва
9320             Москва
9321    Санкт-Петербург
9322               Сочи
9323             Москва
9324             Москва
9325             Москва
9326             Москва
Name: Город, dtype: object

##### Страна

In [ ]:
Paveletskaya_city_df['Страна'].tail(20)

9307       NaN
9308       NaN
9309       NaN
9310       NaN
9311       NaN
9312       NaN
9313       NaN
9314       NaN
9315       NaN
9316       NaN
9317       NaN
9318    Россия
9319    Россия
9320    Россия
9321    Россия
9322    Россия
9323    Россия
9324    Россия
9325    Россия
9326    Россия
Name: Страна, dtype: object

In [ ]:
print(Paveletskaya_city_df['Страна'].count())
print(Paveletskaya_city_df['Страна'].shape[0])

4730
9327


In [ ]:
# Избавляемся от пустот в столбце 'Страна'
Paveletskaya_city_df['Страна'] = Paveletskaya_city_df['Страна'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Страна'].count())
print(Paveletskaya_city_df['Страна'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Страна'][Paveletskaya_city_df['Страна'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Страна'].tail(20)

9307       нет
9308       нет
9309       нет
9310       нет
9311       нет
9312       нет
9313       нет
9314       нет
9315       нет
9316       нет
9317       нет
9318    Россия
9319    Россия
9320    Россия
9321    Россия
9322    Россия
9323    Россия
9324    Россия
9325    Россия
9326    Россия
Name: Страна, dtype: object

##### Сайт

In [ ]:
Paveletskaya_city_df['Сайт'].tail(20)

9307    pavcity.ru
9308    pavcity.ru
9309    pavcity.ru
9310    pavcity.ru
9311    pavcity.ru
9312    pavcity.ru
9313    pavcity.ru
9314    pavcity.ru
9315    pavcity.ru
9316    pavcity.ru
9317    pavcity.ru
9318    pavcity.ru
9319    pavcity.ru
9320    pavcity.ru
9321    pavcity.ru
9322    pavcity.ru
9323    pavcity.ru
9324    pavcity.ru
9325    pavcity.ru
9326    pavcity.ru
Name: Сайт, dtype: object

In [ ]:
print(Paveletskaya_city_df['Сайт'].count())
print(Paveletskaya_city_df['Сайт'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Сайт'][Paveletskaya_city_df['Сайт'] == ''].count()

0

##### Оценка сотрудника

In [ ]:
Paveletskaya_city_df['Оценка сотрудника'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Оценка сотрудника, dtype: object

In [ ]:
print(Paveletskaya_city_df['Оценка сотрудника'].count())
print(Paveletskaya_city_df['Оценка сотрудника'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Оценка сотрудника'
Paveletskaya_city_df['Оценка сотрудника'] = Paveletskaya_city_df['Оценка сотрудника'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Оценка сотрудника'].count())
print(Paveletskaya_city_df['Оценка сотрудника'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Оценка сотрудника'][Paveletskaya_city_df['Оценка сотрудника'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Оценка сотрудника'
Paveletskaya_city_df['Оценка сотрудника'] = Paveletskaya_city_df['Оценка сотрудника'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Оценка сотрудника'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Оценка сотрудника, dtype: object

In [ ]:
Paveletskaya_city_df['Оценка сотрудника'][Paveletskaya_city_df['Оценка сотрудника'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Оценка сотрудника'][Paveletskaya_city_df['Оценка сотрудника'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['Оценка сотрудника'][Paveletskaya_city_df['Оценка сотрудника'] != 'нет'].count()

0

##### Номер сотрудника

In [ ]:
Paveletskaya_city_df['Номер сотрудника'].head()

0      0155649<1
1    74955141111
2      0155649<1
3      0155649<1
4    74955141111
Name: Номер сотрудника, dtype: object

In [ ]:
print(Paveletskaya_city_df['Номер сотрудника'].count())
print(Paveletskaya_city_df['Номер сотрудника'].shape[0])

9309
9327


In [ ]:
# Избавляемся от пустот в столбце 'Номер сотрудника'
Paveletskaya_city_df['Номер сотрудника'] = Paveletskaya_city_df['Номер сотрудника'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Номер сотрудника'].count())
print(Paveletskaya_city_df['Номер сотрудника'].shape[0])

9327
9327


In [ ]:
# Обрезаем '<1' в столбце 'Номер сотрудника'
Paveletskaya_city_df['Номер сотрудника'] = Paveletskaya_city_df['Номер сотрудника'].apply(lambda x: x.rstrip('<1'))

In [ ]:
# Обрезаем '@trunk.uiscom.ru:9090' в столбце 'Номер сотрудника'
Paveletskaya_city_df['Номер сотрудника'] = Paveletskaya_city_df['Номер сотрудника'].apply(lambda x: x.rstrip('@trunk.uiscom.ru:9090'))

In [ ]:
Paveletskaya_city_df['Номер сотрудника'][Paveletskaya_city_df['Номер сотрудника'] == ''].count()

18

In [ ]:
# Меняем '' на '0' в столбце 'Номер сотрудника'
Paveletskaya_city_df['Номер сотрудника'] = Paveletskaya_city_df['Номер сотрудника'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Номер сотрудника'][Paveletskaya_city_df['Номер сотрудника'] == ''].count()

0

In [ ]:
Paveletskaya_city_calls_match_df['Номер сотрудника'].head()

959    74950216267
961    74955141111
897    74950216267
876    74950216267
982    74950216267
Name: Номер сотрудника, dtype: object

In [ ]:
# Приводим тип данных столбца 'Номер сотрудника'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Номер сотрудника': int})
Paveletskaya_city_calls_match_df = Paveletskaya_city_calls_match_df.astype({'Номер сотрудника': int})

In [ ]:
Paveletskaya_city_df['Номер сотрудника'].head()

0      15564
1    7495514
2      15564
3      15564
4    7495514
Name: Номер сотрудника, dtype: int64

In [ ]:
Paveletskaya_city_calls_match_df['Номер сотрудника'].head()

959    74950216267
961    74955141111
897    74950216267
876    74950216267
982    74950216267
Name: Номер сотрудника, dtype: int64

In [ ]:
Paveletskaya_city_df['Номер сотрудника'].tail(20)

9307    74950216267
9308        7495514
9309    74950216267
9310    74950216267
9311    74950216267
9312    74950216267
9313    74950216267
9314    74950216267
9315        7495514
9316        7495514
9317    74950216267
9318          15564
9319          15564
9320          15564
9321        7495514
9322        7495514
9323        7495514
9324        7495514
9325        7495514
9326        7495514
Name: Номер сотрудника, dtype: int64

In [ ]:
Paveletskaya_city_calls_match_df['Номер сотрудника'].tail(20)

110    74950216267
116    74955141111
105    74950216267
127    74950216267
28     74950216267
23     74955141111
187    74955141111
197    74950216267
14     74955141111
86     74955141111
114    74955141111
176    74955141111
123    74950216267
134    74950216267
178    74950216267
108    74955141111
62     74950216267
35     74950216267
173    74955141111
190    74950216267
Name: Номер сотрудника, dtype: int64

##### Трансфер

In [ ]:
Paveletskaya_city_df['Трансфер'].head()

0    Нет
1    Нет
2    Нет
3    Нет
4    Нет
Name: Трансфер, dtype: object

In [ ]:
print(Paveletskaya_city_df['Трансфер'].count())
print(Paveletskaya_city_df['Трансфер'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Трансфер'].tail(20)

9307    Нет
9308    Нет
9309    Нет
9310    Нет
9311    Нет
9312    Нет
9313    Нет
9314    Нет
9315    Нет
9316    Нет
9317    Нет
9318    Нет
9319    Нет
9320    Нет
9321    Нет
9322    Нет
9323    Нет
9324    Нет
9325    Нет
9326    Нет
Name: Трансфер, dtype: object

In [ ]:
Paveletskaya_city_df['Трансфер'][Paveletskaya_city_df['Трансфер'] == ''].count()

0

##### Внутренний звонок

In [ ]:
Paveletskaya_city_df['Внутренний звонок'].tail(20)

9307    Нет
9308    Нет
9309    Нет
9310    Нет
9311    Нет
9312    Нет
9313    Нет
9314    Нет
9315    Нет
9316    Нет
9317    Нет
9318    Нет
9319    Нет
9320    Нет
9321    Нет
9322    Нет
9323    Нет
9324    Нет
9325    Нет
9326    Нет
Name: Внутренний звонок, dtype: object

In [ ]:
print(Paveletskaya_city_df['Внутренний звонок'].count())
print(Paveletskaya_city_df['Внутренний звонок'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Внутренний звонок'][Paveletskaya_city_df['Внутренний звонок'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Внутренний звонок'][Paveletskaya_city_df['Внутренний звонок'] == 'Нет'].count()

9327

In [ ]:
Paveletskaya_city_df['Внутренний звонок'][Paveletskaya_city_df['Внутренний звонок'] != 'Нет'].count()

0

##### ФИО контакта

In [ ]:
Paveletskaya_city_df['ФИО контакта'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: ФИО контакта, dtype: object

In [ ]:
print(Paveletskaya_city_df['ФИО контакта'].count())
print(Paveletskaya_city_df['ФИО контакта'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'ФИО контакта'
Paveletskaya_city_df['ФИО контакта'] = Paveletskaya_city_df['ФИО контакта'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['ФИО контакта'].count())
print(Paveletskaya_city_df['ФИО контакта'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['ФИО контакта'][Paveletskaya_city_df['ФИО контакта'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'ФИО контакта'
Paveletskaya_city_df['ФИО контакта'] = Paveletskaya_city_df['ФИО контакта'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['ФИО контакта'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: ФИО контакта, dtype: object

In [ ]:
Paveletskaya_city_df['ФИО контакта'][Paveletskaya_city_df['ФИО контакта'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['ФИО контакта'][Paveletskaya_city_df['ФИО контакта'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['ФИО контакта'][Paveletskaya_city_df['ФИО контакта'] != 'нет'].count()

0

##### Имя тренера

In [ ]:
Paveletskaya_city_df['Имя тренера'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Имя тренера, dtype: object

In [ ]:
print(Paveletskaya_city_df['Имя тренера'].count())
print(Paveletskaya_city_df['Имя тренера'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Имя тренера'
Paveletskaya_city_df['Имя тренера'] = Paveletskaya_city_df['Имя тренера'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Имя тренера'].count())
print(Paveletskaya_city_df['Имя тренера'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Имя тренера'][Paveletskaya_city_df['Имя тренера'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Имя тренера'
Paveletskaya_city_df['Имя тренера'] = Paveletskaya_city_df['Имя тренера'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Имя тренера'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Имя тренера, dtype: object

In [ ]:
Paveletskaya_city_df['Имя тренера'][Paveletskaya_city_df['Имя тренера'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Имя тренера'][Paveletskaya_city_df['Имя тренера'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['Имя тренера'][Paveletskaya_city_df['Имя тренера'] != 'нет'].count()

0

##### Сценарий

In [ ]:
Paveletskaya_city_df['Сценарий'].head()

0     ОЗ (SIP<1)
1    74955141111
2     ОЗ (SIP<1)
3     ОЗ (SIP<1)
4    74955141111
Name: Сценарий, dtype: object

In [ ]:
print(Paveletskaya_city_df['Сценарий'].count())
print(Paveletskaya_city_df['Сценарий'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Сценарий'][Paveletskaya_city_df['Сценарий'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Сценарий'].tail(20)

9307        SIP URI
9308    74955141111
9309        SIP URI
9310        SIP URI
9311        SIP URI
9312        SIP URI
9313        SIP URI
9314        SIP URI
9315    74955141111
9316    74955141111
9317        SIP URI
9318     ОЗ (SIP<1)
9319     ОЗ (SIP<1)
9320     ОЗ (SIP<1)
9321    74955141111
9322    74955141111
9323    74955141111
9324    74955141111
9325    74955141111
9326    74955141111
Name: Сценарий, dtype: object

##### Операции

In [ ]:
Paveletskaya_city_df['Операции'].head()

0    Переадресация на сотрудника 1
1                    Переадресация
2    Переадресация на сотрудника 1
3    Переадресация на сотрудника 1
4                    Переадресация
Name: Операции, dtype: object

In [ ]:
print(Paveletskaya_city_df['Операции'].count())
print(Paveletskaya_city_df['Операции'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Операции'][Paveletskaya_city_df['Операции'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Операции'].tail(20)

9307    Переадресация на сотрудника 1
9308                    Переадресация
9309    Переадресация на сотрудника 1
9310    Переадресация на сотрудника 1
9311    Переадресация на сотрудника 1
9312    Переадресация на сотрудника 1
9313    Переадресация на сотрудника 1
9314    Переадресация на сотрудника 1
9315                    Переадресация
9316                    Переадресация
9317    Переадресация на сотрудника 1
9318    Переадресация на сотрудника 1
9319    Переадресация на сотрудника 1
9320    Переадресация на сотрудника 1
9321                    Переадресация
9322                    Переадресация
9323                    Переадресация
9324                    Переадресация
9325                    Переадресация
9326                    Переадресация
Name: Операции, dtype: object

##### Идентификатор сессии звонка

In [ ]:
Paveletskaya_city_df['Идентификатор сессии звонка'].head()

0    3124221769
1    3127090775
2    3124131804
3    3124131189
4    3124119339
Name: Идентификатор сессии звонка, dtype: object

In [ ]:
print(Paveletskaya_city_df['Идентификатор сессии звонка'].count())
print(Paveletskaya_city_df['Идентификатор сессии звонка'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Идентификатор сессии звонка'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Идентификатор сессии звонка': int})

In [ ]:
Paveletskaya_city_df['Идентификатор сессии звонка'].head()

0    3124221769
1    3127090775
2    3124131804
3    3124131189
4    3124119339
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Paveletskaya_city_calls_match_df['Идентификатор сессии звонка'].head()

959    3029352265
961    3029390475
897    3029596520
876    3029638181
982    3029730813
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Paveletskaya_city_df['Идентификатор сессии звонка'].tail(20)

9307    2923626736
9308    2915421683
9309    2915298435
9310    2915887777
9311    2915849569
9312    2915829649
9313    2923295348
9314    2915780417
9315    2914969339
9316    2922837124
9317    2906141158
9318    3123529104
9319    3123446119
9320    3113100339
9321    3101609830
9322    3088450090
9323    3075505209
9324    2979927191
9325    2960180570
9326    2975586733
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
Paveletskaya_city_calls_match_df['Идентификатор сессии звонка'].tail(20)

110    3100336231
116    3100336696
105    3100406017
127    3100448162
28     3100471575
23     3100548655
187    3100725217
197    3101262202
14     3101368620
86     3101609830
114    3101610190
176    3101740720
123    3102211595
134    3102793510
178    3103516800
108    3103703425
62     3103919710
35     3104451495
173    3104889660
190    3105745185
Name: Идентификатор сессии звонка, dtype: int64

##### Группа, выбранная посетителем

In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
print(Paveletskaya_city_df['Группа, выбранная посетителем'].count())
print(Paveletskaya_city_df['Группа, выбранная посетителем'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Группа, выбранная посетителем'
Paveletskaya_city_df['Группа, выбранная посетителем'] = Paveletskaya_city_df['Группа, выбранная посетителем'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Группа, выбранная посетителем'].count())
print(Paveletskaya_city_df['Группа, выбранная посетителем'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'][Paveletskaya_city_df['Группа, выбранная посетителем'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Группа, выбранная посетителем'
Paveletskaya_city_df['Группа, выбранная посетителем'] = Paveletskaya_city_df['Группа, выбранная посетителем'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'][Paveletskaya_city_df['Группа, выбранная посетителем'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'][Paveletskaya_city_df['Группа, выбранная посетителем'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['Группа, выбранная посетителем'][Paveletskaya_city_df['Группа, выбранная посетителем'] != 'нет'].count()

0

##### Длительность обработки

In [ ]:
Paveletskaya_city_df['Длительность обработки'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Длительность обработки, dtype: object

In [ ]:
print(Paveletskaya_city_df['Длительность обработки'].count())
print(Paveletskaya_city_df['Длительность обработки'].shape[0])

56
9327


In [ ]:
# Избавляемся от пустот в столбце 'Длительность обработки'
Paveletskaya_city_df['Длительность обработки'] = Paveletskaya_city_df['Длительность обработки'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Длительность обработки'].count())
print(Paveletskaya_city_df['Длительность обработки'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Длительность обработки'][Paveletskaya_city_df['Длительность обработки'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Длительность обработки'
Paveletskaya_city_df['Длительность обработки'] = Paveletskaya_city_df['Длительность обработки'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Длительность обработки'][Paveletskaya_city_df['Длительность обработки'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Длительность обработки'][Paveletskaya_city_df['Длительность обработки'] != '0']

131      948514.0
227      154906.0
261        5505.0
704           5.0
743         709.0
872         147.0
1147         23.0
1285    1050322.0
1322        303.0
1323        315.0
1324       1368.0
1325       1380.0
1326       1440.0
1327       1456.0
1329       2186.0
1330       2204.0
1332       2738.0
1333       2753.0
1334       2784.0
1335       2830.0
1336       2885.0
1337       2911.0
1338       2924.0
1339       2946.0
1523        153.0
1623       4082.0
1655       6606.0
3679         86.0
4428        361.0
4513        695.0
4619       1046.0
4770     615462.0
4777     615492.0
5118    2324874.0
5121        108.0
5262     696385.0
5416     361999.0
5429       1714.0
6237        597.0
8868     608291.0
8869     608320.0
9000       1655.0
9035        101.0
9138      58069.0
9188       1968.0
9224     613174.0
9299        561.0
Name: Длительность обработки, dtype: object

In [ ]:
# Приводим тип данных столбца 'Длительность обработки'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Длительность обработки': int})

In [ ]:
Paveletskaya_city_df['Длительность обработки'].tail(20)

9307    0
9308    0
9309    0
9310    0
9311    0
9312    0
9313    0
9314    0
9315    0
9316    0
9317    0
9318    0
9319    0
9320    0
9321    0
9322    0
9323    0
9324    0
9325    0
9326    0
Name: Длительность обработки, dtype: int64

##### Комментарий

In [ ]:
Paveletskaya_city_df['Комментарий'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Комментарий, dtype: object

In [ ]:
print(Paveletskaya_city_df['Комментарий'].count())
print(Paveletskaya_city_df['Комментарий'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Комментарий'
Paveletskaya_city_df['Комментарий'] = Paveletskaya_city_df['Комментарий'].fillna('нет комментария')

In [ ]:
print(Paveletskaya_city_df['Комментарий'].count())
print(Paveletskaya_city_df['Комментарий'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Комментарий'].head()

0    нет комментария
1    нет комментария
2    нет комментария
3    нет комментария
4    нет комментария
Name: Комментарий, dtype: object

In [ ]:
Paveletskaya_city_df['Комментарий'].tail(20)

9307    нет комментария
9308    нет комментария
9309    нет комментария
9310    нет комментария
9311    нет комментария
9312    нет комментария
9313    нет комментария
9314    нет комментария
9315    нет комментария
9316    нет комментария
9317    нет комментария
9318                   
9319                   
9320                   
9321                   
9322                   
9323                   
9324                   
9325                   
9326                   
Name: Комментарий, dtype: object

In [ ]:
Paveletskaya_city_df['Комментарий'][Paveletskaya_city_df['Комментарий'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет комментария' в столбце 'Комментарий'
Paveletskaya_city_df['Комментарий'] = Paveletskaya_city_df['Комментарий'].apply(lambda x: 'нет комментария' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Комментарий'][Paveletskaya_city_df['Комментарий'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Комментарий'].tail(20)

9307    нет комментария
9308    нет комментария
9309    нет комментария
9310    нет комментария
9311    нет комментария
9312    нет комментария
9313    нет комментария
9314    нет комментария
9315    нет комментария
9316    нет комментария
9317    нет комментария
9318    нет комментария
9319    нет комментария
9320    нет комментария
9321    нет комментария
9322    нет комментария
9323    нет комментария
9324    нет комментария
9325    нет комментария
9326    нет комментария
Name: Комментарий, dtype: object

##### Контакт из CRM

In [ ]:
Paveletskaya_city_df['Контакт из CRM'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Контакт из CRM, dtype: object

In [ ]:
print(Paveletskaya_city_df['Контакт из CRM'].count())
print(Paveletskaya_city_df['Контакт из CRM'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Контакт из CRM'
Paveletskaya_city_df['Контакт из CRM'] = Paveletskaya_city_df['Контакт из CRM'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Контакт из CRM'].count())
print(Paveletskaya_city_df['Контакт из CRM'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Контакт из CRM'][Paveletskaya_city_df['Контакт из CRM'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Контакт из CRM'
Paveletskaya_city_df['Контакт из CRM'] = Paveletskaya_city_df['Контакт из CRM'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Контакт из CRM'][Paveletskaya_city_df['Контакт из CRM'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Контакт из CRM'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Контакт из CRM, dtype: object

In [ ]:
Paveletskaya_city_df['Контакт из CRM'][Paveletskaya_city_df['Контакт из CRM'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['Контакт из CRM'][Paveletskaya_city_df['Контакт из CRM'] != 'нет'].count()

0

##### Поисковая система

In [ ]:
Paveletskaya_city_df['Поисковая система'].tail(20)

9307       NaN
9308       NaN
9309       NaN
9310       NaN
9311       NaN
9312       NaN
9313       NaN
9314       NaN
9315       NaN
9316       NaN
9317       NaN
9318    yandex
9319          
9320    yandex
9321    yandex
9322    yandex
9323    yandex
9324    yandex
9325    yandex
9326    yandex
Name: Поисковая система, dtype: object

In [ ]:
print(Paveletskaya_city_df['Поисковая система'].count())
print(Paveletskaya_city_df['Поисковая система'].shape[0])

1223
9327


In [ ]:
# Избавляемся от пустот в столбце 'Поисковая система'
Paveletskaya_city_df['Поисковая система'] = Paveletskaya_city_df['Поисковая система'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Поисковая система'].count())
print(Paveletskaya_city_df['Поисковая система'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Поисковая система'][Paveletskaya_city_df['Поисковая система'] == ''].count()

1

In [ ]:
# Меняем '' на 'нет' в столбце 'Поисковая система'
Paveletskaya_city_df['Поисковая система'] = Paveletskaya_city_df['Поисковая система'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Поисковая система'][Paveletskaya_city_df['Поисковая система'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Поисковая система'].tail(20)

9307       нет
9308       нет
9309       нет
9310       нет
9311       нет
9312       нет
9313       нет
9314       нет
9315       нет
9316       нет
9317       нет
9318    yandex
9319       нет
9320    yandex
9321    yandex
9322    yandex
9323    yandex
9324    yandex
9325    yandex
9326    yandex
Name: Поисковая система, dtype: object

##### Поисковый запрос

In [ ]:
Paveletskaya_city_df['Поисковый запрос'].head(20)

0                        жк павелецкая сити 3 очередь
1                                                 NaN
2                       павелецкая сити 3 и 4 очереди
3                       павелецкая сити 3 и 4 очереди
4                                  жк павелецкая сити
5     павелецкая сити цены на квартиры от застройщика
6                         апартаменты в замоскворечье
7                                                 NaN
8                                                 NaN
9                                                 NaN
10                                                NaN
11                                                NaN
12                                                NaN
13                                                NaN
14                                                NaN
15                                                NaN
16                         павелецкая сити веб камера
17                                                NaN
18                          

In [ ]:
print(Paveletskaya_city_df['Поисковый запрос'].count())
print(Paveletskaya_city_df['Поисковый запрос'].shape[0])

4048
9327


In [ ]:
# Избавляемся от пустот в столбце 'Поисковый запрос'
Paveletskaya_city_df['Поисковый запрос'] = Paveletskaya_city_df['Поисковый запрос'].fillna('нет запроса')

In [ ]:
print(Paveletskaya_city_df['Поисковый запрос'].count())
print(Paveletskaya_city_df['Поисковый запрос'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Поисковый запрос'][Paveletskaya_city_df['Поисковый запрос'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Поисковый запрос'].tail(20)

9307                            нет запроса
9308                            нет запроса
9309                            нет запроса
9310                            нет запроса
9311                            нет запроса
9312                            нет запроса
9313                            нет запроса
9314                            нет запроса
9315                            нет запроса
9316                            нет запроса
9317                            нет запроса
9318                     жк павелецкая сити
9319    купить квартиру серпуховская москва
9320        павелецкая сити купить квартиру
9321                     жк павелецкая сити
9322              москва жк павелецкая сити
9323                     жк павелецкая сити
9324                        павелецкий сити
9325              жк павелецкий сити москва
9326              жк павелецкий сити москва
Name: Поисковый запрос, dtype: object

##### Реферер

In [ ]:
Paveletskaya_city_df['Реферер'].tail(20)

9307                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [ ]:
print(Paveletskaya_city_df['Реферер'].count())
print(Paveletskaya_city_df['Реферер'].shape[0])

1429
9327


In [ ]:
# Избавляемся от пустот в столбце 'Реферер'
Paveletskaya_city_df['Реферер'] = Paveletskaya_city_df['Реферер'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Реферер'].count())
print(Paveletskaya_city_df['Реферер'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Реферер'][Paveletskaya_city_df['Реферер'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Реферер'].tail(20)

9307                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

##### Домен реферера

In [ ]:
Paveletskaya_city_df['Домен реферера'].tail(20)

9307          NaN
9308          NaN
9309          NaN
9310          NaN
9311          NaN
9312          NaN
9313          NaN
9314          NaN
9315          NaN
9316          NaN
9317          NaN
9318    yandex.ru
9319    yandex.ru
9320    yandex.ru
9321    yandex.ru
9322    yandex.ru
9323    yandex.ru
9324    yandex.ru
9325    yandex.ru
9326    yandex.ru
Name: Домен реферера, dtype: object

In [ ]:
print(Paveletskaya_city_df['Домен реферера'].count())
print(Paveletskaya_city_df['Домен реферера'].shape[0])

1429
9327


In [ ]:
# Избавляемся от пустот в столбце 'Домен реферера'
Paveletskaya_city_df['Домен реферера'] = Paveletskaya_city_df['Домен реферера'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Домен реферера'].count())
print(Paveletskaya_city_df['Домен реферера'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Домен реферера'][Paveletskaya_city_df['Домен реферера'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Домен реферера'].tail(20)

9307          нет
9308          нет
9309          нет
9310          нет
9311          нет
9312          нет
9313          нет
9314          нет
9315          нет
9316          нет
9317          нет
9318    yandex.ru
9319    yandex.ru
9320    yandex.ru
9321    yandex.ru
9322    yandex.ru
9323    yandex.ru
9324    yandex.ru
9325    yandex.ru
9326    yandex.ru
Name: Домен реферера, dtype: object

##### Область

In [ ]:
Paveletskaya_city_df['Область'].tail(20)

9307                   NaN
9308                   NaN
9309                   NaN
9310                   NaN
9311                   NaN
9312                   NaN
9313                   NaN
9314                   NaN
9315                   NaN
9316                   NaN
9317                   NaN
9318                Москва
9319                Москва
9320                Москва
9321       Санкт-Петербург
9322    Краснодарский край
9323                Москва
9324                Москва
9325                Москва
9326                Москва
Name: Область, dtype: object

In [ ]:
print(Paveletskaya_city_df['Область'].count())
print(Paveletskaya_city_df['Область'].shape[0])

4672
9327


In [ ]:
# Избавляемся от пустот в столбце 'Область'
Paveletskaya_city_df['Область'] = Paveletskaya_city_df['Область'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Область'].count())
print(Paveletskaya_city_df['Область'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Область'][Paveletskaya_city_df['Область'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Область'].tail(20)

9307                   нет
9308                   нет
9309                   нет
9310                   нет
9311                   нет
9312                   нет
9313                   нет
9314                   нет
9315                   нет
9316                   нет
9317                   нет
9318                Москва
9319                Москва
9320                Москва
9321       Санкт-Петербург
9322    Краснодарский край
9323                Москва
9324                Москва
9325                Москва
9326                Москва
Name: Область, dtype: object

##### Посадочная страница

In [ ]:
Paveletskaya_city_df['Посадочная страница'].tail(20)

9307                                                                                                                                                               NaN
9308                                                                                                                                                               NaN
9309                                                                                                                                                               NaN
9310                                                                                                                                                               NaN
9311                                                                                                                                                               NaN
9312                                                                                                                                                               Na

In [ ]:
print(Paveletskaya_city_df['Посадочная страница'].count())
print(Paveletskaya_city_df['Посадочная страница'].shape[0])

4885
9327


In [ ]:
# Избавляемся от пустот в столбце 'Посадочная страница'
Paveletskaya_city_df['Посадочная страница'] = Paveletskaya_city_df['Посадочная страница'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Посадочная страница'].count())
print(Paveletskaya_city_df['Посадочная страница'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Посадочная страница'][Paveletskaya_city_df['Посадочная страница'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Посадочная страница'].tail(20)

9307                                                                                                                                                               нет
9308                                                                                                                                                               нет
9309                                                                                                                                                               нет
9310                                                                                                                                                               нет
9311                                                                                                                                                               нет
9312                                                                                                                                                               не

##### Количество посещений

In [ ]:
Paveletskaya_city_df['Количество посещений'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318      5
9319      1
9320      2
9321      1
9322      1
9323      1
9324      1
9325      1
9326      1
Name: Количество посещений, dtype: object

In [ ]:
print(Paveletskaya_city_df['Количество посещений'].count())
print(Paveletskaya_city_df['Количество посещений'].shape[0])

7497
9327


In [ ]:
# Избавляемся от пустот в столбце 'Количество посещений'
Paveletskaya_city_df['Количество посещений'] = Paveletskaya_city_df['Количество посещений'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Количество посещений'].count())
print(Paveletskaya_city_df['Количество посещений'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Количество посещений'][Paveletskaya_city_df['Количество посещений'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Количество посещений'
Paveletskaya_city_df['Количество посещений'] = Paveletskaya_city_df['Количество посещений'].apply(lambda x: x.strip('{}'))

In [ ]:
# Приводим тип данных столбца 'Количество посещений'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'Количество посещений': int})

In [ ]:
Paveletskaya_city_df['Количество посещений'].tail(20)

9307    0
9308    0
9309    0
9310    0
9311    0
9312    0
9313    0
9314    0
9315    0
9316    0
9317    0
9318    5
9319    1
9320    2
9321    1
9322    1
9323    1
9324    1
9325    1
9326    1
Name: Количество посещений, dtype: int64

##### Тип посетителя

In [ ]:
Paveletskaya_city_df['Тип посетителя'].tail(20)

9307    Не заполнен
9308    Не заполнен
9309    Не заполнен
9310    Не заполнен
9311    Не заполнен
9312    Не заполнен
9313    Не заполнен
9314    Не заполнен
9315    Не заполнен
9316    Не заполнен
9317    Не заполнен
9318    Вернувшийся
9319          Новый
9320    Вернувшийся
9321          Новый
9322          Новый
9323          Новый
9324          Новый
9325          Новый
9326          Новый
Name: Тип посетителя, dtype: object

In [ ]:
print(Paveletskaya_city_df['Тип посетителя'].count())
print(Paveletskaya_city_df['Тип посетителя'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Тип посетителя'][Paveletskaya_city_df['Тип посетителя'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Тип посетителя'
Paveletskaya_city_df['Тип посетителя'] = Paveletskaya_city_df['Тип посетителя'].apply(lambda x: x.strip('{}'))

In [ ]:
Paveletskaya_city_df['Тип посетителя'].tail(20)

9307    Не заполнен
9308    Не заполнен
9309    Не заполнен
9310    Не заполнен
9311    Не заполнен
9312    Не заполнен
9313    Не заполнен
9314    Не заполнен
9315    Не заполнен
9316    Не заполнен
9317    Не заполнен
9318    Вернувшийся
9319          Новый
9320    Вернувшийся
9321          Новый
9322          Новый
9323          Новый
9324          Новый
9325          Новый
9326          Новый
Name: Тип посетителя, dtype: object

##### Client ID Google Analytics

In [ ]:
Paveletskaya_city_df['Client ID Google Analytics'].tail(20)

9307                      NaN
9308                      NaN
9309                      NaN
9310                      NaN
9311                      NaN
9312                      NaN
9313                      NaN
9314                      NaN
9315                      NaN
9316                      NaN
9317                      NaN
9318    1100269332.1688472726
9319     688312054.1690805516
9320    1100269332.1688472726
9321    1948809817.1689673358
9322    2019620928.1689024386
9323    2063306502.1688848437
9324     153357971.1655370258
9325     860602033.1685522741
9326     860602033.1685522741
Name: Client ID Google Analytics, dtype: object

In [ ]:
print(Paveletskaya_city_df['Client ID Google Analytics'].count())
print(Paveletskaya_city_df['Client ID Google Analytics'].shape[0])

2350
9327


In [ ]:
# Избавляемся от пустот в столбце 'Client ID Google Analytics'
Paveletskaya_city_df['Client ID Google Analytics'] = Paveletskaya_city_df['Client ID Google Analytics'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Client ID Google Analytics'].count())
print(Paveletskaya_city_df['Client ID Google Analytics'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Client ID Google Analytics'][Paveletskaya_city_df['Client ID Google Analytics'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Client ID Google Analytics'].tail(20)

9307                        0
9308                        0
9309                        0
9310                        0
9311                        0
9312                        0
9313                        0
9314                        0
9315                        0
9316                        0
9317                        0
9318    1100269332.1688472726
9319     688312054.1690805516
9320    1100269332.1688472726
9321    1948809817.1689673358
9322    2019620928.1689024386
9323    2063306502.1688848437
9324     153357971.1655370258
9325     860602033.1685522741
9326     860602033.1685522741
Name: Client ID Google Analytics, dtype: object

##### ID ресурса Google Analytics

In [ ]:
Paveletskaya_city_df['ID ресурса Google Analytics'].tail(20)

9307              NaN
9308              NaN
9309              NaN
9310              NaN
9311              NaN
9312              NaN
9313              NaN
9314              NaN
9315              NaN
9316              NaN
9317              NaN
9318    UA-43866177-8
9319    UA-43866177-8
9320    UA-43866177-8
9321    UA-43866177-8
9322    UA-43866177-8
9323    UA-43866177-8
9324    UA-43866177-8
9325    UA-43866177-8
9326    UA-43866177-8
Name: ID ресурса Google Analytics, dtype: object

In [ ]:
print(Paveletskaya_city_df['ID ресурса Google Analytics'].count())
print(Paveletskaya_city_df['ID ресурса Google Analytics'].shape[0])

4896
9327


In [ ]:
# Избавляемся от пустот в столбце 'ID ресурса Google Analytics'
Paveletskaya_city_df['ID ресурса Google Analytics'] = Paveletskaya_city_df['ID ресурса Google Analytics'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['ID ресурса Google Analytics'].count())
print(Paveletskaya_city_df['ID ресурса Google Analytics'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['ID ресурса Google Analytics'][Paveletskaya_city_df['ID ресурса Google Analytics'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['ID ресурса Google Analytics'].tail(20)

9307              нет
9308              нет
9309              нет
9310              нет
9311              нет
9312              нет
9313              нет
9314              нет
9315              нет
9316              нет
9317              нет
9318    UA-43866177-8
9319    UA-43866177-8
9320    UA-43866177-8
9321    UA-43866177-8
9322    UA-43866177-8
9323    UA-43866177-8
9324    UA-43866177-8
9325    UA-43866177-8
9326    UA-43866177-8
Name: ID ресурса Google Analytics, dtype: object

##### ID счетчика Яндекс.Метрика

In [ ]:
Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].tail(20)

9307         NaN
9308         NaN
9309         NaN
9310         NaN
9311         NaN
9312         NaN
9313         NaN
9314         NaN
9315         NaN
9316         NaN
9317         NaN
9318    57251161
9319    57251161
9320    57251161
9321    57251161
9322    57251161
9323    57251161
9324    57251161
9325    57251161
9326    57251161
Name: ID счетчика Яндекс.Метрика, dtype: object

In [ ]:
print(Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].count())
print(Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].shape[0])

4896
9327


In [ ]:
# Избавляемся от пустот в столбце 'ID счетчика Яндекс.Метрика'
Paveletskaya_city_df['ID счетчика Яндекс.Метрика'] = Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].count())
print(Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['ID счетчика Яндекс.Метрика'][Paveletskaya_city_df['ID счетчика Яндекс.Метрика'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'ID счетчика Яндекс.Метрика'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'ID счетчика Яндекс.Метрика': int})

In [ ]:
Paveletskaya_city_df['ID счетчика Яндекс.Метрика'].tail(20)

9307           0
9308           0
9309           0
9310           0
9311           0
9312           0
9313           0
9314           0
9315           0
9316           0
9317           0
9318    57251161
9319    57251161
9320    57251161
9321    57251161
9322    57251161
9323    57251161
9324    57251161
9325    57251161
9326    57251161
Name: ID счетчика Яндекс.Метрика, dtype: int64

##### UTM-метка Referrer

In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: UTM-метка Referrer, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Referrer'].count())
print(Paveletskaya_city_df['UTM-метка Referrer'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Referrer'
Paveletskaya_city_df['UTM-метка Referrer'] = Paveletskaya_city_df['UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Referrer'].count())
print(Paveletskaya_city_df['UTM-метка Referrer'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'][Paveletskaya_city_df['UTM-метка Referrer'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Referrer'
Paveletskaya_city_df['UTM-метка Referrer'] = Paveletskaya_city_df['UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'][Paveletskaya_city_df['UTM-метка Referrer'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: UTM-метка Referrer, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'][Paveletskaya_city_df['UTM-метка Referrer'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['UTM-метка Referrer'][Paveletskaya_city_df['UTM-метка Referrer'] != 'нет'].count()

0

##### UTM-метка Expid

In [ ]:
Paveletskaya_city_df['UTM-метка Expid'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: UTM-метка Expid, dtype: object

In [ ]:
print(Paveletskaya_city_df['UTM-метка Expid'].count())
print(Paveletskaya_city_df['UTM-метка Expid'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Expid'
Paveletskaya_city_df['UTM-метка Expid'] = Paveletskaya_city_df['UTM-метка Expid'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['UTM-метка Expid'].count())
print(Paveletskaya_city_df['UTM-метка Expid'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['UTM-метка Expid'][Paveletskaya_city_df['UTM-метка Expid'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'UTM-метка Expid'
Paveletskaya_city_df['UTM-метка Expid'] = Paveletskaya_city_df['UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['UTM-метка Expid'][Paveletskaya_city_df['UTM-метка Expid'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['UTM-метка Expid'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: UTM-метка Expid, dtype: object

In [ ]:
Paveletskaya_city_df['UTM-метка Expid'][Paveletskaya_city_df['UTM-метка Expid'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['UTM-метка Expid'][Paveletskaya_city_df['UTM-метка Expid'] != 'нет'].count()

0

##### AdLense-метка ef_id

In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: AdLense-метка ef_id, dtype: object

In [ ]:
print(Paveletskaya_city_df['AdLense-метка ef_id'].count())
print(Paveletskaya_city_df['AdLense-метка ef_id'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'AdLense-метка ef_id'
Paveletskaya_city_df['AdLense-метка ef_id'] = Paveletskaya_city_df['AdLense-метка ef_id'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['AdLense-метка ef_id'].count())
print(Paveletskaya_city_df['AdLense-метка ef_id'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'][Paveletskaya_city_df['AdLense-метка ef_id'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'AdLense-метка ef_id'
Paveletskaya_city_df['AdLense-метка ef_id'] = Paveletskaya_city_df['AdLense-метка ef_id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'][Paveletskaya_city_df['AdLense-метка ef_id'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'][Paveletskaya_city_df['AdLense-метка ef_id'] == 'нет'].count()

9327

In [ ]:
Paveletskaya_city_df['AdLense-метка ef_id'][Paveletskaya_city_df['AdLense-метка ef_id'] != 'нет'].count()

0

##### Метка yclid

In [ ]:
Paveletskaya_city_df['Метка yclid'].head()

0     4365243841418695380
1                     NaN
2     4364010354768405684
3     4364010354768405684
4    15015208040228192255
Name: Метка yclid, dtype: object

In [ ]:
print(Paveletskaya_city_df['Метка yclid'].count())
print(Paveletskaya_city_df['Метка yclid'].shape[0])

5278
9327


In [ ]:
# Меняем '' на '0' в столбце 'Метка yclid'
Paveletskaya_city_df['Метка yclid'] = Paveletskaya_city_df['Метка yclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Метка yclid'][Paveletskaya_city_df['Метка yclid'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Метка yclid'].tail(20)

9307                      0
9308                      0
9309                      0
9310                      0
9311                      0
9312                      0
9313                      0
9314                      0
9315                      0
9316                      0
9317                      0
9318    4360518919195532071
9319    4360166453783364663
9320    4245844585502284583
9321    4063378035915430152
9322    3893253813977028657
9323    3847129068197977069
9324    3021291462438230095
9325    2975318696343967834
9326    2975318696343967834
Name: Метка yclid, dtype: object

##### Метка ymclid

In [ ]:
Paveletskaya_city_df['Метка ymclid'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Метка ymclid, dtype: object

In [ ]:
print(Paveletskaya_city_df['Метка ymclid'].count())
print(Paveletskaya_city_df['Метка ymclid'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Метка ymclid'
Paveletskaya_city_df['Метка ymclid'] = Paveletskaya_city_df['Метка ymclid'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Метка ymclid'].count())
print(Paveletskaya_city_df['Метка ymclid'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Метка ymclid'][Paveletskaya_city_df['Метка ymclid'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Метка ymclid'
Paveletskaya_city_df['Метка ymclid'] = Paveletskaya_city_df['Метка ymclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Метка ymclid'][Paveletskaya_city_df['Метка ymclid'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Метка ymclid'].head()

0    0
1    0
2    0
3    0
4    0
Name: Метка ymclid, dtype: object

In [ ]:
Paveletskaya_city_df['Метка ymclid'].tail(20)

9307    0
9308    0
9309    0
9310    0
9311    0
9312    0
9313    0
9314    0
9315    0
9316    0
9317    0
9318    0
9319    0
9320    0
9321    0
9322    0
9323    0
9324    0
9325    0
9326    0
Name: Метка ymclid, dtype: object

In [ ]:
Paveletskaya_city_df['Метка ymclid'][Paveletskaya_city_df['Метка ymclid'] == '0'].count()

9327

In [ ]:
Paveletskaya_city_df['Метка ymclid'][Paveletskaya_city_df['Метка ymclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка ymclid'
Paveletskaya_city_df['Метка ymclid'] = Paveletskaya_city_df['Метка ymclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_city_df['Метка ymclid'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Метка ymclid, dtype: object

##### Метка gclid

In [ ]:
Paveletskaya_city_df['Метка gclid'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Метка gclid, dtype: object

In [ ]:
print(Paveletskaya_city_df['Метка gclid'].count())
print(Paveletskaya_city_df['Метка gclid'].shape[0])

11
9327


In [ ]:
# Избавляемся от пустот в столбце 'Метка gclid'
Paveletskaya_city_df['Метка gclid'] = Paveletskaya_city_df['Метка gclid'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Метка gclid'].count())
print(Paveletskaya_city_df['Метка gclid'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Метка gclid'][Paveletskaya_city_df['Метка gclid'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Метка gclid'
Paveletskaya_city_df['Метка gclid'] = Paveletskaya_city_df['Метка gclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Метка gclid'][Paveletskaya_city_df['Метка gclid'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Метка gclid'][Paveletskaya_city_df['Метка gclid'] == '0'].count()

9325

In [ ]:
Paveletskaya_city_df['Метка gclid'][Paveletskaya_city_df['Метка gclid'] != '0'].count()

2

In [ ]:
Paveletskaya_city_df['Метка gclid'][Paveletskaya_city_df['Метка gclid'] != '0']

88      EAIaIQobChMI_qObhvDE6gIVKSB7Ch1ozQpVEAAYASAAEgLgFfD_BwE
9081    EAIaIQobChMI8LaUnsre6gIVBhd7Ch0nlgvpEAAYASAAEgJylfD_BwE
Name: Метка gclid, dtype: object

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка gclid'
Paveletskaya_city_df['Метка gclid'] = Paveletskaya_city_df['Метка gclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_city_df['Метка gclid'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Метка gclid, dtype: object

##### Метка from

In [ ]:
Paveletskaya_city_df['Метка from'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: Метка from, dtype: object

In [ ]:
print(Paveletskaya_city_df['Метка from'].count())
print(Paveletskaya_city_df['Метка from'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Метка from'
Paveletskaya_city_df['Метка from'] = Paveletskaya_city_df['Метка from'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['Метка from'].count())
print(Paveletskaya_city_df['Метка from'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Метка from'][Paveletskaya_city_df['Метка from'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'Метка from'
Paveletskaya_city_df['Метка from'] = Paveletskaya_city_df['Метка from'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Метка from'][Paveletskaya_city_df['Метка from'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Метка from'][Paveletskaya_city_df['Метка from'] == '0'].count()

9327

In [ ]:
Paveletskaya_city_df['Метка from'][Paveletskaya_city_df['Метка from'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка from'
Paveletskaya_city_df['Метка from'] = Paveletskaya_city_df['Метка from'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_city_df['Метка from'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Метка from, dtype: object

##### Длительность разговора

In [ ]:
Paveletskaya_city_df['Длительность разговора'].head()

0    00:00:00
1    00:04:04
2    00:00:00
3    00:00:05
4    00:05:07
Name: Длительность разговора, dtype: object

In [ ]:
print(Paveletskaya_city_df['Длительность разговора'].count())
print(Paveletskaya_city_df['Длительность разговора'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Длительность разговора'
Paveletskaya_city_df['Длительность разговора'] = pd.to_datetime(Paveletskaya_city_df['Длительность разговора']).dt.time

In [ ]:
Paveletskaya_city_df['Длительность разговора'].head()

0    00:00:00
1    00:04:04
2    00:00:00
3    00:00:05
4    00:05:07
Name: Длительность разговора, dtype: object

In [ ]:
Paveletskaya_city_df['Длительность разговора'].tail(20)

9307    00:04:23
9308    00:06:59
9309    00:03:57
9310    00:09:02
9311    00:00:49
9312    00:05:03
9313    00:07:48
9314    00:00:41
9315    00:05:20
9316    00:00:15
9317    00:17:40
9318    00:05:58
9319    00:00:00
9320    00:29:39
9321    00:10:11
9322    00:00:32
9323    00:00:55
9324    00:05:20
9325    00:02:58
9326    00:00:52
Name: Длительность разговора, dtype: object

##### Длительность постобработки

In [ ]:
Paveletskaya_city_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
print(Paveletskaya_city_df['Длительность постобработки'].count())
print(Paveletskaya_city_df['Длительность постобработки'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Длительность постобработки'
Paveletskaya_city_df['Длительность постобработки'] = pd.to_datetime(Paveletskaya_city_df['Длительность постобработки']).dt.time

In [ ]:
Paveletskaya_city_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
Paveletskaya_city_df['Длительность постобработки'].tail(20)

9307    00:00:00
9308    00:00:00
9309    00:00:00
9310    00:00:00
9311    00:00:00
9312    00:00:00
9313    00:00:00
9314    00:00:00
9315    00:00:00
9316    00:00:00
9317    00:00:00
9318    00:00:00
9319    00:00:00
9320    00:00:00
9321    00:00:00
9322    00:00:00
9323    00:00:00
9324    00:00:00
9325    00:00:00
9326    00:00:00
Name: Длительность постобработки, dtype: object

##### Чистая длительность разговора

In [ ]:
Paveletskaya_city_df['Чистая длительность разговора'].head()

0    00:00:00
1    00:04:04
2    00:00:00
3    00:00:05
4    00:05:07
Name: Чистая длительность разговора, dtype: object

In [ ]:
print(Paveletskaya_city_df['Чистая длительность разговора'].count())
print(Paveletskaya_city_df['Чистая длительность разговора'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Чистая длительность разговора'
Paveletskaya_city_df['Чистая длительность разговора'] = pd.to_datetime(Paveletskaya_city_df['Чистая длительность разговора']).dt.time

In [ ]:
Paveletskaya_city_df['Чистая длительность разговора'].head()

0    00:00:00
1    00:04:04
2    00:00:00
3    00:00:05
4    00:05:07
Name: Чистая длительность разговора, dtype: object

In [ ]:
Paveletskaya_city_df['Чистая длительность разговора'].tail(20)

9307    00:04:23
9308    00:06:58
9309    00:03:57
9310    00:09:02
9311    00:00:49
9312    00:05:03
9313    00:07:48
9314    00:00:41
9315    00:05:20
9316    00:00:15
9317    00:17:40
9318    00:05:57
9319    00:00:00
9320    00:29:39
9321    00:10:10
9322    00:00:31
9323    00:00:55
9324    00:05:20
9325    00:02:58
9326    00:00:52
Name: Чистая длительность разговора, dtype: object

##### Полная длительность ожидания

In [ ]:
Paveletskaya_city_df['Полная длительность ожидания'].head()

0    00:00:37
1    00:00:02
2    00:00:36
3    00:00:40
4    00:00:01
Name: Полная длительность ожидания, dtype: object

In [ ]:
print(Paveletskaya_city_df['Полная длительность ожидания'].count())
print(Paveletskaya_city_df['Полная длительность ожидания'].shape[0])

9327
9327


In [ ]:
# Приводим тип данных столбца 'Полная длительность ожидания'
Paveletskaya_city_df['Полная длительность ожидания'] = pd.to_datetime(Paveletskaya_city_df['Полная длительность ожидания']).dt.time

In [ ]:
Paveletskaya_city_df['Полная длительность ожидания'].head()

0    00:00:37
1    00:00:02
2    00:00:36
3    00:00:40
4    00:00:01
Name: Полная длительность ожидания, dtype: object

In [ ]:
Paveletskaya_city_df['Полная длительность ожидания'].tail(20)

9307    00:00:01
9308    00:00:02
9309    00:00:01
9310    00:00:01
9311    00:00:01
9312    00:00:01
9313    00:00:01
9314    00:00:01
9315    00:00:01
9316    00:00:03
9317    00:00:01
9318    00:00:36
9319    00:00:36
9320    00:00:39
9321    00:00:02
9322    00:00:02
9323    00:00:01
9324    00:00:02
9325    00:00:02
9326    00:00:01
Name: Полная длительность ожидания, dtype: object

##### Расширенная UTM-метка Source

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Source'].head()

0    yandex
1       NaN
2    yandex
3    yandex
4    yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Source'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Source'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Source'
Paveletskaya_city_df['Расширенная UTM-метка Source'] = Paveletskaya_city_df['Расширенная UTM-метка Source'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Source'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Source'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Source'][Paveletskaya_city_df['Расширенная UTM-метка Source'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Source'].tail(20)

9307       нет
9308       нет
9309       нет
9310       нет
9311       нет
9312       нет
9313       нет
9314       нет
9315       нет
9316       нет
9317       нет
9318    yandex
9319    yandex
9320    yandex
9321    yandex
9322    yandex
9323    yandex
9324    yandex
9325    yandex
9326    yandex
Name: Расширенная UTM-метка Source, dtype: object

##### Расширенная UTM-метка Campaign

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Campaign'].head()

0       m108_pavcity_brand_net_mmo|88281925
1                                       NaN
2       m108_pavcity_brand_net_mmo|88281925
3       m108_pavcity_brand_net_mmo|88281925
4    m108_pavcity_brand_search_mmo|87584869
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Campaign'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Campaign'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Campaign'
Paveletskaya_city_df['Расширенная UTM-метка Campaign'] = Paveletskaya_city_df['Расширенная UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Campaign'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Campaign'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Campaign'][Paveletskaya_city_df['Расширенная UTM-метка Campaign'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Campaign'].tail(20)

9307                                        нет
9308                                        нет
9309                                        нет
9310                                        нет
9311                                        нет
9312                                        нет
9313                                        нет
9314                                        нет
9315                                        нет
9316                                        нет
9317                                        нет
9318      m108_pavcity_brand_search_rf|88705684
9319     m108_pavcity_loc_alll_net_mmo|88841281
9320    m108_pavelcity_brand_search_rf|88705684
9321                  brand_search_mmo|87584869
9322                   brand_search_rf|88705684
9323                  brand_search_mmo|87584869
9324                  brand_search_mmo|87584869
9325                ya_paveletskaya_msk_s_brand
9326                ya_paveletskaya_msk_s_brand
Name: Расширенная UTM-метка Campaign, dt

##### Расширенная UTM-метка Medium

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Medium'].head()

0    cpc
1    NaN
2    cpc
3    cpc
4    cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Medium'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Medium'].shape[0])

4695
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Medium'
Paveletskaya_city_df['Расширенная UTM-метка Medium'] = Paveletskaya_city_df['Расширенная UTM-метка Medium'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Medium'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Medium'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Medium'][Paveletskaya_city_df['Расширенная UTM-метка Medium'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Medium'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    cpc
9319    cpc
9320    cpc
9321    cpc
9322    cpc
9323    cpc
9324    cpc
9325    cpc
9326    cpc
Name: Расширенная UTM-метка Medium, dtype: object

##### Расширенная UTM-метка Content

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Content'].head()

0    id|44896430327_44896430327|cid|88281925|gid|5203088556|aid|14221067105|adp|no|pos|none0|src|context_fashionpronews.ru|dvc|mobile|coef|0
1                                                                                                                                        NaN
2          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
3          id|44896430393_44896430393|cid|88281925|gid|5203088559|aid|14221067219|adp|no|pos|none0|src|context_9186748.ru|dvc|desktop|coef|0
4               id|44890473306_44890473306|cid|87584869|gid|5202782758|aid|14219372961|adp|no|pos|premium1|src|search_none|dvc|mobile|coef|0
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Content'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Content'].shape[0])

4677
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Content'
Paveletskaya_city_df['Расширенная UTM-метка Content'] = Paveletskaya_city_df['Расширенная UTM-метка Content'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Content'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Content'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Content'][Paveletskaya_city_df['Расширенная UTM-метка Content'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Content'].tail(20)

9307                                                                                                                               нет
9308                                                                                                                               нет
9309                                                                                                                               нет
9310                                                                                                                               нет
9311                                                                                                                               нет
9312                                                                                                                               нет
9313                                                                                                                               нет
9314                                                   

##### Расширенная UTM-метка Term

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Term'].head()

0     жк павелецкая сити 3 очередь
1                              NaN
2    павелецкая сити 3 и 4 очереди
3    павелецкая сити 3 и 4 очереди
4               жк павелецкая сити
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Term'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Term'].shape[0])

2791
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Term'
Paveletskaya_city_df['Расширенная UTM-метка Term'] = Paveletskaya_city_df['Расширенная UTM-метка Term'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Term'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Term'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Term'][Paveletskaya_city_df['Расширенная UTM-метка Term'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Term'].tail(20)

9307                                    нет
9308                                    нет
9309                                    нет
9310                                    нет
9311                                    нет
9312                                    нет
9313                                    нет
9314                                    нет
9315                                    нет
9316                                    нет
9317                                    нет
9318                     жк павелецкая сити
9319    купить квартиру серпуховская москва
9320        павелецкая сити купить квартиру
9321                     жк павелецкая сити
9322              москва жк павелецкая сити
9323                     жк павелецкая сити
9324                        павелецкий сити
9325              жк павелецкий сити москва
9326              жк павелецкий сити москва
Name: Расширенная UTM-метка Term, dtype: object

##### Расширенная UTM-метка Referrer

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Referrer'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Referrer, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Referrer'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Referrer'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Referrer'
Paveletskaya_city_df['Расширенная UTM-метка Referrer'] = Paveletskaya_city_df['Расширенная UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Referrer'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Referrer'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Referrer'][Paveletskaya_city_df['Расширенная UTM-метка Referrer'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Referrer'
Paveletskaya_city_df['Расширенная UTM-метка Referrer'] = Paveletskaya_city_df['Расширенная UTM-метка Referrer'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Referrer'][Paveletskaya_city_df['Расширенная UTM-метка Referrer'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Referrer'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Расширенная UTM-метка Referrer, dtype: object

##### Расширенная UTM-метка Expid

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Expid'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: Расширенная UTM-метка Expid, dtype: object

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Expid'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Expid'].shape[0])

9
9327


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Expid'
Paveletskaya_city_df['Расширенная UTM-метка Expid'] = Paveletskaya_city_df['Расширенная UTM-метка Expid'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['Расширенная UTM-метка Expid'].count())
print(Paveletskaya_city_df['Расширенная UTM-метка Expid'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Expid'][Paveletskaya_city_df['Расширенная UTM-метка Expid'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'Расширенная UTM-метка Expid'
Paveletskaya_city_df['Расширенная UTM-метка Expid'] = Paveletskaya_city_df['Расширенная UTM-метка Expid'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Expid'][Paveletskaya_city_df['Расширенная UTM-метка Expid'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['Расширенная UTM-метка Expid'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: Расширенная UTM-метка Expid, dtype: object

##### OS-метка service-name

In [ ]:
Paveletskaya_city_df['OS-метка service-name'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: OS-метка service-name, dtype: object

In [ ]:
print(Paveletskaya_city_df['OS-метка service-name'].count())
print(Paveletskaya_city_df['OS-метка service-name'].shape[0])

975
9327


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка service-name'
Paveletskaya_city_df['OS-метка service-name'] = Paveletskaya_city_df['OS-метка service-name'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['OS-метка service-name'].count())
print(Paveletskaya_city_df['OS-метка service-name'].shape[0])

9327
9327


In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка service-name'
Paveletskaya_city_df['OS-метка service-name'] = Paveletskaya_city_df['OS-метка service-name'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['OS-метка service-name'][Paveletskaya_city_df['OS-метка service-name'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['OS-метка service-name'][Paveletskaya_city_df['OS-метка service-name'] == 'нет'].count()

8361

In [ ]:
Paveletskaya_city_df['OS-метка service-name'][Paveletskaya_city_df['OS-метка service-name'] != 'нет'].count()

966

In [ ]:
Paveletskaya_city_df['OS-метка service-name'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: OS-метка service-name, dtype: object

##### OS-метка campaign-id

In [ ]:
Paveletskaya_city_df['OS-метка campaign-id'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: OS-метка campaign-id, dtype: object

In [ ]:
print(Paveletskaya_city_df['OS-метка campaign-id'].count())
print(Paveletskaya_city_df['OS-метка campaign-id'].shape[0])

975
9327


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка campaign-id'
Paveletskaya_city_df['OS-метка campaign-id'] = Paveletskaya_city_df['OS-метка campaign-id'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['OS-метка campaign-id'].count())
print(Paveletskaya_city_df['OS-метка campaign-id'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['OS-метка campaign-id'][Paveletskaya_city_df['OS-метка campaign-id'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'OS-метка campaign-id'
Paveletskaya_city_df['OS-метка campaign-id'] = Paveletskaya_city_df['OS-метка campaign-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['OS-метка campaign-id'][Paveletskaya_city_df['OS-метка campaign-id'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'OS-метка campaign-id'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'OS-метка campaign-id': int})

In [ ]:
Paveletskaya_city_df['OS-метка campaign-id'].tail(20)

9307    0
9308    0
9309    0
9310    0
9311    0
9312    0
9313    0
9314    0
9315    0
9316    0
9317    0
9318    0
9319    0
9320    0
9321    0
9322    0
9323    0
9324    0
9325    0
9326    0
Name: OS-метка campaign-id, dtype: int64

##### OS-метка ad-id

In [ ]:
Paveletskaya_city_df['OS-метка ad-id'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: OS-метка ad-id, dtype: object

In [ ]:
print(Paveletskaya_city_df['OS-метка ad-id'].count())
print(Paveletskaya_city_df['OS-метка ad-id'].shape[0])

975
9327


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка ad-id'
Paveletskaya_city_df['OS-метка ad-id'] = Paveletskaya_city_df['OS-метка ad-id'].fillna('0')

In [ ]:
print(Paveletskaya_city_df['OS-метка ad-id'].count())
print(Paveletskaya_city_df['OS-метка ad-id'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['OS-метка ad-id'][Paveletskaya_city_df['OS-метка ad-id'] == ''].count()

9

In [ ]:
# Меняем '' на '0' в столбце 'OS-метка ad-id'
Paveletskaya_city_df['OS-метка ad-id'] = Paveletskaya_city_df['OS-метка ad-id'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_city_df['OS-метка ad-id'][Paveletskaya_city_df['OS-метка ad-id'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'OS-метка ad-id'
Paveletskaya_city_df = Paveletskaya_city_df.astype({'OS-метка ad-id': int})

In [ ]:
Paveletskaya_city_df['OS-метка ad-id'].tail(20)

9307    0
9308    0
9309    0
9310    0
9311    0
9312    0
9313    0
9314    0
9315    0
9316    0
9317    0
9318    0
9319    0
9320    0
9321    0
9322    0
9323    0
9324    0
9325    0
9326    0
Name: OS-метка ad-id, dtype: int64

##### OS-метка source-id

In [ ]:
Paveletskaya_city_df['OS-метка source-id'].tail(20)

9307    NaN
9308    NaN
9309    NaN
9310    NaN
9311    NaN
9312    NaN
9313    NaN
9314    NaN
9315    NaN
9316    NaN
9317    NaN
9318       
9319       
9320       
9321       
9322       
9323       
9324       
9325       
9326       
Name: OS-метка source-id, dtype: object

In [ ]:
print(Paveletskaya_city_df['OS-метка source-id'].count())
print(Paveletskaya_city_df['OS-метка source-id'].shape[0])

975
9327


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка source-id'
Paveletskaya_city_df['OS-метка source-id'] = Paveletskaya_city_df['OS-метка source-id'].fillna('нет')

In [ ]:
print(Paveletskaya_city_df['OS-метка source-id'].count())
print(Paveletskaya_city_df['OS-метка source-id'].shape[0])

9327
9327


In [ ]:
Paveletskaya_city_df['OS-метка source-id'][Paveletskaya_city_df['OS-метка source-id'] == ''].count()

9

In [ ]:
# Меняем '' на 'нет' в столбце 'OS-метка source-id'
Paveletskaya_city_df['OS-метка source-id'] = Paveletskaya_city_df['OS-метка source-id'].apply(lambda x: 'нет' if x == '' else x)

In [ ]:
Paveletskaya_city_df['OS-метка source-id'][Paveletskaya_city_df['OS-метка source-id'] == ''].count()

0

In [ ]:
Paveletskaya_city_df['OS-метка source-id'][Paveletskaya_city_df['OS-метка source-id'] == 'нет'].count()

8361

In [ ]:
Paveletskaya_city_df['OS-метка source-id'][Paveletskaya_city_df['OS-метка source-id'] != 'нет'].count()

966

In [ ]:
Paveletskaya_city_df['OS-метка source-id'].tail(20)

9307    нет
9308    нет
9309    нет
9310    нет
9311    нет
9312    нет
9313    нет
9314    нет
9315    нет
9316    нет
9317    нет
9318    нет
9319    нет
9320    нет
9321    нет
9322    нет
9323    нет
9324    нет
9325    нет
9326    нет
Name: OS-метка source-id, dtype: object

##### Типы данных

In [ ]:
# Смотрим типы данных
Paveletskaya_city_calls_match_df.dtypes

Дата и время                   datetime64[ns]
Номер абонента                          int64
Номер сотрудника                        int64
Идентификатор сессии звонка             int64
Файл записи звонка                     object
dtype: object

In [ ]:
# Смотрим типы данных
Paveletskaya_city_df.dtypes

Статус                                             object
Тип                                                object
Дата и время                               datetime64[ns]
Номер абонента                                      int64
Длительность ожидания ответа                       object
Сотрудник                                          object
Рекламная кампания                                 object
Номер обращения                                     int64
Длительность звонка                                object
Теги                                               object
ID посетителя                                       int64
UTM-метка Campaign                                 object
Город                                              object
Страна                                             object
Страна номера абонента                             object
UTM-метка Content                                  object
Тип устройства                                     object
Сайт          

##### Добавление столбцов

In [ ]:
# Упорядочиваем столбцы в Paveletskaya_city_df как в Primavera_df
Paveletskaya_city_df = Paveletskaya_city_df[Primavera_df.columns[:-1]]

In [ ]:
# Смотрим несовпадающие столбцы Paveletskaya_city_calls_match_df vs Paveletskaya_city_df
for column in Paveletskaya_city_calls_match_df.columns.tolist():
  if column not in Paveletskaya_city_df.columns.tolist():
    print(f'Paveletskaya_city_calls_match_df[{column}], {Paveletskaya_city_calls_match_df[column].shape[0]} строк, тип данных: {Paveletskaya_city_calls_match_df[column].dtypes}')

Paveletskaya_city_calls_match_df[Файл записи звонка], 992 строк, тип данных: object


In [ ]:
# Сортируем Paveletskaya_city_df по столбцу 'Идентификатор сессии звонка'
Paveletskaya_city_df = Paveletskaya_city_df.sort_values('Идентификатор сессии звонка')

In [ ]:
# Индексируем по порядку
Paveletskaya_city_df.index = range(Paveletskaya_city_df.shape[0])

In [ ]:
# Добавляем относительные пути в столбец 'Файл записи звонка' в Paveletskaya_city_calls_match_df
Paveletskaya_city_calls_match_df['Файл записи звонка'] = Paveletskaya_city_calls_match_df['Файл записи звонка'].apply(lambda x: 'Записи звонков/Записи звонков_павелецкая сити/' + x)
Paveletskaya_city_calls_match_df['Файл записи звонка']

959    Записи звонков/Записи звонков_павелецкая сити/2023-06-17_13-44-17.539114_from_79066743768_to_74950216267_session_3029352265_talk.mp3
961    Записи звонков/Записи звонков_павелецкая сити/2023-06-17_14-08-51.827169_from_74993205445_to_74955141111_session_3029390475_talk.mp3
897    Записи звонков/Записи звонков_павелецкая сити/2023-06-17_16-16-05.571252_from_79680303488_to_74950216267_session_3029596520_talk.mp3
876    Записи звонков/Записи звонков_павелецкая сити/2023-06-17_12-54-28.210462_from_79803286601_to_74950216267_session_3029638181_talk.mp3
982    Записи звонков/Записи звонков_павелецкая сити/2023-06-17_14-21-31.493157_from_79259041087_to_74950216267_session_3029730813_talk.mp3
                                                                       ...                                                                 
108    Записи звонков/Записи звонков_павелецкая сити/2023-07-19_12-17-04.090745_from_79343331086_to_74955141111_session_3103703425_talk.mp3
62     Записи звонко

In [ ]:
# Индексируем по порядку
Paveletskaya_city_calls_match_df.index = range(Paveletskaya_city_calls_match_df.shape[0])

In [ ]:
# Добавляем столбец 'Файл записи звонка' в Paveletskaya_city_df
Paveletskaya_city_df = Paveletskaya_city_df.merge(Paveletskaya_city_calls_match_df[['Идентификатор сессии звонка','Файл записи звонка']], how='outer', on=['Идентификатор сессии звонка'])

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Paveletskaya_city_df['Файл записи звонка'].count())
print(Paveletskaya_city_df['Файл записи звонка'].shape[0])

992
9327


In [ ]:
# Запоняем пустоты в столбце 'Файл записи звонка' значением 'нет файла'
Paveletskaya_city_df['Файл записи звонка'] = Paveletskaya_city_df['Файл записи звонка'].fillna('нет файла')

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Paveletskaya_city_df['Файл записи звонка'].count())
print(Paveletskaya_city_df['Файл записи звонка'].shape[0])

9327
9327


In [ ]:
# Нет файлов
Paveletskaya_city_df['Файл записи звонка'][Paveletskaya_city_df['Файл записи звонка'] == 'нет файла'].count()

8335

In [ ]:
# Есть файлы
Paveletskaya_city_df['Файл записи звонка'][Paveletskaya_city_df['Файл записи звонка'] != 'нет файла'].count()

992

##### Сохранение датафреймов

In [ ]:
# Сохраняем Paveletskaya_city_df в файл
with open(df_path + 'Paveletskaya_city_df.pkl', 'wb') as f:
  pkl.dump(Paveletskaya_city_df, f)

In [ ]:
# Сохраняем Paveletskaya_city_calls_match_df в файл
with open(df_path + 'Paveletskaya_city_calls_match_df.pkl', 'wb') as f:
  pkl.dump(Paveletskaya_city_calls_match_df, f)

#### Paveletskaya

##### Статус

In [ ]:
Paveletskaya_df['Статус'].head()

0    Потерянный
1      Принятый
2      Принятый
3      Принятый
4      Принятый
Name: Статус, dtype: object

In [ ]:
print(Paveletskaya_df['Статус'].count())
print(Paveletskaya_df['Статус'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Статус'].tail(20)

1191    Потерянный
1192      Принятый
1193      Принятый
1194      Принятый
1195      Принятый
1196    Потерянный
1197      Принятый
1198      Принятый
1199      Принятый
1200      Принятый
1201    Потерянный
1202      Принятый
1203      Принятый
1204      Принятый
1205      Принятый
1206      Принятый
1207    Потерянный
1208      Принятый
1209      Принятый
1210      Принятый
Name: Статус, dtype: object

In [ ]:
Paveletskaya_df['Статус'][Paveletskaya_df['Статус'] == ''].count()

0

##### Тип

In [ ]:
Paveletskaya_df['Тип'].head()

0    Динамический коллтрекинг
1    Динамический коллтрекинг
2    Динамический коллтрекинг
3    Динамический коллтрекинг
4    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
print(Paveletskaya_df['Тип'].count())
print(Paveletskaya_df['Тип'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Тип'].tail(20)

1191    Динамический коллтрекинг
1192    Динамический коллтрекинг
1193    Динамический коллтрекинг
1194    Динамический коллтрекинг
1195    Динамический коллтрекинг
1196    Динамический коллтрекинг
1197    Динамический коллтрекинг
1198    Динамический коллтрекинг
1199    Динамический коллтрекинг
1200    Динамический коллтрекинг
1201    Динамический коллтрекинг
1202    Динамический коллтрекинг
1203    Динамический коллтрекинг
1204    Динамический коллтрекинг
1205    Динамический коллтрекинг
1206    Динамический коллтрекинг
1207    Динамический коллтрекинг
1208    Динамический коллтрекинг
1209    Динамический коллтрекинг
1210    Динамический коллтрекинг
Name: Тип, dtype: object

In [ ]:
Paveletskaya_df['Тип'][Paveletskaya_df['Тип'] == ''].count()

0

##### Дата и время

In [ ]:
Paveletskaya_df['Дата и время'].head()

0    2023-07-27 14:53:43
1    2023-07-26 12:57:19
2    2023-07-20 19:14:10
3    2023-07-18 14:54:11
4    2023-07-18 13:51:58
Name: Дата и время, dtype: object

In [ ]:
print(Paveletskaya_df['Дата и время'].count())
print(Paveletskaya_df['Дата и время'].shape[0])

1211
1211


In [ ]:
Paveletskaya_calls_match_df['Дата и время'].head()

965    2023-06-19 11:27:46
819    2023-06-19 20:04:39
770    2023-06-24 15:26:51
630    2023-06-27 15:04:43
342    2023-07-11 18:13:13
Name: Дата и время, dtype: object

In [ ]:
print(Paveletskaya_calls_match_df['Дата и время'].count())
print(Paveletskaya_calls_match_df['Дата и время'].shape[0])

8
8


In [ ]:
# Приводим тип данных столбца 'Дата и время'
Paveletskaya_df['Дата и время'] = pd.to_datetime(Paveletskaya_df['Дата и время'])

In [ ]:
Paveletskaya_calls_match_df['Дата и время']= pd.to_datetime(Paveletskaya_calls_match_df['Дата и время'])

In [ ]:
Paveletskaya_calls_match_df['Дата и время'].head()

965   2023-06-19 11:27:46
819   2023-06-19 20:04:39
770   2023-06-24 15:26:51
630   2023-06-27 15:04:43
342   2023-07-11 18:13:13
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Paveletskaya_df['Дата и время'].head()

0   2023-07-27 14:53:43
1   2023-07-26 12:57:19
2   2023-07-20 19:14:10
3   2023-07-18 14:54:11
4   2023-07-18 13:51:58
Name: Дата и время, dtype: datetime64[ns]

In [ ]:
Paveletskaya_df['Дата и время'].tail(20)

1191   2023-05-09 17:54:56
1192   2023-05-09 17:54:37
1193   2023-05-09 12:46:56
1194   2023-05-09 12:46:18
1195   2023-05-09 08:20:56
1196   2023-05-09 08:00:23
1197   2023-05-08 08:52:34
1198   2023-05-08 07:40:07
1199   2023-05-07 21:11:05
1200   2023-05-06 15:25:36
1201   2023-05-06 15:25:30
1202   2023-05-03 22:30:03
1203   2023-05-03 20:46:13
1204   2023-05-03 16:44:20
1205   2023-05-03 16:43:56
1206   2023-05-03 16:43:18
1207   2023-05-03 06:28:01
1208   2023-05-03 06:27:25
1209   2023-05-02 19:52:09
1210   2023-05-02 02:04:42
Name: Дата и время, dtype: datetime64[ns]

##### Номер абонента

In [ ]:
Paveletskaya_df['Номер абонента'].head()

0     79894365488
1    375295142037
2     79331115364
3     79680573141
4     78005055934
Name: Номер абонента, dtype: int64

In [ ]:
print(Paveletskaya_df['Номер абонента'].count())
print(Paveletskaya_df['Номер абонента'].shape[0])

1211
1211


In [ ]:
Paveletskaya_calls_match_df['Номер абонента'].head()

965    79781935179
819    79274574090
770    79778960807
630    79036175926
342    79274147009
Name: Номер абонента, dtype: object

In [ ]:
print(Paveletskaya_calls_match_df['Номер абонента'].count())
print(Paveletskaya_calls_match_df['Номер абонента'].shape[0])

8
8


In [ ]:
# Приводим тип данных столбца 'Номер абонента'
Paveletskaya_df = Paveletskaya_df.astype({'Номер абонента': int})

In [ ]:
Paveletskaya_calls_match_df = Paveletskaya_calls_match_df.astype({'Номер абонента': int})

In [ ]:
Paveletskaya_calls_match_df['Номер абонента'].head()

965    79781935179
819    79274574090
770    79778960807
630    79036175926
342    79274147009
Name: Номер абонента, dtype: int64

In [ ]:
Paveletskaya_df['Номер абонента'].head()

0     79894365488
1    375295142037
2     79331115364
3     79680573141
4     78005055934
Name: Номер абонента, dtype: int64

In [ ]:
Paveletskaya_df['Номер абонента'].tail(20)

1191      79505554249
1192      79505554249
1193      79389102781
1194      79389102781
1195      79029277703
1196      79036626069
1197    8617840105599
1198      79320608806
1199      79774940464
1200      79855217394
1201      79855217394
1202      79872958996
1203      79168259133
1204      79279382701
1205      79377806862
1206      79373021536
1207      77058938377
1208      77058938377
1209      77779012504
1210      79025771096
Name: Номер абонента, dtype: int64

##### Рекламная кампания

In [ ]:
Paveletskaya_df['Рекламная кампания'].head()

0    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
2    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
3    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
4    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
Name: Рекламная кампания, dtype: object

In [ ]:
print(Paveletskaya_df['Рекламная кампания'].count())
print(Paveletskaya_df['Рекламная кампания'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Рекламная кампания'].tail(20)

1191    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1192    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1193    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1194    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1195    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1196    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1197    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1198    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1199    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1200    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1201    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1202    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1203    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1204    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1205    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1206    Ar

In [ ]:
Paveletskaya_df['Рекламная кампания'][Paveletskaya_df['Рекламная кампания'] == ''].count()

0

##### Длительность звонка

In [ ]:
Paveletskaya_df['Длительность звонка'].head()

0    00:00:01
1    00:00:10
2    00:00:23
3    00:01:00
4    00:00:02
Name: Длительность звонка, dtype: object

In [ ]:
print(Paveletskaya_df['Длительность звонка'].count())
print(Paveletskaya_df['Длительность звонка'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Длительность звонка'
Paveletskaya_df['Длительность звонка'] = pd.to_datetime(Paveletskaya_df['Длительность звонка']).dt.time

In [ ]:
Paveletskaya_df['Длительность звонка'].head()

0    00:00:01
1    00:00:10
2    00:00:23
3    00:01:00
4    00:00:02
Name: Длительность звонка, dtype: object

In [ ]:
Paveletskaya_df['Длительность звонка'].tail(20)

1191    00:00:00
1192    00:00:14
1193    00:00:07
1194    00:00:08
1195    00:00:45
1196    00:00:01
1197    00:00:05
1198    00:00:06
1199    00:00:07
1200    00:00:07
1201    00:00:01
1202    00:00:03
1203    00:00:08
1204    00:00:14
1205    00:00:14
1206    00:00:14
1207    00:00:00
1208    00:00:14
1209    00:01:00
1210    00:00:06
Name: Длительность звонка, dtype: object

##### Длительность ожидания ответа

In [ ]:
Paveletskaya_df['Длительность ожидания ответа'].head()

0    00:00:01
1    00:00:03
2    00:00:01
3    00:00:01
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
print(Paveletskaya_df['Длительность ожидания ответа'].count())
print(Paveletskaya_df['Длительность ожидания ответа'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Длительность ожидания ответа'
Paveletskaya_df['Длительность ожидания ответа'] = pd.to_datetime(Paveletskaya_df['Длительность ожидания ответа']).dt.time

In [ ]:
Paveletskaya_df['Длительность ожидания ответа'].head()

0    00:00:01
1    00:00:03
2    00:00:01
3    00:00:01
4    00:00:01
Name: Длительность ожидания ответа, dtype: object

In [ ]:
Paveletskaya_df['Длительность ожидания ответа'].tail(20)

1191    00:00:00
1192    00:00:01
1193    00:00:02
1194    00:00:02
1195    00:00:01
1196    00:00:01
1197    00:00:03
1198    00:00:01
1199    00:00:01
1200    00:00:01
1201    00:00:01
1202    00:00:01
1203    00:00:01
1204    00:00:02
1205    00:00:02
1206    00:00:02
1207    00:00:00
1208    00:00:03
1209    00:00:02
1210    00:00:02
Name: Длительность ожидания ответа, dtype: object

##### Сотрудник

In [ ]:
Paveletskaya_df['Сотрудник'].head()

0    7.495514e+10
1    7.495514e+10
2    7.495514e+10
3    7.495514e+10
4    7.495514e+10
Name: Сотрудник, dtype: float64

In [ ]:
print(Paveletskaya_df['Сотрудник'].count())
print(Paveletskaya_df['Сотрудник'].shape[0])

62
1211


In [ ]:
# Избавляемся от пустот в столбце 'Сотрудник'
Paveletskaya_df['Сотрудник'] = Paveletskaya_df['Сотрудник'].fillna('нет сотрудника')

In [ ]:
print(Paveletskaya_df['Сотрудник'].count())
print(Paveletskaya_df['Сотрудник'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Сотрудник'].tail(20)

1191    нет сотрудника
1192     74955141111.0
1193     74955141111.0
1194     74955141111.0
1195     74955141111.0
1196     74955141111.0
1197     74955141111.0
1198     74955141111.0
1199     74955141111.0
1200     74955141111.0
1201     74955141111.0
1202     74955141111.0
1203     74955141111.0
1204     74955141111.0
1205     74955141111.0
1206     74955141111.0
1207    нет сотрудника
1208     74955141111.0
1209     74955141111.0
1210     74955141111.0
Name: Сотрудник, dtype: object

In [ ]:
Paveletskaya_df['Сотрудник'][Paveletskaya_df['Сотрудник'] == ''].count()

0

##### UTM-метка Campaign

In [ ]:
Paveletskaya_df['UTM-метка Campaign'].head()

0             ya_paveletskaya_kz_n_geo
1             ya_paveletskaya_kz_n_geo
2    ya_paveletskaya_msk_r_retargeting
3             ya_paveletskaya_kz_n_geo
4             ya_paveletskaya_kz_n_geo
Name: UTM-метка Campaign, dtype: object

In [ ]:
print(Paveletskaya_df['UTM-метка Campaign'].count())
print(Paveletskaya_df['UTM-метка Campaign'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Campaign'
Paveletskaya_df['UTM-метка Campaign'] = Paveletskaya_df['UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Campaign'].count())
print(Paveletskaya_df['UTM-метка Campaign'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Campaign'].tail(20)

1191         ya_paveletskaya_rf_n_general
1192         ya_paveletskaya_rf_n_general
1193         ya_paveletskaya_rf_n_general
1194         ya_paveletskaya_rf_n_general
1195         ya_paveletskaya_rf_n_general
1196    ya_paveletskaya_msk_r_retargeting
1197    ya_paveletskaya_msk_r_retargeting
1198         ya_paveletskaya_rf_n_general
1199    ya_paveletskaya_msk_r_retargeting
1200    ya_paveletskaya_msk_r_retargeting
1201    ya_paveletskaya_msk_r_retargeting
1202         ya_paveletskaya_rf_n_general
1203    ya_paveletskaya_msk_r_retargeting
1204     ya_paveletskaya_msk_n_konkurenty
1205             ya_paveletskaya_kz_n_geo
1206    ya_paveletskaya_msk_r_retargeting
1207             ya_paveletskaya_kz_n_geo
1208             ya_paveletskaya_kz_n_geo
1209             ya_paveletskaya_kz_n_geo
1210         ya_paveletskaya_rf_n_general
Name: UTM-метка Campaign, dtype: object

In [ ]:
Paveletskaya_df['UTM-метка Campaign'][Paveletskaya_df['UTM-метка Campaign'] == ''].count()

0

##### Теги

In [ ]:
Paveletskaya_df['Теги'].head()

0    Не обработано - Нецелевой контакт
1                    Нецелевой контакт
2                    Нецелевой контакт
3                                  NaN
4                    Нецелевой контакт
Name: Теги, dtype: object

In [ ]:
print(Paveletskaya_df['Теги'].count())
print(Paveletskaya_df['Теги'].shape[0])

1189
1211


In [ ]:
# Избавляемся от пустот в столбце 'Теги'
Paveletskaya_df['Теги'] = Paveletskaya_df['Теги'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Теги'].count())
print(Paveletskaya_df['Теги'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Теги'][Paveletskaya_df['Теги'] == ''].count()

0

In [ ]:
Paveletskaya_df['Теги'].tail(20)

1191    Не обработано - Нецелевой контакт
1192                    Нецелевой контакт
1193                    Нецелевой контакт
1194                    Нецелевой контакт
1195                                  нет
1196    Не обработано - Нецелевой контакт
1197                    Нецелевой контакт
1198                    Нецелевой контакт
1199                    Нецелевой контакт
1200                    Нецелевой контакт
1201       Обработано - Нецелевой контакт
1202                    Нецелевой контакт
1203                    Нецелевой контакт
1204                    Нецелевой контакт
1205                    Нецелевой контакт
1206                    Нецелевой контакт
1207    Не обработано - Нецелевой контакт
1208                    Нецелевой контакт
1209                                  нет
1210                    Нецелевой контакт
Name: Теги, dtype: object

##### Номер обращения

In [ ]:
Paveletskaya_df['Номер обращения'].head()

0    1
1    1
2    1
3    1
4    2
Name: Номер обращения, dtype: int64

In [ ]:
print(Paveletskaya_df['Номер обращения'].count())
print(Paveletskaya_df['Номер обращения'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Номер обращения'
Paveletskaya_df = Paveletskaya_df.astype({'Номер обращения': int})

In [ ]:
Paveletskaya_df['Номер обращения'].tail(20)

1191    2
1192    1
1193    2
1194    1
1195    1
1196    1
1197    1
1198    1
1199    1
1200    2
1201    1
1202    1
1203    1
1204    1
1205    1
1206    1
1207    2
1208    1
1209    1
1210    4
Name: Номер обращения, dtype: int64

##### UTM-метка Term

In [ ]:
Paveletskaya_df['UTM-метка Term'].tail(20)

1191                        ---autotargeting
1192                        ---autotargeting
1193             купить квартиру рядом школа
1194             купить квартиру рядом школа
1195                 купить квартиру паркинг
1196                                     NaN
1197                                     NaN
1198                        ---autotargeting
1199                                     NaN
1200                                     NaN
1201                                     NaN
1202                        ---autotargeting
1203                                     NaN
1204              sydney city жк цена москва
1205    коммерческая недвижимость москва юао
1206                                     NaN
1207                        ---autotargeting
1208                        ---autotargeting
1209                  комплекс москва москва
1210                     жилье бизнес-класса
Name: UTM-метка Term, dtype: object

In [ ]:
print(Paveletskaya_df['UTM-метка Term'].count())
print(Paveletskaya_df['UTM-метка Term'].shape[0])

1200
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Term'
Paveletskaya_df['UTM-метка Term'] = Paveletskaya_df['UTM-метка Term'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Term'].count())
print(Paveletskaya_df['UTM-метка Term'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Term'][Paveletskaya_df['UTM-метка Term'] == ''].count()

0

In [ ]:
Paveletskaya_df['UTM-метка Term'][Paveletskaya_df['UTM-метка Term'] == 'нет'].count()

11

In [ ]:
Paveletskaya_df['UTM-метка Term'][Paveletskaya_df['UTM-метка Term'] != 'нет'].count()

1200

In [ ]:
Paveletskaya_df['UTM-метка Term'].tail(20)

1191                        ---autotargeting
1192                        ---autotargeting
1193             купить квартиру рядом школа
1194             купить квартиру рядом школа
1195                 купить квартиру паркинг
1196                                     нет
1197                                     нет
1198                        ---autotargeting
1199                                     нет
1200                                     нет
1201                                     нет
1202                        ---autotargeting
1203                                     нет
1204              sydney city жк цена москва
1205    коммерческая недвижимость москва юао
1206                                     нет
1207                        ---autotargeting
1208                        ---autotargeting
1209                  комплекс москва москва
1210                     жилье бизнес-класса
Name: UTM-метка Term, dtype: object

##### Причина завершения

In [ ]:
Paveletskaya_df['Причина завершения'].head()

0    Абонент разорвал соединение
1    Абонент разорвал соединение
2    Абонент разорвал соединение
3    Абонент разорвал соединение
4    Абонент разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
print(Paveletskaya_df['Причина завершения'].count())
print(Paveletskaya_df['Причина завершения'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Причина завершения'].tail(20)

1191      Абонент разорвал соединение
1192      Абонент разорвал соединение
1193      Абонент разорвал соединение
1194      Абонент разорвал соединение
1195      Абонент разорвал соединение
1196      Абонент разорвал соединение
1197      Абонент разорвал соединение
1198      Абонент разорвал соединение
1199      Абонент разорвал соединение
1200      Абонент разорвал соединение
1201      Абонент разорвал соединение
1202      Абонент разорвал соединение
1203      Абонент разорвал соединение
1204      Абонент разорвал соединение
1205      Абонент разорвал соединение
1206      Абонент разорвал соединение
1207      Абонент разорвал соединение
1208      Абонент разорвал соединение
1209    Сотрудник разорвал соединение
1210      Абонент разорвал соединение
Name: Причина завершения, dtype: object

In [ ]:
Paveletskaya_df['Причина завершения'][Paveletskaya_df['Причина завершения'] == ''].count()

0

##### Виртуальный номер

In [ ]:
Paveletskaya_df['Виртуальный номер'].head()

0    74954312102
1    74954313519
2    74954312103
3    74954311950
4    74953240218
Name: Виртуальный номер, dtype: int64

In [ ]:
print(Paveletskaya_df['Виртуальный номер'].count())
print(Paveletskaya_df['Виртуальный номер'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Виртуальный номер'
Paveletskaya_df = Paveletskaya_df.astype({'Виртуальный номер': int})

In [ ]:
Paveletskaya_df['Виртуальный номер'].tail(20)

1191    74954312059
1192    74954312059
1193    74954870140
1194    74954870140
1195    74954313542
1196    74954311958
1197    74954312102
1198    74954314633
1199    74953240218
1200    74954871156
1201    74954871156
1202    74953242915
1203    74954871152
1204    74954871184
1205    74954871152
1206    74954871102
1207    74954312103
1208    74954312103
1209    74950327048
1210    74954313519
Name: Виртуальный номер, dtype: int64

##### UTM-метка Source

In [ ]:
Paveletskaya_df['UTM-метка Source'].head()

0    yandex
1    yandex
2    yandex
3    yandex
4    yandex
Name: UTM-метка Source, dtype: object

In [ ]:
print(Paveletskaya_df['UTM-метка Source'].count())
print(Paveletskaya_df['UTM-метка Source'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Source'
Paveletskaya_df['UTM-метка Source'] = Paveletskaya_df['UTM-метка Source'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Source'].count())
print(Paveletskaya_df['UTM-метка Source'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Source'][Paveletskaya_df['UTM-метка Source'] == ''].count()

0

In [ ]:
Paveletskaya_df['UTM-метка Source'].tail(20)

1191    yandex
1192    yandex
1193    yandex
1194    yandex
1195    yandex
1196    yandex
1197    yandex
1198    yandex
1199    yandex
1200    yandex
1201    yandex
1202    yandex
1203    yandex
1204    yandex
1205    yandex
1206    yandex
1207    yandex
1208    yandex
1209    yandex
1210    yandex
Name: UTM-метка Source, dtype: object

##### UTM-метка Medium

In [ ]:
Paveletskaya_df['UTM-метка Medium'].head()

0    cpc
1    cpc
2    cpc
3    cpc
4    cpc
Name: UTM-метка Medium, dtype: object

In [ ]:
print(Paveletskaya_df['UTM-метка Medium'].count())
print(Paveletskaya_df['UTM-метка Medium'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Medium'
Paveletskaya_df['UTM-метка Medium'] = Paveletskaya_df['UTM-метка Medium'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Medium'].count())
print(Paveletskaya_df['UTM-метка Medium'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Medium'][Paveletskaya_df['UTM-метка Medium'] == ''].count()

0

In [ ]:
Paveletskaya_df['UTM-метка Medium'].tail(20)

1191    cpc
1192    cpc
1193    cpc
1194    cpc
1195    cpc
1196    cpc
1197    cpc
1198    cpc
1199    cpc
1200    cpc
1201    cpc
1202    cpc
1203    cpc
1204    cpc
1205    cpc
1206    cpc
1207    cpc
1208    cpc
1209    cpc
1210    cpc
Name: UTM-метка Medium, dtype: object

##### UTM-метка Content

In [ ]:
Paveletskaya_df['UTM-метка Content'].head()

0    v2||64424254||11012716629||33325751517||жилые комплексы на юге москвы||0||none||com.imo.android.imoim||context||no||re_33325751517||turbo
1                 v2||64424254||13819492116||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo
2                                               v2||50264576||11707219000||42082330||||0||none||ex.24smi.info||context||no||re_42082330||turbo
3                 v2||64424254||13819492123||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo
4             v2||64424254||11012716629||33325751427||купить двушку на юге москвы||0||none||seeziskids.com||context||no||re_33325751427||turbo
Name: UTM-метка Content, dtype: object

In [ ]:
print(Paveletskaya_df['UTM-метка Content'].count())
print(Paveletskaya_df['UTM-метка Content'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Content'
Paveletskaya_df['UTM-метка Content'] = Paveletskaya_df['UTM-метка Content'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Content'].count())
print(Paveletskaya_df['UTM-метка Content'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Content'].tail(20)

1191                             v2||53841693||11012736873||37976139827||---autotargeting||1||none||kinopoisk.ru||context||no||re_37976139827||turbo
1192                             v2||53841693||11012736873||37976139827||---autotargeting||1||none||kinopoisk.ru||context||no||re_37976139827||turbo
1193                v2||53841693||11012736871||44050210577||купить квартиру рядом школа||0||none||game.yandex.ru||context||no||re_44050210577||turbo
1194                v2||53841693||11012736871||44050210577||купить квартиру рядом школа||0||none||game.yandex.ru||context||no||re_44050210577||turbo
1195                    v2||53841693||11012736871||44050210576||купить квартиру паркинг||0||none||game.yandex.ru||context||no||re_44050210576||turbo
1196                                       v2||50264576||11707218990||42082330||||1||none||com.blotclubam.bazarblot||context||no||re_42082330||turbo
1197                                                        v2||50264576||11707218991||42082330||||0||none

In [ ]:
Paveletskaya_df['UTM-метка Content'][Paveletskaya_df['UTM-метка Content'] == ''].count()

0

##### Client ID Яндекс.Метрика

In [ ]:
Paveletskaya_df['Client ID Яндекс.Метрика'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Client ID Яндекс.Метрика, dtype: float64

In [ ]:
print(Paveletskaya_df['Client ID Яндекс.Метрика'].count())
print(Paveletskaya_df['Client ID Яндекс.Метрика'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Client ID Яндекс.Метрика'
Paveletskaya_df['Client ID Яндекс.Метрика'] = Paveletskaya_df['Client ID Яндекс.Метрика'].fillna('0')

In [ ]:
print(Paveletskaya_df['Client ID Яндекс.Метрика'].count())
print(Paveletskaya_df['Client ID Яндекс.Метрика'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Client ID Яндекс.Метрика'][Paveletskaya_df['Client ID Яндекс.Метрика'] == ''].count()

0

In [ ]:
Paveletskaya_df['Client ID Яндекс.Метрика'].tail(20)

1191    0
1192    0
1193    0
1194    0
1195    0
1196    0
1197    0
1198    0
1199    0
1200    0
1201    0
1202    0
1203    0
1204    0
1205    0
1206    0
1207    0
1208    0
1209    0
1210    0
Name: Client ID Яндекс.Метрика, dtype: object

##### Первая рекламная кампания

In [ ]:
Paveletskaya_df['Первая рекламная кампания'].tail(20)

1191    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1192    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1193    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1194    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1195    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1196    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1197    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1198    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1199    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1200    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1201    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1202    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1203    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1204    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1205    Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо)
1206    Ar

In [ ]:
print(Paveletskaya_df['Первая рекламная кампания'].count())
print(Paveletskaya_df['Первая рекламная кампания'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Первая рекламная кампания'][Paveletskaya_df['Первая рекламная кампания'] == ''].count()

0

##### ID посетителя

In [ ]:
Paveletskaya_df['ID посетителя'].head()

0    7.095819e+09
1    7.147024e+09
2    7.470856e+09
3    7.107143e+09
4    7.046435e+09
Name: ID посетителя, dtype: float64

In [ ]:
print(Paveletskaya_df['ID посетителя'].count())
print(Paveletskaya_df['ID посетителя'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'ID посетителя'
Paveletskaya_df['ID посетителя'] = Paveletskaya_df['ID посетителя'].fillna('0')

In [ ]:
print(Paveletskaya_df['ID посетителя'].count())
print(Paveletskaya_df['ID посетителя'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'ID посетителя'
Paveletskaya_df = Paveletskaya_df.astype({'ID посетителя': int})

In [ ]:
Paveletskaya_df['ID посетителя'].tail(20)

1191    7112460877
1192    7112460877
1193    7111847277
1194    7111847277
1195    7111378409
1196    7061663097
1197    7071896021
1198    7109035261
1199    7108381637
1200    7105514749
1201    7105514749
1202    7098740261
1203    7044942105
1204    7097894401
1205    7097847893
1206    7097843989
1207    7096342433
1208    7096342433
1209    7071742277
1210    7089602681
Name: ID посетителя, dtype: int64

##### Страна номера абонента

In [ ]:
Paveletskaya_df['Страна номера абонента'].tail(20)

1191           РФ
1192           РФ
1193           РФ
1194           РФ
1195           РФ
1196           РФ
1197        Китай
1198           РФ
1199           РФ
1200           РФ
1201           РФ
1202           РФ
1203           РФ
1204           РФ
1205           РФ
1206           РФ
1207    Казахстан
1208    Казахстан
1209    Казахстан
1210           РФ
Name: Страна номера абонента, dtype: object

In [ ]:
print(Paveletskaya_df['Страна номера абонента'].count())
print(Paveletskaya_df['Страна номера абонента'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Страна номера абонента'][Paveletskaya_df['Страна номера абонента'] == ''].count()

0

##### Регион номера абонента

In [ ]:
Paveletskaya_df['Регион номера абонента'].tail(20)

1191            Свердловская обл.
1192            Свердловская обл.
1193             Республика Чечня
1194             Республика Чечня
1195            Красноярский край
1196                    г. Москва
1197        Китай / не определено
1198            Ямало-Ненецкий АО
1199                    г. Москва
1200                    г. Москва
1201                    г. Москва
1202         Республика Татарстан
1203                    г. Москва
1204      Республика Башкортостан
1205      Республика Башкортостан
1206      Республика Башкортостан
1207    Казахстан / не определено
1208    Казахстан / не определено
1209    Казахстан / не определено
1210               Иркутская обл.
Name: Регион номера абонента, dtype: object

In [ ]:
print(Paveletskaya_df['Регион номера абонента'].count())
print(Paveletskaya_df['Регион номера абонента'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Регион номера абонента'][Paveletskaya_df['Регион номера абонента'] == ''].count()

0

##### Идентификатор звонка во внешней системе

In [ ]:
Paveletskaya_df['Идентификатор звонка во внешней системе'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Идентификатор звонка во внешней системе, dtype: float64

In [ ]:
print(Paveletskaya_df['Идентификатор звонка во внешней системе'].count())
print(Paveletskaya_df['Идентификатор звонка во внешней системе'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Идентификатор звонка во внешней системе'
Paveletskaya_df['Идентификатор звонка во внешней системе'] = Paveletskaya_df['Идентификатор звонка во внешней системе'].fillna('0')

In [ ]:
print(Paveletskaya_df['Идентификатор звонка во внешней системе'].count())
print(Paveletskaya_df['Идентификатор звонка во внешней системе'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Идентификатор звонка во внешней системе'][Paveletskaya_df['Идентификатор звонка во внешней системе'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'Идентификатор звонка во внешней системе'
Paveletskaya_df = Paveletskaya_df.astype({'Идентификатор звонка во внешней системе': int})

In [ ]:
Paveletskaya_df['Идентификатор звонка во внешней системе'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: Идентификатор звонка во внешней системе, dtype: int64

##### Тип устройства

In [ ]:
Paveletskaya_df['Тип устройства'].tail(20)

1191    Смартфон
1192    Смартфон
1193    Смартфон
1194    Смартфон
1195    Смартфон
1196    Смартфон
1197    Смартфон
1198    Смартфон
1199    Смартфон
1200    Смартфон
1201    Смартфон
1202    Смартфон
1203    Смартфон
1204    Смартфон
1205    Смартфон
1206    Смартфон
1207    Смартфон
1208    Смартфон
1209    Смартфон
1210    Смартфон
Name: Тип устройства, dtype: object

In [ ]:
print(Paveletskaya_df['Тип устройства'].count())
print(Paveletskaya_df['Тип устройства'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Тип устройства'][Paveletskaya_df['Тип устройства'] == ''].count()

0

##### URL страницы обращения

In [ ]:
Paveletskaya_df['URL страницы обращения'].tail(20)

1191    https://pavcity.turbo.site/page2987565/
1192    https://pavcity.turbo.site/page2987565/
1193    https://pavcity.turbo.site/page2987565/
1194    https://pavcity.turbo.site/page2987565/
1195    https://pavcity.turbo.site/page2987565/
1196    https://pavcity.turbo.site/page2987565/
1197    https://pavcity.turbo.site/page2987565/
1198    https://pavcity.turbo.site/page2987565/
1199    https://pavcity.turbo.site/page2987565/
1200    https://pavcity.turbo.site/page2987565/
1201    https://pavcity.turbo.site/page2987565/
1202    https://pavcity.turbo.site/page2987565/
1203    https://pavcity.turbo.site/page2987565/
1204    https://pavcity.turbo.site/page2987565/
1205    https://pavcity.turbo.site/page2987565/
1206    https://pavcity.turbo.site/page2987565/
1207    https://pavcity.turbo.site/page2987565/
1208    https://pavcity.turbo.site/page2987565/
1209    https://pavcity.turbo.site/page2987565/
1210    https://pavcity.turbo.site/page2987565/
Name: URL страницы обращения, dtype: obj

In [ ]:
print(Paveletskaya_df['URL страницы обращения'].count())
print(Paveletskaya_df['URL страницы обращения'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'URL страницы обращения'
Paveletskaya_df['URL страницы обращения'] = Paveletskaya_df['URL страницы обращения'].fillna('нет')

In [ ]:
print(Paveletskaya_df['URL страницы обращения'].count())
print(Paveletskaya_df['URL страницы обращения'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['URL страницы обращения'][Paveletskaya_df['URL страницы обращения'] == ''].count()

0

In [ ]:
Paveletskaya_df['URL страницы обращения'].head(20)

0     https://pavcity.turbo.site/page2987565/
1     https://pavcity.turbo.site/page2987565/
2     https://pavcity.turbo.site/page2987565/
3     https://pavcity.turbo.site/page2987565/
4     https://pavcity.turbo.site/page2987565/
5                                         нет
6     https://pavcity.turbo.site/page2987565/
7      https://pavcity.turbo.site/page2269112
8                                         нет
9     https://pavcity.turbo.site/page2987565/
10    https://pavcity.turbo.site/page2987565/
11    https://pavcity.turbo.site/page2987565/
12     https://pavcity.turbo.site/page2269112
13    https://pavcity.turbo.site/page2987565/
14    https://pavcity.turbo.site/page2987565/
15    https://pavcity.turbo.site/page2987565/
16    https://pavcity.turbo.site/page2987565/
17    https://pavcity.turbo.site/page2987565/
18    https://pavcity.turbo.site/page2987565/
19    https://pavcity.turbo.site/page2987565/
Name: URL страницы обращения, dtype: object

##### Город

In [ ]:
Paveletskaya_df['Город'].tail(20)

1191              NaN
1192              NaN
1193          Грозный
1194          Грозный
1195              NaN
1196           Москва
1197           Москва
1198    Новый Уренгой
1199           Москва
1200           Москва
1201           Москва
1202              Уфа
1203           Москва
1204              NaN
1205          Шымкент
1206           Москва
1207           Алматы
1208           Алматы
1209           Астана
1210              NaN
Name: Город, dtype: object

In [ ]:
print(Paveletskaya_df['Город'].count())
print(Paveletskaya_df['Город'].shape[0])

1201
1211


In [ ]:
# Избавляемся от пустот в столбце 'Город'
Paveletskaya_df['Город'] = Paveletskaya_df['Город'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Город'].count())
print(Paveletskaya_df['Город'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Город'][Paveletskaya_df['Город'] == ''].count()

0

In [ ]:
Paveletskaya_df['Город'].tail(20)

1191              нет
1192              нет
1193          Грозный
1194          Грозный
1195              нет
1196           Москва
1197           Москва
1198    Новый Уренгой
1199           Москва
1200           Москва
1201           Москва
1202              Уфа
1203           Москва
1204              нет
1205          Шымкент
1206           Москва
1207           Алматы
1208           Алматы
1209           Астана
1210              нет
Name: Город, dtype: object

##### Страна

In [ ]:
Paveletskaya_df['Страна'].tail(20)

1191       Россия
1192       Россия
1193       Россия
1194       Россия
1195       Россия
1196       Россия
1197       Россия
1198       Россия
1199       Россия
1200       Россия
1201       Россия
1202       Россия
1203       Россия
1204       Россия
1205    Казахстан
1206       Россия
1207    Казахстан
1208    Казахстан
1209    Казахстан
1210       Россия
Name: Страна, dtype: object

In [ ]:
print(Paveletskaya_df['Страна'].count())
print(Paveletskaya_df['Страна'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Страна'
Paveletskaya_df['Страна'] = Paveletskaya_df['Страна'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Страна'].count())
print(Paveletskaya_df['Страна'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Страна'][Paveletskaya_df['Страна'] == ''].count()

0

In [ ]:
Paveletskaya_df['Страна'].head(20)

0         Польша
1         Турция
2         Россия
3      Казахстан
4      Казахстан
5            нет
6      Казахстан
7      Казахстан
8            нет
9      Казахстан
10    Узбекистан
11     Казахстан
12     Казахстан
13     Казахстан
14     Казахстан
15     Казахстан
16    Узбекистан
17     Казахстан
18     Казахстан
19     Казахстан
Name: Страна, dtype: object

##### Сайт

In [ ]:
Paveletskaya_df['Сайт'].tail(20)

1191    pavcity.turbo.site
1192    pavcity.turbo.site
1193    pavcity.turbo.site
1194    pavcity.turbo.site
1195    pavcity.turbo.site
1196    pavcity.turbo.site
1197    pavcity.turbo.site
1198    pavcity.turbo.site
1199    pavcity.turbo.site
1200    pavcity.turbo.site
1201    pavcity.turbo.site
1202    pavcity.turbo.site
1203    pavcity.turbo.site
1204    pavcity.turbo.site
1205    pavcity.turbo.site
1206    pavcity.turbo.site
1207    pavcity.turbo.site
1208    pavcity.turbo.site
1209    pavcity.turbo.site
1210    pavcity.turbo.site
Name: Сайт, dtype: object

In [ ]:
print(Paveletskaya_df['Сайт'].count())
print(Paveletskaya_df['Сайт'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Сайт'][Paveletskaya_df['Сайт'] == ''].count()

0

##### Оценка сотрудника

In [ ]:
Paveletskaya_df['Оценка сотрудника'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Оценка сотрудника, dtype: float64

In [ ]:
print(Paveletskaya_df['Оценка сотрудника'].count())
print(Paveletskaya_df['Оценка сотрудника'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Оценка сотрудника'
Paveletskaya_df['Оценка сотрудника'] = Paveletskaya_df['Оценка сотрудника'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Оценка сотрудника'].count())
print(Paveletskaya_df['Оценка сотрудника'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Оценка сотрудника'][Paveletskaya_df['Оценка сотрудника'] == ''].count()

0

In [ ]:
Paveletskaya_df['Оценка сотрудника'][Paveletskaya_df['Оценка сотрудника'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['Оценка сотрудника'][Paveletskaya_df['Оценка сотрудника'] != 'нет'].count()

0

In [ ]:
Paveletskaya_df['Оценка сотрудника'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Оценка сотрудника, dtype: object

##### Номер сотрудника

In [ ]:
Paveletskaya_df['Номер сотрудника'].head()

0    74955141111
1    74955141111
2    74955141111
3    74955141111
4    74955141111
Name: Номер сотрудника, dtype: int64

In [ ]:
print(Paveletskaya_df['Номер сотрудника'].count())
print(Paveletskaya_df['Номер сотрудника'].shape[0])

1211
1211


In [ ]:
Paveletskaya_calls_match_df['Номер сотрудника'].head()

965    74955141111
819    74955141111
770    74955141111
630    74955141111
342    74955141111
Name: Номер сотрудника, dtype: object

In [ ]:
print(Paveletskaya_calls_match_df['Номер сотрудника'].count())
print(Paveletskaya_calls_match_df['Номер сотрудника'].shape[0])

8
8


In [ ]:
# Приводим тип данных столбца 'Номер сотрудника'
Paveletskaya_df = Paveletskaya_df.astype({'Номер сотрудника': int})

In [ ]:
Paveletskaya_calls_match_df = Paveletskaya_calls_match_df.astype({'Номер сотрудника': int})

In [ ]:
Paveletskaya_calls_match_df['Номер сотрудника'].head()

965    74955141111
819    74955141111
770    74955141111
630    74955141111
342    74955141111
Name: Номер сотрудника, dtype: int64

In [ ]:
Paveletskaya_df['Номер сотрудника'].tail(20)

1191    74955141111
1192    74955141111
1193    74955141111
1194    74955141111
1195    74955141111
1196    74955141111
1197    74955141111
1198    74955141111
1199    74955141111
1200    74955141111
1201    74955141111
1202    74955141111
1203    74955141111
1204    74955141111
1205    74955141111
1206    74955141111
1207    74955141111
1208    74955141111
1209    74955141111
1210    74955141111
Name: Номер сотрудника, dtype: int64

##### Трансфер

In [ ]:
Paveletskaya_df['Трансфер'].head()

0    Нет
1    Нет
2    Нет
3    Нет
4    Нет
Name: Трансфер, dtype: object

In [ ]:
print(Paveletskaya_df['Трансфер'].count())
print(Paveletskaya_df['Трансфер'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Трансфер'][Paveletskaya_df['Трансфер'] == ''].count()

0

##### Внутренний звонок

In [ ]:
Paveletskaya_df['Внутренний звонок'].tail(20)

1191    Нет
1192    Нет
1193    Нет
1194    Нет
1195    Нет
1196    Нет
1197    Нет
1198    Нет
1199    Нет
1200    Нет
1201    Нет
1202    Нет
1203    Нет
1204    Нет
1205    Нет
1206    Нет
1207    Нет
1208    Нет
1209    Нет
1210    Нет
Name: Внутренний звонок, dtype: object

In [ ]:
print(Paveletskaya_df['Внутренний звонок'].count())
print(Paveletskaya_df['Внутренний звонок'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Внутренний звонок'][Paveletskaya_df['Внутренний звонок'] == ''].count()

0

In [ ]:
Paveletskaya_df['Внутренний звонок'][Paveletskaya_df['Внутренний звонок'] == 'Нет'].count()

1211

In [ ]:
Paveletskaya_df['Внутренний звонок'][Paveletskaya_df['Внутренний звонок'] != 'Нет'].count()

0

##### ФИО контакта

In [ ]:
Paveletskaya_df['ФИО контакта'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: ФИО контакта, dtype: float64

In [ ]:
print(Paveletskaya_df['ФИО контакта'].count())
print(Paveletskaya_df['ФИО контакта'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'ФИО контакта'
Paveletskaya_df['ФИО контакта'] = Paveletskaya_df['ФИО контакта'].fillna('нет')

In [ ]:
print(Paveletskaya_df['ФИО контакта'].count())
print(Paveletskaya_df['ФИО контакта'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['ФИО контакта'][Paveletskaya_df['ФИО контакта'] == ''].count()

0

In [ ]:
Paveletskaya_df['ФИО контакта'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: ФИО контакта, dtype: object

In [ ]:
Paveletskaya_df['ФИО контакта'][Paveletskaya_df['ФИО контакта'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['ФИО контакта'][Paveletskaya_df['ФИО контакта'] != 'нет'].count()

0

##### Имя тренера

In [ ]:
Paveletskaya_df['Имя тренера'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Имя тренера, dtype: float64

In [ ]:
print(Paveletskaya_df['Имя тренера'].count())
print(Paveletskaya_df['Имя тренера'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Имя тренера'
Paveletskaya_df['Имя тренера'] = Paveletskaya_df['Имя тренера'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Имя тренера'].count())
print(Paveletskaya_df['Имя тренера'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Имя тренера'][Paveletskaya_df['Имя тренера'] == ''].count()

0

In [ ]:
Paveletskaya_df['Имя тренера'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Имя тренера, dtype: object

In [ ]:
Paveletskaya_df['Имя тренера'][Paveletskaya_df['Имя тренера'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['Имя тренера'][Paveletskaya_df['Имя тренера'] != 'нет'].count()

0

##### Сценарий

In [ ]:
Paveletskaya_df['Сценарий'].head()

0    74955141111
1    74955141111
2    74955141111
3    74955141111
4    74955141111
Name: Сценарий, dtype: int64

In [ ]:
print(Paveletskaya_df['Сценарий'].count())
print(Paveletskaya_df['Сценарий'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Сценарий'
Paveletskaya_df = Paveletskaya_df.astype({'Сценарий': str})

In [ ]:
Paveletskaya_df['Сценарий'].tail(20)

1191    74955141111
1192    74955141111
1193    74955141111
1194    74955141111
1195    74955141111
1196    74955141111
1197    74955141111
1198    74955141111
1199    74955141111
1200    74955141111
1201    74955141111
1202    74955141111
1203    74955141111
1204    74955141111
1205    74955141111
1206    74955141111
1207    74955141111
1208    74955141111
1209    74955141111
1210    74955141111
Name: Сценарий, dtype: object

In [ ]:
Paveletskaya_df['Сценарий'][Paveletskaya_df['Сценарий'] == ''].count()

0

##### Операции

In [ ]:
Paveletskaya_df['Операции'].head()

0    Переадресация
1    Переадресация
2    Переадресация
3    Переадресация
4    Переадресация
Name: Операции, dtype: object

In [ ]:
print(Paveletskaya_df['Операции'].count())
print(Paveletskaya_df['Операции'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Операции'].tail(20)

1191    Переадресация
1192    Переадресация
1193    Переадресация
1194    Переадресация
1195    Переадресация
1196    Переадресация
1197    Переадресация
1198    Переадресация
1199    Переадресация
1200    Переадресация
1201    Переадресация
1202    Переадресация
1203    Переадресация
1204    Переадресация
1205    Переадресация
1206    Переадресация
1207    Переадресация
1208    Переадресация
1209    Переадресация
1210    Переадресация
Name: Операции, dtype: object

In [ ]:
Paveletskaya_df['Операции'][Paveletskaya_df['Операции'] == ''].count()

0

##### Идентификатор сессии звонка

In [ ]:
Paveletskaya_df['Идентификатор сессии звонка'].head()

0    3119627257
1    3117258020
2    3100917048
3    3098289147
4    3098031427
Name: Идентификатор сессии звонка, dtype: int64

In [ ]:
print(Paveletskaya_df['Идентификатор сессии звонка'].count())
print(Paveletskaya_df['Идентификатор сессии звонка'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Идентификатор сессии звонка'].tail(20)

1191    2929683551
1192    2939731732
1193    2939339696
1194    2919219530
1195    2929069427
1196    2929055699
1197    2927887963
1198    2928398037
1199    2937423628
1200    2926702493
1201    2916300430
1202    2929336168
1203    2920529967
1204    2920656525
1205    2920065451
1206    2920652145
1207    2918275207
1208    2918275147
1209    2918676957
1210    2915798223
Name: Идентификатор сессии звонка, dtype: int64

##### Группа, выбранная посетителем

In [ ]:
Paveletskaya_df['Группа, выбранная посетителем'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Группа, выбранная посетителем, dtype: float64

In [ ]:
print(Paveletskaya_df['Группа, выбранная посетителем'].count())
print(Paveletskaya_df['Группа, выбранная посетителем'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Группа, выбранная посетителем'
Paveletskaya_df['Группа, выбранная посетителем'] = Paveletskaya_df['Группа, выбранная посетителем'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Группа, выбранная посетителем'].count())
print(Paveletskaya_df['Группа, выбранная посетителем'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Группа, выбранная посетителем'][Paveletskaya_df['Группа, выбранная посетителем'] == ''].count()

0

In [ ]:
Paveletskaya_df['Группа, выбранная посетителем'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Группа, выбранная посетителем, dtype: object

In [ ]:
Paveletskaya_df['Группа, выбранная посетителем'][Paveletskaya_df['Группа, выбранная посетителем'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['Группа, выбранная посетителем'][Paveletskaya_df['Группа, выбранная посетителем'] != 'нет'].count()

0

##### Длительность обработки

In [ ]:
Paveletskaya_df['Длительность обработки'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Длительность обработки, dtype: float64

In [ ]:
print(Paveletskaya_df['Длительность обработки'].count())
print(Paveletskaya_df['Длительность обработки'].shape[0])

3
1211


In [ ]:
# Избавляемся от пустот в столбце 'Длительность обработки'
Paveletskaya_df['Длительность обработки'] = Paveletskaya_df['Длительность обработки'].fillna('0')

In [ ]:
print(Paveletskaya_df['Длительность обработки'].count())
print(Paveletskaya_df['Длительность обработки'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Длительность обработки'][Paveletskaya_df['Длительность обработки'] == ''].count()

0

In [ ]:
Paveletskaya_df['Длительность обработки'][Paveletskaya_df['Длительность обработки'] != '0']

603       83869.0
604     1363272.0
1201         14.0
Name: Длительность обработки, dtype: object

In [ ]:
# Приводим тип данных столбца 'Длительность обработки'
Paveletskaya_df = Paveletskaya_df.astype({'Длительность обработки': int})

In [ ]:
Paveletskaya_df['Длительность обработки'].tail(20)

1191     0
1192     0
1193     0
1194     0
1195     0
1196     0
1197     0
1198     0
1199     0
1200     0
1201    14
1202     0
1203     0
1204     0
1205     0
1206     0
1207     0
1208     0
1209     0
1210     0
Name: Длительность обработки, dtype: int64

##### Комментарий

In [ ]:
Paveletskaya_df['Комментарий'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Комментарий, dtype: float64

In [ ]:
print(Paveletskaya_df['Комментарий'].count())
print(Paveletskaya_df['Комментарий'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Комментарий'
Paveletskaya_df['Комментарий'] = Paveletskaya_df['Комментарий'].fillna('нет комментария')

In [ ]:
print(Paveletskaya_df['Комментарий'].count())
print(Paveletskaya_df['Комментарий'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Комментарий'].tail(20)

1191    нет комментария
1192    нет комментария
1193    нет комментария
1194    нет комментария
1195    нет комментария
1196    нет комментария
1197    нет комментария
1198    нет комментария
1199    нет комментария
1200    нет комментария
1201    нет комментария
1202    нет комментария
1203    нет комментария
1204    нет комментария
1205    нет комментария
1206    нет комментария
1207    нет комментария
1208    нет комментария
1209    нет комментария
1210    нет комментария
Name: Комментарий, dtype: object

In [ ]:
Paveletskaya_df['Комментарий'][Paveletskaya_df['Комментарий'] == ''].count()

0

##### Контакт из CRM

In [ ]:
Paveletskaya_df['Контакт из CRM'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Контакт из CRM, dtype: float64

In [ ]:
print(Paveletskaya_df['Контакт из CRM'].count())
print(Paveletskaya_df['Контакт из CRM'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Контакт из CRM'
Paveletskaya_df['Контакт из CRM'] = Paveletskaya_df['Контакт из CRM'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Контакт из CRM'].count())
print(Paveletskaya_df['Контакт из CRM'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Контакт из CRM'][Paveletskaya_df['Контакт из CRM'] == ''].count()

0

In [ ]:
Paveletskaya_df['Контакт из CRM'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Контакт из CRM, dtype: object

In [ ]:
Paveletskaya_df['Контакт из CRM'][Paveletskaya_df['Контакт из CRM'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['Контакт из CRM'][Paveletskaya_df['Контакт из CRM'] != 'нет'].count()

0

##### Поисковая система

In [ ]:
Paveletskaya_df['Поисковая система'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Поисковая система, dtype: float64

In [ ]:
print(Paveletskaya_df['Поисковая система'].count())
print(Paveletskaya_df['Поисковая система'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Поисковая система'
Paveletskaya_df['Поисковая система'] = Paveletskaya_df['Поисковая система'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Поисковая система'].count())
print(Paveletskaya_df['Поисковая система'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Поисковая система'][Paveletskaya_df['Поисковая система'] == ''].count()

0

In [ ]:
Paveletskaya_df['Поисковая система'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Поисковая система, dtype: object

##### Поисковый запрос

In [ ]:
Paveletskaya_df['Поисковый запрос'].head(20)

0                      жилые комплексы на юге москвы
1                                   ---autotargeting
2                                                NaN
3                                   ---autotargeting
4                        купить двушку на юге москвы
5                                                NaN
6                                   ---autotargeting
7                             новостройки юао москва
8                                                NaN
9                                   ---autotargeting
10                                  ---autotargeting
11    продажа жк южный административный округ москвы
12                       жк в юао новостройки москвы
13                                  ---autotargeting
14                                  ---autotargeting
15                                  ---autotargeting
16                                  ---autotargeting
17                                  ---autotargeting
18                                  ---autotar

In [ ]:
print(Paveletskaya_df['Поисковый запрос'].count())
print(Paveletskaya_df['Поисковый запрос'].shape[0])

1200
1211


In [ ]:
# Избавляемся от пустот в столбце 'Поисковый запрос'
Paveletskaya_df['Поисковый запрос'] = Paveletskaya_df['Поисковый запрос'].fillna('нет запроса')

In [ ]:
print(Paveletskaya_df['Поисковый запрос'].count())
print(Paveletskaya_df['Поисковый запрос'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Поисковый запрос'][Paveletskaya_df['Поисковый запрос'] == ''].count()

0

In [ ]:
Paveletskaya_df['Поисковый запрос'].tail(20)

1191                        ---autotargeting
1192                        ---autotargeting
1193             купить квартиру рядом школа
1194             купить квартиру рядом школа
1195                 купить квартиру паркинг
1196                             нет запроса
1197                             нет запроса
1198                        ---autotargeting
1199                             нет запроса
1200                             нет запроса
1201                             нет запроса
1202                        ---autotargeting
1203                             нет запроса
1204              sydney city жк цена москва
1205    коммерческая недвижимость москва юао
1206                             нет запроса
1207                        ---autotargeting
1208                        ---autotargeting
1209                  комплекс москва москва
1210                     жилье бизнес-класса
Name: Поисковый запрос, dtype: object

##### Реферер

In [ ]:
Paveletskaya_df['Реферер'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Реферер, dtype: float64

In [ ]:
print(Paveletskaya_df['Реферер'].count())
print(Paveletskaya_df['Реферер'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Реферер'
Paveletskaya_df['Реферер'] = Paveletskaya_df['Реферер'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Реферер'].count())
print(Paveletskaya_df['Реферер'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Реферер'][Paveletskaya_df['Реферер'] == ''].count()

0

In [ ]:
Paveletskaya_df['Реферер'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Реферер, dtype: object

##### Домен реферера

In [ ]:
Paveletskaya_df['Домен реферера'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Домен реферера, dtype: float64

In [ ]:
print(Paveletskaya_df['Домен реферера'].count())
print(Paveletskaya_df['Домен реферера'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Домен реферера'
Paveletskaya_df['Домен реферера'] = Paveletskaya_df['Домен реферера'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Домен реферера'].count())
print(Paveletskaya_df['Домен реферера'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Домен реферера'][Paveletskaya_df['Домен реферера'] == ''].count()

0

In [ ]:
Paveletskaya_df['Домен реферера'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Домен реферера, dtype: object

##### Область

In [ ]:
Paveletskaya_df['Область'].tail(20)

1191    Челябинская область
1192    Челябинская область
1193                  Чечня
1194                  Чечня
1195                    NaN
1196                 Москва
1197                 Москва
1198      Ямало-Ненецкий АО
1199                 Москва
1200                 Москва
1201                 Москва
1202           Башкортостан
1203                 Москва
1204                    NaN
1205                Шымкент
1206                 Москва
1207                 Алматы
1208                 Алматы
1209           Город Астана
1210                    NaN
Name: Область, dtype: object

In [ ]:
print(Paveletskaya_df['Область'].count())
print(Paveletskaya_df['Область'].shape[0])

1203
1211


In [ ]:
# Избавляемся от пустот в столбце 'Область'
Paveletskaya_df['Область'] = Paveletskaya_df['Область'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Область'].count())
print(Paveletskaya_df['Область'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Область'][Paveletskaya_df['Область'] == ''].count()

0

In [ ]:
Paveletskaya_df['Область'].tail(20)

1191    Челябинская область
1192    Челябинская область
1193                  Чечня
1194                  Чечня
1195                    нет
1196                 Москва
1197                 Москва
1198      Ямало-Ненецкий АО
1199                 Москва
1200                 Москва
1201                 Москва
1202           Башкортостан
1203                 Москва
1204                    нет
1205                Шымкент
1206                 Москва
1207                 Алматы
1208                 Алматы
1209           Город Астана
1210                    нет
Name: Область, dtype: object

##### Посадочная страница

In [ ]:
Paveletskaya_df['Посадочная страница'].tail(20)

1191    https://pavcity.turbo.site/page2987565/
1192    https://pavcity.turbo.site/page2987565/
1193    https://pavcity.turbo.site/page2987565/
1194    https://pavcity.turbo.site/page2987565/
1195    https://pavcity.turbo.site/page2987565/
1196    https://pavcity.turbo.site/page2987565/
1197    https://pavcity.turbo.site/page2987565/
1198    https://pavcity.turbo.site/page2987565/
1199    https://pavcity.turbo.site/page2987565/
1200    https://pavcity.turbo.site/page2987565/
1201    https://pavcity.turbo.site/page2987565/
1202    https://pavcity.turbo.site/page2987565/
1203    https://pavcity.turbo.site/page2987565/
1204    https://pavcity.turbo.site/page2987565/
1205    https://pavcity.turbo.site/page2987565/
1206    https://pavcity.turbo.site/page2987565/
1207    https://pavcity.turbo.site/page2987565/
1208    https://pavcity.turbo.site/page2987565/
1209    https://pavcity.turbo.site/page2987565/
1210    https://pavcity.turbo.site/page2987565/
Name: Посадочная страница, dtype: object

In [ ]:
print(Paveletskaya_df['Посадочная страница'].count())
print(Paveletskaya_df['Посадочная страница'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Посадочная страница'
Paveletskaya_df['Посадочная страница'] = Paveletskaya_df['Посадочная страница'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Посадочная страница'].count())
print(Paveletskaya_df['Посадочная страница'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Посадочная страница'][Paveletskaya_df['Посадочная страница'] == ''].count()

0

In [ ]:
Paveletskaya_df['Посадочная страница'].tail(20)

1191    https://pavcity.turbo.site/page2987565/
1192    https://pavcity.turbo.site/page2987565/
1193    https://pavcity.turbo.site/page2987565/
1194    https://pavcity.turbo.site/page2987565/
1195    https://pavcity.turbo.site/page2987565/
1196    https://pavcity.turbo.site/page2987565/
1197    https://pavcity.turbo.site/page2987565/
1198    https://pavcity.turbo.site/page2987565/
1199    https://pavcity.turbo.site/page2987565/
1200    https://pavcity.turbo.site/page2987565/
1201    https://pavcity.turbo.site/page2987565/
1202    https://pavcity.turbo.site/page2987565/
1203    https://pavcity.turbo.site/page2987565/
1204    https://pavcity.turbo.site/page2987565/
1205    https://pavcity.turbo.site/page2987565/
1206    https://pavcity.turbo.site/page2987565/
1207    https://pavcity.turbo.site/page2987565/
1208    https://pavcity.turbo.site/page2987565/
1209    https://pavcity.turbo.site/page2987565/
1210    https://pavcity.turbo.site/page2987565/
Name: Посадочная страница, dtype: object

##### Количество посещений

In [ ]:
Paveletskaya_df['Количество посещений'].tail(20)

1191    1.0
1192    1.0
1193    1.0
1194    1.0
1195    1.0
1196    2.0
1197    2.0
1198    1.0
1199    1.0
1200    1.0
1201    1.0
1202    1.0
1203    6.0
1204    1.0
1205    1.0
1206    1.0
1207    1.0
1208    1.0
1209    2.0
1210    1.0
Name: Количество посещений, dtype: float64

In [ ]:
print(Paveletskaya_df['Количество посещений'].count())
print(Paveletskaya_df['Количество посещений'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Количество посещений'
Paveletskaya_df['Количество посещений'] = Paveletskaya_df['Количество посещений'].fillna('0')

In [ ]:
print(Paveletskaya_df['Количество посещений'].count())
print(Paveletskaya_df['Количество посещений'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Количество посещений'][Paveletskaya_df['Количество посещений'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'Количество посещений'
Paveletskaya_df = Paveletskaya_df.astype({'Количество посещений': int})

In [ ]:
Paveletskaya_df['Количество посещений'].tail(20)

1191    1
1192    1
1193    1
1194    1
1195    1
1196    2
1197    2
1198    1
1199    1
1200    1
1201    1
1202    1
1203    6
1204    1
1205    1
1206    1
1207    1
1208    1
1209    2
1210    1
Name: Количество посещений, dtype: int64

##### Тип посетителя

In [ ]:
Paveletskaya_df['Тип посетителя'].tail(20)

1191          Новый
1192          Новый
1193          Новый
1194          Новый
1195          Новый
1196    Вернувшийся
1197    Вернувшийся
1198          Новый
1199          Новый
1200          Новый
1201          Новый
1202          Новый
1203    Вернувшийся
1204          Новый
1205          Новый
1206          Новый
1207          Новый
1208          Новый
1209    Вернувшийся
1210          Новый
Name: Тип посетителя, dtype: object

In [ ]:
print(Paveletskaya_df['Тип посетителя'].count())
print(Paveletskaya_df['Тип посетителя'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Тип посетителя'][Paveletskaya_df['Тип посетителя'] == ''].count()

0

In [ ]:
# Убираем '{}' в столбце 'Тип посетителя'
Paveletskaya_df['Тип посетителя'] = Paveletskaya_df['Тип посетителя'].apply(lambda x: x.strip('{}'))

In [ ]:
Paveletskaya_df['Тип посетителя'].tail(20)

1191          Новый
1192          Новый
1193          Новый
1194          Новый
1195          Новый
1196    Вернувшийся
1197    Вернувшийся
1198          Новый
1199          Новый
1200          Новый
1201          Новый
1202          Новый
1203    Вернувшийся
1204          Новый
1205          Новый
1206          Новый
1207          Новый
1208          Новый
1209    Вернувшийся
1210          Новый
Name: Тип посетителя, dtype: object

##### Client ID Google Analytics

In [ ]:
Paveletskaya_df['Client ID Google Analytics'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Client ID Google Analytics, dtype: float64

In [ ]:
print(Paveletskaya_df['Client ID Google Analytics'].count())
print(Paveletskaya_df['Client ID Google Analytics'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Client ID Google Analytics'
Paveletskaya_df['Client ID Google Analytics'] = Paveletskaya_df['Client ID Google Analytics'].fillna('0')

In [ ]:
print(Paveletskaya_df['Client ID Google Analytics'].count())
print(Paveletskaya_df['Client ID Google Analytics'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Client ID Google Analytics'][Paveletskaya_df['Client ID Google Analytics'] == ''].count()

0

In [ ]:
Paveletskaya_df['Client ID Google Analytics'].tail(20)

1191    0
1192    0
1193    0
1194    0
1195    0
1196    0
1197    0
1198    0
1199    0
1200    0
1201    0
1202    0
1203    0
1204    0
1205    0
1206    0
1207    0
1208    0
1209    0
1210    0
Name: Client ID Google Analytics, dtype: object

##### ID ресурса Google Analytics

In [ ]:
Paveletskaya_df['ID ресурса Google Analytics'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: ID ресурса Google Analytics, dtype: float64

In [ ]:
print(Paveletskaya_df['ID ресурса Google Analytics'].count())
print(Paveletskaya_df['ID ресурса Google Analytics'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'ID ресурса Google Analytics'
Paveletskaya_df['ID ресурса Google Analytics'] = Paveletskaya_df['ID ресурса Google Analytics'].fillna('нет')

In [ ]:
print(Paveletskaya_df['ID ресурса Google Analytics'].count())
print(Paveletskaya_df['ID ресурса Google Analytics'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['ID ресурса Google Analytics'][Paveletskaya_df['ID ресурса Google Analytics'] == ''].count()

0

In [ ]:
Paveletskaya_df['ID ресурса Google Analytics'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: ID ресурса Google Analytics, dtype: object

##### ID счетчика Яндекс.Метрика

In [ ]:
Paveletskaya_df['ID счетчика Яндекс.Метрика'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: ID счетчика Яндекс.Метрика, dtype: float64

In [ ]:
print(Paveletskaya_df['ID счетчика Яндекс.Метрика'].count())
print(Paveletskaya_df['ID счетчика Яндекс.Метрика'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'ID счетчика Яндекс.Метрика'
Paveletskaya_df['ID счетчика Яндекс.Метрика'] = Paveletskaya_df['ID счетчика Яндекс.Метрика'].fillna('0')

In [ ]:
print(Paveletskaya_df['ID счетчика Яндекс.Метрика'].count())
print(Paveletskaya_df['ID счетчика Яндекс.Метрика'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['ID счетчика Яндекс.Метрика'][Paveletskaya_df['ID счетчика Яндекс.Метрика'] == ''].count()

0

In [ ]:
# Приводим тип данных столбца 'ID счетчика Яндекс.Метрика'
Paveletskaya_df = Paveletskaya_df.astype({'ID счетчика Яндекс.Метрика': int})

In [ ]:
Paveletskaya_df['ID счетчика Яндекс.Метрика'].tail(20)

1191    0
1192    0
1193    0
1194    0
1195    0
1196    0
1197    0
1198    0
1199    0
1200    0
1201    0
1202    0
1203    0
1204    0
1205    0
1206    0
1207    0
1208    0
1209    0
1210    0
Name: ID счетчика Яндекс.Метрика, dtype: int64

##### UTM-метка Referrer

In [ ]:
Paveletskaya_df['UTM-метка Referrer'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: UTM-метка Referrer, dtype: float64

In [ ]:
print(Paveletskaya_df['UTM-метка Referrer'].count())
print(Paveletskaya_df['UTM-метка Referrer'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Referrer'
Paveletskaya_df['UTM-метка Referrer'] = Paveletskaya_df['UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Referrer'].count())
print(Paveletskaya_df['UTM-метка Referrer'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Referrer'][Paveletskaya_df['UTM-метка Referrer'] == ''].count()

0

In [ ]:
Paveletskaya_df['UTM-метка Referrer'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: UTM-метка Referrer, dtype: object

In [ ]:
Paveletskaya_df['UTM-метка Referrer'][Paveletskaya_df['UTM-метка Referrer'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['UTM-метка Referrer'][Paveletskaya_df['UTM-метка Referrer'] != 'нет'].count()

0

##### UTM-метка Expid

In [ ]:
Paveletskaya_df['UTM-метка Expid'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: UTM-метка Expid, dtype: float64

In [ ]:
print(Paveletskaya_df['UTM-метка Expid'].count())
print(Paveletskaya_df['UTM-метка Expid'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'UTM-метка Expid'
Paveletskaya_df['UTM-метка Expid'] = Paveletskaya_df['UTM-метка Expid'].fillna('нет')

In [ ]:
print(Paveletskaya_df['UTM-метка Expid'].count())
print(Paveletskaya_df['UTM-метка Expid'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['UTM-метка Expid'][Paveletskaya_df['UTM-метка Expid'] == ''].count()

0

In [ ]:
Paveletskaya_df['UTM-метка Expid'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: UTM-метка Expid, dtype: object

In [ ]:
Paveletskaya_df['UTM-метка Expid'][Paveletskaya_df['UTM-метка Expid'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['UTM-метка Expid'][Paveletskaya_df['UTM-метка Expid'] != 'нет'].count()

0

##### AdLense-метка ef_id

In [ ]:
Paveletskaya_df['AdLense-метка ef_id'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: AdLense-метка ef_id, dtype: float64

In [ ]:
print(Paveletskaya_df['AdLense-метка ef_id'].count())
print(Paveletskaya_df['AdLense-метка ef_id'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'AdLense-метка ef_id'
Paveletskaya_df['AdLense-метка ef_id'] = Paveletskaya_df['AdLense-метка ef_id'].fillna('нет')

In [ ]:
print(Paveletskaya_df['AdLense-метка ef_id'].count())
print(Paveletskaya_df['AdLense-метка ef_id'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['AdLense-метка ef_id'][Paveletskaya_df['AdLense-метка ef_id'] == ''].count()

0

In [ ]:
Paveletskaya_df['AdLense-метка ef_id'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: AdLense-метка ef_id, dtype: object

In [ ]:
Paveletskaya_df['AdLense-метка ef_id'][Paveletskaya_df['AdLense-метка ef_id'] == 'нет'].count()

1211

In [ ]:
Paveletskaya_df['AdLense-метка ef_id'][Paveletskaya_df['AdLense-метка ef_id'] != 'нет'].count()

0

##### Метка yclid

In [ ]:
Paveletskaya_df['Метка yclid'].head()

0    2304393096956537234
1    2786043124276584666
2    1071624817303941911
3    2430834524818565452
4    1871823518871913588
Name: Метка yclid, dtype: object

In [ ]:
print(Paveletskaya_df['Метка yclid'].count())
print(Paveletskaya_df['Метка yclid'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Метка yclid'][Paveletskaya_df['Метка yclid'] == ''].count()

3

In [ ]:
# Меняем '' на '0' в столбце 'Метка yclid'
Paveletskaya_df['Метка yclid'] = Paveletskaya_df['Метка yclid'].apply(lambda x: '0' if x == '' else x)

In [ ]:
Paveletskaya_df['Метка yclid'][Paveletskaya_df['Метка yclid'] == ''].count()

0

In [ ]:
Paveletskaya_df['Метка yclid'].tail(20)

1191    2482836476699677730
1192    2482836476699677730
1193    2477985337313137282
1194    2477985337313137282
1195    2473809613675095292
1196    2473478203443045716
1197    2451222710439187700
1198    2450520024821863466
1199    2440617604582282981
1200    2412540979051565902
1201    2412540979051565902
1202    2351269325935286014
1203    2349636746303563022
1204    2345605047298101872
1205    2345351946387658519
1206    2345328960014455981
1207    2335437372109417058
1208    2335437372109417058
1209    2326132470679661353
1210    2273905779376712190
Name: Метка yclid, dtype: object

##### Метка ymclid

In [ ]:
Paveletskaya_df['Метка ymclid'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Метка ymclid, dtype: float64

In [ ]:
print(Paveletskaya_df['Метка ymclid'].count())
print(Paveletskaya_df['Метка ymclid'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Метка ymclid'
Paveletskaya_df['Метка ymclid'] = Paveletskaya_df['Метка ymclid'].fillna('0')

In [ ]:
print(Paveletskaya_df['Метка ymclid'].count())
print(Paveletskaya_df['Метка ymclid'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Метка ymclid'][Paveletskaya_df['Метка ymclid'] == ''].count()

0

In [ ]:
Paveletskaya_df['Метка ymclid'][Paveletskaya_df['Метка ymclid'] == '0'].count()

1211

In [ ]:
Paveletskaya_df['Метка ymclid'][Paveletskaya_df['Метка ymclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка ymclid'
Paveletskaya_df['Метка ymclid'] = Paveletskaya_df['Метка ymclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_df['Метка ymclid'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Метка ymclid, dtype: object

##### Метка gclid

In [ ]:
Paveletskaya_df['Метка gclid'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Метка gclid, dtype: float64

In [ ]:
print(Paveletskaya_df['Метка gclid'].count())
print(Paveletskaya_df['Метка gclid'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Метка gclid'
Paveletskaya_df['Метка gclid'] = Paveletskaya_df['Метка gclid'].fillna('0')

In [ ]:
print(Paveletskaya_df['Метка gclid'].count())
print(Paveletskaya_df['Метка gclid'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Метка gclid'][Paveletskaya_df['Метка gclid'] == ''].count()

0

In [ ]:
Paveletskaya_df['Метка gclid'][Paveletskaya_df['Метка gclid'] == '0'].count()

1211

In [ ]:
Paveletskaya_df['Метка gclid'][Paveletskaya_df['Метка gclid'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка gclid'
Paveletskaya_df['Метка gclid'] = Paveletskaya_df['Метка gclid'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_df['Метка gclid'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Метка gclid, dtype: object

##### Метка from

In [ ]:
Paveletskaya_df['Метка from'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: Метка from, dtype: float64

In [ ]:
print(Paveletskaya_df['Метка from'].count())
print(Paveletskaya_df['Метка from'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Метка from'
Paveletskaya_df['Метка from'] = Paveletskaya_df['Метка from'].fillna('0')

In [ ]:
print(Paveletskaya_df['Метка from'].count())
print(Paveletskaya_df['Метка from'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Метка from'][Paveletskaya_df['Метка from'] == ''].count()

0

In [ ]:
Paveletskaya_df['Метка from'][Paveletskaya_df['Метка from'] == '0'].count()

1211

In [ ]:
Paveletskaya_df['Метка from'][Paveletskaya_df['Метка from'] != '0'].count()

0

In [ ]:
# Меняем '0' на 'нет' в столбце 'Метка from'
Paveletskaya_df['Метка from'] = Paveletskaya_df['Метка from'].apply(lambda x: 'нет' if x == '0' else x)

In [ ]:
Paveletskaya_df['Метка from'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Метка from, dtype: object

##### Длительность разговора

In [ ]:
Paveletskaya_df['Длительность разговора'].head()

0    00:00:00
1    00:00:07
2    00:00:22
3    00:00:59
4    00:00:01
Name: Длительность разговора, dtype: object

In [ ]:
print(Paveletskaya_df['Длительность разговора'].count())
print(Paveletskaya_df['Длительность разговора'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Длительность разговора'
Paveletskaya_df['Длительность разговора'] = pd.to_datetime(Paveletskaya_df['Длительность разговора']).dt.time

In [ ]:
Paveletskaya_df['Длительность разговора'].tail(20)

1191    00:00:00
1192    00:00:13
1193    00:00:05
1194    00:00:06
1195    00:00:44
1196    00:00:00
1197    00:00:02
1198    00:00:05
1199    00:00:06
1200    00:00:06
1201    00:00:00
1202    00:00:02
1203    00:00:07
1204    00:00:12
1205    00:00:12
1206    00:00:12
1207    00:00:00
1208    00:00:11
1209    00:00:58
1210    00:00:04
Name: Длительность разговора, dtype: object

##### Длительность постобработки

In [ ]:
Paveletskaya_df['Длительность постобработки'].head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Длительность постобработки, dtype: object

In [ ]:
print(Paveletskaya_df['Длительность постобработки'].count())
print(Paveletskaya_df['Длительность постобработки'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Длительность постобработки'
Paveletskaya_df['Длительность постобработки'] = pd.to_datetime(Paveletskaya_df['Длительность постобработки']).dt.time

In [ ]:
Paveletskaya_df['Длительность постобработки'].tail(20)

1191    00:00:00
1192    00:00:00
1193    00:00:00
1194    00:00:00
1195    00:00:00
1196    00:00:00
1197    00:00:00
1198    00:00:00
1199    00:00:00
1200    00:00:00
1201    00:00:00
1202    00:00:00
1203    00:00:00
1204    00:00:00
1205    00:00:00
1206    00:00:00
1207    00:00:00
1208    00:00:00
1209    00:00:00
1210    00:00:00
Name: Длительность постобработки, dtype: object

##### Чистая длительность разговора

In [ ]:
Paveletskaya_df['Чистая длительность разговора'].head()

0    00:00:00
1    00:00:07
2    00:00:22
3    00:00:59
4    00:00:01
Name: Чистая длительность разговора, dtype: object

In [ ]:
print(Paveletskaya_df['Чистая длительность разговора'].count())
print(Paveletskaya_df['Чистая длительность разговора'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Чистая длительность разговора'
Paveletskaya_df['Чистая длительность разговора'] = pd.to_datetime(Paveletskaya_df['Чистая длительность разговора']).dt.time

In [ ]:
Paveletskaya_df['Чистая длительность разговора'].tail(20)

1191    00:00:00
1192    00:00:13
1193    00:00:05
1194    00:00:06
1195    00:00:44
1196    00:00:00
1197    00:00:01
1198    00:00:04
1199    00:00:06
1200    00:00:06
1201    00:00:00
1202    00:00:02
1203    00:00:06
1204    00:00:12
1205    00:00:12
1206    00:00:12
1207    00:00:00
1208    00:00:11
1209    00:00:58
1210    00:00:04
Name: Чистая длительность разговора, dtype: object

##### Полная длительность ожидания

In [ ]:
Paveletskaya_df['Полная длительность ожидания'].head()

0    00:00:01
1    00:00:03
2    00:00:01
3    00:00:01
4    00:00:01
Name: Полная длительность ожидания, dtype: object

In [ ]:
print(Paveletskaya_df['Полная длительность ожидания'].count())
print(Paveletskaya_df['Полная длительность ожидания'].shape[0])

1211
1211


In [ ]:
# Приводим тип данных столбца 'Полная длительность ожидания'
Paveletskaya_df['Полная длительность ожидания'] = pd.to_datetime(Paveletskaya_df['Полная длительность ожидания']).dt.time

In [ ]:
Paveletskaya_df['Полная длительность ожидания'].tail(20)

1191    00:00:00
1192    00:00:01
1193    00:00:02
1194    00:00:02
1195    00:00:01
1196    00:00:01
1197    00:00:04
1198    00:00:02
1199    00:00:01
1200    00:00:01
1201    00:00:01
1202    00:00:01
1203    00:00:02
1204    00:00:02
1205    00:00:02
1206    00:00:02
1207    00:00:00
1208    00:00:03
1209    00:00:02
1210    00:00:02
Name: Полная длительность ожидания, dtype: object

##### Расширенная UTM-метка Source

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Source'].head()

0    yandex
1    yandex
2    yandex
3    yandex
4    yandex
Name: Расширенная UTM-метка Source, dtype: object

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Source'].count())
print(Paveletskaya_df['Расширенная UTM-метка Source'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Source'
Paveletskaya_df['Расширенная UTM-метка Source'] = Paveletskaya_df['Расширенная UTM-метка Source'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Source'].count())
print(Paveletskaya_df['Расширенная UTM-метка Source'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Source'][Paveletskaya_df['Расширенная UTM-метка Source'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Source'].tail(20)

1191    yandex
1192    yandex
1193    yandex
1194    yandex
1195    yandex
1196    yandex
1197    yandex
1198    yandex
1199    yandex
1200    yandex
1201    yandex
1202    yandex
1203    yandex
1204    yandex
1205    yandex
1206    yandex
1207    yandex
1208    yandex
1209    yandex
1210    yandex
Name: Расширенная UTM-метка Source, dtype: object

##### Расширенная UTM-метка Campaign

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Campaign'].head()

0             ya_paveletskaya_kz_n_geo
1             ya_paveletskaya_kz_n_geo
2    ya_paveletskaya_msk_r_retargeting
3             ya_paveletskaya_kz_n_geo
4             ya_paveletskaya_kz_n_geo
Name: Расширенная UTM-метка Campaign, dtype: object

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Campaign'].count())
print(Paveletskaya_df['Расширенная UTM-метка Campaign'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Campaign'
Paveletskaya_df['Расширенная UTM-метка Campaign'] = Paveletskaya_df['Расширенная UTM-метка Campaign'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Campaign'].count())
print(Paveletskaya_df['Расширенная UTM-метка Campaign'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Campaign'][Paveletskaya_df['Расширенная UTM-метка Campaign'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Campaign'].tail(20)

1191         ya_paveletskaya_rf_n_general
1192         ya_paveletskaya_rf_n_general
1193         ya_paveletskaya_rf_n_general
1194         ya_paveletskaya_rf_n_general
1195         ya_paveletskaya_rf_n_general
1196    ya_paveletskaya_msk_r_retargeting
1197    ya_paveletskaya_msk_r_retargeting
1198         ya_paveletskaya_rf_n_general
1199    ya_paveletskaya_msk_r_retargeting
1200    ya_paveletskaya_msk_r_retargeting
1201    ya_paveletskaya_msk_r_retargeting
1202         ya_paveletskaya_rf_n_general
1203    ya_paveletskaya_msk_r_retargeting
1204     ya_paveletskaya_msk_n_konkurenty
1205             ya_paveletskaya_kz_n_geo
1206    ya_paveletskaya_msk_r_retargeting
1207             ya_paveletskaya_kz_n_geo
1208             ya_paveletskaya_kz_n_geo
1209             ya_paveletskaya_kz_n_geo
1210         ya_paveletskaya_rf_n_general
Name: Расширенная UTM-метка Campaign, dtype: object

##### Расширенная UTM-метка Medium

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Medium'].head()

0    cpc
1    cpc
2    cpc
3    cpc
4    cpc
Name: Расширенная UTM-метка Medium, dtype: object

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Medium'].count())
print(Paveletskaya_df['Расширенная UTM-метка Medium'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Medium'
Paveletskaya_df['Расширенная UTM-метка Medium'] = Paveletskaya_df['Расширенная UTM-метка Medium'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Medium'].count())
print(Paveletskaya_df['Расширенная UTM-метка Medium'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Medium'][Paveletskaya_df['Расширенная UTM-метка Medium'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Medium'].tail(20)

1191    cpc
1192    cpc
1193    cpc
1194    cpc
1195    cpc
1196    cpc
1197    cpc
1198    cpc
1199    cpc
1200    cpc
1201    cpc
1202    cpc
1203    cpc
1204    cpc
1205    cpc
1206    cpc
1207    cpc
1208    cpc
1209    cpc
1210    cpc
Name: Расширенная UTM-метка Medium, dtype: object

##### Расширенная UTM-метка Content

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Content'].head()

0    v2||64424254||11012716629||33325751517||жилые комплексы на юге москвы||0||none||com.imo.android.imoim||context||no||re_33325751517||turbo
1                 v2||64424254||13819492116||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo
2                                               v2||50264576||11707219000||42082330||||0||none||ex.24smi.info||context||no||re_42082330||turbo
3                 v2||64424254||13819492123||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo
4             v2||64424254||11012716629||33325751427||купить двушку на юге москвы||0||none||seeziskids.com||context||no||re_33325751427||turbo
Name: Расширенная UTM-метка Content, dtype: object

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Content'].count())
print(Paveletskaya_df['Расширенная UTM-метка Content'].shape[0])

1208
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Content'
Paveletskaya_df['Расширенная UTM-метка Content'] = Paveletskaya_df['Расширенная UTM-метка Content'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Content'].count())
print(Paveletskaya_df['Расширенная UTM-метка Content'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Content'][Paveletskaya_df['Расширенная UTM-метка Content'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Content'].tail(20)

1191                             v2||53841693||11012736873||37976139827||---autotargeting||1||none||kinopoisk.ru||context||no||re_37976139827||turbo
1192                             v2||53841693||11012736873||37976139827||---autotargeting||1||none||kinopoisk.ru||context||no||re_37976139827||turbo
1193                v2||53841693||11012736871||44050210577||купить квартиру рядом школа||0||none||game.yandex.ru||context||no||re_44050210577||turbo
1194                v2||53841693||11012736871||44050210577||купить квартиру рядом школа||0||none||game.yandex.ru||context||no||re_44050210577||turbo
1195                    v2||53841693||11012736871||44050210576||купить квартиру паркинг||0||none||game.yandex.ru||context||no||re_44050210576||turbo
1196                                       v2||50264576||11707218990||42082330||||1||none||com.blotclubam.bazarblot||context||no||re_42082330||turbo
1197                                                        v2||50264576||11707218991||42082330||||0||none

##### Расширенная UTM-метка Term

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Term'].head()

0    жилые комплексы на юге москвы
1                 ---autotargeting
2                              NaN
3                 ---autotargeting
4      купить двушку на юге москвы
Name: Расширенная UTM-метка Term, dtype: object

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Term'].count())
print(Paveletskaya_df['Расширенная UTM-метка Term'].shape[0])

1200
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Term'
Paveletskaya_df['Расширенная UTM-метка Term'] = Paveletskaya_df['Расширенная UTM-метка Term'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Term'].count())
print(Paveletskaya_df['Расширенная UTM-метка Term'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Term'][Paveletskaya_df['Расширенная UTM-метка Term'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Term'].tail(20)

1191                        ---autotargeting
1192                        ---autotargeting
1193             купить квартиру рядом школа
1194             купить квартиру рядом школа
1195                 купить квартиру паркинг
1196                                     нет
1197                                     нет
1198                        ---autotargeting
1199                                     нет
1200                                     нет
1201                                     нет
1202                        ---autotargeting
1203                                     нет
1204              sydney city жк цена москва
1205    коммерческая недвижимость москва юао
1206                                     нет
1207                        ---autotargeting
1208                        ---autotargeting
1209                  комплекс москва москва
1210                     жилье бизнес-класса
Name: Расширенная UTM-метка Term, dtype: object

##### Расширенная UTM-метка Referrer

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Referrer'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Расширенная UTM-метка Referrer, dtype: float64

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Referrer'].count())
print(Paveletskaya_df['Расширенная UTM-метка Referrer'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Referrer'
Paveletskaya_df['Расширенная UTM-метка Referrer'] = Paveletskaya_df['Расширенная UTM-метка Referrer'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Referrer'].count())
print(Paveletskaya_df['Расширенная UTM-метка Referrer'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Referrer'][Paveletskaya_df['Расширенная UTM-метка Referrer'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Referrer'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Расширенная UTM-метка Referrer, dtype: object

##### Расширенная UTM-метка Expid

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Expid'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Расширенная UTM-метка Expid, dtype: float64

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Expid'].count())
print(Paveletskaya_df['Расширенная UTM-метка Expid'].shape[0])

0
1211


In [ ]:
# Избавляемся от пустот в столбце 'Расширенная UTM-метка Expid'
Paveletskaya_df['Расширенная UTM-метка Expid'] = Paveletskaya_df['Расширенная UTM-метка Expid'].fillna('нет')

In [ ]:
print(Paveletskaya_df['Расширенная UTM-метка Expid'].count())
print(Paveletskaya_df['Расширенная UTM-метка Expid'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['Расширенная UTM-метка Expid'][Paveletskaya_df['Расширенная UTM-метка Expid'] == ''].count()

0

In [ ]:
Paveletskaya_df['Расширенная UTM-метка Expid'].tail(20)

1191    нет
1192    нет
1193    нет
1194    нет
1195    нет
1196    нет
1197    нет
1198    нет
1199    нет
1200    нет
1201    нет
1202    нет
1203    нет
1204    нет
1205    нет
1206    нет
1207    нет
1208    нет
1209    нет
1210    нет
Name: Расширенная UTM-метка Expid, dtype: object

##### OS-метка service-name

In [ ]:
Paveletskaya_df['OS-метка service-name'].tail(20)

1191    NaN
1192    NaN
1193    NaN
1194    NaN
1195    NaN
1196    NaN
1197    NaN
1198    NaN
1199    NaN
1200    NaN
1201    NaN
1202    NaN
1203    NaN
1204    NaN
1205    NaN
1206    NaN
1207    NaN
1208    NaN
1209    NaN
1210    NaN
Name: OS-метка service-name, dtype: object

In [ ]:
print(Paveletskaya_df['OS-метка service-name'].count())
print(Paveletskaya_df['OS-метка service-name'].shape[0])

4
1211


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка service-name'
Paveletskaya_df['OS-метка service-name'] = Paveletskaya_df['OS-метка service-name'].fillna('нет')

In [ ]:
print(Paveletskaya_df['OS-метка service-name'].count())
print(Paveletskaya_df['OS-метка service-name'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['OS-метка service-name'][Paveletskaya_df['OS-метка service-name'] == ''].count()

0

In [ ]:
Paveletskaya_df['OS-метка service-name'][Paveletskaya_df['OS-метка service-name'] == 'нет'].count()

1207

In [ ]:
Paveletskaya_df['OS-метка service-name'][Paveletskaya_df['OS-метка service-name'] != 'нет'].count()

4

In [ ]:
Paveletskaya_df['OS-метка service-name'][Paveletskaya_df['OS-метка service-name'] != 'нет']

2     direct.yandex.ru
4     direct.yandex.ru
7     direct.yandex.ru
12    direct.yandex.ru
Name: OS-метка service-name, dtype: object

In [ ]:
Paveletskaya_df['OS-метка service-name'].head()

0                 нет
1                 нет
2    direct.yandex.ru
3                 нет
4    direct.yandex.ru
Name: OS-метка service-name, dtype: object

##### OS-метка campaign-id

In [ ]:
Paveletskaya_df['OS-метка campaign-id'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: OS-метка campaign-id, dtype: float64

In [ ]:
print(Paveletskaya_df['OS-метка campaign-id'].count())
print(Paveletskaya_df['OS-метка campaign-id'].shape[0])

4
1211


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка campaign-id'
Paveletskaya_df['OS-метка campaign-id'] = Paveletskaya_df['OS-метка campaign-id'].fillna('0')

In [ ]:
print(Paveletskaya_df['OS-метка campaign-id'].count())
print(Paveletskaya_df['OS-метка campaign-id'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['OS-метка campaign-id'][Paveletskaya_df['OS-метка service-name'] == ''].count()

0

In [ ]:
Paveletskaya_df['OS-метка campaign-id'][Paveletskaya_df['OS-метка campaign-id'] != '0']

2     50264576.0
4     64424254.0
7     64424254.0
12    64424254.0
Name: OS-метка campaign-id, dtype: object

In [ ]:
# Приводим тип данных столбца 'OS-метка campaign-id'
Paveletskaya_df = Paveletskaya_df.astype({'OS-метка campaign-id': int})

In [ ]:
Paveletskaya_df['OS-метка campaign-id'].head()

0           0
1           0
2    50264576
3           0
4    64424254
Name: OS-метка campaign-id, dtype: int64

##### OS-метка ad-id

In [ ]:
Paveletskaya_df['OS-метка ad-id'].tail(20)

1191   NaN
1192   NaN
1193   NaN
1194   NaN
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
1200   NaN
1201   NaN
1202   NaN
1203   NaN
1204   NaN
1205   NaN
1206   NaN
1207   NaN
1208   NaN
1209   NaN
1210   NaN
Name: OS-метка ad-id, dtype: float64

In [ ]:
print(Paveletskaya_df['OS-метка ad-id'].count())
print(Paveletskaya_df['OS-метка ad-id'].shape[0])

4
1211


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка ad-id'
Paveletskaya_df['OS-метка ad-id'] = Paveletskaya_df['OS-метка ad-id'].fillna('0')

In [ ]:
print(Paveletskaya_df['OS-метка ad-id'].count())
print(Paveletskaya_df['OS-метка ad-id'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['OS-метка ad-id'][Paveletskaya_df['OS-метка ad-id'] == ''].count()

0

In [ ]:
Paveletskaya_df['OS-метка ad-id'][Paveletskaya_df['OS-метка ad-id'] != '0']

2     11707219000.0
4     11012716629.0
7     11012716797.0
12    11012716797.0
Name: OS-метка ad-id, dtype: object

In [ ]:
# Приводим тип данных столбца 'OS-метка ad-id'
Paveletskaya_df = Paveletskaya_df.astype({'OS-метка ad-id': int})

In [ ]:
Paveletskaya_df['OS-метка ad-id'].head()

0              0
1              0
2    11707219000
3              0
4    11012716629
Name: OS-метка ad-id, dtype: int64

##### OS-метка source-id

In [ ]:
Paveletskaya_df['OS-метка source-id'].tail(20)

1191    NaN
1192    NaN
1193    NaN
1194    NaN
1195    NaN
1196    NaN
1197    NaN
1198    NaN
1199    NaN
1200    NaN
1201    NaN
1202    NaN
1203    NaN
1204    NaN
1205    NaN
1206    NaN
1207    NaN
1208    NaN
1209    NaN
1210    NaN
Name: OS-метка source-id, dtype: object

In [ ]:
print(Paveletskaya_df['OS-метка source-id'].count())
print(Paveletskaya_df['OS-метка source-id'].shape[0])

4
1211


In [ ]:
# Избавляемся от пустот в столбце 'OS-метка source-id'
Paveletskaya_df['OS-метка source-id'] = Paveletskaya_df['OS-метка source-id'].fillna('нет')

In [ ]:
print(Paveletskaya_df['OS-метка source-id'].count())
print(Paveletskaya_df['OS-метка source-id'].shape[0])

1211
1211


In [ ]:
Paveletskaya_df['OS-метка source-id'][Paveletskaya_df['OS-метка source-id'] == ''].count()

0

In [ ]:
Paveletskaya_df['OS-метка source-id'][Paveletskaya_df['OS-метка source-id'] == 'нет'].count()

1207

In [ ]:
Paveletskaya_df['OS-метка source-id'][Paveletskaya_df['OS-метка source-id'] != 'нет'].count()

4

In [ ]:
Paveletskaya_df['OS-метка source-id'][Paveletskaya_df['OS-метка source-id'] != 'нет']

2           ex.24smi.info:guarantee
4               yandex.ru:guarantee
7     minecraftandroid.ru:guarantee
12              yandex.ru:guarantee
Name: OS-метка source-id, dtype: object

In [ ]:
Paveletskaya_df['OS-метка source-id'].head()

0                        нет
1                        нет
2    ex.24smi.info:guarantee
3                        нет
4        yandex.ru:guarantee
Name: OS-метка source-id, dtype: object

##### Типы данных

In [ ]:
# Смотрим типы данных
Paveletskaya_calls_match_df.dtypes

Дата и время                   datetime64[ns]
Номер абонента                          int64
Номер сотрудника                        int64
Идентификатор сессии звонка             int64
Файл записи звонка                     object
dtype: object

In [ ]:
# Смотрим типы данных
Paveletskaya_df.dtypes

Статус                                             object
Тип                                                object
Дата и время                               datetime64[ns]
Номер абонента                                      int64
Рекламная кампания                                 object
Длительность звонка                                object
Длительность ожидания ответа                       object
Сотрудник                                          object
UTM-метка Campaign                                 object
Теги                                               object
Номер обращения                                     int64
UTM-метка Term                                     object
Причина завершения                                 object
Виртуальный номер                                   int64
UTM-метка Source                                   object
UTM-метка Medium                                   object
UTM-метка Content                                  object
Client ID Янде

##### Добавление столбцов

In [ ]:
# Упорядочиваем столбцы в Paveletskaya_df как в Primavera_df
Paveletskaya_df = Paveletskaya_df[Primavera_df.columns[:-1]]

In [ ]:
# Смотрим несовпадающие столбцы Paveletskaya_calls_match_df vs Paveletskaya_df
for column in Paveletskaya_calls_match_df.columns.tolist():
  if column not in Paveletskaya_df.columns.tolist():
    print(f'Paveletskaya_calls_match_df[{column}], {Paveletskaya_calls_match_df[column].shape[0]} строк, тип данных: {Paveletskaya_calls_match_df[column].dtypes}')

Paveletskaya_calls_match_df[Файл записи звонка], 8 строк, тип данных: object


In [ ]:
# Сортируем Paveletskaya_df по столбцу 'Идентификатор сессии звонка'
Paveletskaya_df = Paveletskaya_df.sort_values('Идентификатор сессии звонка')

In [ ]:
# Индексируем по порядку
Paveletskaya_df.index = range(Paveletskaya_df.shape[0])

In [ ]:
# Добавляем относительные пути в столбец 'Файл записи звонка' в Paveletskaya_calls_match_df
Paveletskaya_calls_match_df['Файл записи звонка'] = Paveletskaya_calls_match_df['Файл записи звонка'].apply(lambda x: 'Записи звонков/Записи звонков_павелецкая сити/' + x)
Paveletskaya_calls_match_df['Файл записи звонка']

965    Записи звонков/Записи звонков_павелецкая сити/2023-06-19_11-27-46.734461_from_79781935179_to_74955141111_session_3032161583_talk.mp3
819    Записи звонков/Записи звонков_павелецкая сити/2023-06-19_20-04-39.492518_from_79274574090_to_74955141111_session_3033747752_talk.mp3
770    Записи звонков/Записи звонков_павелецкая сити/2023-06-24_15-26-51.254630_from_79778960807_to_74955141111_session_3045004274_talk.mp3
630    Записи звонков/Записи звонков_павелецкая сити/2023-06-27_15-04-43.293132_from_79036175926_to_74955141111_session_3051069194_talk.mp3
342    Записи звонков/Записи звонков_павелецкая сити/2023-07-11_18-13-13.999706_from_79274147009_to_74955141111_session_3081712381_talk.mp3
157    Записи звонков/Записи звонков_павелецкая сити/2023-07-14_14-11-49.712590_from_79029502470_to_74955141111_session_3095279810_talk.mp3
130    Записи звонков/Записи звонков_павелецкая сити/2023-07-18_13-51-59.127269_from_78005055934_to_74955141111_session_3098031427_talk.mp3
103    Записи звонко

In [ ]:
# Индексируем по порядку
Paveletskaya_calls_match_df.index = range(Paveletskaya_calls_match_df.shape[0])

In [ ]:
# Добавляем столбец 'Файл записи звонка' в Paveletskaya_df
Paveletskaya_df = Paveletskaya_df.merge(Paveletskaya_calls_match_df[['Идентификатор сессии звонка','Файл записи звонка']], how='outer', on=['Идентификатор сессии звонка'])

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Paveletskaya_df['Файл записи звонка'].count())
print(Paveletskaya_df['Файл записи звонка'].shape[0])

8
1211


In [ ]:
# Запоняем пустоты в столбце 'Файл записи звонка' значением 'нет файла'
Paveletskaya_df['Файл записи звонка'] = Paveletskaya_df['Файл записи звонка'].fillna('нет файла')

In [ ]:
# Смотрим долю непустых записей в столбце 'Файл записи звонка'
print(Paveletskaya_df['Файл записи звонка'].count())
print(Paveletskaya_df['Файл записи звонка'].shape[0])

1211
1211


In [ ]:
# Нет файлов
Paveletskaya_df['Файл записи звонка'][Paveletskaya_df['Файл записи звонка'] == 'нет файла'].count()

1203

In [ ]:
# Есть файлы
Paveletskaya_df['Файл записи звонка'][Paveletskaya_df['Файл записи звонка'] != 'нет файла'].count()

8

##### Сохранение датафреймов

In [ ]:
# Сохраняем Paveletskaya_df в файл
with open(df_path + 'Paveletskaya_df.pkl', 'wb') as f:
  pkl.dump(Paveletskaya_df, f)

In [ ]:
# Сохраняем Paveletskaya_calls_match_df в файл
with open(df_path + 'Paveletskaya_calls_match_df.pkl', 'wb') as f:
  pkl.dump(Paveletskaya_calls_match_df, f)

### Соединение датафреймов

In [ ]:
# Смотрим размеры датафреймов
print(Headliner_df.shape)
print(Primavera_df.shape)
print(Paveletskaya_city_df.shape)
print(Paveletskaya_df.shape)

(4759, 75)
(3506, 75)
(9327, 75)
(1211, 75)


In [ ]:
# Соединяем датафреймы
total_df = pd.concat([Headliner_df, Primavera_df, Paveletskaya_city_df, Paveletskaya_df])
total_df.index = range(total_df.shape[0])

In [ ]:
# Смотрим размер общего датафрейма
total_df.shape

(18803, 75)

In [ ]:
total_df.head()

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Файл записи звонка
0,Принятый,Звонок ВАТС,2023-05-01 09:44:05,79777662131,Медийная реклама // kvartirny-control // Спец,00:08:33,00:00:01,Метриум Вертушка 74997552563,нет,Первичный целевой - 2 очередь - ипотека - 3 очередь - 2-ком,...,нет,нет,нет,нет,нет,нет,0,0,нет,нет файла
1,Принятый,Звонок ВАТС,2023-05-01 11:24:16,79643463441,Базы // Avito,00:00:35,00:00:31,Метриум Вертушка 74997552563,нет,спорный - нет соединения,...,нет,нет,нет,нет,нет,нет,0,0,нет,нет файла
2,Принятый,Динамический коллтрекинг,2023-05-01 12:26:22,74991352444,Яндекс Дисплей,00:00:11,00:00:05,Capital 7 (495) 960 77 74,нет,спорный - нет соединения,...,нет,нет,нет,нет,нет,нет,0,0,нет,нет файла
3,Принятый,Динамический коллтрекинг,2023-05-01 15:19:49,79268860117,Яндекс Директ,00:01:55,00:00:13,Capital 2,brend_search_mmo_apartment_site|85809506,Первичный целевой - 2 очередь - запись на просмотр - 3 очередь - 3-ком - 4-ком,...,cpc,id|44083963593_44083963593|cid|85809506|gid|5166376911|aid|13836463984|adp|no|pos|premium1|src|search_none|dvc|mobile|coef|0,44083963593_44083963593,нет,нет,нет,0,0,нет,нет файла
4,Принятый,Звонок ВАТС,2023-05-01 16:43:57,79859404929,Медийная реклама // Roomberry // звонок с карточки,00:00:57,00:00:00,Кортрос,нет,Лид - ПБ - 2 очередь - 2-ком - Условно целевой,...,нет,нет,нет,нет,нет,нет,0,0,нет,нет файла


In [ ]:
total_df.tail()

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Medium,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Файл записи звонка
18798,Принятый,Динамический коллтрекинг,2023-07-18 13:51:58,78005055934,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:02,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,Нецелевой контакт,...,cpc,v2||64424254||11012716629||33325751427||купить двушку на юге москвы||0||none||seeziskids.com||context||no||re_33325751427||turbo,купить двушку на юге москвы,нет,нет,direct.yandex.ru,64424254,11012716629,yandex.ru:guarantee,Записи звонков/Записи звонков_павелецкая сити/2023-07-18_13-51-59.127269_from_78005055934_to_74955141111_session_3098031427_talk.mp3
18799,Принятый,Динамический коллтрекинг,2023-07-18 14:54:11,79680573141,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:01:00,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,нет,...,cpc,v2||64424254||13819492123||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo,---autotargeting,нет,нет,нет,0,0,нет,Записи звонков/Записи звонков_павелецкая сити/2023-07-18_14-54-12.903688_from_79680573141_to_74955141111_session_3098289147_talk.mp3
18800,Принятый,Динамический коллтрекинг,2023-07-20 19:14:10,79331115364,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:23,00:00:01,74955141111.0,ya_paveletskaya_msk_r_retargeting,Нецелевой контакт,...,cpc,v2||50264576||11707219000||42082330||||0||none||ex.24smi.info||context||no||re_42082330||turbo,нет,нет,нет,direct.yandex.ru,50264576,11707219000,ex.24smi.info:guarantee,нет файла
18801,Принятый,Динамический коллтрекинг,2023-07-26 12:57:19,375295142037,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:10,00:00:03,74955141111.0,ya_paveletskaya_kz_n_geo,Нецелевой контакт,...,cpc,v2||64424254||13819492116||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo,---autotargeting,нет,нет,нет,0,0,нет,нет файла
18802,Потерянный,Динамический коллтрекинг,2023-07-27 14:53:43,79894365488,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:01,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,Не обработано - Нецелевой контакт,...,cpc,v2||64424254||11012716629||33325751517||жилые комплексы на юге москвы||0||none||com.imo.android.imoim||context||no||re_33325751517||turbo,жилые комплексы на юге москвы,нет,нет,нет,0,0,нет,нет файла


In [ ]:
# Сохраняем total_df в файл
with open(df_path + 'total_df.pkl', 'wb') as f:
  pkl.dump(total_df, f)

## Создание сводной таблицы с информацией о звонках, у которых нет файла записи звонка

### Загрузка общего датафрейма с информацией о звонках

In [ ]:
# Загружаем total_df из файла
with open(df_path + 'total_df.pkl', 'rb') as f:
  total_df = pkl.load(f)

### Выборка из общего датафрейма с информацией о звонках

In [ ]:
# Получаем датафрейм total_not_match_df
total_not_match_df = total_df[total_df['Файл записи звонка'] == 'нет файла']

In [ ]:
# Смотрим размер total_not_match_df
total_not_match_df.shape

(14249, 75)

In [ ]:
# Сохраняем total_not_match_df в файл
with open(df_path + 'total_not_match_df.pkl', 'wb') as f:
  pkl.dump(total_not_match_df, f)

In [ ]:
# Сохраняем total_not_match_df в файл Microsoft Excell
with pd.ExcelWriter(table_path + 'Информация о звонках в csv-файлах без файлов записи звонков.xlsx') as excel_writer:
  total_not_match_df.to_excel(excel_writer, sheet_name='Информация о звонках в csv-файлах без файлов записи звонков', index=False)

## Создание сводной таблицы с информацией о звонках, у которых есть файл записи звонка

### Загрузка общего датафрейма с информацией о звонках

In [ ]:
# Загружаем total_df из файла
with open(df_path + 'total_df.pkl', 'rb') as f:
  total_df = pkl.load(f)

### Выборка из общего датафрейма с информацией о звонках

In [ ]:
# Получаем датафрейм total_match_df
total_match_df = total_df[total_df['Файл записи звонка'] != 'нет файла']

In [ ]:
# Смотрим размер total_match_df
total_match_df.shape

(4554, 75)

In [ ]:
# Сохраняем total_match_df в файл
with open(df_path + 'total_match_df.pkl', 'wb') as f:
  pkl.dump(total_match_df, f)

In [ ]:
# Сохраняем total_match_df в файл Microsoft Excell
with pd.ExcelWriter(table_path + 'Информация о звонках в csv-файлах с файлами записи звонков.xlsx') as excel_writer:
  total_match_df.to_excel(excel_writer, sheet_name='Информация о звонках в csv-файлах с файлами записи звонков', index=False)

# 4 неделя

In [ ]:
!rm -r sample_data

In [ ]:
# Подключаем google-диск
from google.colab import drive
drive.mount('/content/drive')
my_drv_path = '/content/drive/MyDrive/'

Mounted at /content/drive


## Библиотеки и переменные

In [ ]:
import pickle as pkl # для сохранения и загрузки переменных
import pandas as pd  # для работы с таблицами

# Параметры отображения датафреймов
pd.options.display.max_rows     = 99
pd.options.display.max_colwidth = 999

df_path           = my_drv_path + 'media108.ru/Датафреймы/' # путь к папке для датафреймов
table_path        = my_drv_path + 'media108.ru/Таблицы/'    # путь к папке для таблиц

## Создание сводной таблицы с информацией о звонках

### Загрузка датафреймов

In [ ]:
# Загружаем Headliner_df из файла
with open(df_path + 'Headliner_df.pkl', 'rb') as f:
  Headliner_df = pkl.load(f)

# Загружаем Primavera_df из файла
with open(df_path + 'Primavera_df.pkl', 'rb') as f:
  Primavera_df = pkl.load(f)

# Загружаем Paveletskaya_city_df из файла
with open(df_path + 'Paveletskaya_city_df.pkl', 'rb') as f:
  Paveletskaya_city_df = pkl.load(f)

# Загружаем Paveletskaya_df из файла
with open(df_path + 'Paveletskaya_df.pkl', 'rb') as f:
  Paveletskaya_df = pkl.load(f)

### Добавление в датафреймы столбца 'Файл информации о звонках'

In [ ]:
# Добавляем в датафреймы столбец 'Файл информации о звонках'
Headliner_df['Файл информации о звонках']         = 'Headliner.csv'
Primavera_df['Файл информации о звонках']         = 'Primavera .csv'
Paveletskaya_city_df['Файл информации о звонках'] = 'Павелецкая_ЖК_«Павелецкая_сити».csv'
Paveletskaya_df['Файл информации о звонках']      = 'Павелецкая (pavcity.turbo.site).csv'

### Соединение датафреймов

In [ ]:
# Смотрим размеры датафреймов
print(Headliner_df.shape)
print(Primavera_df.shape)
print(Paveletskaya_city_df.shape)
print(Paveletskaya_df.shape)

(4759, 76)
(3506, 76)
(9327, 76)
(1211, 76)


In [ ]:
# Соединяем датафреймы
total_df = pd.concat([Headliner_df, Primavera_df, Paveletskaya_city_df, Paveletskaya_df])
total_df.index = range(total_df.shape[0])

In [ ]:
# Смотрим размер общего датафрейма
total_df.shape

(18803, 76)

In [ ]:
total_df.head()

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Файл записи звонка,Файл информации о звонках
0,Принятый,Звонок ВАТС,2023-05-01 09:44:05,79777662131,Медийная реклама // kvartirny-control // Спец,00:08:33,00:00:01,Метриум Вертушка 74997552563,нет,Первичный целевой - 2 очередь - ипотека - 3 очередь - 2-ком,...,нет,нет,нет,нет,нет,0,0,нет,нет файла,Headliner.csv
1,Принятый,Звонок ВАТС,2023-05-01 11:24:16,79643463441,Базы // Avito,00:00:35,00:00:31,Метриум Вертушка 74997552563,нет,спорный - нет соединения,...,нет,нет,нет,нет,нет,0,0,нет,нет файла,Headliner.csv
2,Принятый,Динамический коллтрекинг,2023-05-01 12:26:22,74991352444,Яндекс Дисплей,00:00:11,00:00:05,Capital 7 (495) 960 77 74,нет,спорный - нет соединения,...,нет,нет,нет,нет,нет,0,0,нет,нет файла,Headliner.csv
3,Принятый,Динамический коллтрекинг,2023-05-01 15:19:49,79268860117,Яндекс Директ,00:01:55,00:00:13,Capital 2,brend_search_mmo_apartment_site|85809506,Первичный целевой - 2 очередь - запись на просмотр - 3 очередь - 3-ком - 4-ком,...,id|44083963593_44083963593|cid|85809506|gid|5166376911|aid|13836463984|adp|no|pos|premium1|src|search_none|dvc|mobile|coef|0,44083963593_44083963593,нет,нет,нет,0,0,нет,нет файла,Headliner.csv
4,Принятый,Звонок ВАТС,2023-05-01 16:43:57,79859404929,Медийная реклама // Roomberry // звонок с карточки,00:00:57,00:00:00,Кортрос,нет,Лид - ПБ - 2 очередь - 2-ком - Условно целевой,...,нет,нет,нет,нет,нет,0,0,нет,нет файла,Headliner.csv


In [ ]:
total_df.tail()

,Статус,Тип,Дата и время,Номер абонента,Рекламная кампания,Длительность звонка,Длительность ожидания ответа,Сотрудник,UTM-метка Campaign,Теги,...,Расширенная UTM-метка Content,Расширенная UTM-метка Term,Расширенная UTM-метка Referrer,Расширенная UTM-метка Expid,OS-метка service-name,OS-метка campaign-id,OS-метка ad-id,OS-метка source-id,Файл записи звонка,Файл информации о звонках
18798,Принятый,Динамический коллтрекинг,2023-07-18 13:51:58,78005055934,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:02,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,Нецелевой контакт,...,v2||64424254||11012716629||33325751427||купить двушку на юге москвы||0||none||seeziskids.com||context||no||re_33325751427||turbo,купить двушку на юге москвы,нет,нет,direct.yandex.ru,64424254,11012716629,yandex.ru:guarantee,Записи звонков/Записи звонков_павелецкая сити/2023-07-18_13-51-59.127269_from_78005055934_to_74955141111_session_3098031427_talk.mp3,Павелецкая (pavcity.turbo.site).csv
18799,Принятый,Динамический коллтрекинг,2023-07-18 14:54:11,79680573141,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:01:00,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,нет,...,v2||64424254||13819492123||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo,---autotargeting,нет,нет,нет,0,0,нет,Записи звонков/Записи звонков_павелецкая сити/2023-07-18_14-54-12.903688_from_79680573141_to_74955141111_session_3098289147_talk.mp3,Павелецкая (pavcity.turbo.site).csv
18800,Принятый,Динамический коллтрекинг,2023-07-20 19:14:10,79331115364,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:23,00:00:01,74955141111.0,ya_paveletskaya_msk_r_retargeting,Нецелевой контакт,...,v2||50264576||11707219000||42082330||||0||none||ex.24smi.info||context||no||re_42082330||turbo,нет,нет,нет,direct.yandex.ru,50264576,11707219000,ex.24smi.info:guarantee,нет файла,Павелецкая (pavcity.turbo.site).csv
18801,Принятый,Динамический коллтрекинг,2023-07-26 12:57:19,375295142037,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:10,00:00:03,74955141111.0,ya_paveletskaya_kz_n_geo,Нецелевой контакт,...,v2||64424254||13819492116||44050215064||---autotargeting||0||none||com.imo.android.imoim||context||no||re_44050215064||turbo,---autotargeting,нет,нет,нет,0,0,нет,нет файла,Павелецкая (pavcity.turbo.site).csv
18802,Потерянный,Динамический коллтрекинг,2023-07-27 14:53:43,79894365488,Artics | Павелецкая Сити | direct | Яндекс.Директ (турбо),00:00:01,00:00:01,74955141111.0,ya_paveletskaya_kz_n_geo,Не обработано - Нецелевой контакт,...,v2||64424254||11012716629||33325751517||жилые комплексы на юге москвы||0||none||com.imo.android.imoim||context||no||re_33325751517||turbo,жилые комплексы на юге москвы,нет,нет,нет,0,0,нет,нет файла,Павелецкая (pavcity.turbo.site).csv


In [ ]:
# Сохраняем total_df в файл
with open(df_path + 'total_df.pkl', 'wb') as f:
  pkl.dump(total_df, f)

## Создание сводной таблицы с информацией о звонках, у которых нет файла записи звонка

### Загрузка общего датафрейма с информацией о звонках

In [ ]:
# Загружаем total_df из файла
with open(df_path + 'total_df.pkl', 'rb') as f:
  total_df = pkl.load(f)

### Выборка из общего датафрейма с информацией о звонках

In [ ]:
# Получаем датафрейм total_not_match_df
total_not_match_df = total_df[total_df['Файл записи звонка'] == 'нет файла']

In [ ]:
# Смотрим размер total_not_match_df
total_not_match_df.shape

(14241, 76)

In [ ]:
# Сохраняем total_not_match_df в файл
with open(df_path + 'total_not_match_df.pkl', 'wb') as f:
  pkl.dump(total_not_match_df, f)

In [ ]:
# Сохраняем total_not_match_df в файл Microsoft Excell
with pd.ExcelWriter(table_path + 'Информация о звонках в csv-файлах без файлов записи звонков.xlsx') as excel_writer:
  total_not_match_df.to_excel(excel_writer, sheet_name='Информация о звонках в csv-файлах без файлов записи звонков', index=False)

## Создание сводной таблицы с информацией о звонках, у которых есть файл записи звонка

### Загрузка общего датафрейма с информацией о звонках

In [ ]:
# Загружаем total_df из файла
with open(df_path + 'total_df.pkl', 'rb') as f:
  total_df = pkl.load(f)

### Выборка из общего датафрейма с информацией о звонках

In [ ]:
# Получаем датафрейм total_match_df
total_match_df = total_df[total_df['Файл записи звонка'] != 'нет файла']

In [ ]:
# Смотрим размер total_match_df
total_match_df.shape

(4562, 76)

In [ ]:
# Сохраняем total_match_df в файл
with open(df_path + 'total_match_df.pkl', 'wb') as f:
  pkl.dump(total_match_df, f)

In [ ]:
# Сохраняем total_match_df в файл Microsoft Excell
with pd.ExcelWriter(table_path + 'Информация о звонках в csv-файлах с файлами записи звонков.xlsx') as excel_writer:
  total_match_df.to_excel(excel_writer, sheet_name='Информация о звонках в csv-файлах с файлами записи звонков', index=False)